A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
test_RoughHeston_20200616_E_vanilla_HestonCalib220530.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 3;

priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

%SX5E
% targetDateint = [20190228;20190221;20190131;20190128;20190116;20190108;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];


%HSCEI
% targetDateint = [20190228;20190222;20190219;20190211;20190131;20190129;20190122;20190116;20190109;20190108;20190104;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];

valueDateH = H_Date('2020-07-08');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [30
91
182
365
730
% 1095
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2020-07-08');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;

    
    %% SX5E Model Building Start
    %% zeroCurve
%     rawData = [0.00273973	0.00371595
% 0.08333300	0.00394045
% 0.16666700	0.00408717
% 0.25000000	0.00423357
% 0.50000000	0.00613963
% 0.75000000	0.00583531
% 1.00000000	0.00547714
% 1.25000000	0.00514254
% 1.50000000	0.00481191
% 1.75000000	0.00448580
% 2.00000000	0.00416366
% 2.25000000	0.00406663
% 2.50000000	0.00396960
% 2.75000000	0.00387351
% 3.00000000	0.00377741
% 3.25000000	0.00381506
% 3.50000000	0.00385271
% 3.75000000	0.00389075
% 4.00000000	0.00392878
%     ];

rawData = [0.00273973	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For SX5E
    rawData = 1.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SX5E

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
112	0.555139	0.485217	0.404103	0.3382	0.283587	0.232006	0.18761	0.167861	0.171241	0.183719	0.198589
203	0.502264	0.432085	0.36965	0.317463	0.274925	0.234267	0.1959	0.168019	0.159601	0.162847	0.171302
294	0.458209	0.398372	0.346175	0.303478	0.267304	0.232795	0.198822	0.172078	0.160959	0.159914	0.16477
385	0.42799	0.376421	0.3317	0.294388	0.262087	0.231102	0.201085	0.176483	0.165888	0.162597	0.164797
567	0.387911	0.346273	0.310367	0.279611	0.252505	0.226926	0.202399	0.181555	0.169772	0.16446	0.163933
756	0.36288	0.327345	0.296277	0.26861	0.243774	0.221585	0.202259	0.186415	0.174872	0.168036	0.165384
    ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','SX5EVolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SX5E
%     rawData = [0	0.0303
%     ];

    rawData = [0	0.0
        ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EDividendCurve'});
    dividendCurveMktData = MktData(mktData);
    dividendCurveMktData.params('dividendType') = 'dividendYield';
    
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SX5E

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;
   

   %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsDupire= containers.Map({'modelName'},{'Black'});
    SX5EBlack = EQBlack(EQModel(Model(AllMktData,modelParamsDupire))); 
    maturities = SX5EBlack.volSurface.maturities;
    strikes =    SX5EBlack.volSurface.strikes;
    impVolSurface = SX5EBlack.volSurface.vol;
    spot = SX5EBlack.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));

    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    SX5EImpliedVolSurface =SX5EBlack.volSurface;

    localVolSurfaceDupire.expiry = maturities;
    localVolSurfaceDupire.strike = strikes;
    localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
%     localVolSurfaceDupire.strikeType = 'fwdMoneyness';
    localVolSurfaceDupire.strikeType = 'spotMoneyness';

    % localVolSurfaceDupire.strikeType = 'fixedStrike';
    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurfaceDupire,'Dupire'});
    %% SS SABR vol calibration setting
    mcOneTimeStepTest = 3;
    NMCTest = 2e5;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

    modelParamsDupire('NMC') = NMCTest;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    modelParamsDupire('maxIter') = 1000;

    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    % modelParamsDupire('slope') = 1.0;
    modelParamsDupire('slope') = 0.5;

    %% SS SABR params

%     modelParamsDupire('alpha_0') = 0.3;
%     modelParamsDupire('alpha_inf') = 0.243;
%     modelParamsDupire('rho_SS')   = -0.778;
%     modelParamsDupire('nu') = 0.901;
    
    modelParamsDupire('alpha_0') = 0.195616389248541;
    modelParamsDupire('alpha_inf') = 0.196132174974662;
    modelParamsDupire('rho_SS')   = -0.135998259486906;
    modelParamsDupire('nu') = 0.347183843323165;
    
    modelParamsDupire('beta') = 1.0;
    modelParamsDupire('lambda') = 0.7;
    modelParamsDupire('omega')   = 0.6;
    modelParamsDupire('time_shift') = 0.1667;

    %% Ornstein-Uhlenbeck params
    modelParamsDupire('speed_OU') = 0.1;
    modelParamsDupire('level_OU') = 0.5;
    modelParamsDupire('vol_OU')   = 0.3;
    modelParamsDupire('x0_OU') = 1.0;

    % modelParamsDupire('lagrangeOrder') = 4;
    % 
    % modelParamsDupire('HermiteNormalXBar')  =[-2.334414218
    % -0.741963784
    % 0.741963784
    % 2.334414218
    % ];

    modelParamsDupire('lagrangeOrder') = 9;

    modelParamsDupire('HermiteNormalXBar')  =[-4.512745864
-3.205429004
-2.076847978
-1.023255664
0
1.023255664
2.076847978
3.205429004
4.512745864
];

 modelParamsDupire('gridStreching_sigma') = 1.37143571;

    %     modelParamsDupire('maturityTimes') = [ 0.05
%     0.25
%     0.5
%     1
%     2
%     3
%     4
%     ];

    modelParamsDupire('maturityTimes') = maturities/365.0;


    modelParamsDupire('lb') = [1.00E-08
    1.00E-08
    -0.9999
    1.00E-08
    1.00E-08
    1.00E-08
    % 1.00E-08
    ];

    %alpah_0
    %alpha_inf
    %rho
    %nu
    %lambda
    %omega

    modelParamsDupire('ub') = [100
    100
    0.9999
    100
    100.0
    100
    % 100
    ];


    calibratedParams = [0.308809422
    0.216171194
    -0.545484863
    0.765996606
    1.034002631
    0.630720371
    0.462011155
    ];


    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParamsDupire('calibrationErrorType') = 'priceError';

%     modelParamsDupire('reference') = 3234.07;
    modelParamsDupire('reference') = 1.0;
    
    %% Heston modelparams

%     modelParams= containers.Map({'localVolSurface','modelName'}, ...
%                             {localVolSurface,'Heston'});

    % we use fixed kappa  
%     modelParamsDupire('kappa') = 2.0;
    modelParamsDupire('kappa') = 4.24786769742683;
    modelParamsDupire('theta') = 0.0406289993407702;
    modelParamsDupire('rho')   = -0.141521862284196;
    modelParamsDupire('sigma') = 0.422008324;
    modelParamsDupire('v0')    = 0.0389493800866629;
    
        modelParamsDummy = [
    3.080265858
0.078074529
-0.780027041
1.41323435
0.080569293
        ];
    
    
    modelParams('kappa') = modelParamsDummy(1);
    modelParams('theta') = modelParamsDummy(2);
    modelParams('rho')    = modelParamsDummy(3);
    modelParams('sigma') = modelParamsDummy(4);
    modelParams('v0')   = modelParamsDummy(5);

    modelParamsDupire('H')    = 0.1;
     
    
    modelParamsDupire('tol') = 1e-10;
    modelParamsDupire('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParamsDupire('quadrature_nodesAndWeights') = laguerre192;


    modelParamsDupire('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParamsDupire('quadrature_NumOfWeights') = 32;

    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

%     modelParamsDupire('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
%     modelParamsDupire('modelPriceFormula') = 'GatheralR';
%     modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33';
    modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33Amax';
    
%     modelParamsDupire('modelPriceFormula') = 'EER18Sigurdroemer';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParamsDupire('strikeUseBound') = strikeUseBound;

    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParamsDupire('useQuasiRandomYN') = 'NO';
    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParamsDupire('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParamsDupire('smoothWeightKappa') = 0.001;




    % SX5EDupire = EQCOMDupireSSSABR(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
    % SX5EDupire = EQCOMDupireSSSABRRaw(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
%     SX5EDupire = EQCOMDupireSSSABRRawNormalCLVHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));
    SX5EDupire = EQCOMDupireSSSABRRawNormalCLVRoughHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    %% localVolSurfaceParams
    detFwd    = zeros(length(expiry),1);
    atmFwdVol = zeros(length(expiry),1);
    atmSpotVol = zeros(length(expiry),1);
    fwdMoneynessVolSurface = zeros(length(expiry),length(strikes));
    spotVolSurface = zeros(length(expiry),length(strikes));
    for idxDum=1:length(expiry)
        detFwd(idxDum) = SX5EDupire.FwdWithoutSpot(expiry(idxDum));
        dummyOut = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum),expiry(idxDum)/365.0);
        atmFwdVol(idxDum) = dummyOut.vol;
        
        dummyOut1 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot,expiry(idxDum)/365.0);
        atmSpotVol(idxDum) = dummyOut1.vol;
        
        for idxDum2=1:length(strikes)
            dummyOut2 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum)*strikes(idxDum2),expiry(idxDum)/365.0);
            fwdMoneynessVolSurface(idxDum,idxDum2) = dummyOut2.vol;
            
            dummyOut3 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot*strikes(idxDum2),expiry(idxDum)/365.0);
            spotVolSurface(idxDum,idxDum2) = dummyOut3.vol;
        end
    end
    
    atmVols = zeros(length(expiry),2);
    for idxDum=1:length(expiry)
        atmVols(idxDum,1) = expiry(idxDum);
        atmVols(idxDum,2) = atmFwdVol(idxDum);
    end
    
%     atmVols = [30	0.357
%     91	0.34
%     182	0.3165
%     365	0.284
%     730	0.247
%     1095	0.228
%     ];

    refExpiryH = localVolSurfaceDupire.expiry;
    settleFutures = zeros(length(refExpiryH),1);
    settleFuturesRef = zeros(length(refExpiryH),1);

    referenceSpot = SX5EDupire.modelParams('reference');

    spotRawData = SX5EDupire.spot.params('rawData');
    for idxhhh=1:length(refExpiryH)
    %     settleFutures(idxhh) = SX5EDupire.FwdWithoutSpot(refExpiryH(idxhh));
        settleFutures(idxhhh) = spotRawData;
    %     settleFuturesRef(idxhh) = SX5EDupire.Fwd(referenceSpot,refExpiryH(idxhh));
        settleFuturesRef(idxhhh) = modelParamsDupire('reference');
    end

    modelParamsDupire('settleFutures') = settleFutures;
    modelParamsDupire('settleFuturesRef') = settleFuturesRef;
    modelParamsDupire('atmVols') = atmVols;

    SX5EDupire.settleFutures = settleFutures;
    SX5EDupire.settleFuturesRef = settleFuturesRef;
    SX5EDupire.atmVols = atmVols;

    tic
    useSSSABRAutoCallScheduleFlag = true;
    
    LocalVolEndDate = vanillaParams.startDate.AddDate(targetMaturities(end),'day');
    autoCallScheduleInt =[FngDate(LocalVolEndDate)	0.875	0.0405
    ];

    manualScheduleYN = 1;
    dailyTimeStepsYN = 'NO';
    stepdownInfo = containers.Map({'dailyTimeStepsYN',...
                                    'autoCallScheduleInt','manualScheduleYN'}, ...
                            {dailyTimeStepsYN,...
                            autoCallScheduleInt,manualScheduleYN});

    stepdownParams.params = stepdownInfo;  

%     outLocalVolSS = SX5EDupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    outLocalVolSS = SX5EDupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    
%     out = SX5EDupire.CalibrateToVolSurface(SX5EBlack,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = SX5EDupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    % pricingOutLV = SX5EDupire.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = SX5EDupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
%     localVolSurface.volProxy = out.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParams)));
%     SX5EHeston = EQCOMDupireSSSABRRawNormalCLVHeston(EQModel(Model(AllMktData,modelParamsDupire)));

%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalKappaFixed');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'global');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalHFixed');
    
%       outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemer');
      outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemerHFixed');
      
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalGA');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParamsDupire('kappa') = modelParamsDummy(1);
    modelParamsDupire('theta') = modelParamsDummy(2);
    modelParamsDupire('rho')    = modelParamsDummy(3);
    modelParamsDupire('sigma') = modelParamsDummy(4);
    modelParamsDupire('v0')   = modelParamsDummy(5);

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    pricingOutHeston = SX5EDupire.computeForwardStartingVanillaMCHeston(valueDateH,vanillaParams);
    % pricingOutHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = SX5EDupire.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

%     modelParamsDupire('kappa') = modelParamsDummy(1);
%     modelParamsDupire('theta') = modelParamsDummy(2);
%     modelParamsDupire('rho')    = modelParamsDummy(3);
%     modelParamsDupire('sigma') = modelParamsDummy(4)*mixingWeight;
%     modelParamsDupire('v0')   = modelParamsDummy(5);
%     
%     SX5EHestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    SX5EDupire.sigma = modelParamsDummy(4) * mixingWeight;
    
    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = SX5EDupire.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = SX5EDupire.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = SX5EDupire.computeForwardStartingVanillaMCHestonSLV(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = SX5EDupire.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end

aaa = 1.0;

ccc = 1.0;
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalGA');

% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaFixed');
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaConstrained');
bbb = 1.0;

priceSimul = zeros(length(expiry),101);
impVolSimul = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);

% price simulation
bbb =  1.0;
spot =  SX5EHestonSLV.spot.params('rawData');

for j=1:length(expiry)
    fwd = SX5EHestonSLV.Fwd(spot,expiry(j));
    df = SX5EHestonSLV.zeroCurve.DF(expiry(j)/365.0);
    for k=1:length(strikes)
        tic
        relStrike = strikes(k);
        if relStrike <= 1.0
            vanillaParams.params('callPutFlag') = 'P';
            callputStringhsw = 'Put';
        else
            vanillaParams.params('callPutFlag') = 'C';
            callputStringhsw = 'Call';
        end
        
        priceSimul(j,k)  = SX5EHeston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
        impVolSimul(j,k) = SX5EHeston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        SX5EHestonSLV.modelParams('MCScheme') = 'QE';
        SX5EHeston.modelParams('MCScheme') = 'QE';

        vanillaParams.params('strike') = fwd*relStrike;
        vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
        
        pricingOutMCHestonSLV = SX5EHestonSLV.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
        impVolSimulQESLV(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        pricingOutMCHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
        impVolSimulQE(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        toc
    end
end


fff = 1.0;

In [ ]:
test_RoughHeston_20200608_E_vanilla_HestonCalib220530.m


clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 3;

priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

%SX5E
% targetDateint = [20190228;20190221;20190131;20190128;20190116;20190108;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];


%HSCEI
% targetDateint = [20190228;20190222;20190219;20190211;20190131;20190129;20190122;20190116;20190109;20190108;20190104;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];

valueDateH = H_Date('2020-07-08');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [30
91
182
365
730
% 1095
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2020-07-08');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;

    
    %% SX5E Model Building Start
    %% zeroCurve
%     rawData = [0.00273973	0.00371595
% 0.08333300	0.00394045
% 0.16666700	0.00408717
% 0.25000000	0.00423357
% 0.50000000	0.00613963
% 0.75000000	0.00583531
% 1.00000000	0.00547714
% 1.25000000	0.00514254
% 1.50000000	0.00481191
% 1.75000000	0.00448580
% 2.00000000	0.00416366
% 2.25000000	0.00406663
% 2.50000000	0.00396960
% 2.75000000	0.00387351
% 3.00000000	0.00377741
% 3.25000000	0.00381506
% 3.50000000	0.00385271
% 3.75000000	0.00389075
% 4.00000000	0.00392878
%     ];

rawData = [0.00273973	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For SX5E
    rawData = 1.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SX5E
%     rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.772935544	0.660794663	0.560134289	0.466982478	0.3786614	0.294696393	0.227406437	0.212764901	0.231856547	0.257759106	0.283989413
% 91	0.64653478	0.560555897	0.483733518	0.413219942	0.347314997	0.285909471	0.233960992	0.207079785	0.208453133	0.221892285	0.238619703
% 182	0.552759946	0.485667786	0.426044378	0.37177818	0.321697021	0.275690437	0.235822475	0.208170248	0.198039125	0.200806015	0.209403502
% 365	0.461746233	0.412244058	0.36858374	0.329259583	0.29345123	0.26095042	0.232357252	0.209398474	0.194397922	0.187919169	0.187806662
% 730	0.383471893	0.348399204	0.317745566	0.29043906	0.265869603	0.243759238	0.224127195	0.207284638	0.193758099	0.184014699	0.178066211
% 1095	0.346099384	0.317848635	0.293299804	0.271568435	0.252131066	0.234699663	0.219162731	0.205551542	0.194000167	0.184671543	0.177647363
%     ];

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
30	0.331083003	0.29058414	0.256378003	0.22822257	0.207404552	0.196392987	0.196136294	0.203716755	0.215149667	0.227933788	0.240894057
91	0.290920149	0.261085991	0.236697005	0.217532087	0.204061972	0.196873201	0.195723121	0.199198657	0.20550901	0.21326486	0.221616654
182	0.263239929	0.241298399	0.223968702	0.210882014	0.201952352	0.19703482	0.195666263	0.197069072	0.200383539	0.2048831	0.210046601
365	0.239422421	0.224822532	0.213744953	0.205671075	0.200226572	0.197052991	0.195761823	0.195949415	0.19723705	0.199303184	0.201895308
730	0.222209005	0.213328989	0.206807937	0.202137928	0.198949287	0.196948426	0.195891832	0.19557575	0.195831542	0.196522176	0.197538381
% 1095	0.215097926	0.20867642	0.204012109	0.200671445	0.19835158	0.196827815	0.195927348	0.195514793	0.195483246	0.19574812	0.196242523
    ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','SX5EVolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SX5E
%     rawData = [0	0.0303
%     ];

    rawData = [0	0.0
        ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EDividendCurve'});
    dividendCurveMktData = MktData(mktData);
    dividendCurveMktData.params('dividendType') = 'dividendYield';
    
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SX5E

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;
   

   %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsDupire= containers.Map({'modelName'},{'Black'});
    SX5EBlack = EQBlack(EQModel(Model(AllMktData,modelParamsDupire))); 
    maturities = SX5EBlack.volSurface.maturities;
    strikes =    SX5EBlack.volSurface.strikes;
    impVolSurface = SX5EBlack.volSurface.vol;
    spot = SX5EBlack.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));

    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    SX5EImpliedVolSurface =SX5EBlack.volSurface;

    localVolSurfaceDupire.expiry = maturities;
    localVolSurfaceDupire.strike = strikes;
    localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
%     localVolSurfaceDupire.strikeType = 'fwdMoneyness';
    localVolSurfaceDupire.strikeType = 'spotMoneyness';

    % localVolSurfaceDupire.strikeType = 'fixedStrike';
    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurfaceDupire,'Dupire'});
    %% SS SABR vol calibration setting
    mcOneTimeStepTest = 3;
    NMCTest = 2e5;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

    modelParamsDupire('NMC') = NMCTest;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    modelParamsDupire('maxIter') = 1000;

    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    % modelParamsDupire('slope') = 1.0;
    modelParamsDupire('slope') = 0.5;

    %% SS SABR params

%     modelParamsDupire('alpha_0') = 0.3;
%     modelParamsDupire('alpha_inf') = 0.243;
%     modelParamsDupire('rho_SS')   = -0.778;
%     modelParamsDupire('nu') = 0.901;
    
    modelParamsDupire('alpha_0') = 0.195616389248541;
    modelParamsDupire('alpha_inf') = 0.196132174974662;
    modelParamsDupire('rho_SS')   = -0.135998259486906;
    modelParamsDupire('nu') = 0.347183843323165;
    
    modelParamsDupire('beta') = 1.0;
    modelParamsDupire('lambda') = 0.7;
    modelParamsDupire('omega')   = 0.6;
    modelParamsDupire('time_shift') = 0.1667;

    %% Ornstein-Uhlenbeck params
    modelParamsDupire('speed_OU') = 0.1;
    modelParamsDupire('level_OU') = 0.5;
    modelParamsDupire('vol_OU')   = 0.3;
    modelParamsDupire('x0_OU') = 1.0;

    % modelParamsDupire('lagrangeOrder') = 4;
    % 
    % modelParamsDupire('HermiteNormalXBar')  =[-2.334414218
    % -0.741963784
    % 0.741963784
    % 2.334414218
    % ];

    modelParamsDupire('lagrangeOrder') = 9;

    modelParamsDupire('HermiteNormalXBar')  =[-4.512745864
-3.205429004
-2.076847978
-1.023255664
0
1.023255664
2.076847978
3.205429004
4.512745864
];

 modelParamsDupire('gridStreching_sigma') = 1.37143571;

    %     modelParamsDupire('maturityTimes') = [ 0.05
%     0.25
%     0.5
%     1
%     2
%     3
%     4
%     ];

    modelParamsDupire('maturityTimes') = maturities/365.0;


    modelParamsDupire('lb') = [1.00E-08
    1.00E-08
    -0.9999
    1.00E-08
    1.00E-08
    1.00E-08
    % 1.00E-08
    ];

    %alpah_0
    %alpha_inf
    %rho
    %nu
    %lambda
    %omega

    modelParamsDupire('ub') = [100
    100
    0.9999
    100
    100.0
    100
    % 100
    ];


    calibratedParams = [0.308809422
    0.216171194
    -0.545484863
    0.765996606
    1.034002631
    0.630720371
    0.462011155
    ];


    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParamsDupire('calibrationErrorType') = 'priceError';

%     modelParamsDupire('reference') = 3234.07;
    modelParamsDupire('reference') = 1.0;
    
    %% Heston modelparams

%     modelParams= containers.Map({'localVolSurface','modelName'}, ...
%                             {localVolSurface,'Heston'});

    % we use fixed kappa  
%     modelParamsDupire('kappa') = 2.0;
    modelParamsDupire('kappa') = 4.24786769742683;

    modelParamsDupire('theta') = 0.0406289993407702;
    modelParamsDupire('rho')   = -0.141521862284196;
    modelParamsDupire('sigma') = 0.422008324;
    modelParamsDupire('v0')    = 0.0389493800866629;

    modelParamsDupire('H')    = 0.1;
     
    
    modelParamsDupire('tol') = 1e-10;
    modelParamsDupire('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParamsDupire('quadrature_nodesAndWeights') = laguerre192;


    modelParamsDupire('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParamsDupire('quadrature_NumOfWeights') = 32;

    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

%     modelParamsDupire('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
%     modelParamsDupire('modelPriceFormula') = 'GatheralR';
%     modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33';
    modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33Amax';
    
%     modelParamsDupire('modelPriceFormula') = 'EER18Sigurdroemer';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParamsDupire('strikeUseBound') = strikeUseBound;

    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParamsDupire('useQuasiRandomYN') = 'NO';
    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParamsDupire('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParamsDupire('smoothWeightKappa') = 0.001;

%     modelParamsDummy = [2.0
% 0.074403
% -0.535078
% 1.180945
% 0.0858942
%         ];
%     
%     
%     modelParams('kappa') = modelParamsDummy(1);
%     modelParams('theta') = modelParamsDummy(2);
%     modelParams('rho')    = modelParamsDummy(3);
%     modelParams('sigma') = modelParamsDummy(4);
%     modelParams('v0')   = modelParamsDummy(5);


    % SX5EDupire = EQCOMDupireSSSABR(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
    % SX5EDupire = EQCOMDupireSSSABRRaw(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
%     SX5EDupire = EQCOMDupireSSSABRRawNormalCLVHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));
    SX5EDupire = EQCOMDupireSSSABRRawNormalCLVRoughHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    %% localVolSurfaceParams
    detFwd    = zeros(length(expiry),1);
    atmFwdVol = zeros(length(expiry),1);
    atmSpotVol = zeros(length(expiry),1);
    fwdMoneynessVolSurface = zeros(length(expiry),length(strikes));
    spotVolSurface = zeros(length(expiry),length(strikes));
    for idxDum=1:length(expiry)
        detFwd(idxDum) = SX5EDupire.FwdWithoutSpot(expiry(idxDum));
        dummyOut = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum),expiry(idxDum)/365.0);
        atmFwdVol(idxDum) = dummyOut.vol;
        
        dummyOut1 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot,expiry(idxDum)/365.0);
        atmSpotVol(idxDum) = dummyOut1.vol;
        
        for idxDum2=1:length(strikes)
            dummyOut2 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum)*strikes(idxDum2),expiry(idxDum)/365.0);
            fwdMoneynessVolSurface(idxDum,idxDum2) = dummyOut2.vol;
            
            dummyOut3 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot*strikes(idxDum2),expiry(idxDum)/365.0);
            spotVolSurface(idxDum,idxDum2) = dummyOut3.vol;
        end
    end
    
    atmVols = zeros(length(expiry),2);
    for idxDum=1:length(expiry)
        atmVols(idxDum,1) = expiry(idxDum);
        atmVols(idxDum,2) = atmFwdVol(idxDum);
    end
    
%     atmVols = [30	0.357
%     91	0.34
%     182	0.3165
%     365	0.284
%     730	0.247
%     1095	0.228
%     ];

    refExpiryH = localVolSurfaceDupire.expiry;
    settleFutures = zeros(length(refExpiryH),1);
    settleFuturesRef = zeros(length(refExpiryH),1);

    referenceSpot = SX5EDupire.modelParams('reference');

    spotRawData = SX5EDupire.spot.params('rawData');
    for idxhhh=1:length(refExpiryH)
    %     settleFutures(idxhh) = SX5EDupire.FwdWithoutSpot(refExpiryH(idxhh));
        settleFutures(idxhhh) = spotRawData;
    %     settleFuturesRef(idxhh) = SX5EDupire.Fwd(referenceSpot,refExpiryH(idxhh));
        settleFuturesRef(idxhhh) = modelParamsDupire('reference');
    end

    modelParamsDupire('settleFutures') = settleFutures;
    modelParamsDupire('settleFuturesRef') = settleFuturesRef;
    modelParamsDupire('atmVols') = atmVols;

    SX5EDupire.settleFutures = settleFutures;
    SX5EDupire.settleFuturesRef = settleFuturesRef;
    SX5EDupire.atmVols = atmVols;

    tic
    useSSSABRAutoCallScheduleFlag = true;
    
    LocalVolEndDate = vanillaParams.startDate.AddDate(targetMaturities(end),'day');
    autoCallScheduleInt =[FngDate(LocalVolEndDate)	0.875	0.0405
    ];

    manualScheduleYN = 1;
    dailyTimeStepsYN = 'NO';
    stepdownInfo = containers.Map({'dailyTimeStepsYN',...
                                    'autoCallScheduleInt','manualScheduleYN'}, ...
                            {dailyTimeStepsYN,...
                            autoCallScheduleInt,manualScheduleYN});

    stepdownParams.params = stepdownInfo;  

%     outLocalVolSS = SX5EDupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    outLocalVolSS = SX5EDupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    
%     out = SX5EDupire.CalibrateToVolSurface(SX5EBlack,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = SX5EDupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    % pricingOutLV = SX5EDupire.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = SX5EDupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
%     localVolSurface.volProxy = out.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParams)));
%     SX5EHeston = EQCOMDupireSSSABRRawNormalCLVHeston(EQModel(Model(AllMktData,modelParamsDupire)));

%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalKappaFixed');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'global');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalHFixed');
    
      outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemer');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalGA');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParamsDupire('kappa') = modelParamsDummy(1);
    modelParamsDupire('theta') = modelParamsDummy(2);
    modelParamsDupire('rho')    = modelParamsDummy(3);
    modelParamsDupire('sigma') = modelParamsDummy(4);
    modelParamsDupire('v0')   = modelParamsDummy(5);

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    pricingOutHeston = SX5EDupire.computeForwardStartingVanillaMCHeston(valueDateH,vanillaParams);
    % pricingOutHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = SX5EDupire.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

%     modelParamsDupire('kappa') = modelParamsDummy(1);
%     modelParamsDupire('theta') = modelParamsDummy(2);
%     modelParamsDupire('rho')    = modelParamsDummy(3);
%     modelParamsDupire('sigma') = modelParamsDummy(4)*mixingWeight;
%     modelParamsDupire('v0')   = modelParamsDummy(5);
%     
%     SX5EHestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    SX5EDupire.sigma = modelParamsDummy(4) * mixingWeight;
    
    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = SX5EDupire.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = SX5EDupire.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = SX5EDupire.computeForwardStartingVanillaMCHestonSLV(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = SX5EDupire.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end

aaa = 1.0;

ccc = 1.0;
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalGA');

% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaFixed');
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaConstrained');
bbb = 1.0;

priceSimul = zeros(length(expiry),101);
impVolSimul = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);

% price simulation
bbb =  1.0;
spot =  SX5EHestonSLV.spot.params('rawData');

for j=1:length(expiry)
    fwd = SX5EHestonSLV.Fwd(spot,expiry(j));
    df = SX5EHestonSLV.zeroCurve.DF(expiry(j)/365.0);
    for k=1:length(strikes)
        tic
        relStrike = strikes(k);
        if relStrike <= 1.0
            vanillaParams.params('callPutFlag') = 'P';
            callputStringhsw = 'Put';
        else
            vanillaParams.params('callPutFlag') = 'C';
            callputStringhsw = 'Call';
        end
        
        priceSimul(j,k)  = SX5EHeston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
        impVolSimul(j,k) = SX5EHeston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        SX5EHestonSLV.modelParams('MCScheme') = 'QE';
        SX5EHeston.modelParams('MCScheme') = 'QE';

        vanillaParams.params('strike') = fwd*relStrike;
        vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
        
        pricingOutMCHestonSLV = SX5EHestonSLV.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
        impVolSimulQESLV(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        pricingOutMCHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
        impVolSimulQE(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        toc
    end
end


fff = 1.0;

In [ ]:

test_RoughHeston_20200608_E_vanilla_HestonCalib220405.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 3;

priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

%SX5E
% targetDateint = [20190228;20190221;20190131;20190128;20190116;20190108;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];


%HSCEI
% targetDateint = [20190228;20190222;20190219;20190211;20190131;20190129;20190122;20190116;20190109;20190108;20190104;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];

valueDateH = H_Date('2020-07-08');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [30
91
182
365
730
% 1095
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2020-07-08');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;

    
    %% SX5E Model Building Start
    %% zeroCurve
%     rawData = [0.00273973	0.00371595
% 0.08333300	0.00394045
% 0.16666700	0.00408717
% 0.25000000	0.00423357
% 0.50000000	0.00613963
% 0.75000000	0.00583531
% 1.00000000	0.00547714
% 1.25000000	0.00514254
% 1.50000000	0.00481191
% 1.75000000	0.00448580
% 2.00000000	0.00416366
% 2.25000000	0.00406663
% 2.50000000	0.00396960
% 2.75000000	0.00387351
% 3.00000000	0.00377741
% 3.25000000	0.00381506
% 3.50000000	0.00385271
% 3.75000000	0.00389075
% 4.00000000	0.00392878
%     ];

rawData = [0.00273973	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For SX5E
    rawData = 1.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SX5E
%     rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.772935544	0.660794663	0.560134289	0.466982478	0.3786614	0.294696393	0.227406437	0.212764901	0.231856547	0.257759106	0.283989413
% 91	0.64653478	0.560555897	0.483733518	0.413219942	0.347314997	0.285909471	0.233960992	0.207079785	0.208453133	0.221892285	0.238619703
% 182	0.552759946	0.485667786	0.426044378	0.37177818	0.321697021	0.275690437	0.235822475	0.208170248	0.198039125	0.200806015	0.209403502
% 365	0.461746233	0.412244058	0.36858374	0.329259583	0.29345123	0.26095042	0.232357252	0.209398474	0.194397922	0.187919169	0.187806662
% 730	0.383471893	0.348399204	0.317745566	0.29043906	0.265869603	0.243759238	0.224127195	0.207284638	0.193758099	0.184014699	0.178066211
% 1095	0.346099384	0.317848635	0.293299804	0.271568435	0.252131066	0.234699663	0.219162731	0.205551542	0.194000167	0.184671543	0.177647363
%     ];

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
30	0.331083003	0.29058414	0.256378003	0.22822257	0.207404552	0.196392987	0.196136294	0.203716755	0.215149667	0.227933788	0.240894057
91	0.290920149	0.261085991	0.236697005	0.217532087	0.204061972	0.196873201	0.195723121	0.199198657	0.20550901	0.21326486	0.221616654
182	0.263239929	0.241298399	0.223968702	0.210882014	0.201952352	0.19703482	0.195666263	0.197069072	0.200383539	0.2048831	0.210046601
365	0.239422421	0.224822532	0.213744953	0.205671075	0.200226572	0.197052991	0.195761823	0.195949415	0.19723705	0.199303184	0.201895308
730	0.222209005	0.213328989	0.206807937	0.202137928	0.198949287	0.196948426	0.195891832	0.19557575	0.195831542	0.196522176	0.197538381
% 1095	0.215097926	0.20867642	0.204012109	0.200671445	0.19835158	0.196827815	0.195927348	0.195514793	0.195483246	0.19574812	0.196242523
    ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','SX5EVolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SX5E
%     rawData = [0	0.0303
%     ];

    rawData = [0	0.0
        ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EDividendCurve'});
    dividendCurveMktData = MktData(mktData);
    dividendCurveMktData.params('dividendType') = 'dividendYield';
    
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SX5E

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;
   

   %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsDupire= containers.Map({'modelName'},{'Black'});
    SX5EBlack = EQBlack(EQModel(Model(AllMktData,modelParamsDupire))); 
    maturities = SX5EBlack.volSurface.maturities;
    strikes =    SX5EBlack.volSurface.strikes;
    impVolSurface = SX5EBlack.volSurface.vol;
    spot = SX5EBlack.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));

    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    SX5EImpliedVolSurface =SX5EBlack.volSurface;

    localVolSurfaceDupire.expiry = maturities;
    localVolSurfaceDupire.strike = strikes;
    localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
%     localVolSurfaceDupire.strikeType = 'fwdMoneyness';
    localVolSurfaceDupire.strikeType = 'spotMoneyness';

    % localVolSurfaceDupire.strikeType = 'fixedStrike';
    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurfaceDupire,'Dupire'});
    %% SS SABR vol calibration setting
    mcOneTimeStepTest = 3;
    NMCTest = 2e5;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

    modelParamsDupire('NMC') = NMCTest;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    modelParamsDupire('maxIter') = 1000;

    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    % modelParamsDupire('slope') = 1.0;
    modelParamsDupire('slope') = 0.5;

    %% SS SABR params

%     modelParamsDupire('alpha_0') = 0.3;
%     modelParamsDupire('alpha_inf') = 0.243;
%     modelParamsDupire('rho_SS')   = -0.778;
%     modelParamsDupire('nu') = 0.901;
    
    modelParamsDupire('alpha_0') = 0.195616389248541;
    modelParamsDupire('alpha_inf') = 0.196132174974662;
    modelParamsDupire('rho_SS')   = -0.135998259486906;
    modelParamsDupire('nu') = 0.347183843323165;
    
    modelParamsDupire('beta') = 1.0;
    modelParamsDupire('lambda') = 0.7;
    modelParamsDupire('omega')   = 0.6;
    modelParamsDupire('time_shift') = 0.1667;

    %% Ornstein-Uhlenbeck params
    modelParamsDupire('speed_OU') = 0.1;
    modelParamsDupire('level_OU') = 0.5;
    modelParamsDupire('vol_OU')   = 0.3;
    modelParamsDupire('x0_OU') = 1.0;

    % modelParamsDupire('lagrangeOrder') = 4;
    % 
    % modelParamsDupire('HermiteNormalXBar')  =[-2.334414218
    % -0.741963784
    % 0.741963784
    % 2.334414218
    % ];

    modelParamsDupire('lagrangeOrder') = 9;

    modelParamsDupire('HermiteNormalXBar')  =[-4.512745864
-3.205429004
-2.076847978
-1.023255664
0
1.023255664
2.076847978
3.205429004
4.512745864
];

 modelParamsDupire('gridStreching_sigma') = 1.37143571;

    %     modelParamsDupire('maturityTimes') = [ 0.05
%     0.25
%     0.5
%     1
%     2
%     3
%     4
%     ];

    modelParamsDupire('maturityTimes') = maturities/365.0;


    modelParamsDupire('lb') = [1.00E-08
    1.00E-08
    -0.9999
    1.00E-08
    1.00E-08
    1.00E-08
    % 1.00E-08
    ];

    %alpah_0
    %alpha_inf
    %rho
    %nu
    %lambda
    %omega

    modelParamsDupire('ub') = [100
    100
    0.9999
    100
    100.0
    100
    % 100
    ];


    calibratedParams = [0.308809422
    0.216171194
    -0.545484863
    0.765996606
    1.034002631
    0.630720371
    0.462011155
    ];


    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParamsDupire('calibrationErrorType') = 'priceError';

%     modelParamsDupire('reference') = 3234.07;
    modelParamsDupire('reference') = 1.0;
    
    %% Heston modelparams

%     modelParams= containers.Map({'localVolSurface','modelName'}, ...
%                             {localVolSurface,'Heston'});

    % we use fixed kappa  
%     modelParamsDupire('kappa') = 2.0;
    modelParamsDupire('kappa') = 4.24786769742683;

    modelParamsDupire('theta') = 0.0406289993407702;
    modelParamsDupire('rho')   = -0.141521862284196;
    modelParamsDupire('sigma') = 0.422008324;
    modelParamsDupire('v0')    = 0.0389493800866629;

    modelParamsDupire('H')    = 0.1;
     
    
    modelParamsDupire('tol') = 1e-10;
    modelParamsDupire('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParamsDupire('quadrature_nodesAndWeights') = laguerre192;


    modelParamsDupire('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParamsDupire('quadrature_NumOfWeights') = 32;

    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

%     modelParamsDupire('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
%     modelParamsDupire('modelPriceFormula') = 'GatheralR';
%     modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33';
    modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33Amax';
    
%     modelParamsDupire('modelPriceFormula') = 'EER18Sigurdroemer';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParamsDupire('strikeUseBound') = strikeUseBound;

    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParamsDupire('useQuasiRandomYN') = 'NO';
    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParamsDupire('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParamsDupire('smoothWeightKappa') = 0.001;

%     modelParamsDummy = [2.0
% 0.074403
% -0.535078
% 1.180945
% 0.0858942
%         ];
%     
%     
%     modelParams('kappa') = modelParamsDummy(1);
%     modelParams('theta') = modelParamsDummy(2);
%     modelParams('rho')    = modelParamsDummy(3);
%     modelParams('sigma') = modelParamsDummy(4);
%     modelParams('v0')   = modelParamsDummy(5);


    % SX5EDupire = EQCOMDupireSSSABR(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
    % SX5EDupire = EQCOMDupireSSSABRRaw(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
%     SX5EDupire = EQCOMDupireSSSABRRawNormalCLVHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));
    SX5EDupire = EQCOMDupireSSSABRRawNormalCLVRoughHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    %% localVolSurfaceParams
    detFwd    = zeros(length(expiry),1);
    atmFwdVol = zeros(length(expiry),1);
    atmSpotVol = zeros(length(expiry),1);
    fwdMoneynessVolSurface = zeros(length(expiry),length(strikes));
    spotVolSurface = zeros(length(expiry),length(strikes));
    for idxDum=1:length(expiry)
        detFwd(idxDum) = SX5EDupire.FwdWithoutSpot(expiry(idxDum));
        dummyOut = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum),expiry(idxDum)/365.0);
        atmFwdVol(idxDum) = dummyOut.vol;
        
        dummyOut1 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot,expiry(idxDum)/365.0);
        atmSpotVol(idxDum) = dummyOut1.vol;
        
        for idxDum2=1:length(strikes)
            dummyOut2 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum)*strikes(idxDum2),expiry(idxDum)/365.0);
            fwdMoneynessVolSurface(idxDum,idxDum2) = dummyOut2.vol;
            
            dummyOut3 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot*strikes(idxDum2),expiry(idxDum)/365.0);
            spotVolSurface(idxDum,idxDum2) = dummyOut3.vol;
        end
    end
    
    atmVols = zeros(length(expiry),2);
    for idxDum=1:length(expiry)
        atmVols(idxDum,1) = expiry(idxDum);
        atmVols(idxDum,2) = atmFwdVol(idxDum);
    end
    
%     atmVols = [30	0.357
%     91	0.34
%     182	0.3165
%     365	0.284
%     730	0.247
%     1095	0.228
%     ];

    refExpiryH = localVolSurfaceDupire.expiry;
    settleFutures = zeros(length(refExpiryH),1);
    settleFuturesRef = zeros(length(refExpiryH),1);

    referenceSpot = SX5EDupire.modelParams('reference');

    spotRawData = SX5EDupire.spot.params('rawData');
    for idxhhh=1:length(refExpiryH)
    %     settleFutures(idxhh) = SX5EDupire.FwdWithoutSpot(refExpiryH(idxhh));
        settleFutures(idxhhh) = spotRawData;
    %     settleFuturesRef(idxhh) = SX5EDupire.Fwd(referenceSpot,refExpiryH(idxhh));
        settleFuturesRef(idxhhh) = modelParamsDupire('reference');
    end

    modelParamsDupire('settleFutures') = settleFutures;
    modelParamsDupire('settleFuturesRef') = settleFuturesRef;
    modelParamsDupire('atmVols') = atmVols;

    SX5EDupire.settleFutures = settleFutures;
    SX5EDupire.settleFuturesRef = settleFuturesRef;
    SX5EDupire.atmVols = atmVols;

    tic
    useSSSABRAutoCallScheduleFlag = true;
    
    LocalVolEndDate = vanillaParams.startDate.AddDate(targetMaturities(end),'day');
    autoCallScheduleInt =[FngDate(LocalVolEndDate)	0.875	0.0405
    ];

    manualScheduleYN = 1;
    dailyTimeStepsYN = 'NO';
    stepdownInfo = containers.Map({'dailyTimeStepsYN',...
                                    'autoCallScheduleInt','manualScheduleYN'}, ...
                            {dailyTimeStepsYN,...
                            autoCallScheduleInt,manualScheduleYN});

    stepdownParams.params = stepdownInfo;  

%     outLocalVolSS = SX5EDupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    outLocalVolSS = SX5EDupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    
%     out = SX5EDupire.CalibrateToVolSurface(SX5EBlack,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = SX5EDupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    % pricingOutLV = SX5EDupire.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = SX5EDupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
%     localVolSurface.volProxy = out.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParams)));
%     SX5EHeston = EQCOMDupireSSSABRRawNormalCLVHeston(EQModel(Model(AllMktData,modelParamsDupire)));

%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalKappaFixed');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'global');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalHFixed');
    
      outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemer');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalGA');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParamsDupire('kappa') = modelParamsDummy(1);
    modelParamsDupire('theta') = modelParamsDummy(2);
    modelParamsDupire('rho')    = modelParamsDummy(3);
    modelParamsDupire('sigma') = modelParamsDummy(4);
    modelParamsDupire('v0')   = modelParamsDummy(5);

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    pricingOutHeston = SX5EDupire.computeForwardStartingVanillaMCHeston(valueDateH,vanillaParams);
    % pricingOutHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = SX5EDupire.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

%     modelParamsDupire('kappa') = modelParamsDummy(1);
%     modelParamsDupire('theta') = modelParamsDummy(2);
%     modelParamsDupire('rho')    = modelParamsDummy(3);
%     modelParamsDupire('sigma') = modelParamsDummy(4)*mixingWeight;
%     modelParamsDupire('v0')   = modelParamsDummy(5);
%     
%     SX5EHestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    SX5EDupire.sigma = modelParamsDummy(4) * mixingWeight;
    
    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = SX5EDupire.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = SX5EDupire.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = SX5EDupire.computeForwardStartingVanillaMCHestonSLV(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = SX5EDupire.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end

aaa = 1.0;

ccc = 1.0;
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalGA');

% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaFixed');
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaConstrained');
bbb = 1.0;

priceSimul = zeros(length(expiry),101);
impVolSimul = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);

% price simulation
bbb =  1.0;
spot =  SX5EHestonSLV.spot.params('rawData');

for j=1:length(expiry)
    fwd = SX5EHestonSLV.Fwd(spot,expiry(j));
    df = SX5EHestonSLV.zeroCurve.DF(expiry(j)/365.0);
    for k=1:length(strikes)
        tic
        relStrike = strikes(k);
        if relStrike <= 1.0
            vanillaParams.params('callPutFlag') = 'P';
            callputStringhsw = 'Put';
        else
            vanillaParams.params('callPutFlag') = 'C';
            callputStringhsw = 'Call';
        end
        
        priceSimul(j,k)  = SX5EHeston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
        impVolSimul(j,k) = SX5EHeston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        SX5EHestonSLV.modelParams('MCScheme') = 'QE';
        SX5EHeston.modelParams('MCScheme') = 'QE';

        vanillaParams.params('strike') = fwd*relStrike;
        vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
        
        pricingOutMCHestonSLV = SX5EHestonSLV.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
        impVolSimulQESLV(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        pricingOutMCHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
        impVolSimulQE(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        toc
    end
end


fff = 1.0;

In [ ]:
test_RoughHeston_20200608_E_vanilla_220405_crosscheckG.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 3;

priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

%SX5E
% targetDateint = [20190228;20190221;20190131;20190128;20190116;20190108;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];


%HSCEI
% targetDateint = [20190228;20190222;20190219;20190211;20190131;20190129;20190122;20190116;20190109;20190108;20190104;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];

valueDateH = H_Date('2020-07-08');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [30
91
182
365
730
% 1095
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2020-07-08');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;

    
    %% SX5E Model Building Start
    %% zeroCurve
%     rawData = [0.00273973	0.00371595
% 0.08333300	0.00394045
% 0.16666700	0.00408717
% 0.25000000	0.00423357
% 0.50000000	0.00613963
% 0.75000000	0.00583531
% 1.00000000	0.00547714
% 1.25000000	0.00514254
% 1.50000000	0.00481191
% 1.75000000	0.00448580
% 2.00000000	0.00416366
% 2.25000000	0.00406663
% 2.50000000	0.00396960
% 2.75000000	0.00387351
% 3.00000000	0.00377741
% 3.25000000	0.00381506
% 3.50000000	0.00385271
% 3.75000000	0.00389075
% 4.00000000	0.00392878
%     ];

rawData = [0.00273973	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For SX5E
    rawData = 1.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SX5E
%     rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.772935544	0.660794663	0.560134289	0.466982478	0.3786614	0.294696393	0.227406437	0.212764901	0.231856547	0.257759106	0.283989413
% 91	0.64653478	0.560555897	0.483733518	0.413219942	0.347314997	0.285909471	0.233960992	0.207079785	0.208453133	0.221892285	0.238619703
% 182	0.552759946	0.485667786	0.426044378	0.37177818	0.321697021	0.275690437	0.235822475	0.208170248	0.198039125	0.200806015	0.209403502
% 365	0.461746233	0.412244058	0.36858374	0.329259583	0.29345123	0.26095042	0.232357252	0.209398474	0.194397922	0.187919169	0.187806662
% 730	0.383471893	0.348399204	0.317745566	0.29043906	0.265869603	0.243759238	0.224127195	0.207284638	0.193758099	0.184014699	0.178066211
% 1095	0.346099384	0.317848635	0.293299804	0.271568435	0.252131066	0.234699663	0.219162731	0.205551542	0.194000167	0.184671543	0.177647363
%     ];

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.331083003	0.29058414	0.256378003	0.22822257	0.207404552	0.196392987	0.196136294	0.203716755	0.215149667	0.227933788	0.240894057
91	0.290920149	0.261085991	0.236697005	0.217532087	0.204061972	0.196873201	0.195723121	0.199198657	0.20550901	0.21326486	0.221616654
182	0.263239929	0.241298399	0.223968702	0.210882014	0.201952352	0.19703482	0.195666263	0.197069072	0.200383539	0.2048831	0.210046601
365	0.239422421	0.224822532	0.213744953	0.205671075	0.200226572	0.197052991	0.195761823	0.195949415	0.19723705	0.199303184	0.201895308
730	0.222209005	0.213328989	0.206807937	0.202137928	0.198949287	0.196948426	0.195891832	0.19557575	0.195831542	0.196522176	0.197538381
% 1095	0.215097926	0.20867642	0.204012109	0.200671445	0.19835158	0.196827815	0.195927348	0.195514793	0.195483246	0.19574812	0.196242523
    ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','SX5EVolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SX5E
%     rawData = [0	0.0303
%     ];

    rawData = [0	0.0
        ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EDividendCurve'});
    dividendCurveMktData = MktData(mktData);
    dividendCurveMktData.params('dividendType') = 'dividendYield';
    
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SX5E

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;
   

   %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsDupire= containers.Map({'modelName'},{'Black'});
    SX5EBlack = EQBlack(EQModel(Model(AllMktData,modelParamsDupire))); 
    maturities = SX5EBlack.volSurface.maturities;
    strikes =    SX5EBlack.volSurface.strikes;
    impVolSurface = SX5EBlack.volSurface.vol;
    spot = SX5EBlack.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));

    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    SX5EImpliedVolSurface =SX5EBlack.volSurface;

    localVolSurfaceDupire.expiry = maturities;
    localVolSurfaceDupire.strike = strikes;
    localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
%     localVolSurfaceDupire.strikeType = 'fwdMoneyness';
    localVolSurfaceDupire.strikeType = 'spotMoneyness';

    % localVolSurfaceDupire.strikeType = 'fixedStrike';
    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurfaceDupire,'Dupire'});
    %% SS SABR vol calibration setting
    mcOneTimeStepTest = 3;
    NMCTest = 2e5;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

    modelParamsDupire('NMC') = NMCTest;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    modelParamsDupire('maxIter') = 1000;

    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    % modelParamsDupire('slope') = 1.0;
    modelParamsDupire('slope') = 0.5;

    %% SS SABR params

%     modelParamsDupire('alpha_0') = 0.3;
%     modelParamsDupire('alpha_inf') = 0.243;
%     modelParamsDupire('rho_SS')   = -0.778;
%     modelParamsDupire('nu') = 0.901;
    
    modelParamsDupire('alpha_0') = 0.195616389248541;
    modelParamsDupire('alpha_inf') = 0.196132174974662;
    modelParamsDupire('rho_SS')   = -0.135998259486906;
    modelParamsDupire('nu') = 0.347183843323165;
    
    modelParamsDupire('beta') = 1.0;
    modelParamsDupire('lambda') = 0.7;
    modelParamsDupire('omega')   = 0.6;
    modelParamsDupire('time_shift') = 0.1667;

    %% Ornstein-Uhlenbeck params
    modelParamsDupire('speed_OU') = 0.1;
    modelParamsDupire('level_OU') = 0.5;
    modelParamsDupire('vol_OU')   = 0.3;
    modelParamsDupire('x0_OU') = 1.0;

    % modelParamsDupire('lagrangeOrder') = 4;
    % 
    % modelParamsDupire('HermiteNormalXBar')  =[-2.334414218
    % -0.741963784
    % 0.741963784
    % 2.334414218
    % ];

    modelParamsDupire('lagrangeOrder') = 9;

    modelParamsDupire('HermiteNormalXBar')  =[-4.512745864
-3.205429004
-2.076847978
-1.023255664
0
1.023255664
2.076847978
3.205429004
4.512745864
];

 modelParamsDupire('gridStreching_sigma') = 1.37143571;

    %     modelParamsDupire('maturityTimes') = [ 0.05
%     0.25
%     0.5
%     1
%     2
%     3
%     4
%     ];

    modelParamsDupire('maturityTimes') = maturities/365.0;


    modelParamsDupire('lb') = [1.00E-08
    1.00E-08
    -0.9999
    1.00E-08
    1.00E-08
    1.00E-08
    % 1.00E-08
    ];

    %alpah_0
    %alpha_inf
    %rho
    %nu
    %lambda
    %omega

    modelParamsDupire('ub') = [100
    100
    0.9999
    100
    100.0
    100
    % 100
    ];


    calibratedParams = [0.308809422
    0.216171194
    -0.545484863
    0.765996606
    1.034002631
    0.630720371
    0.462011155
    ];


    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParamsDupire('calibrationErrorType') = 'priceError';

%     modelParamsDupire('reference') = 3234.07;
    modelParamsDupire('reference') = 1.0;
    
    %% Heston modelparams

%     modelParams= containers.Map({'localVolSurface','modelName'}, ...
%                             {localVolSurface,'Heston'});

    % we use fixed kappa  
    modelParamsDupire('kappa') = 0.0;

    modelParamsDupire('theta') = 0.025;
    modelParamsDupire('rho')   = -0.65;
%     modelParamsDupire('rho')   = -0.7;
%     modelParamsDupire('rho')   = -0.9;
%     modelParamsDupire('rho')   = -0.8;
    
    
    
    modelParamsDupire('sigma') = 0.4089;
%     modelParamsDupire('sigma') = 0.7;
%     modelParamsDupire('sigma') = 0.9;
    
    
    
    modelParamsDupire('v0')    = 0.025;

    modelParamsDupire('H')    = 0.05;
%      modelParamsDupire('H')    = 0.08;
%      modelParamsDupire('H')    = 0.05;
     
    
    modelParamsDupire('tol') = 1e-10;
    modelParamsDupire('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParamsDupire('quadrature_nodesAndWeights') = laguerre192;


    modelParamsDupire('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParamsDupire('quadrature_NumOfWeights') = 32;

    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

%     modelParamsDupire('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
%     modelParamsDupire('modelPriceFormula') = 'GatheralR';
%     modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33';
    modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33Amax';
    
%     modelParamsDupire('modelPriceFormula') = 'EER18Sigurdroemer';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParamsDupire('strikeUseBound') = strikeUseBound;

    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParamsDupire('useQuasiRandomYN') = 'NO';
    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParamsDupire('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParamsDupire('smoothWeightKappa') = 0.001;

%     modelParamsDummy = [2.0
% 0.074403
% -0.535078
% 1.180945
% 0.0858942
%         ];
%     
%     
%     modelParams('kappa') = modelParamsDummy(1);
%     modelParams('theta') = modelParamsDummy(2);
%     modelParams('rho')    = modelParamsDummy(3);
%     modelParams('sigma') = modelParamsDummy(4);
%     modelParams('v0')   = modelParamsDummy(5);


    % SX5EDupire = EQCOMDupireSSSABR(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
    % SX5EDupire = EQCOMDupireSSSABRRaw(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
%     SX5EDupire = EQCOMDupireSSSABRRawNormalCLVHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));
    SX5EDupire = EQCOMDupireSSSABRRawNormalCLVRoughHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    %% localVolSurfaceParams
    detFwd    = zeros(length(expiry),1);
    atmFwdVol = zeros(length(expiry),1);
    atmSpotVol = zeros(length(expiry),1);
    fwdMoneynessVolSurface = zeros(length(expiry),length(strikes));
    spotVolSurface = zeros(length(expiry),length(strikes));
    for idxDum=1:length(expiry)
        detFwd(idxDum) = SX5EDupire.FwdWithoutSpot(expiry(idxDum));
        dummyOut = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum),expiry(idxDum)/365.0);
        atmFwdVol(idxDum) = dummyOut.vol;
        
        dummyOut1 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot,expiry(idxDum)/365.0);
        atmSpotVol(idxDum) = dummyOut1.vol;
        
        for idxDum2=1:length(strikes)
            dummyOut2 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum)*strikes(idxDum2),expiry(idxDum)/365.0);
            fwdMoneynessVolSurface(idxDum,idxDum2) = dummyOut2.vol;
            
            dummyOut3 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot*strikes(idxDum2),expiry(idxDum)/365.0);
            spotVolSurface(idxDum,idxDum2) = dummyOut3.vol;
        end
    end
    
    atmVols = zeros(length(expiry),2);
    for idxDum=1:length(expiry)
        atmVols(idxDum,1) = expiry(idxDum);
        atmVols(idxDum,2) = atmFwdVol(idxDum);
    end
    
%     atmVols = [30	0.357
%     91	0.34
%     182	0.3165
%     365	0.284
%     730	0.247
%     1095	0.228
%     ];

    refExpiryH = localVolSurfaceDupire.expiry;
    settleFutures = zeros(length(refExpiryH),1);
    settleFuturesRef = zeros(length(refExpiryH),1);

    referenceSpot = SX5EDupire.modelParams('reference');

    spotRawData = SX5EDupire.spot.params('rawData');
    for idxhhh=1:length(refExpiryH)
    %     settleFutures(idxhh) = SX5EDupire.FwdWithoutSpot(refExpiryH(idxhh));
        settleFutures(idxhhh) = spotRawData;
    %     settleFuturesRef(idxhh) = SX5EDupire.Fwd(referenceSpot,refExpiryH(idxhh));
        settleFuturesRef(idxhhh) = modelParamsDupire('reference');
    end

    modelParamsDupire('settleFutures') = settleFutures;
    modelParamsDupire('settleFuturesRef') = settleFuturesRef;
    modelParamsDupire('atmVols') = atmVols;

    SX5EDupire.settleFutures = settleFutures;
    SX5EDupire.settleFuturesRef = settleFuturesRef;
    SX5EDupire.atmVols = atmVols;

    tic
    useSSSABRAutoCallScheduleFlag = true;
    
    LocalVolEndDate = vanillaParams.startDate.AddDate(targetMaturities(end),'day');
    autoCallScheduleInt =[FngDate(LocalVolEndDate)	0.875	0.0405
    ];

    manualScheduleYN = 1;
    dailyTimeStepsYN = 'NO';
    stepdownInfo = containers.Map({'dailyTimeStepsYN',...
                                    'autoCallScheduleInt','manualScheduleYN'}, ...
                            {dailyTimeStepsYN,...
                            autoCallScheduleInt,manualScheduleYN});

    stepdownParams.params = stepdownInfo;  

%     outLocalVolSS = SX5EDupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    outLocalVolSS = SX5EDupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    
%     out = SX5EDupire.CalibrateToVolSurface(SX5EBlack,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = SX5EDupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    % pricingOutLV = SX5EDupire.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = SX5EDupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
%     localVolSurface.volProxy = out.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParams)));
%     SX5EHeston = EQCOMDupireSSSABRRawNormalCLVHeston(EQModel(Model(AllMktData,modelParamsDupire)));

%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalKappaFixed');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'global');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalHFixed');
    
      outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemer');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalGA');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParamsDupire('kappa') = modelParamsDummy(1);
    modelParamsDupire('theta') = modelParamsDummy(2);
    modelParamsDupire('rho')    = modelParamsDummy(3);
    modelParamsDupire('sigma') = modelParamsDummy(4);
    modelParamsDupire('v0')   = modelParamsDummy(5);

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    pricingOutHeston = SX5EDupire.computeForwardStartingVanillaMCHeston(valueDateH,vanillaParams);
    % pricingOutHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = SX5EDupire.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

%     modelParamsDupire('kappa') = modelParamsDummy(1);
%     modelParamsDupire('theta') = modelParamsDummy(2);
%     modelParamsDupire('rho')    = modelParamsDummy(3);
%     modelParamsDupire('sigma') = modelParamsDummy(4)*mixingWeight;
%     modelParamsDupire('v0')   = modelParamsDummy(5);
%     
%     SX5EHestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    SX5EDupire.sigma = modelParamsDummy(4) * mixingWeight;
    
    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = SX5EDupire.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = SX5EDupire.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = SX5EDupire.computeForwardStartingVanillaMCHestonSLV(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = SX5EDupire.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end

aaa = 1.0;

ccc = 1.0;
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalGA');

% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaFixed');
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaConstrained');
bbb = 1.0;

priceSimul = zeros(length(expiry),101);
impVolSimul = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);

% price simulation
bbb =  1.0;
spot =  SX5EHestonSLV.spot.params('rawData');

for j=1:length(expiry)
    fwd = SX5EHestonSLV.Fwd(spot,expiry(j));
    df = SX5EHestonSLV.zeroCurve.DF(expiry(j)/365.0);
    for k=1:length(strikes)
        tic
        relStrike = strikes(k);
        if relStrike <= 1.0
            vanillaParams.params('callPutFlag') = 'P';
            callputStringhsw = 'Put';
        else
            vanillaParams.params('callPutFlag') = 'C';
            callputStringhsw = 'Call';
        end
        
        priceSimul(j,k)  = SX5EHeston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
        impVolSimul(j,k) = SX5EHeston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        SX5EHestonSLV.modelParams('MCScheme') = 'QE';
        SX5EHeston.modelParams('MCScheme') = 'QE';

        vanillaParams.params('strike') = fwd*relStrike;
        vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
        
        pricingOutMCHestonSLV = SX5EHestonSLV.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
        impVolSimulQESLV(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        pricingOutMCHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
        impVolSimulQE(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        toc
    end
end


fff = 1.0;

In [ ]:

test_RoughHeston_20200608_E_vanilla_220405.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 3;

priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

%SX5E
% targetDateint = [20190228;20190221;20190131;20190128;20190116;20190108;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];


%HSCEI
% targetDateint = [20190228;20190222;20190219;20190211;20190131;20190129;20190122;20190116;20190109;20190108;20190104;20181228;20181221;20181130;20181025;20180928;20180927;20180831;20180830;20180817;20180731;20180730;20180712];

valueDateH = H_Date('2020-07-08');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [30
91
182
365
730
% 1095
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2020-07-08');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;

    
    %% SX5E Model Building Start
    %% zeroCurve
%     rawData = [0.00273973	0.00371595
% 0.08333300	0.00394045
% 0.16666700	0.00408717
% 0.25000000	0.00423357
% 0.50000000	0.00613963
% 0.75000000	0.00583531
% 1.00000000	0.00547714
% 1.25000000	0.00514254
% 1.50000000	0.00481191
% 1.75000000	0.00448580
% 2.00000000	0.00416366
% 2.25000000	0.00406663
% 2.50000000	0.00396960
% 2.75000000	0.00387351
% 3.00000000	0.00377741
% 3.25000000	0.00381506
% 3.50000000	0.00385271
% 3.75000000	0.00389075
% 4.00000000	0.00392878
%     ];

rawData = [0.00273973	0.0
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For SX5E
    rawData = 1.0;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor SX5E
%     rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 30	0.772935544	0.660794663	0.560134289	0.466982478	0.3786614	0.294696393	0.227406437	0.212764901	0.231856547	0.257759106	0.283989413
% 91	0.64653478	0.560555897	0.483733518	0.413219942	0.347314997	0.285909471	0.233960992	0.207079785	0.208453133	0.221892285	0.238619703
% 182	0.552759946	0.485667786	0.426044378	0.37177818	0.321697021	0.275690437	0.235822475	0.208170248	0.198039125	0.200806015	0.209403502
% 365	0.461746233	0.412244058	0.36858374	0.329259583	0.29345123	0.26095042	0.232357252	0.209398474	0.194397922	0.187919169	0.187806662
% 730	0.383471893	0.348399204	0.317745566	0.29043906	0.265869603	0.243759238	0.224127195	0.207284638	0.193758099	0.184014699	0.178066211
% 1095	0.346099384	0.317848635	0.293299804	0.271568435	0.252131066	0.234699663	0.219162731	0.205551542	0.194000167	0.184671543	0.177647363
%     ];

rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
30	0.331083003	0.29058414	0.256378003	0.22822257	0.207404552	0.196392987	0.196136294	0.203716755	0.215149667	0.227933788	0.240894057
91	0.290920149	0.261085991	0.236697005	0.217532087	0.204061972	0.196873201	0.195723121	0.199198657	0.20550901	0.21326486	0.221616654
182	0.263239929	0.241298399	0.223968702	0.210882014	0.201952352	0.19703482	0.195666263	0.197069072	0.200383539	0.2048831	0.210046601
365	0.239422421	0.224822532	0.213744953	0.205671075	0.200226572	0.197052991	0.195761823	0.195949415	0.19723705	0.199303184	0.201895308
730	0.222209005	0.213328989	0.206807937	0.202137928	0.198949287	0.196948426	0.195891832	0.19557575	0.195831542	0.196522176	0.197538381
% 1095	0.215097926	0.20867642	0.204012109	0.200671445	0.19835158	0.196827815	0.195927348	0.195514793	0.195483246	0.19574812	0.196242523
    ];

    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','SX5EVolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';


    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor SX5E
%     rawData = [0	0.0303
%     ];

    rawData = [0	0.0
        ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EDividendCurve'});
    dividendCurveMktData = MktData(mktData);
    dividendCurveMktData.params('dividendType') = 'dividendYield';
    
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For SX5E

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','SX5EepoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;
   

   %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParamsDupire= containers.Map({'modelName'},{'Black'});
    SX5EBlack = EQBlack(EQModel(Model(AllMktData,modelParamsDupire))); 
    maturities = SX5EBlack.volSurface.maturities;
    strikes =    SX5EBlack.volSurface.strikes;
    impVolSurface = SX5EBlack.volSurface.vol;
    spot = SX5EBlack.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));

    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    SX5EImpliedVolSurface =SX5EBlack.volSurface;

    localVolSurfaceDupire.expiry = maturities;
    localVolSurfaceDupire.strike = strikes;
    localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
%     localVolSurfaceDupire.strikeType = 'fwdMoneyness';
    localVolSurfaceDupire.strikeType = 'spotMoneyness';

    % localVolSurfaceDupire.strikeType = 'fixedStrike';
    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurfaceDupire,'Dupire'});
    %% SS SABR vol calibration setting
    mcOneTimeStepTest = 3;
    NMCTest = 2e5;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

    modelParamsDupire('NMC') = NMCTest;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    modelParamsDupire('maxIter') = 1000;

    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    % modelParamsDupire('slope') = 1.0;
    modelParamsDupire('slope') = 0.5;

    %% SS SABR params

%     modelParamsDupire('alpha_0') = 0.3;
%     modelParamsDupire('alpha_inf') = 0.243;
%     modelParamsDupire('rho_SS')   = -0.778;
%     modelParamsDupire('nu') = 0.901;
    
    modelParamsDupire('alpha_0') = 0.195616389248541;
    modelParamsDupire('alpha_inf') = 0.196132174974662;
    modelParamsDupire('rho_SS')   = -0.135998259486906;
    modelParamsDupire('nu') = 0.347183843323165;
    
    modelParamsDupire('beta') = 1.0;
    modelParamsDupire('lambda') = 0.7;
    modelParamsDupire('omega')   = 0.6;
    modelParamsDupire('time_shift') = 0.1667;

    %% Ornstein-Uhlenbeck params
    modelParamsDupire('speed_OU') = 0.1;
    modelParamsDupire('level_OU') = 0.5;
    modelParamsDupire('vol_OU')   = 0.3;
    modelParamsDupire('x0_OU') = 1.0;

    % modelParamsDupire('lagrangeOrder') = 4;
    % 
    % modelParamsDupire('HermiteNormalXBar')  =[-2.334414218
    % -0.741963784
    % 0.741963784
    % 2.334414218
    % ];

    modelParamsDupire('lagrangeOrder') = 9;

    modelParamsDupire('HermiteNormalXBar')  =[-4.512745864
-3.205429004
-2.076847978
-1.023255664
0
1.023255664
2.076847978
3.205429004
4.512745864
];

 modelParamsDupire('gridStreching_sigma') = 1.37143571;

    %     modelParamsDupire('maturityTimes') = [ 0.05
%     0.25
%     0.5
%     1
%     2
%     3
%     4
%     ];

    modelParamsDupire('maturityTimes') = maturities/365.0;


    modelParamsDupire('lb') = [1.00E-08
    1.00E-08
    -0.9999
    1.00E-08
    1.00E-08
    1.00E-08
    % 1.00E-08
    ];

    %alpah_0
    %alpha_inf
    %rho
    %nu
    %lambda
    %omega

    modelParamsDupire('ub') = [100
    100
    0.9999
    100
    100.0
    100
    % 100
    ];


    calibratedParams = [0.308809422
    0.216171194
    -0.545484863
    0.765996606
    1.034002631
    0.630720371
    0.462011155
    ];


    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParamsDupire('calibrationErrorType') = 'priceError';

%     modelParamsDupire('reference') = 3234.07;
    modelParamsDupire('reference') = 1.0;
    
    %% Heston modelparams

%     modelParams= containers.Map({'localVolSurface','modelName'}, ...
%                             {localVolSurface,'Heston'});

    % we use fixed kappa  
    modelParamsDupire('kappa') = 2.0;
    modelParamsDupire('kappa') = 0.0;
    

    modelParamsDupire('theta') = 0.038570205;
    modelParamsDupire('rho')   = -0.5;
%     modelParamsDupire('rho')   = -0.7;
%     modelParamsDupire('rho')   = -0.9;
%     modelParamsDupire('rho')   = -0.8;
    
    
    
    modelParamsDupire('sigma') = 0.5;
%     modelParamsDupire('sigma') = 0.7;
%     modelParamsDupire('sigma') = 0.9;
    
    
    
    modelParamsDupire('v0')    = 0.038570205;

    modelParamsDupire('H')    = 0.1;
%      modelParamsDupire('H')    = 0.08;
%      modelParamsDupire('H')    = 0.05;
     
    
    modelParamsDupire('tol') = 1e-10;
    modelParamsDupire('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParamsDupire('quadrature_nodesAndWeights') = laguerre192;


    modelParamsDupire('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParamsDupire('quadrature_NumOfWeights') = 32;

    modelParamsDupire('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

%     modelParamsDupire('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
%     modelParamsDupire('modelPriceFormula') = 'GatheralR';
%     modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33';
    modelParamsDupire('modelPriceFormula') = 'GatheralPadeApprox33Amax';
    
%     modelParamsDupire('modelPriceFormula') = 'EER18Sigurdroemer';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParamsDupire('strikeUseBound') = strikeUseBound;

    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParamsDupire('useQuasiRandomYN') = 'NO';
    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParamsDupire('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParamsDupire('smoothWeightKappa') = 0.001;

%     modelParamsDummy = [2.0
% 0.074403
% -0.535078
% 1.180945
% 0.0858942
%         ];
%     
%     
%     modelParams('kappa') = modelParamsDummy(1);
%     modelParams('theta') = modelParamsDummy(2);
%     modelParams('rho')    = modelParamsDummy(3);
%     modelParams('sigma') = modelParamsDummy(4);
%     modelParams('v0')   = modelParamsDummy(5);


    % SX5EDupire = EQCOMDupireSSSABR(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
    % SX5EDupire = EQCOMDupireSSSABRRaw(EQModel(Model(SX5EAllMktData,modelParamsDupire)));
%     SX5EDupire = EQCOMDupireSSSABRRawNormalCLVHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));
    SX5EDupire = EQCOMDupireSSSABRRawNormalCLVRoughHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    %% localVolSurfaceParams
    detFwd    = zeros(length(expiry),1);
    atmFwdVol = zeros(length(expiry),1);
    atmSpotVol = zeros(length(expiry),1);
    fwdMoneynessVolSurface = zeros(length(expiry),length(strikes));
    spotVolSurface = zeros(length(expiry),length(strikes));
    for idxDum=1:length(expiry)
        detFwd(idxDum) = SX5EDupire.FwdWithoutSpot(expiry(idxDum));
        dummyOut = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum),expiry(idxDum)/365.0);
        atmFwdVol(idxDum) = dummyOut.vol;
        
        dummyOut1 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot,expiry(idxDum)/365.0);
        atmSpotVol(idxDum) = dummyOut1.vol;
        
        for idxDum2=1:length(strikes)
            dummyOut2 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),detFwd(idxDum)*strikes(idxDum2),expiry(idxDum)/365.0);
            fwdMoneynessVolSurface(idxDum,idxDum2) = dummyOut2.vol;
            
            dummyOut3 = SX5EDupire.SABR_ImpVol(detFwd(idxDum),spot*strikes(idxDum2),expiry(idxDum)/365.0);
            spotVolSurface(idxDum,idxDum2) = dummyOut3.vol;
        end
    end
    
    atmVols = zeros(length(expiry),2);
    for idxDum=1:length(expiry)
        atmVols(idxDum,1) = expiry(idxDum);
        atmVols(idxDum,2) = atmFwdVol(idxDum);
    end
    
%     atmVols = [30	0.357
%     91	0.34
%     182	0.3165
%     365	0.284
%     730	0.247
%     1095	0.228
%     ];

    refExpiryH = localVolSurfaceDupire.expiry;
    settleFutures = zeros(length(refExpiryH),1);
    settleFuturesRef = zeros(length(refExpiryH),1);

    referenceSpot = SX5EDupire.modelParams('reference');

    spotRawData = SX5EDupire.spot.params('rawData');
    for idxhhh=1:length(refExpiryH)
    %     settleFutures(idxhh) = SX5EDupire.FwdWithoutSpot(refExpiryH(idxhh));
        settleFutures(idxhhh) = spotRawData;
    %     settleFuturesRef(idxhh) = SX5EDupire.Fwd(referenceSpot,refExpiryH(idxhh));
        settleFuturesRef(idxhhh) = modelParamsDupire('reference');
    end

    modelParamsDupire('settleFutures') = settleFutures;
    modelParamsDupire('settleFuturesRef') = settleFuturesRef;
    modelParamsDupire('atmVols') = atmVols;

    SX5EDupire.settleFutures = settleFutures;
    SX5EDupire.settleFuturesRef = settleFuturesRef;
    SX5EDupire.atmVols = atmVols;

    tic
    useSSSABRAutoCallScheduleFlag = true;
    
    LocalVolEndDate = vanillaParams.startDate.AddDate(targetMaturities(end),'day');
    autoCallScheduleInt =[FngDate(LocalVolEndDate)	0.875	0.0405
    ];

    manualScheduleYN = 1;
    dailyTimeStepsYN = 'NO';
    stepdownInfo = containers.Map({'dailyTimeStepsYN',...
                                    'autoCallScheduleInt','manualScheduleYN'}, ...
                            {dailyTimeStepsYN,...
                            autoCallScheduleInt,manualScheduleYN});

    stepdownParams.params = stepdownInfo;  

%     outLocalVolSS = SX5EDupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    outLocalVolSS = SX5EDupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
    
%     out = SX5EDupire.CalibrateToVolSurface(SX5EBlack,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = SX5EDupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    % pricingOutLV = SX5EDupire.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = SX5EDupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
%     localVolSurface.volProxy = out.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParams)));
%     SX5EHeston = EQCOMDupireSSSABRRawNormalCLVHeston(EQModel(Model(AllMktData,modelParamsDupire)));

%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalKappaFixed');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'global');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalHFixed');
    
      outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalSigurdroemer');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'globalGA');
%     outHeston = SX5EDupire.CalibrateToVolSurfaceHeston(SX5EBlack,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParamsDupire('kappa') = modelParamsDummy(1);
    modelParamsDupire('theta') = modelParamsDummy(2);
    modelParamsDupire('rho')    = modelParamsDummy(3);
    modelParamsDupire('sigma') = modelParamsDummy(4);
    modelParamsDupire('v0')   = modelParamsDummy(5);

%     SX5EHeston = EQHeston(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    pricingOutHeston = SX5EDupire.computeForwardStartingVanillaMCHeston(valueDateH,vanillaParams);
    % pricingOutHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = SX5EDupire.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

%     modelParamsDupire('kappa') = modelParamsDummy(1);
%     modelParamsDupire('theta') = modelParamsDummy(2);
%     modelParamsDupire('rho')    = modelParamsDummy(3);
%     modelParamsDupire('sigma') = modelParamsDummy(4)*mixingWeight;
%     modelParamsDupire('v0')   = modelParamsDummy(5);
%     
%     SX5EHestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParamsDupire)));

    SX5EDupire.sigma = modelParamsDummy(4) * mixingWeight;
    
    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = SX5EDupire.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = SX5EDupire.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = SX5EDupire.computeForwardStartingVanillaMCHestonSLV(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = SX5EDupire.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end

aaa = 1.0;

ccc = 1.0;
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalGA');

% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaFixed');
% out = SX5EHeston.CalibrateToVolSurface(SX5EBlack,'globalKappaConstrained');
bbb = 1.0;

priceSimul = zeros(length(expiry),101);
impVolSimul = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);


priceSimulQE = zeros(length(expiry),101);
impVolSimulQE = zeros(length(expiry),101);

% price simulation
bbb =  1.0;
spot =  SX5EHestonSLV.spot.params('rawData');

for j=1:length(expiry)
    fwd = SX5EHestonSLV.Fwd(spot,expiry(j));
    df = SX5EHestonSLV.zeroCurve.DF(expiry(j)/365.0);
    for k=1:length(strikes)
        tic
        relStrike = strikes(k);
        if relStrike <= 1.0
            vanillaParams.params('callPutFlag') = 'P';
            callputStringhsw = 'Put';
        else
            vanillaParams.params('callPutFlag') = 'C';
            callputStringhsw = 'Call';
        end
        
        priceSimul(j,k)  = SX5EHeston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
        impVolSimul(j,k) = SX5EHeston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        SX5EHestonSLV.modelParams('MCScheme') = 'QE';
        SX5EHeston.modelParams('MCScheme') = 'QE';

        vanillaParams.params('strike') = fwd*relStrike;
        vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
        
        pricingOutMCHestonSLV = SX5EHestonSLV.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
        impVolSimulQESLV(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        pricingOutMCHeston = SX5EHeston.computeVanillaMC(valueDateH,vanillaParams);
        priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
        impVolSimulQE(j,k) = SX5EHestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
        
        toc
    end
end


fff = 1.0;

In [ ]:
test_riskyFixedBond_HazardRateBootStrapping.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);

simulNPV = zeros(1000,1);
simulfairRate = zeros(1000,1);

%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
dataType= {'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
'RiskyFixedBond'
};

dataTenor = {'1Y'
'2Y'
'3Y'
'5Y'
};
% 
% % % 우리은행 AAA
% % dataRate = {0.01926
% % 0.02043
% % 0.02129
% % 0.02301
% % };
% 
% % % % 신한카드 AA+
% % dataRate = {0.02039
% % 0.02199
% % 0.02302
% % 0.02507
% % };
% 
% % 벤츠 파이넨셜 A+
dataRate = {0.02082
0.02394
0.02849
0.03151
};
% 
% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
dataIn = {};
dataIn.dataType = dataType;
dataIn.dataTenor = dataTenor;
dataIn.dataRate = dataRate;
    

% other static setting for bootstrapping

riskyFixedBondParams = {};
riskyFixedBondParams.recoveryRate = 0.4;

 %% import and construct a discountCurve
% global dictionary(map) init
globalDict = containers.Map();

%% calendar
calendarNames = {'KRNT','KRBK'};
jointCalendar = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

globalDict('jointCalendar') = jointCalendar;

%% MarketData YLD
%yldCurveSetting
yiledCurveSettings = containers.Map();

%KRW_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act365';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';

yieldCurveParams.fixingDays = 1;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');
yieldCurveParams.fixedLegTenor = '3M';
yieldCurveParams.fixedLegDayCounter = 'Act365';
yieldCurveParams.fixedLegConvention = 'ModifiedFollowing';
yieldCurveParams.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;4;5;7;10;12;15;20;25;30];
yiledCurveSettings('KRW_IRS') = yieldCurveParams;

%KRW_KTB
yieldCurveParamsDisc = {};
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;
yieldCurveParamsDisc.fixingCalendar = globalDict('jointCalendar');

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
yieldCurveParamsDisc.targetTimeBucket = [0.0027397;0.25;0.5;0.75;1;2;3;5;10;20;30];

yiledCurveSettings('KRW_KTB') = yieldCurveParamsDisc;

%USD_IRS
yieldCurveParams = {};
yieldCurveParams.floatingLegTenor = '3M';
yieldCurveParams.floatingLegDayCounter = 'Act360';
yieldCurveParams.floatingLegConvention = 'ModifiedFollowing';
yieldCurveParams.fixingDays = 2;
yieldCurveParams.fixingCalendar = globalDict('jointCalendar');

yieldCurveParams.fixedLegTenor = '6M';
yieldCurveParams.fixedLegDayCounter = '30360';
yieldCurveParams.fixedLegConvention = 'Following';
yieldCurveParams.targetTimeBucket = [0.002739726;0.083333333;0.166666667;0.25;0.5;1;2;3;4;5;7;10;12;15;20;25;30];

yiledCurveSettings('USD_IRS') = yieldCurveParams;

globalDict('yieldCurveSettings') = yiledCurveSettings;
 



curveName = 'KRW_KTB';
ycData = GetYieldCurveFromOctagon(valueDate,curveName,globalDict,true);

    
rawData = ycData.zcCurve.zeroCurve.params('rawData');

% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

aaa = 1.0;

% yieldCurveParamsDisc = {};
%  %KTB
% yieldCurveParamsDisc.floatingLegTenor = '3M';
% yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
% yieldCurveParamsDisc.floatingLegConvention = 'Following';
% yieldCurveParamsDisc.fixingDays = 1;
% 
% yieldCurveParamsDisc.fixedLegTenor = '6M';
% yieldCurveParamsDisc.fixedLegDayCounter = '30360';
% yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';
% 
% targetTimeBucket = [0.0027397
% 0.25
% 0.5
% 0.75
% 1
% 2
% 3
% 5
% 10
% 20
% 30];
% 
% yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;
% 
% curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData,yieldCurveParamsDisc);
%     
% rawData = curveOutDisc(:,1:2);
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end

% hazardRateCurveParams : bootStrapping input arg
hazardRateCurveParams = {};
hazardRateCurveParams.recoveryRate = 0.4;

for idxSimul =1:400

dateH = H_Date(valueDate).AddDate(idxSimul-1,'day');   

mktDataDisc = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,dateH,'KTB','discountZero'});
discountCurveMktData = MktData(mktDataDisc);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);

hazardRateCurveParams.discountCurve=discountCurve;

riskyFixedBondParams.discountCurve=discountCurve;


% survProbCurve
rawData = [0.002739726	0.999972603
0.252054795	0.997503122
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawData,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurve =  SurvProbCurve(survProbCurveMktData);

%for riskyFixedBond 
hazardRateCurveParams.settlesAccrual = false;
hazardRateCurveParams.paysAtDefaultTime = false;

hazardRateCurveParams.engineType = 'Integrate';
% hazardRateCurveParams.engineType = 'Mid';

%riskyFixedBond settings
hazardRateCurveParams.riskyFixedBondSettlementDays = 1;
hazardRateCurveParams.riskyFixedBondBasis = 'Act365';
hazardRateCurveParams.riskyFixedBondConvention = 'ModifiedFollowing';
hazardRateCurveParams.riskyFixedBondPeriod = '3M';
hazardRateCurveParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

targetTimeBucket = [ 0.0027397 
];

hazardRateCurveParams.targetTimeBucket = targetTimeBucket;
curveOut = HazardRateCurveBootStrap('KRW',dateH,dataIn,hazardRateCurveParams);

% survProbCurve
rawDataSurv = zeros(length(curveOut.survProb),2);
rawDataSurv(:,1) = curveOut.times(:);
rawDataSurv(:,2) = curveOut.survProb(:);

% rawDataSurv = [1.01369863	0.993992919
% 2.010958904	0.98113753
% 3.008219178	0.953832109
% 5.005479452	0.913660385
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawDataSurv,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurve =  SurvProbCurve(survProbCurveMktData);

riskyFixedBondParams.survProbCurve = survProbCurve;
%riskyFixedBondParams settings
riskyFixedBondParams.riskyFixedBondSettlementDays = 1;
riskyFixedBondParams.riskyFixedBondBasis = 'Act365';
riskyFixedBondParams.riskyFixedBondConvention = 'ModifiedFollowing';
riskyFixedBondParams.riskyFixedBondPeriod = '3M';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'BackwardCDS';
riskyFixedBondParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

riskyFixedBondParams.settlesAccrual = false;
riskyFixedBondParams.paysAtDefaultTime = false;

riskyFixedBondParams.engineType = 'Integrate';
% riskyFixedBondParams.engineType = 'Mid';

riskyFixedBondParams.evaluationDate = riskyFixedBondParams.discountCurve.asOfDate;

riskyFixedBondParams.efffectiveStartDate = H_Date('2018-09-12');

riskyFixedBondParams.startDate = riskyFixedBondParams.efffectiveStartDate;
riskyFixedBondParams.endDate = AddTenor(riskyFixedBondParams.efffectiveStartDate,'5Y');
jointCalendar = MakeCalendar(dateH,{},'30Y');
riskyFixedBondParams.schedule = Schedule(riskyFixedBondParams.startDate,riskyFixedBondParams.endDate,riskyFixedBondParams.riskyFixedBondPeriod,jointCalendar, riskyFixedBondParams.riskyFixedBondConvention,...
                              riskyFixedBondParams.riskyFixedBondDateGenerationRule,false);

riskyFixedBondParams.type = 'Buyer';  
riskyFixedBondParams.nominal = 10000.0;
riskyFixedBondParams.premiumRate = 0.03151;

riskyFixedBond = RiskyFixedBond(riskyFixedBondParams.type,riskyFixedBondParams.engineType,riskyFixedBondParams.nominal,...
                                riskyFixedBondParams.schedule,riskyFixedBondParams.premiumRate,riskyFixedBondParams.riskyFixedBondBasis,...
                                riskyFixedBondParams.riskyFixedBondConvention,riskyFixedBondParams.settlesAccrual,riskyFixedBondParams.paysAtDefaultTime,...
                                riskyFixedBondParams.efffectiveStartDate);
                            

outInfo = riskyFixedBond.Calculate(riskyFixedBondParams.recoveryRate,riskyFixedBondParams.survProbCurve,riskyFixedBondParams.discountCurve);

simulNPV(idxSimul) = outInfo.NPV;
simulfairRate(idxSimul) = outInfo.fairRate;


end

aaa = 1.0;
% %crossCheck
% dataDiff = zeros(length(dataRate),1);
% 
% if strcmp(dataType{1},'RiskyFixedBond')
% 
%     dataType= {'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     };
%     
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedParCDSSPread(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
% 
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedYTM(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%  
%     end
%     
% else
%     
%     dataType= {'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     };
% 
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedYTM(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
%     
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedParCDSSPread(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%     end
%     
% end


aaa =1.0;


In [ ]:

test_riskyFixedBond_20180911_BankAAA_KTB_New.m

clc
clear
valueDate = '2018-09-11';

dateH0 = H_Date(valueDate);

dateH = H_Date(valueDate);

simulNPV = zeros(1000,1);
simulfairRate = zeros(1000,1);

%% import and construct a hazardRateCurve
% 
% 
% dataType= {'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% 'SpreadCDS'
% };
% 
% dataType= {'RiskyFixedBond'
% 'RiskyFixedBond'
% 'RiskyFixedBond'
% 'RiskyFixedBond'
% };
% 
% dataTenor = {'1Y'
% '2Y'
% '3Y'
% '5Y'
% };
% 
% % % 우리은행 AAA
% % dataRate = {0.01926
% % 0.02043
% % 0.02129
% % 0.02301
% % };
% 
% % % % 신한카드 AA+
% % dataRate = {0.02039
% % 0.02199
% % 0.02302
% % 0.02507
% % };
% 
% % 벤츠 파이넨셜 A+
% dataRate = {0.02082
% 0.02394
% 0.02849
% 0.03151
% };
% 
% % dataRate = {0.02978608428583750000
% % 0.02987540501789370000
% % 0.02989337047641950000
% % };

% market data IN
% dataIn = {};
% dataIn.dataType = dataType;
% dataIn.dataTenor = dataTenor;
% dataIn.dataRate = dataRate;
    
%
calendarNames = {'KRNT','KRBK'};
jointCalendar1 = GetCalendarsFromOctagon(valueDate,calendarNames,'30Y');

% other static setting for bootstrapping

riskyFixedBondParams = {};
riskyFixedBondParams.recoveryRate = 0.4;

 %% import and construct a discountCurve
curveName = 'KRW_KTB';
ycData = GetYieldCurveFromOctagon(valueDate,curveName,[],false);
yieldCurveParamsDisc = {};

yieldCurveParamsDisc.fixingCalendar = jointCalendar1;
%KTB
yieldCurveParamsDisc.floatingLegTenor = '3M';
yieldCurveParamsDisc.floatingLegDayCounter = 'Act365';
yieldCurveParamsDisc.floatingLegConvention = 'Following';
yieldCurveParamsDisc.fixingDays = 1;

yieldCurveParamsDisc.fixedLegTenor = '6M';
yieldCurveParamsDisc.fixedLegDayCounter = '30360';
yieldCurveParamsDisc.fixedLegConvention = 'Unadjusted';

targetTimeBucket = [0.0027397
0.25
0.5
0.75
1
2
3
5
10
20
30];

yieldCurveParamsDisc.targetTimeBucket = targetTimeBucket;

tic
curveOutDisc = YieldCurveBootStrap('KTB',dateH,ycData.ycData,yieldCurveParamsDisc);
toc

rawData = curveOutDisc.bootStrapOut.forwardCurve.params('rawData');

% rawData = curveOutDisc(:,1:2);
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end


for idxSimul =1:400

dateH = H_Date(valueDate).AddDate(idxSimul-1,'day');   

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,dateH,'KTB','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);

riskyFixedBondParams.discountCurve=discountCurve;

% survProbCurve
rawDataSurv = [1.01369863	0.993992919
2.010958904	0.98113753
3.008219178	0.953832109
5.005479452	0.913660385
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
{rawDataSurv,dateH,'KRW','KRWSurvProb'});
survProbCurveMktData = MktData(mktData);
survProbCurveMktData.params('convention') = {};
survProbCurve =  SurvProbCurve(survProbCurveMktData);

riskyFixedBondParams.survProbCurve = survProbCurve;
%riskyFixedBondParams settings
riskyFixedBondParams.riskyFixedBondSettlementDays = 1;
riskyFixedBondParams.riskyFixedBondBasis = 'Act365';
riskyFixedBondParams.riskyFixedBondConvention = 'ModifiedFollowing';
riskyFixedBondParams.riskyFixedBondPeriod = '3M';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'TwentiethIMM';
% riskyFixedBondParams.spreadCDSDateGenerationRule = 'BackwardCDS';
riskyFixedBondParams.riskyFixedBondDateGenerationRule = 'ForwardCDS';

riskyFixedBondParams.settlesAccrual = false;
riskyFixedBondParams.paysAtDefaultTime = false;

riskyFixedBondParams.engineType = 'Integrate';
% riskyFixedBondParams.engineType = 'Mid';

riskyFixedBondParams.evaluationDate = riskyFixedBondParams.discountCurve.asOfDate;

riskyFixedBondParams.efffectiveStartDate = H_Date('2018-09-12');

riskyFixedBondParams.startDate = riskyFixedBondParams.efffectiveStartDate;
riskyFixedBondParams.endDate = AddTenor(riskyFixedBondParams.efffectiveStartDate,'5Y');
jointCalendar = MakeCalendar(dateH,{},'30Y');
riskyFixedBondParams.schedule = Schedule(riskyFixedBondParams.startDate,riskyFixedBondParams.endDate,riskyFixedBondParams.riskyFixedBondPeriod,jointCalendar, riskyFixedBondParams.riskyFixedBondConvention,...
                              riskyFixedBondParams.riskyFixedBondDateGenerationRule,false);

riskyFixedBondParams.type = 'Buyer';  
riskyFixedBondParams.nominal = 10000.0;
riskyFixedBondParams.premiumRate = 0.03151;

riskyFixedBond = RiskyFixedBond(riskyFixedBondParams.type,riskyFixedBondParams.engineType,riskyFixedBondParams.nominal,...
                                riskyFixedBondParams.schedule,riskyFixedBondParams.premiumRate,riskyFixedBondParams.riskyFixedBondBasis,...
                                riskyFixedBondParams.riskyFixedBondConvention,riskyFixedBondParams.settlesAccrual,riskyFixedBondParams.paysAtDefaultTime,...
                                riskyFixedBondParams.efffectiveStartDate);
                            

outInfo = riskyFixedBond.Calculate(riskyFixedBondParams.recoveryRate,riskyFixedBondParams.survProbCurve,riskyFixedBondParams.discountCurve);

simulNPV(idxSimul) = outInfo.NPV;
simulfairRate(idxSimul) = outInfo.fairRate;


end

aaa = 1.0;
% %crossCheck
% dataDiff = zeros(length(dataRate),1);
% 
% if strcmp(dataType{1},'RiskyFixedBond')
% 
%     dataType= {'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     'SpreadCDS'
%     };
%     
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedParCDSSPread(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
% 
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedYTM(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%  
%     end
%     
% else
%     
%     dataType= {'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     'RiskyFixedBond'
%     };
% 
%     dataRateOrg = dataRate;
%     
%     for i=1:length(dataRate)
%         dataRate{i} = curveOut.impliedYTM(i);
%     end
%     
%     dataIn.dataType = dataType;
%     dataIn.dataRate = dataRate;
%     
%     curveOutCrossCheck = HazardRateCurveBootStrap('KRW',dateH,dataIn,riskyFixedBondParams);
%     
%     dataRateNew = dataRateOrg;
%     
%     for i=1:length(dataRate)
%         dataRateNew{i} = curveOutCrossCheck.impliedParCDSSPread(i);
%         dataDiff(i) = dataRateNew{i} - dataRateOrg{i}; 
%     end
%     
% end


aaa =1.0;


In [ ]:
test_PutShortStrategyIndex1S_Reghai_KOSPI200_Generic_20180626.m

clc
clear
valueDate = '2018-06-26';

%% import and construct a zeroCurve
rawData = [0.0027397	0.014999692
0.25	0.016422129
0.5	0.017141194
0.75	0.017876828
1	0.018350015
2	0.019798517
3	0.020565804
4	0.021153781
5	0.02156353
7	0.02202487
10	0.022737327
12	0.023170456
15	0.023462116
20	0.023846646
25	0.023809452
30	0.023783578
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a spot For KOSPI200
rawData = 302.76;
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200

rawData = [0	0.3	0.4	0.6	0.8	0.9	0.95	0.975	1	1.025	1.05	1.1	1.2	1.3
30	0.6467	0.5444	0.3709	0.2918	0.214	0.1796	0.1636	0.1486	0.1367	0.1309	0.1345	0.1645	0.1937
61	0.4939	0.4243	0.3234	0.2506	0.1919	0.164	0.1512	0.1394	0.1289	0.1236	0.1223	0.1282	0.1494
92	0.4363	0.3694	0.3157	0.2302	0.1814	0.1594	0.1501	0.1415	0.1332	0.1264	0.1214	0.126	0.1378
183	0.3978	0.3728	0.3009	0.209	0.1701	0.1501	0.1417	0.1352	0.1307	0.1279	0.1246	0.13	0.1507
273	0.3923	0.3628	0.2841	0.1991	0.1638	0.15	0.1445	0.1402	0.137	0.1348	0.1334	0.138	0.1481
365	0.374	0.3414	0.2629	0.1892	0.1627	0.1526	0.1483	0.1447	0.1419	0.1399	0.1385	0.1419	0.1484
548	0.3548	0.3189	0.2404	0.1799	0.1629	0.1561	0.1531	0.1504	0.1482	0.1464	0.1445	0.1447	0.1467
731	0.3292	0.2937	0.2277	0.1771	0.1629	0.158	0.1559	0.154	0.1521	0.1504	0.147	0.1424	0.143
1096	0.3042	0.2688	0.2057	0.1649	0.1536	0.1492	0.1473	0.1457	0.1443	0.1433	0.1423	0.1434	0.1463
1461	0.2727	0.2452	0.1946	0.162	0.1529	0.1495	0.1481	0.1468	0.1457	0.1449	0.1438	0.1441	0.1462
1826	0.2546	0.2317	0.1889	0.1618	0.1537	0.1507	0.1494	0.1483	0.1474	0.1467	0.1459	0.146	0.1478
2557	0.2497	0.2242	0.1848	0.1628	0.1559	0.1532	0.1521	0.1512	0.1504	0.1499	0.1493	0.1497	0.1512
3653	0.2454	0.2189	0.1848	0.166	0.1601	0.1581	0.1572	0.1566	0.156	0.1556	0.155	0.155	0.1561
];

rawData = [0	0.3	0.4	0.6	0.8	0.9	0.95	0.975	1	1.025	1.05	1.1	1.2	1.3
30	0.6467	0.5444	0.3709	0.2918	0.214	0.1796	0.1636	0.1486	0.1367	0.1309	0.1345	0.1645	0.1937
61	0.4939	0.4243	0.3234	0.2506	0.1919	0.164	0.1512	0.1394	0.1289	0.1236	0.1223	0.1282	0.1494
92	0.4363	0.3694	0.3157	0.2302	0.1814	0.1594	0.1501	0.1415	0.1332	0.1264	0.1214	0.126	0.1378
183	0.3978	0.3728	0.3009	0.209	0.1701	0.1501	0.1417	0.1352	0.1307	0.1279	0.1246	0.13	0.1507
273	0.3923	0.3628	0.2841	0.1991	0.1638	0.15	0.1445	0.1402	0.137	0.1348	0.1334	0.138	0.1481
365	0.374	0.3414	0.2629	0.1892	0.1627	0.1526	0.1483	0.1447	0.1419	0.1399	0.1385	0.1419	0.1484
548	0.3548	0.3189	0.2404	0.1799	0.1629	0.1561	0.1531	0.1504	0.1482	0.1464	0.1445	0.1447	0.1467
731	0.3292	0.2937	0.2277	0.1771	0.1629	0.158	0.1559	0.154	0.1521	0.1504	0.147	0.1424	0.143
% 1096	0.3042	0.2688	0.2057	0.1649	0.1536	0.1492	0.1473	0.1457	0.1443	0.1433	0.1423	0.1434	0.1463
% 1461	0.2727	0.2452	0.1946	0.162	0.1529	0.1495	0.1481	0.1468	0.1457	0.1449	0.1438	0.1441	0.1462
% 1826	0.2546	0.2317	0.1889	0.1618	0.1537	0.1507	0.1494	0.1483	0.1474	0.1467	0.1459	0.146	0.1478
% 2557	0.2497	0.2242	0.1848	0.1628	0.1559	0.1532	0.1521	0.1512	0.1504	0.1499	0.1493	0.1497	0.1512
% 3653	0.2454	0.2189	0.1848	0.166	0.1601	0.1581	0.1572	0.1566	0.156	0.1556	0.155	0.155	0.1561
];

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'spotMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [30	-0.033919693
61	-0.007869551
92	-0.014659114
183	-0.042552562
273	-0.033097396
365	-0.011457884
548	-0.026099792
731	-0.023565502
1096	-0.024741147
1461	-0.024592228
1826	-0.024811274
2557	-0.024690073
3653	-0.024200761
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

fwd = zeros(length(maturities),1);
df = zeros(length(maturities),1);

if strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'fwdMoneyNess')
    
    for i=1:length(maturities)
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end

elseif strcmp(KOSPI200Black.volSurface.params('moneyNessType'),'spotMoneyNess')
    
    for i=1:length(maturities)
    %     fwd(i) = KOSPI200Black.Fwd(spot,expiry(i));
    %     df(i) = KOSPI200Black.zeroCurve.DF(expiry(i)/365.0);
        fwdFactor(i) = KOSPI200Black.FwdFactor(expiry(i));
        for j=1:length(strikes)
            fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
            blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));

            if fwdMoneyness(i,j) <= 1
    %   blackOTMPrices2 : the usual black formula
    %   blackOTMPrice   : Peter Jackel's Let's be rational
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
            else
                blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
                blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
            end
        end
     end
end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;

modelParams('NMC') = 2^17;
% modelParams('NMC') = 2^16;

% modelParams('NMC') = 2^10;
% modelParams('NMC') = 2^10;

%nestedMC params
modelParams('nestedNMC') = 1000;
modelParams('nestedMaxTimeStepsSize') = 10; % daily step -> maximum one month
modelParams('nestedMCOneTimeStep') = 7;

% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic
out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

maxIdx = 18;
simulMean = zeros(maxIdx,1);
simulStd = zeros(maxIdx,1);
simulMeanAnn = zeros(maxIdx,1);
simulStdAnn = zeros(maxIdx,1);

simulIndexNPV = zeros(maxIdx,1);
simulOptionNPV = zeros(maxIdx,1);
for idx=10:maxIdx
    %% Pricing putShortStrategy 1S
    %Steptown info
    tempN = 2^idx;
    KOSPI200Dupire.modelParams('NMC') = tempN;
    
    putShortStrategyParams.payCcy = 'KRW';
    putShortStrategyParams.startDate = H_Date('2018-06-26');
    putShortStrategyParams.endDate = H_Date('2019-06-26');
    
%     putShortStrategyParams.endDate = AddDate(H_Date('2018-06-26'),idx,'month');
    
    putShortStrategyParams.maturity = 1;

    valueDateH = H_Date(valueDate);

    nominal = 10000;
%     basePrice = 302.76;
    basePrice = 1447.37;
    lowerBarrier = 0.9;

    callStrike = 1.0;
    gearing = 1.0;
    
                                  %reset  strike
    denominatorResetScheduleInt = [20180614	0.97
20180712	0.97
20180809	0.97
20180913	0.97
20181011	0.97
20181108	0.97
20181213	0.97
20190111	0.97
20190208	0.97
20190308	0.97
20190412	0.97
20190510	0.97
20190614	0.97
    ];

%     newSize = 0;
%     for i=1:size(denominatorResetScheduleInt,1)
%         if DateDiff(H_Date(denominatorResetScheduleInt(i,1)),putShortStrategyParams.endDate) <=0
%             newSize =  newSize +1;
%         else
%             break;
%         end
%     end
% 
%     denominatorResetScheduleInt = denominatorResetScheduleInt(1:newSize,:);
    
    matuT = DateDiff(putShortStrategyParams.endDate,putShortStrategyParams.startDate)/365.0;


    dailyTimeStepsYN = 'YES';
    % fixingInfo
    
    KOSPI200Fixing = containers.Map();
    KOSPI200FixingDates = {'2018-06-14'
'2018-07-12'
'2018-08-09'
'2018-09-13'
'2018-10-11'
'2018-11-08'
'2018-12-13'
'2019-01-11'
'2019-02-08'
'2019-03-08'
'2019-04-12'
'2019-05-10'
'2019-06-14'
'2019-07-12'
        };
    
    for ii=1:length(KOSPI200FixingDates)
        KOSPI200Fixing(KOSPI200FixingDates{ii}) = 311.2;
    end
    
    allFixings = containers.Map({'KOSPI200'},{KOSPI200Fixing});
    
    PutShortStrategyIndexFixing = containers.Map();
    PutShortStrategyIndexFixingDates = {'2017-06-22'
'2017-06-23'
'2017-06-26'
'2017-06-27'
'2017-06-28'
'2017-06-29'
'2017-06-30'
'2017-07-03'
'2017-07-04'
'2017-07-05'
'2017-07-06'
'2017-07-07'
'2017-07-10'
'2017-07-11'
'2017-07-12'
'2017-07-13'
'2017-07-14'
'2017-07-17'
'2017-07-18'
'2017-07-19'
'2017-07-20'
'2017-07-21'
'2017-07-24'
'2017-07-25'
'2017-07-26'
'2017-07-27'
'2017-07-28'
'2017-07-31'
'2017-08-01'
'2017-08-02'
'2017-08-03'
'2017-08-04'
'2017-08-07'
'2017-08-08'
'2017-08-09'
'2017-08-10'
'2017-08-11'
'2017-08-14'
'2017-08-16'
'2017-08-17'
'2017-08-18'
'2017-08-21'
'2017-08-22'
'2017-08-23'
'2017-08-24'
'2017-08-25'
'2017-08-28'
'2017-08-29'
'2017-08-30'
'2017-08-31'
'2017-09-01'
'2017-09-04'
'2017-09-05'
'2017-09-06'
'2017-09-07'
'2017-09-08'
'2017-09-11'
'2017-09-12'
'2017-09-13'
'2017-09-14'
'2017-09-15'
'2017-09-18'
'2017-09-19'
'2017-09-20'
'2017-09-21'
'2017-09-22'
'2017-09-25'
'2017-09-26'
'2017-09-27'
'2017-09-28'
'2017-09-29'
'2017-10-10'
'2017-10-11'
'2017-10-12'
'2017-10-13'
'2017-10-16'
'2017-10-17'
'2017-10-18'
'2017-10-19'
'2017-10-20'
'2017-10-23'
'2017-10-24'
'2017-10-25'
'2017-10-26'
'2017-10-27'
'2017-10-30'
'2017-10-31'
'2017-11-01'
'2017-11-02'
'2017-11-03'
'2017-11-06'
'2017-11-07'
'2017-11-08'
'2017-11-09'
'2017-11-10'
'2017-11-13'
'2017-11-14'
'2017-11-15'
'2017-11-16'
'2017-11-17'
'2017-11-20'
'2017-11-21'
'2017-11-22'
'2017-11-23'
'2017-11-24'
'2017-11-27'
'2017-11-28'
'2017-11-29'
'2017-11-30'
'2017-12-01'
'2017-12-04'
'2017-12-05'
'2017-12-06'
'2017-12-07'
'2017-12-08'
'2017-12-11'
'2017-12-12'
'2017-12-13'
'2017-12-14'
'2017-12-15'
'2017-12-18'
'2017-12-19'
'2017-12-20'
'2017-12-21'
'2017-12-22'
'2017-12-26'
'2017-12-27'
'2017-12-28'
'2018-01-02'
'2018-01-03'
'2018-01-04'
'2018-01-05'
'2018-01-08'
'2018-01-09'
'2018-01-10'
'2018-01-11'
'2018-01-12'
'2018-01-15'
'2018-01-16'
'2018-01-17'
'2018-01-18'
'2018-01-19'
'2018-01-22'
'2018-01-23'
'2018-01-24'
'2018-01-25'
'2018-01-26'
'2018-01-29'
'2018-01-30'
'2018-01-31'
'2018-02-01'
'2018-02-02'
'2018-02-05'
'2018-02-06'
'2018-02-07'
'2018-02-08'
'2018-02-09'
'2018-02-12'
'2018-02-13'
'2018-02-14'
'2018-02-19'
'2018-02-20'
'2018-02-21'
'2018-02-22'
'2018-02-23'
'2018-02-26'
'2018-02-27'
'2018-02-28'
'2018-03-02'
'2018-03-05'
'2018-03-06'
'2018-03-07'
'2018-03-08'
'2018-03-09'
'2018-03-12'
'2018-03-13'
'2018-03-14'
'2018-03-15'
'2018-03-16'
'2018-03-19'
'2018-03-20'
'2018-03-21'
'2018-03-22'
'2018-03-23'
'2018-03-26'
'2018-03-27'
'2018-03-28'
'2018-03-29'
'2018-03-30'
'2018-04-02'
'2018-04-03'
'2018-04-04'
'2018-04-05'
'2018-04-06'
'2018-04-09'
'2018-04-10'
'2018-04-11'
'2018-04-12'
'2018-04-13'
'2018-04-16'
'2018-04-17'
'2018-04-18'
'2018-04-19'
'2018-04-20'
'2018-04-23'
'2018-04-24'
'2018-04-25'
'2018-04-26'
'2018-04-27'
'2018-04-30'
'2018-05-02'
'2018-05-03'
'2018-05-04'
'2018-05-08'
'2018-05-09'
'2018-05-10'
'2018-05-11'
'2018-05-14'
'2018-05-15'
'2018-05-16'
'2018-05-17'
'2018-05-18'
'2018-05-21'
'2018-05-23'
'2018-05-24'
'2018-05-25'
'2018-05-28'
'2018-05-29'
'2018-05-30'
'2018-05-31'
'2018-06-01'
'2018-06-04'
'2018-06-05'
'2018-06-07'
'2018-06-08'
'2018-06-11'
'2018-06-12'
'2018-06-14'
'2018-06-15'
'2018-06-18'
'2018-06-19'
'2018-06-20'
'2018-06-21'
'2018-06-22'
'2018-06-25'
'2018-06-26'
        };
    
    PutShortStrategyIndexFixingValues = [1384.26
1384.58
1385
1385.21
1385.12
1385.51
1385.45
1385.73
1384.81
1385.63
1385.84
1385.93
1386.33
1386.45
1386.5
1386.59
1387.25
1387.84
1388.11
1388.51
1389
1389.42
1389.72
1389.55
1389.6
1389.76
1386.34
1387.8
1389.67
1390
1387.28
1389.3
1389.95
1390.48
1388.92
1391.08
1382.51
1390.23
1395.04
1398.28
1397.31
1397.44
1399.32
1399.46
1400.83
1401.06
1400.41
1399.6
1401.19
1400.13
1400.63
1397.89
1397.51
1397.67
1401.25
1401.21
1403.52
1404.05
1404.24
1404.49
1405.55
1407.88
1408.13
1408.11
1408.43
1407.65
1408.43
1408.12
1408.42
1408.65
1408.59
1410.69
1410.87
1410.96
1411.83
1412.37
1412.63
1412.77
1411.55
1413.58
1414.33
1414.27
1414.58
1413.64
1415.37
1416.12
1416.65
1416.89
1416.94
1417.14
1417.31
1417.45
1417.5
1417.59
1418.25
1417.24
1417.37
1415.51
1418.63
1418.7
1417.83
1419.63
1420.93
1420.39
1421.12
1415.51
1418.44
1418.69
1409.68
1409.63
1416.29
1418.79
1410.73
1408.99
1412.31
1413.62
1411.83
1420.08
1418.83
1418.96
1420.32
1420.16
1420.44
1413.81
1417.22
1416.15
1419.89
1422.6
1423.47
1423.77
1423.79
1424.18
1424.45
1424.62
1424.68
1424.78
1425.41
1425.79
1427.14
1426.67
1426.64
1426.92
1426.16
1428.15
1428.19
1428.67
1428.91
1429.2
1429.08
1429.16
1429.33
1428.86
1427.85
1419.05
1402.07
1409.73
1392.96
1401.78
1407.51
1411.57
1416.78
1414.63
1416.39
1414.99
1418.61
1419.47
1419.48
1419.13
1416.84
1413.97
1420.25
1420.7
1421.26
1426.17
1429.04
1429.94
1429.93
1430.56
1430.46
1429.46
1430.13
1430.28
1431.25
1419.78
1425.01
1427.4
1421.38
1427.29
1429.25
1429.02
1428.9
1423.06
1432
1430.76
1434.42
1435.71
1436.25
1436.7
1441
1441.82
1442.66
1444.54
1445.29
1444.91
1445.28
1444.56
1443.28
1445.34
1446.23
1446.87
1447.06
1447.12
1447.27
1447.56
1447.71
1447.82
1450.37
1450.32
1446.7
1447.67
1446.11
1448.31
1449.11
1451.06
1451.07
1452.2
1453.66
1451.5
1438.45
1443.61
1448.95
1452.26
1453.28
1455.58
1454.08
1456
1456.43
1457.06
1455.67
1450.89
1440.97
1448.65
1442.5
1448.45
1448.82
1447.37
        ];
    
    for ii=1:length(PutShortStrategyIndexFixingDates)
        PutShortStrategyIndexFixing(PutShortStrategyIndexFixingDates{ii}) = PutShortStrategyIndexFixingValues(ii);
    end
    
    allFixings('PutShortStrategyIndex') = PutShortStrategyIndexFixing;
    
    putShortStrategyInfo = containers.Map({'nominal','lowerBarrier','basePrice', ...
                                   'callStrike','gearing', ...
                                    'dailyTimeStepsYN',...
                                    'denominatorResetScheduleInt','allFixings'}, ...
                            {nominal,lowerBarrier,basePrice, ...
                            callStrike, gearing,...
                            dailyTimeStepsYN,...
                            denominatorResetScheduleInt,allFixings});

    putShortStrategyParams.params = putShortStrategyInfo;     

    tic
%     pricingOutMCMC = KOSPI200Dupire.computePutShortStrategy1SMCMC(valueDateH,putShortStrategyParams);
    pricingOutMCMC = KOSPI200Dupire.computePutShortStrategy1SMC(valueDateH,putShortStrategyParams);
    toc
    
    simulIndexNPV(idx) = pricingOutMCMC.indexValue.npv;
    simulOptionNPV(idx) = pricingOutMCMC.optionValue.npv;
    
    simulMean(idx) = mean(pricingOutMCMC.indexValue.payoff/basePrice);
    simulStd(idx) = std(pricingOutMCMC.indexValue.payoff/basePrice);
    simulMeanAnn(idx) = log(simulMean(idx))/matuT;
    simulStdAnn(idx) = simulStd(idx)/sqrt(matuT);
end    

 


bbb= 1.0;



In [ ]:
test_PutShortStrategyIndex1S_Reghai_KOSPI200.m


clc
clear
valueDate = '2017-09-05';

%% import and construct a zeroCurve
rawData = [0.002739726	0.012399789
0.25 	0.013831759
0.50 	0.013997163
0.75 	0.014424661
1.00 	0.014852168
1.25 	0.015131926
1.50 	0.015410048
1.75 	0.01568817
2.00 	0.015966292
2.25 	0.016165259
2.50 	0.016361535
2.75 	0.016557811
3.00 	0.016754088
3.25 	0.016888608
3.50 	0.017021745
3.75 	0.017154882
4.00 	0.017288018
4.25 	0.017409599
4.50 	0.017530922
4.75 	0.017652244
5.00 	0.017773566
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a spot For KOSPI200
rawData = 304.06;
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWSpot'});
spotMktData = MktData(mktData);
AllMktData('spot') = spotMktData;

%% import and construct an impliedVolSurface For KOSPI200

rawData = [0	0.70 	0.75 	0.80 	0.85 	0.90 	0.95 	1.00 	1.05 	1.10 	1.15 	1.20 	1.25 	1.30 
30	0.382439	0.345916	0.303095	0.261172	0.224037	0.173528	0.125353	0.106002	0.127818	0.167276	0.199944	0.227108	0.249932
61	0.348244	0.310325	0.271628	0.234333	0.196454	0.161561	0.135375	0.104921	0.107317	0.122378	0.140717	0.158741	0.175702
91	0.314318	0.277755	0.24066	0.204726	0.173888	0.152398	0.134139	0.117248	0.117854	0.128676	0.140786	0.152564	0.163112
183	0.246586	0.217115	0.192976	0.176045	0.163392	0.151517	0.137817	0.121389	0.11961	0.127195	0.133098	0.13736	0.140671
274	0.222498	0.205463	0.192914	0.182743	0.172567	0.160658	0.146826	0.127344	0.124535	0.134437	0.141484	0.146701	0.150687
365	0.209748	0.199005	0.190259	0.181874	0.172487	0.16143	0.148926	0.132726	0.12775	0.137468	0.144796	0.150396	0.154789
548	0.20234	0.195465	0.188641	0.18113	0.172441	0.162671	0.152049	0.139373	0.132846	0.140597	0.14805	0.153977	0.158799
730	0.194813	0.189184	0.183338	0.17716	0.170622	0.163512	0.155195	0.144498	0.129391	0.136032	0.143091	0.148834	0.153582
913	0.191654	0.187255	0.182873	0.178297	0.173223	0.16728	0.159958	0.150549	0.13789	0.134733	0.141248	0.146602	0.151075
1095	0.193952	0.189764	0.185509	0.180995	0.17599	0.170184	0.163177	0.154425	0.143059	0.139839	0.146051	0.151199	0.155547
];

mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess','KRW','KOSPI200VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
% impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';

AllMktData('impliedVolSurface') = impliedVolSurface;

%% import and construct a dividend CurveFor KOSPI200
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200DividendCurve'});
dividendCurveMktData = MktData(mktData);
AllMktData('dividendCurve') = dividendCurveMktData;

%% import and construct a repoRate Curve For KOSPI200

rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
AllMktData('repoRateCurve') = repoRateCurveMktData;

%% import and construct a forward Curve For KOSPI200
rawData = [0	0
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KOSPI200ForwardCurve'});

forwardCurveMktData = MktData(mktData);
AllMktData('forwardCurve') = forwardCurveMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});


KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
maturities = KOSPI200Black.volSurface.maturities;
strikes =    KOSPI200Black.volSurface.strikes;
impVolSurface = KOSPI200Black.volSurface.vol;
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));


for i=1:length(maturities)

    for j=1:length(strikes)
        if strikes(j) <= 0
            dummyStd = impliedVolSurface.volSurfaceR(i,j)*sqrt(expiry(i)/365.0);
            fwdMoneyness(i,j) = exp(norminv(-1.0*strikes(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
        else
            dummyStd = impliedVolSurface.volSurfaceR(i,j)*sqrt(expiry(i)/365.0);
            fwdMoneyness(i,j) = exp(-norminv(1.0*strikes(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
        end

        blackVolInput(i,j) = impliedVolSurface.volSurfaceR(i,j);

        if fwdMoneyness(i,j) <= 1
%   blackOTMPrices2 : the usual black formula
%   blackOTMPrice   : Peter Jackel's Let's be rational
            blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'P');
            blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,-1);
        else
            blackOTMPrices2(i,j) = KOSPI200Black.BlackVanillaFwd(expiry(i),1.0,fwdMoneyness(i,j),blackVolInput(i,j),'C');
            blackOTMPrices(i,j) = blackPrice(1,fwdMoneyness(i,j),blackVolInput(i,j),expiry(i)/365.0,1);
        end
    end
 end


%% create a DupireModel with initial modelParams
%Initialize local volatility
KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;
localVolSurface = KOSPI200ImpliedVolSurface;

modelParams= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurface,'Dupire'});


%% vol calibration grid setting
modelParams('Kmin') = 0.1;
modelParams('Kmax') = 2.5;
modelParams('Ns') = 241;
% modelParams('Ns') = 961;

dK = (modelParams('Kmax') - modelParams('Kmin'))/(modelParams('Ns')-1);
modelParams('dK') = dK;

Ks = zeros(modelParams('Ns'),1);
for i=1:modelParams('Ns')
    Ks(i) = modelParams('Kmin') + modelParams('dK')*(i-1);
end

modelParams('Ks') = Ks;

% restricted interval for calibration target

modelParams('numOfCutoffTenor') = length(maturities);

modelParams('lowerCutoff') = 0.0;
modelParams('upperCutoff') = 2.0;

lowerCutoffVector = modelParams('lowerCutoff')*ones(length(maturities),1);
upperCutoffVector = modelParams('upperCutoff')*ones(length(maturities),1);

modelParams('lowerCutoffVector') = lowerCutoffVector;
modelParams('upperCutoffVector') = upperCutoffVector;


% GFType : (Calibration) Forward Scheme option
%              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
%              0  = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation
%              1  = one step matrix exponential , scaling and squaring with Pade approximation
%              2  = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%              3  = one step Huge Decomposition & multiplication from right RG
%              4  = multi step Huge Decomposition & multiplication from right RG


%%
% % backGFType : (Pricing) backward scheme FD(PDE,GF) option
% %              -1 = one step matrix exponential , expmReghai, simple matrix exponential 
% %              0 = one step matrix exponential , expmdemo1,scaling and squaring with Pade approximation 
% %              1 = one step matrix exponential , scaling and squaring with Pade approximation
% %              2 = multisteps(daily) matrix exponential , scaling and squaring with Pade approximation
%%            21= one step matrix exponential(fwd), daily matrix exponential(backward)
% %              3 =  one step Huge Decomposition & one step Thomas
% %              4 = multi step Huge Decomposition & multistep thomas
% %            : 14= one step matrix exponential(fwd), daily Thomas(backward)
% %              24= multisteps(daily) matrix exponential(fwd) dailyThomas(backward)
% %              34=  one step Huge, daily Thomas(backward)
% 
GFType = -1;
%     backGFType = 14;
backGFType = -21;

% GFType = 4;
% backGFType = 4;

modelParams('GFType') = GFType;
modelParams('backGFType') = backGFType;


% mc time steps in days 
% modelParams('mcOneTimeStep') = 1095;
modelParams('mcOneTimeStep') = 7;
modelParams('NMC') = 2^16;
% modelParams('NMC') = 2^14;

modelParams('useQuasiRandomYN') = 'true';

%pricingSchemeType = 1 for PDE oneTimesStep
%pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
%pricingSchemeType = 4 for one time markov chain monte carlo

modelParams('pricingSchemeType') = 1;

% optimizer setting

modelParams('tol') = 0.0001;
%     modelParams('maxIter') = 1000;
modelParams('maxIter') = 100;

%    modelParams('UseLetsBeRational') = 'YES';
modelParams('UseLetsBeRational') = 'YES';
modelParams('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParams)));


% out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end

%     if idx == 1
%         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrap');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForward');
% %         out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
%     end
tic
out = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
toc
KOSPI200Dupire.fwdMoneyness = out.fwdMoneyness;
KOSPI200Dupire.localVolSurface.volProxy = out.volProxy;
KOSPI200Dupire.localVolSurface.ipos = out.ipos;
KOSPI200Dupire.localVolSurface.localVol = out.localVol;
KOSPI200Dupire.localVolSurface.probMesh = out.probMesh;
KOSPI200Dupire.localVolSurface.condProb = out.condProb;

KOSPI200Dupire.localVolSurface.Ks = out.Ks;
KOSPI200Dupire.localVolSurface.Pricingidx = out.Pricingidx;
KOSPI200Dupire.localVolSurface.expiry = out.expiry;
KOSPI200Dupire.localVolSurface.strike = out.strike;
KOSPI200Dupire.localVolSurface.marketStrikes = out.marketStrikes;

maxIdx = 12;
simulMean = zeros(maxIdx,1);
simulStd = zeros(maxIdx,1);
simulMeanAnn = zeros(maxIdx,1);
simulStdAnn = zeros(maxIdx,1);

for idx=1:maxIdx
    %% Pricing putShortStrategy 1S
    %Steptown info


    putShortStrategyParams.payCcy = 'KRW';
    putShortStrategyParams.startDate = H_Date('2017-09-05');
    putShortStrategyParams.maturity = 1;

    valueDateH = H_Date(valueDate);

    nominal = 10000;
    coupon = 0.076;
    couponFreq = 0.5;
    basePrice = 61.64;
    %     lowerBarrier = 0.5;
    lowerBarrier = 0.47;

    callStrike0 = 0.5;
    callValue0 = 1.0 + 0.228;

    callStrike1 = 0.5;
    callValue1 =  0.5;

    % for dummy


    KIYN = false;
    %     KIYN = true;
                        %resetStart resetEnd  strike
    resetScheduleInt = [20170905	20171005	0.97
    20171005	20171105	0.97
    20171105	20171205	0.97
    20171205	20180105	0.97
    20180105	20180205	0.97
    20180205	20180305	0.97
    20180305	20180405	0.97
    20180405	20180505	0.97
    20180505	20180605	0.97
    20180605	20180705	0.97
    20180705	20180805	0.97
    20180805	20180905	0.97
    20180905	20180905	0.97
    ];

    resetScheduleInt = resetScheduleInt(1:1+idx,:);
    matuT = DateDiff(H_Date(resetScheduleInt(end,1)),H_Date(resetScheduleInt(1,1)))/365.0;
    % autoCallScheduleInt =[20180828	0.95	0.038
    % 20190228	0.95	0.076
    % 20190828	0.9	0.114
    % 20200228	0.9	0.152
    % 20200828	0.85	0.19
    % 20210226	0.6	0.228
    % ];

    overHedgeShift = -0.1;
    overHedgeSpread = 3.0;
    manualScheduleYN = 1;

    SSpayoffYN = 'YES';

    dailyTimeStepsYN = 'YES';
    indexFixing = 1.0;
    pastSFixing = 304.06;
    putShortStrategyInfo = containers.Map({'nominal','coupon','couponFreq','lowerBarrier','basePrice', ...
                                   'callStrike0','callValue0','callStrike1','callValue1', ...
                                    'KIYN', 'SSpayoffYN','dailyTimeStepsYN',...
                                    'resetScheduleInt','overHedgeShift','overHedgeSpread','manualScheduleYN','indexFixing','pastSFixing'}, ...
                            {nominal,coupon,couponFreq,lowerBarrier,basePrice, ...
                            callStrike0, callValue0, callStrike1, callValue1, ...
                            KIYN, SSpayoffYN,dailyTimeStepsYN,...
                            resetScheduleInt,overHedgeShift,overHedgeSpread,manualScheduleYN,indexFixing,pastSFixing});

    putShortStrategyParams.params = putShortStrategyInfo;     

    % tic;
    % pricingOutGF = KOSPI200Dupire.computePutShortStrategy1SGF(valueDateH,putShortStrategyParams);
    % toc;

    % tic
%     pricingOutMC = KOSPI200Dupire.computePutShortStrategy1SMC(valueDateH,putShortStrategyParams);
    % toc

    tic
    pricingOutMCMC = KOSPI200Dupire.computePutShortStrategy1SMCMC(valueDateH,putShortStrategyParams);
    toc
    
    simulMean(idx) = mean(pricingOutMCMC.indexValue.payoff);
    simulStd(idx) = std(pricingOutMCMC.indexValue.payoff);
    simulMeanAnn(idx) = log(mean(pricingOutMCMC.indexValue.payoff))/matuT;
    simulStdAnn(idx) = std(pricingOutMCMC.indexValue.payoff)/sqrt(matuT);
end    

 


bbb= 1.0;



In [ ]:

test_LiborFallback_VanillaSwap_20230119_SOFRCurveFront_2019.m

clc
clear
tic

valueDate = '2023-01-19';

useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-04'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-05-01'
'2026-05-05'
'2026-09-24'
'2026-09-25'
'2026-10-09'
'2026-12-25'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
    end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.00175380});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.00175380;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = [
0.002739726	0.043594619
0.019178082	0.043702012
0.038356164	0.044326444
0.057534247	0.044974705
0.083333333	0.04540879
0.166666667	0.04590952
0.25	0.046758874
0.5	0.047993824
0.75	0.048204779
1	0.047696907
2	0.040839682
3	0.036049391
4	0.03353264
5	0.032141073
6	0.031326467
7	0.030835602
8	0.030547674
9	0.030431235
10	0.030416947
12	0.030538286
15	0.030785077
20	0.030464768
25	0.029213652
30	0.02795406
40	0.025006251
50	0.021974912
];

rawData_Libor3M_DC = [
0.087671232876712	0.001098799851025
0.167123287671233	0.001492765168950
0.252054794520548	0.001777759972902
0.509589041095890	0.002093881108158
1.000000000000000	0.002082797416369
2.005479452054790	0.002757637808099
3.005479452054790	0.004599469233654
4.002739726027400	0.006983466216479
5.002739726027400	0.009201864211354
7.010958904109590	0.012826605809096
10.005479452054800	0.016117892407056
12.008219178082200	0.017531896005676
15.010958904109600	0.018870232807240
20.016438356164400	0.019987631397089
25.021917808219200	0.020322045675130
30.024657534246600	0.020377680903766
50.032876712328800	0.018354038457619
];

rawData_Libor3M_SC = [
0.002739726	0.043690217
0.083333333	0.045139798
0.166666667	0.046757083
0.25	0.048153085
0.5	0.050617731
0.75	0.050246323
1	0.049764664
1.5	0.046545703
2	0.043276196
2.5	0.04084174
3	0.038430807
3.5	0.037219362
4	0.036041706
4.5	0.035357371
5	0.034692713
5.5	0.034371505
6	0.034061972
6.5	0.033752439
7	0.033442906
7.5	0.033370461
8	0.033306077
8.5	0.033241692
9	0.033177307
9.5	0.033112922
10	0.033048537
10.5	0.033082554
11	0.033121083
11.5	0.033159611
12	0.033198139
12.5	0.033237542
13	0.033276979
13.5	0.033316416
14	0.033355853
14.5	0.03339529
15	0.033434727
15.5	0.03340911
16	0.033379721
16.5	0.033350332
17	0.033320943
17.5	0.033291554
18	0.033262165
18.5	0.033232776
19	0.033203387
19.5	0.033173998
20	0.033144609
20.5	0.033022127
21	0.032894816
21.5	0.032767505
22	0.032640194
22.5	0.032512883
23	0.032385572
23.5	0.032258261
24	0.03213095
24.5	0.032003639
25	0.031876328
25.5	0.031745005
26	0.031613448
26.5	0.031481892
27	0.031350336
27.5	0.031218779
28	0.031087223
28.5	0.030955667
29	0.030824111
29.5	0.030692554
30	0.030560998
30.5	0.030404257
31	0.030245742
31.5	0.030087228
32	0.029928714
32.5	0.029770199
33	0.029611685
33.5	0.029453171
34	0.029294657
34.5	0.029136142
35	0.028977628
35.5	0.028819114
36	0.0286606
36.5	0.028502085
37	0.028343571
37.5	0.028185057
38	0.028026542
38.5	0.027868028
39	0.027709514
39.5	0.027551
40	0.027392485
40.5	0.027233971
41	0.027075457
41.5	0.026916943
42	0.026758428
42.5	0.026599914
43	0.0264414
43.5	0.026282885
44	0.026124371
44.5	0.025965857
45	0.025807343
45.5	0.025648828
46	0.025490314
46.5	0.0253318
47	0.025173286
47.5	0.025014771
];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
referenceDate = H_Date('2015-04-28');
% 

% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '6Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 2;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2033-06-30');

% startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
%                                     fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);
                                
startDate = H_Date('2019-04-30');


useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth); 
            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);

                    
% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;
floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

floatingLeg = FloatingRateLeg(floatingSchedule,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                   
numOfFloatingCoupons = length(floatingLeg.couponRates);

floatingCouponFallbackStartIdx = 1;
for idx=1:numOfFloatingCoupons
    tempDateDiff = DateDiff(floatingLeg.couponRates{idx}.fixingDate,liborTermDate);
    if tempDateDiff <=0
        floatingCouponFallbackStartIdx = floatingCouponFallbackStartIdx +1;
    else
        break;
    end
end

numOfFallbackCoupons = numOfFloatingCoupons - floatingCouponFallbackStartIdx + 1;

if numOfFallbackCoupons <=0
    disp('no fallback coupons !');
    aaa = 1.0;
    error('no fallback coupon');
end


rateRecordDate = cell(numOfFloatingCoupons,1);
accrualSpotDate = cell(numOfFloatingCoupons,1);
accrualStartDate = cell(numOfFloatingCoupons,1);
accrualEndDate = cell(numOfFloatingCoupons,1);
accrualEnd2BDDate = cell(numOfFloatingCoupons,1);
floatingRatePaymentDate = cell(numOfFloatingCoupons,1);
dateDiffDays = zeros(numOfFloatingCoupons,1);

rateRecordDateStr = cell(numOfFloatingCoupons,1);
accrualSpotDateStr = cell(numOfFloatingCoupons,1);
accrualStartDateStr = cell(numOfFloatingCoupons,1);
accrualEndDateStr = cell(numOfFloatingCoupons,1);
accrualEnd2BDDateStr = cell(numOfFloatingCoupons,1);
floatingRatePaymentDateStr = cell(numOfFloatingCoupons,1);

for idx=1:numOfFloatingCoupons
    rateRecordDate{idx} = floatingLeg.couponRates{idx}.fixingDate;
    accrualSpotDate{idx} = USGTCalendar.Advance(rateRecordDate{idx},overnightIndex.convention,...
                                    fallBackSpotLag,'day',overnightIndex.endOfMonth);

    accrualStartDate{idx} = USGTCalendar.Advance(accrualSpotDate{idx},overnightIndex.convention,...
                                    -1*fallBackOffsetLag,'day',overnightIndex.endOfMonth);
                                
    accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},overnightIndex.convention,...
                                    str2double(floatingLegTenor(1:end-1)),'month',overnightIndex.endOfMonth);

    accrualEnd2BDDate{idx} = calcCalendar.Advance(accrualEndDate{idx},'Following', 2,'day',false);
                           
    floatingRatePaymentDate{idx} = floatingLeg.couponRates{idx}.paymentDate;
    
    dateDiffDays(idx) = DateDiff(floatingRatePaymentDate{idx},accrualEnd2BDDate{idx});
    
end

for idx=1:numOfFloatingCoupons
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    accrualEnd2BDDateStr{idx} = StrDate(accrualEnd2BDDate{idx},'str');
    floatingRatePaymentDateStr{idx} = StrDate(floatingRatePaymentDate{idx},'str');
    
end

new_rateRecordDate = cell(numOfFloatingCoupons,1);
new_accrualSpotDate = cell(numOfFloatingCoupons,1);
new_accrualStartDate = cell(numOfFloatingCoupons,1);
new_accrualEndDate = cell(numOfFloatingCoupons,1);
new_accrualEnd2BDDate = cell(numOfFloatingCoupons,1);
new_floatingRatePaymentDate = cell(numOfFloatingCoupons,1);
new_dateDiffDays = zeros(numOfFloatingCoupons,1);

new_rateRecordDateStr = cell(numOfFloatingCoupons,1);
new_accrualSpotDateStr = cell(numOfFloatingCoupons,1);
new_accrualStartDateStr = cell(numOfFloatingCoupons,1);
new_accrualEndDateStr = cell(numOfFloatingCoupons,1);
new_accrualEnd2BDDateStr = cell(numOfFloatingCoupons,1);
new_floatingRatePaymentDateStr = cell(numOfFloatingCoupons,1);

for idx=1:numOfFloatingCoupons
    new_rateRecordDate{idx} = rateRecordDate{idx};
    new_accrualSpotDate{idx} = accrualSpotDate{idx};
    new_accrualStartDate{idx} = accrualStartDate{idx};
    new_accrualEndDate{idx} = accrualEndDate{idx};
    new_accrualEnd2BDDate{idx} = accrualEnd2BDDate{idx};
    new_floatingRatePaymentDate{idx} = floatingRatePaymentDate{idx};
    new_dateDiffDays(idx) = dateDiffDays(idx);
    
    temp_rateRecordDate = new_rateRecordDate{idx};
    temp_accrualSpotDate = new_accrualSpotDate{idx};
    temp_accrualStartDate = new_accrualStartDate{idx};
    temp_accrualEndDate = new_accrualEndDate{idx};
    temp_accrualEnd2BDDate = new_accrualEnd2BDDate{idx};
    
    if new_dateDiffDays(idx) < 0
        dummyDiffDays = new_dateDiffDays(idx);
        temp_rateRecordDate = new_rateRecordDate{idx};
        while dummyDiffDays <0
            if idx == 23
                eee = 1.0;
            end
            
            temp_rateRecordDate = GBENCalendar.Advance(temp_rateRecordDate,'Following', -1,'day',false);
            if DateDiff(new_rateRecordDate{idx},temp_rateRecordDate) < 0
                ccc = 1.0;
                error('temp_rateRecordDate should be previous of new_rateRecordDate')
            end
            
            temp_accrualSpotDate = USGTCalendar.Advance(temp_rateRecordDate,overnightIndex.convention,...
                                    fallBackSpotLag,'day',overnightIndex.endOfMonth);

            temp_accrualStartDate = USGTCalendar.Advance(temp_accrualSpotDate,overnightIndex.convention,...
                                            -1*fallBackOffsetLag,'day',overnightIndex.endOfMonth);

            temp_accrualEndDate = USGTCalendar.Advance(temp_accrualStartDate,overnightIndex.convention,...
                                            str2double(floatingLegTenor(1:end-1)),'month',overnightIndex.endOfMonth);

            temp_accrualEnd2BDDate = calcCalendar.Advance(temp_accrualEndDate,'Following', 2,'day',false);
            temp_dateDiffDays = DateDiff(floatingRatePaymentDate{idx},temp_accrualEnd2BDDate);
            if dummyDiffDays > temp_dateDiffDays
                aaa = 1.0;
                error('infinitely recurring loop!')
                
            end
            
            dummyDiffDays = temp_dateDiffDays;
            
        end
    
        new_rateRecordDate{idx} = temp_rateRecordDate;
        new_accrualSpotDate{idx} = temp_accrualSpotDate;
        new_accrualStartDate{idx} = temp_accrualStartDate;
        new_accrualEndDate{idx} = temp_accrualEndDate;
        new_accrualEnd2BDDate{idx} = temp_accrualEnd2BDDate;
        new_dateDiffDays(idx) = DateDiff(floatingRatePaymentDate{idx},temp_accrualEnd2BDDate);
        
        if new_dateDiffDays < 0
            bbb = 1.0;
            error('End2BDDate should be before or the same as paymentDate')
        end
    end
    
end

for idx=1:numOfFloatingCoupons
    new_rateRecordDateStr{idx} = StrDate(new_rateRecordDate{idx},'str');
    new_accrualSpotDateStr{idx} = StrDate(new_accrualSpotDate{idx},'str');
    new_accrualStartDateStr{idx} = StrDate(new_accrualStartDate{idx},'str');
    new_accrualEndDateStr{idx} = StrDate(new_accrualEndDate{idx},'str');
    new_accrualEnd2BDDateStr{idx} = StrDate(new_accrualEnd2BDDate{idx},'str');
    new_floatingRatePaymentDateStr{idx} = StrDate(new_floatingRatePaymentDate{idx},'str');
    
end


floatingLeg_RFR_Schedule = ScheduleWTermConvention(startDate,endDate,floatingLeg_RFR_Tenor,calcCalendar,...
                            floatingLeg_RFR_Convention,floatingLeg_RFR_Convention, 'Backward',useEndOfMonth);



vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
                          fixedSchedule,fixedRate,fixedDayCounter,...  
                          floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
                          floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);

out = vanillaSwap.Calculate(discountCurve_SOFR);

fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:

test_LiborFallback_VanillaSwap_20230119_SOFRCurveFront_0125.m

clc
clear
tic

valueDate = '2023-01-19';


% we do not apply 2BD Rule

apply2BDFallbackRuleFlag = false;


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-04'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-05-01'
'2026-05-05'
'2026-09-24'
'2026-09-25'
'2026-10-09'
'2026-12-25'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
dummySOFRFixing = {
'2022-10-27'	0.0304
'2022-10-28'	0.0305
'2022-10-31'	0.0305
'2022-11-01'	0.0305
'2022-11-02'	0.0305
'2022-11-03'	0.0380
'2022-11-04'	0.0380
'2022-11-07'	0.0378
'2022-11-08'	0.0378
'2022-11-09'	0.0378
'2022-11-10'	0.0378
'2022-11-11'	0.0378
'2022-11-14'	0.0379
'2022-11-15'	0.0380
'2022-11-16'	0.0381
'2022-11-17'	0.0380
'2022-11-18'	0.0380
'2022-11-21'	0.0380
'2022-11-22'	0.0380
'2022-11-23'	0.0379
'2022-11-25'	0.0380
'2022-11-28'	0.0380
'2022-11-29'	0.0381
'2022-11-30'	0.0382
'2022-12-01'	0.0382
'2022-12-02'	0.0381
'2022-12-05'	0.0381
'2022-12-06'	0.0380
'2022-12-07'	0.0380
'2022-12-08'	0.0380
'2022-12-09'	0.0380
'2022-12-12'	0.0380
'2022-12-13'	0.0380
'2022-12-14'	0.0380
'2022-12-15'	0.0432
'2022-12-16'	0.0432
'2022-12-19'	0.0430
'2022-12-20'	0.0430
'2022-12-21'	0.0430
'2022-12-22'	0.0430
'2022-12-23'	0.0430
'2022-12-26'	0.0430
'2022-12-27'	0.0430
'2022-12-28'	0.0430
'2022-12-29'	0.0430
'2022-12-30'	0.0430
'2023-01-02'	0.0430
'2023-01-03'	0.0431
'2023-01-04'	0.0430
'2023-01-05'	0.0431
'2023-01-06'	0.0431
'2023-01-09'	0.0431
'2023-01-10'	0.0431
'2023-01-11'	0.0430
'2023-01-12'	0.0430
'2023-01-13'	0.0430
'2023-01-16'	0.0430
'2023-01-17'	0.0431
'2023-01-18'	0.0430
'2023-01-19'	0.0431
'2023-01-20'	0.0430
'2023-01-23'	0.0430
'2023-01-24'	0.0430
'2023-01-25'	0.0430
};

irIndexFixing_SOFR = containers.Map({dummySOFRFixing{1,1}},{dummySOFRFixing{1,2}});
for idxhong=2:size(dummySOFRFixing,1)
    irIndexFixing_SOFR(dummySOFRFixing{idxhong,1}) = dummySOFRFixing{idxhong,2};
end

aaa = 1.0;

% tempDate = H_Date('2021-05-04');
% irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
% for idxhong=1:10000
%     tempDate = AddDate(tempDate,1,'day');
%     irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
% end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.0441471});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.0441471;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

%Samsung Front System  weird??
% rawData_SOFR = [
% 0.00273972603	0.04359461870
% 0.01917808219	0.04370201200
% 0.03835616438	0.04432644440
% 0.05753424658	0.04497470510
% 0.08333333333	0.04540878960
% 0.16666666667	0.04590952010
% 0.25000000000	0.04675887430
% 0.50000000000	0.04799382350
% 0.75000000000	0.04820477850
% 1.00000000000	0.04769690700
% 2.00000000000	0.04083968150
% 3.00000000000	0.03604939100
% 4.00000000000	0.03353264000
% 5.00000000000	0.03214107290
% 6.00000000000	0.03132646730
% 7.00000000000	0.03083560170
% 8.00000000000	0.03054767440
% 9.00000000000	0.03043123480
% 10.00000000000	0.03041694650
% 12.00000000000	0.03053828620
% 15.00000000000	0.03078507670
% 20.00000000000	0.03046476800
% 25.00000000000	0.02921365170
% 30.00000000000	0.02795405990
% 40.00000000000	0.02500625100
% 50.00000000000	0.02197491160
% ];

%Bloomberg
rawData_SOFR = [
0.01369863	0.043667058
0.030136986	0.043762482
0.049315068	0.04435542
0.068493151	0.044978813
0.095890411	0.045400395
0.17260274	0.045910712
0.260273973	0.046757468
0.509589041	0.048002924
0.75890411	0.048213335
1.010958904	0.047669389
2.01369863	0.040868784
3.01369863	0.036047358
4.019178082	0.033549637
5.016438356	0.03215024
6.016438356	0.031343975
7.016438356	0.030849127
8.016438356	0.03055894
9.016438356	0.030449211
10.02191781	0.030429693
12.01917808	0.030560795
15.02739726	0.030796081
20.02465753	0.030487161
25.02739726	0.029234585
30.03287671	0.027956203
40.03835616	0.02512122
50.04657534	0.022007343
];

rawData_Libor3M_DC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];

rawData_Libor3M_SC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
referenceDate = H_Date('2015-04-28');
% 

% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '6Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 0;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2022-06-30');

% liborTermDate = H_Date('2033-06-30');

% startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
%                                     fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);
                                
startDate = H_Date('2019-04-30');


useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth);
 


            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);

                    
% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;

fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedLegConvention,payLag);



floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

floatingLeg = FloatingRateLeg(floatingSchedule,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                   
numOfFloatingCoupons = length(floatingLeg.couponRates);

floatingCouponFallbackStartIdx = 1;
for idx=1:numOfFloatingCoupons
    tempDateDiff = DateDiff(floatingLeg.couponRates{idx}.fixingDate,liborTermDate);
    if tempDateDiff <=0
        floatingCouponFallbackStartIdx = floatingCouponFallbackStartIdx +1;
    else
        break;
    end
end

% we construct Libor Floating Leg(floatingLegLIbor) & 
% Libor Fallback FloatingLeg(overnightLeg_ARR)
% separately without redundancy

% constructing floatingLegLibor start
floatingLegLiborSize = floatingCouponFallbackStartIdx-1;
floatingLegLIbor = FloatingRateLegWithUsedScheduleSize(floatingSchedule,floatingLegLiborSize,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                                  
% constructing floatingLegLibor end

% constructing overnightLeg_ARR start

numOfFallbackCoupons = numOfFloatingCoupons - floatingCouponFallbackStartIdx + 1;

if numOfFallbackCoupons <=0
    disp('no fallback coupons !');
    aaa = 1.0;
    error('no fallback coupon');
end


rateRecordDate = cell(numOfFallbackCoupons,1);
accrualSpotDate = cell(numOfFallbackCoupons,1);
accrualStartDate = cell(numOfFallbackCoupons,1);
accrualEndDate = cell(numOfFallbackCoupons,1);
accrualEnd2BDDate = cell(numOfFallbackCoupons,1);
floatingRatePaymentDate = cell(numOfFallbackCoupons,1);
dateDiffDays = zeros(numOfFallbackCoupons,1);

rateRecordDateStr = cell(numOfFallbackCoupons,1);
accrualSpotDateStr = cell(numOfFallbackCoupons,1);
accrualStartDateStr = cell(numOfFallbackCoupons,1);
accrualEndDateStr = cell(numOfFallbackCoupons,1);
accrualEnd2BDDateStr = cell(numOfFallbackCoupons,1);
floatingRatePaymentDateStr = cell(numOfFallbackCoupons,1);

for idx=1:numOfFallbackCoupons
    idx2 = idx + floatingCouponFallbackStartIdx -1;
    rateRecordDate{idx} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.fixingDate;
    accrualSpotDate{idx} = USGTCalendar.Advance(rateRecordDate{idx},overnightIndex.convention,...
                                    fallBackSpotLag,'day',overnightIndex.endOfMonth);

    accrualStartDate{idx} = USGTCalendar.Advance(accrualSpotDate{idx},overnightIndex.convention,...
                                    -1*fallBackOffsetLag,'day',overnightIndex.endOfMonth);
                                
    accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},overnightIndex.convention,...
                                    str2double(floatingLegTenor(1:end-1)),'month',overnightIndex.endOfMonth);

    accrualEnd2BDDate{idx} = calcCalendar.Advance(accrualEndDate{idx},'Following', 2,'day',false);
                           
    floatingRatePaymentDate{idx} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.paymentDate;
    
    dateDiffDays(idx) = DateDiff(floatingRatePaymentDate{idx},accrualEnd2BDDate{idx});
    
end

for idx=1:numOfFallbackCoupons
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    accrualEnd2BDDateStr{idx} = StrDate(accrualEnd2BDDate{idx},'str');
    floatingRatePaymentDateStr{idx} = StrDate(floatingRatePaymentDate{idx},'str');
    
end


new_rateRecordDate = cell(numOfFallbackCoupons,1);
new_accrualSpotDate = cell(numOfFallbackCoupons,1);
new_accrualStartDate = cell(numOfFallbackCoupons,1);
new_accrualEndDate = cell(numOfFallbackCoupons,1);
new_accrualEnd2BDDate = cell(numOfFallbackCoupons,1);
new_floatingRatePaymentDate = cell(numOfFallbackCoupons,1);
new_dateDiffDays = zeros(numOfFallbackCoupons,1);

new_rateRecordDateStr = cell(numOfFallbackCoupons,1);
new_accrualSpotDateStr = cell(numOfFallbackCoupons,1);
new_accrualStartDateStr = cell(numOfFallbackCoupons,1);
new_accrualEndDateStr = cell(numOfFallbackCoupons,1);
new_accrualEnd2BDDateStr = cell(numOfFallbackCoupons,1);
new_floatingRatePaymentDateStr = cell(numOfFallbackCoupons,1);


if apply2BDFallbackRuleFlag
    for idx=1:numOfFallbackCoupons
        new_rateRecordDate{idx} = rateRecordDate{idx};
        new_accrualSpotDate{idx} = accrualSpotDate{idx};
        new_accrualStartDate{idx} = accrualStartDate{idx};
        new_accrualEndDate{idx} = accrualEndDate{idx};
        new_accrualEnd2BDDate{idx} = accrualEnd2BDDate{idx};
        new_floatingRatePaymentDate{idx} = floatingRatePaymentDate{idx};
        new_dateDiffDays(idx) = dateDiffDays(idx);

        temp_rateRecordDate = new_rateRecordDate{idx};
        temp_accrualSpotDate = new_accrualSpotDate{idx};
        temp_accrualStartDate = new_accrualStartDate{idx};
        temp_accrualEndDate = new_accrualEndDate{idx};
        temp_accrualEnd2BDDate = new_accrualEnd2BDDate{idx};

        if new_dateDiffDays(idx) < 0
            dummyDiffDays = new_dateDiffDays(idx);
            temp_rateRecordDate = new_rateRecordDate{idx};
            while dummyDiffDays <0
                if idx == 23
                    eee = 1.0;
                end

                temp_rateRecordDate = GBENCalendar.Advance(temp_rateRecordDate,'Following', -1,'day',false);
                if DateDiff(new_rateRecordDate{idx},temp_rateRecordDate) < 0
                    ccc = 1.0;
                    error('temp_rateRecordDate should be previous of new_rateRecordDate')
                end

                temp_accrualSpotDate = USGTCalendar.Advance(temp_rateRecordDate,overnightIndex.convention,...
                                        fallBackSpotLag,'day',overnightIndex.endOfMonth);

                temp_accrualStartDate = USGTCalendar.Advance(temp_accrualSpotDate,overnightIndex.convention,...
                                                -1*fallBackOffsetLag,'day',overnightIndex.endOfMonth);

                temp_accrualEndDate = USGTCalendar.Advance(temp_accrualStartDate,overnightIndex.convention,...
                                                str2double(floatingLegTenor(1:end-1)),'month',overnightIndex.endOfMonth);

                temp_accrualEnd2BDDate = calcCalendar.Advance(temp_accrualEndDate,'Following', 2,'day',false);
                temp_dateDiffDays = DateDiff(floatingRatePaymentDate{idx},temp_accrualEnd2BDDate);
                if dummyDiffDays > temp_dateDiffDays
                    aaa = 1.0;
                    error('infinitely recurring loop!')

                end

                dummyDiffDays = temp_dateDiffDays;

            end

            new_rateRecordDate{idx} = temp_rateRecordDate;
            new_accrualSpotDate{idx} = temp_accrualSpotDate;
            new_accrualStartDate{idx} = temp_accrualStartDate;
            new_accrualEndDate{idx} = temp_accrualEndDate;
            new_accrualEnd2BDDate{idx} = temp_accrualEnd2BDDate;
            new_dateDiffDays(idx) = DateDiff(floatingRatePaymentDate{idx},temp_accrualEnd2BDDate);

            if new_dateDiffDays < 0
                bbb = 1.0;
                error('End2BDDate should be before or the same as paymentDate')
            end
        end

    end

else

    for idx=1:numOfFallbackCoupons
        new_rateRecordDate{idx} = rateRecordDate{idx};
        new_accrualSpotDate{idx} = accrualSpotDate{idx};
        new_accrualStartDate{idx} = accrualStartDate{idx};
        new_accrualEndDate{idx} = accrualEndDate{idx};
        new_accrualEnd2BDDate{idx} = accrualEnd2BDDate{idx};
        new_floatingRatePaymentDate{idx} = floatingRatePaymentDate{idx};
        new_dateDiffDays(idx) = dateDiffDays(idx);
    end

end


for idx=1:numOfFallbackCoupons
    new_rateRecordDateStr{idx} = StrDate(new_rateRecordDate{idx},'str');
    new_accrualSpotDateStr{idx} = StrDate(new_accrualSpotDate{idx},'str');
    new_accrualStartDateStr{idx} = StrDate(new_accrualStartDate{idx},'str');
    new_accrualEndDateStr{idx} = StrDate(new_accrualEndDate{idx},'str');
    new_accrualEnd2BDDateStr{idx} = StrDate(new_accrualEnd2BDDate{idx},'str');
    new_floatingRatePaymentDateStr{idx} = StrDate(new_floatingRatePaymentDate{idx},'str');
    
end




manualSchedule = cell(numOfFallbackCoupons,3);
liborFloatingSchedule = cell(numOfFallbackCoupons,2);

for idx=1:numOfFallbackCoupons
    manualSchedule{idx,1} = new_accrualStartDate{idx};
    manualSchedule{idx,2} = new_accrualEndDate{idx};
    manualSchedule{idx,3} = new_floatingRatePaymentDate{idx};
    
    liborFloatingSchedule{idx,1} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.accrualStartDate;
    liborFloatingSchedule{idx,2} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.accrualEndDate;
    
end


% adjustingFactor = DayCount_Libor/DayCount_RFR
% for usd adjustingFactor = 1.0

adjustingFactor = 1.0;

floatingLeg_RFR = OvernightLegARRWithManualSchedule(manualSchedule,liborFloatingSchedule,overnightIndex,1.0,...
                                      floatingLeg_RFR_Spread,nominal,...
                                      floatingLegDayCounter,floatingLeg_RFR_DayCounter,floatingLeg_RFR_Convention,calcCalendar,adjustingFactor);

% constructing overnightLeg_ARR end

aaa = 1.0;



% vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
%                           fixedSchedule,fixedRate,fixedDayCounter,...  
%                           floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
%                           floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);


vanillaSwap = VanillaSwapLiborFallback(type,nominal,liborTermDate,...
                                  fixedLeg,...  
                                  floatingLegLIbor,libor3M_Index_fwd,...
                                  floatingLeg_RFR,overnightIndex);

out = vanillaSwap.Calculate(discountCurve_SOFR);
out3 = vanillaSwap.Calculate3(discountCurve_SOFR,discountCurve_SOFR,discountCurve_SOFR);


fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:
test_LiborFallback_VanillaSwap_20230119_SOFRCurveFront.m

clc
clear
tic

valueDate = '2023-01-19';

useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-04-30'
'2022-01-31'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
    end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.00175380});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.00175380;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = [
0.002739726	0.043594619
0.019178082	0.043702012
0.038356164	0.044326444
0.057534247	0.044974705
0.083333333	0.04540879
0.166666667	0.04590952
0.25	0.046758874
0.5	0.047993824
0.75	0.048204779
1	0.047696907
2	0.040839682
3	0.036049391
4	0.03353264
5	0.032141073
6	0.031326467
7	0.030835602
8	0.030547674
9	0.030431235
10	0.030416947
12	0.030538286
15	0.030785077
20	0.030464768
25	0.029213652
30	0.02795406
40	0.025006251
50	0.021974912
];

rawData_Libor3M_DC = [
0.087671232876712	0.001098799851025
0.167123287671233	0.001492765168950
0.252054794520548	0.001777759972902
0.509589041095890	0.002093881108158
1.000000000000000	0.002082797416369
2.005479452054790	0.002757637808099
3.005479452054790	0.004599469233654
4.002739726027400	0.006983466216479
5.002739726027400	0.009201864211354
7.010958904109590	0.012826605809096
10.005479452054800	0.016117892407056
12.008219178082200	0.017531896005676
15.010958904109600	0.018870232807240
20.016438356164400	0.019987631397089
25.021917808219200	0.020322045675130
30.024657534246600	0.020377680903766
50.032876712328800	0.018354038457619
];

rawData_Libor3M_SC = [
0.002739726	0.043690217
0.083333333	0.045139798
0.166666667	0.046757083
0.25	0.048153085
0.5	0.050617731
0.75	0.050246323
1	0.049764664
1.5	0.046545703
2	0.043276196
2.5	0.04084174
3	0.038430807
3.5	0.037219362
4	0.036041706
4.5	0.035357371
5	0.034692713
5.5	0.034371505
6	0.034061972
6.5	0.033752439
7	0.033442906
7.5	0.033370461
8	0.033306077
8.5	0.033241692
9	0.033177307
9.5	0.033112922
10	0.033048537
10.5	0.033082554
11	0.033121083
11.5	0.033159611
12	0.033198139
12.5	0.033237542
13	0.033276979
13.5	0.033316416
14	0.033355853
14.5	0.03339529
15	0.033434727
15.5	0.03340911
16	0.033379721
16.5	0.033350332
17	0.033320943
17.5	0.033291554
18	0.033262165
18.5	0.033232776
19	0.033203387
19.5	0.033173998
20	0.033144609
20.5	0.033022127
21	0.032894816
21.5	0.032767505
22	0.032640194
22.5	0.032512883
23	0.032385572
23.5	0.032258261
24	0.03213095
24.5	0.032003639
25	0.031876328
25.5	0.031745005
26	0.031613448
26.5	0.031481892
27	0.031350336
27.5	0.031218779
28	0.031087223
28.5	0.030955667
29	0.030824111
29.5	0.030692554
30	0.030560998
30.5	0.030404257
31	0.030245742
31.5	0.030087228
32	0.029928714
32.5	0.029770199
33	0.029611685
33.5	0.029453171
34	0.029294657
34.5	0.029136142
35	0.028977628
35.5	0.028819114
36	0.0286606
36.5	0.028502085
37	0.028343571
37.5	0.028185057
38	0.028026542
38.5	0.027868028
39	0.027709514
39.5	0.027551
40	0.027392485
40.5	0.027233971
41	0.027075457
41.5	0.026916943
42	0.026758428
42.5	0.026599914
43	0.0264414
43.5	0.026282885
44	0.026124371
44.5	0.025965857
45	0.025807343
45.5	0.025648828
46	0.025490314
46.5	0.0253318
47	0.025173286
47.5	0.025014771
];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
referenceDate = H_Date('2015-04-28');


% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '10Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 2;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2033-06-30');

startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
                                    fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);

useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth); 
            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);
                    
floatingLeg_RFR_Schedule = ScheduleWTermConvention(startDate,endDate,floatingLeg_RFR_Tenor,calcCalendar,...
                            floatingLeg_RFR_Convention,floatingLeg_RFR_Convention, 'Backward',useEndOfMonth);

% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;
floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
                          fixedSchedule,fixedRate,fixedDayCounter,...  
                          floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
                          floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);

out = vanillaSwap.Calculate(discountCurve_SOFR);

fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:
test_LiborFallback_VanillaSwap_20230119.m


clc
clear
tic

valueDate = '2023-01-19';

useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-04-30'
'2022-01-31'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
    end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.00175380});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.00175380;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = [
0.002739726027	0.000101388875
0.019178082192	0.000192790616
0.038356164384	0.000188582651
0.057534246575	0.000190610066
0.087671232877	0.000186047094
0.167123287671	0.000282614854
0.252054794521	0.000344707247
0.339726027397	0.000367004898
0.419178082192	0.000400452499
0.509589041096	0.000415650422
0.586301369863	0.000427300637
0.671232876712	0.000435908443
0.758904109589	0.000446542384
0.835616438356	0.000470858745
0.917808219178	0.000496692325
1.000000000000	0.000516949692
1.506849315068	0.000720697544
2.005479452055	0.001120059319
3.002739726027	0.002714418174
4.002739726027	0.004933516134
5.002739726027	0.007049886611
6.002739726027	0.008952079387
7.010958904110	0.010559045064
8.008219178082	0.011830012220
9.005479452055	0.012885165460
10.005479452055	0.013741279433
12.008219178082	0.015097592964
15.010958904110	0.016378720092
20.013698630137	0.017426478939
25.019178082192	0.017726708805
30.024657534247	0.017768042382
40.027397260274	0.016871633528
50.032876712329	0.015711420383
];

rawData_Libor3M_DC = [
0.087671232876712	0.001098799851025
0.167123287671233	0.001492765168950
0.252054794520548	0.001777759972902
0.509589041095890	0.002093881108158
1.000000000000000	0.002082797416369
2.005479452054790	0.002757637808099
3.005479452054790	0.004599469233654
4.002739726027400	0.006983466216479
5.002739726027400	0.009201864211354
7.010958904109590	0.012826605809096
10.005479452054800	0.016117892407056
12.008219178082200	0.017531896005676
15.010958904109600	0.018870232807240
20.016438356164400	0.019987631397089
25.021917808219200	0.020322045675130
30.024657534246600	0.020377680903766
50.032876712328800	0.018354038457619
];

rawData_Libor3M_SC = [
0.002739726	0.043690217
0.083333333	0.045139798
0.166666667	0.046757083
0.25	0.048153085
0.5	0.050617731
0.75	0.050246323
1	0.049764664
1.5	0.046545703
2	0.043276196
2.5	0.04084174
3	0.038430807
3.5	0.037219362
4	0.036041706
4.5	0.035357371
5	0.034692713
5.5	0.034371505
6	0.034061972
6.5	0.033752439
7	0.033442906
7.5	0.033370461
8	0.033306077
8.5	0.033241692
9	0.033177307
9.5	0.033112922
10	0.033048537
10.5	0.033082554
11	0.033121083
11.5	0.033159611
12	0.033198139
12.5	0.033237542
13	0.033276979
13.5	0.033316416
14	0.033355853
14.5	0.03339529
15	0.033434727
15.5	0.03340911
16	0.033379721
16.5	0.033350332
17	0.033320943
17.5	0.033291554
18	0.033262165
18.5	0.033232776
19	0.033203387
19.5	0.033173998
20	0.033144609
20.5	0.033022127
21	0.032894816
21.5	0.032767505
22	0.032640194
22.5	0.032512883
23	0.032385572
23.5	0.032258261
24	0.03213095
24.5	0.032003639
25	0.031876328
25.5	0.031745005
26	0.031613448
26.5	0.031481892
27	0.031350336
27.5	0.031218779
28	0.031087223
28.5	0.030955667
29	0.030824111
29.5	0.030692554
30	0.030560998
30.5	0.030404257
31	0.030245742
31.5	0.030087228
32	0.029928714
32.5	0.029770199
33	0.029611685
33.5	0.029453171
34	0.029294657
34.5	0.029136142
35	0.028977628
35.5	0.028819114
36	0.0286606
36.5	0.028502085
37	0.028343571
37.5	0.028185057
38	0.028026542
38.5	0.027868028
39	0.027709514
39.5	0.027551
40	0.027392485
40.5	0.027233971
41	0.027075457
41.5	0.026916943
42	0.026758428
42.5	0.026599914
43	0.0264414
43.5	0.026282885
44	0.026124371
44.5	0.025965857
45	0.025807343
45.5	0.025648828
46	0.025490314
46.5	0.0253318
47	0.025173286
47.5	0.025014771
];

% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
referenceDate = H_Date('2015-04-28');


% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '10Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 2;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2033-06-30');

startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
                                    fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);

useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth); 
            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);
                    
floatingLeg_RFR_Schedule = ScheduleWTermConvention(startDate,endDate,floatingLeg_RFR_Tenor,calcCalendar,...
                            floatingLeg_RFR_Convention,floatingLeg_RFR_Convention, 'Backward',useEndOfMonth);

% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;
floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
                          fixedSchedule,fixedRate,fixedDayCounter,...  
                          floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
                          floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);

out = vanillaSwap.Calculate(discountCurve_SOFR);

fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:
test_LiborFallback_VanillaSwap.m

clc
clear
tic

valueDate = '2022-01-19';

useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
    end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.00175380});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.00175380;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = [
0.002739726027	0.000101388875
0.019178082192	0.000192790616
0.038356164384	0.000188582651
0.057534246575	0.000190610066
0.087671232877	0.000186047094
0.167123287671	0.000282614854
0.252054794521	0.000344707247
0.339726027397	0.000367004898
0.419178082192	0.000400452499
0.509589041096	0.000415650422
0.586301369863	0.000427300637
0.671232876712	0.000435908443
0.758904109589	0.000446542384
0.835616438356	0.000470858745
0.917808219178	0.000496692325
1.000000000000	0.000516949692
1.506849315068	0.000720697544
2.005479452055	0.001120059319
3.002739726027	0.002714418174
4.002739726027	0.004933516134
5.002739726027	0.007049886611
6.002739726027	0.008952079387
7.010958904110	0.010559045064
8.008219178082	0.011830012220
9.005479452055	0.012885165460
10.005479452055	0.013741279433
12.008219178082	0.015097592964
15.010958904110	0.016378720092
20.013698630137	0.017426478939
25.019178082192	0.017726708805
30.024657534247	0.017768042382
40.027397260274	0.016871633528
50.032876712329	0.015711420383
];

rawData_Libor3M_DC = [
0.087671232876712	0.001098799851025
0.167123287671233	0.001492765168950
0.252054794520548	0.001777759972902
0.509589041095890	0.002093881108158
1.000000000000000	0.002082797416369
2.005479452054790	0.002757637808099
3.005479452054790	0.004599469233654
4.002739726027400	0.006983466216479
5.002739726027400	0.009201864211354
7.010958904109590	0.012826605809096
10.005479452054800	0.016117892407056
12.008219178082200	0.017531896005676
15.010958904109600	0.018870232807240
20.016438356164400	0.019987631397089
25.021917808219200	0.020322045675130
30.024657534246600	0.020377680903766
50.032876712328800	0.018354038457619
];

rawData_Libor3M_SC = [
0.002739726	0.043690217
0.083333333	0.045139798
0.166666667	0.046757083
0.25	0.048153085
0.5	0.050617731
0.75	0.050246323
1	0.049764664
1.5	0.046545703
2	0.043276196
2.5	0.04084174
3	0.038430807
3.5	0.037219362
4	0.036041706
4.5	0.035357371
5	0.034692713
5.5	0.034371505
6	0.034061972
6.5	0.033752439
7	0.033442906
7.5	0.033370461
8	0.033306077
8.5	0.033241692
9	0.033177307
9.5	0.033112922
10	0.033048537
10.5	0.033082554
11	0.033121083
11.5	0.033159611
12	0.033198139
12.5	0.033237542
13	0.033276979
13.5	0.033316416
14	0.033355853
14.5	0.03339529
15	0.033434727
15.5	0.03340911
16	0.033379721
16.5	0.033350332
17	0.033320943
17.5	0.033291554
18	0.033262165
18.5	0.033232776
19	0.033203387
19.5	0.033173998
20	0.033144609
20.5	0.033022127
21	0.032894816
21.5	0.032767505
22	0.032640194
22.5	0.032512883
23	0.032385572
23.5	0.032258261
24	0.03213095
24.5	0.032003639
25	0.031876328
25.5	0.031745005
26	0.031613448
26.5	0.031481892
27	0.031350336
27.5	0.031218779
28	0.031087223
28.5	0.030955667
29	0.030824111
29.5	0.030692554
30	0.030560998
30.5	0.030404257
31	0.030245742
31.5	0.030087228
32	0.029928714
32.5	0.029770199
33	0.029611685
33.5	0.029453171
34	0.029294657
34.5	0.029136142
35	0.028977628
35.5	0.028819114
36	0.0286606
36.5	0.028502085
37	0.028343571
37.5	0.028185057
38	0.028026542
38.5	0.027868028
39	0.027709514
39.5	0.027551
40	0.027392485
40.5	0.027233971
41	0.027075457
41.5	0.026916943
42	0.026758428
42.5	0.026599914
43	0.0264414
43.5	0.026282885
44	0.026124371
44.5	0.025965857
45	0.025807343
45.5	0.025648828
46	0.025490314
46.5	0.0253318
47	0.025173286
47.5	0.025014771
];

% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
referenceDate = valueDateH;


% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '3Y';
rateData = 0.00460200;


% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 2;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2033-06-30');

startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
                                    spotLag,'day',libor3M_Index_fwd.endOfMonth);

useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(valueDateH,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth); 
            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);
                    
floatingLeg_RFR_Schedule = ScheduleWTermConvention(startDate,endDate,floatingLeg_RFR_Tenor,calcCalendar,...
                            floatingLeg_RFR_Convention,floatingLeg_RFR_Convention, 'Backward',useEndOfMonth);

% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;
floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
                          fixedSchedule,fixedRate,fixedDayCounter,...  
                          floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
                          floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);

out = vanillaSwap.Calculate(discountCurve_SOFR);

fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:
test_LiborFallback_CrossCheck_20230119_SOFRCurveFront_Simple.m

clc
clear
tic

% valueDate = '2023-01-19';
valueDate = '2023-01-27';


% we do not apply 2BD Rule

apply2BDFallbackRuleFlag = false;


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-04'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-05-01'
'2026-05-05'
'2026-09-24'
'2026-09-25'
'2026-10-09'
'2026-12-25'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2020-01-01'
'2020-04-10'
'2020-04-13'
'2020-05-08'
'2020-05-25'
'2020-08-31'
'2020-12-25'
'2020-12-28'
'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-09-19'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-08'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2020-01-01'
'2020-01-20'
'2020-02-17'
'2020-04-10'
'2020-05-25'
'2020-07-03'
'2020-09-07'
'2020-10-12'
'2020-11-11'
'2020-11-26'
'2020-12-25'
'2021-01-01'
'2021-01-18'
'2021-02-15'
'2021-05-31'
'2021-07-05'
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-06-19'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-06-18'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-06-19'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-06-19'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-06-19'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-06-19'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-06-18'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-06-20'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-06-19'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-06-19'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-06-19'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-06-19'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-06-18'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-06-20'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-06-19'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-06-19'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-06-19'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-06-19'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-06-20'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-06-19'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-06-19'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-06-19'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-06-19'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-06-18'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-06-20'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-06-19'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-06-19'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-06-19'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-06-19'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-06-18'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-06-19'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-06-19'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-06-19'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-06-19'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-06-18'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-21'
'2061-04-08'
'2061-05-30'
'2061-06-20'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-06-19'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-06-19'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-06-19'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-06-19'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-06-18'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-06-20'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-06-19'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-06-19'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-06-19'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-06-19'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2020-01-01'
'2020-01-20'
'2020-02-17'
'2020-05-25'
'2020-09-07'
'2020-10-12'
'2020-11-11'
'2020-11-26'
'2020-12-25'
'2021-01-01'
'2021-01-18'
'2021-02-15'
'2021-05-31'
'2021-07-05'
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-06-19'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-06-19'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-06-19'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-06-19'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-06-19'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-06-20'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-06-19'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-06-19'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-06-19'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-06-19'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-06-20'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-06-19'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-06-19'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-06-19'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-06-19'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-06-20'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-06-19'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-06-19'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-06-19'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-06-19'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-06-20'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-06-19'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-06-19'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-06-19'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-06-19'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-06-19'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-06-19'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-06-19'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-06-19'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-06-20'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-06-19'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-06-19'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-06-19'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-06-19'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-06-20'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-06-19'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-06-19'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-06-19'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-06-19'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
dummySOFRFixing = {
'2020-05-01'	0.00030
'2020-05-04'	0.00050
'2020-05-05'	0.00050
'2020-05-06'	0.00050
'2020-05-07'	0.00050
'2020-05-08'	0.00060
'2020-05-11'	0.00060
'2020-05-12'	0.00060
'2020-05-13'	0.00040
'2020-05-14'	0.00020
'2020-05-15'	0.00050
'2020-05-18'	0.00040
'2020-05-19'	0.00040
'2020-05-20'	0.00010
'2020-05-21'	0.00020
'2020-05-22'	0.00040
'2020-05-26'	0.00060
'2020-05-27'	0.00060
'2020-05-28'	0.00060
'2020-05-29'	0.00060
'2020-06-01'	0.00060
'2020-06-02'	0.00070
'2020-06-03'	0.00060
'2020-06-04'	0.00070
'2020-06-05'	0.00070
'2020-06-08'	0.00070
'2020-06-09'	0.00080
'2020-06-10'	0.00070
'2020-06-11'	0.00080
'2020-06-12'	0.00080
'2020-06-15'	0.00090
'2020-06-16'	0.00090
'2020-06-17'	0.00090
'2020-06-18'	0.00090
'2020-06-19'	0.00090
'2020-06-22'	0.00080
'2020-06-23'	0.00070
'2020-06-24'	0.00080
'2020-06-25'	0.00090
'2020-06-26'	0.00080
'2020-06-29'	0.00080
'2020-06-30'	0.00100
'2020-07-01'	0.00110
'2020-07-02'	0.00110
'2020-07-06'	0.00100
'2020-07-07'	0.00100
'2020-07-08'	0.00100
'2020-07-09'	0.00100
'2020-07-10'	0.00100
'2020-07-13'	0.00100
'2020-07-14'	0.00110
'2020-07-15'	0.00130
'2020-07-16'	0.00120
'2020-07-17'	0.00120
'2020-07-20'	0.00120
'2020-07-21'	0.00110
'2020-07-22'	0.00100
'2020-07-23'	0.00100
'2020-07-24'	0.00100
'2020-07-27'	0.00100
'2020-07-28'	0.00100
'2020-07-29'	0.00090
'2020-07-30'	0.00100
'2020-07-31'	0.00100
'2020-08-03'	0.00100
'2020-08-04'	0.00090
'2020-08-05'	0.00090
'2020-08-06'	0.00090
'2020-08-07'	0.00090
'2020-08-10'	0.00090
'2020-08-11'	0.00100
'2020-08-12'	0.00090
'2020-08-13'	0.00090
'2020-08-14'	0.00090
'2020-08-17'	0.00100
'2020-08-18'	0.00090
'2020-08-19'	0.00090
'2020-08-20'	0.00070
'2020-08-21'	0.00070
'2020-08-24'	0.00080
'2020-08-25'	0.00080
'2020-08-26'	0.00070
'2020-08-27'	0.00070
'2020-08-28'	0.00070
'2020-08-31'	0.00090
'2020-09-01'	0.00090
'2020-09-02'	0.00090
'2020-09-03'	0.00100
'2020-09-04'	0.00090
'2020-09-08'	0.00090
'2020-09-09'	0.00090
'2020-09-10'	0.00090
'2020-09-11'	0.00090
'2020-09-14'	0.00090
'2020-09-15'	0.00100
'2020-09-16'	0.00100
'2020-09-17'	0.00100
'2020-09-18'	0.00090
'2020-09-21'	0.00080
'2020-09-22'	0.00070
'2020-09-23'	0.00060
'2020-09-24'	0.00070
'2020-09-25'	0.00080
'2020-09-28'	0.00080
'2020-09-29'	0.00070
'2020-09-30'	0.00080
'2020-10-01'	0.00080
'2020-10-02'	0.00100
'2020-10-05'	0.00090
'2020-10-06'	0.00100
'2020-10-07'	0.00090
'2020-10-08'	0.00080
'2020-10-09'	0.00090
'2020-10-13'	0.00090
'2020-10-14'	0.00090
'2020-10-15'	0.00100
'2020-10-16'	0.00090
'2020-10-19'	0.00090
'2020-10-20'	0.00080
'2020-10-21'	0.00070
'2020-10-22'	0.00070
'2020-10-23'	0.00080
'2020-10-26'	0.00090
'2020-10-27'	0.00090
'2020-10-28'	0.00080
'2020-10-29'	0.00090
'2020-10-30'	0.00090
'2020-11-02'	0.00110
'2020-11-03'	0.00100
'2020-11-04'	0.00100
'2020-11-05'	0.00110
'2020-11-06'	0.00100
'2020-11-09'	0.00100
'2020-11-10'	0.00100
'2020-11-12'	0.00090
'2020-11-13'	0.00090
'2020-11-16'	0.00100
'2020-11-17'	0.00090
'2020-11-18'	0.00070
'2020-11-19'	0.00060
'2020-11-20'	0.00050
'2020-11-23'	0.00050
'2020-11-24'	0.00070
'2020-11-25'	0.00080
'2020-11-27'	0.00080
'2020-11-30'	0.00090
'2020-12-01'	0.00080
'2020-12-02'	0.00080
'2020-12-03'	0.00080
'2020-12-04'	0.00090
'2020-12-07'	0.00080
'2020-12-08'	0.00070
'2020-12-09'	0.00080
'2020-12-10'	0.00080
'2020-12-11'	0.00080
'2020-12-14'	0.00080
'2020-12-15'	0.00090
'2020-12-16'	0.00090
'2020-12-17'	0.00090
'2020-12-18'	0.00090
'2020-12-21'	0.00090
'2020-12-22'	0.00070
'2020-12-23'	0.00060
'2020-12-24'	0.00080
'2020-12-28'	0.00090
'2020-12-29'	0.00100
'2020-12-30'	0.00090
'2020-12-31'	0.00070
'2021-01-04'	0.00100
'2021-01-05'	0.00110
'2021-01-06'	0.00100
'2021-01-07'	0.00100
'2021-01-08'	0.00090
'2021-01-11'	0.00090
'2021-01-12'	0.00080
'2021-01-13'	0.00080
'2021-01-14'	0.00080
'2021-01-15'	0.00080
'2021-01-19'	0.00070
'2021-01-20'	0.00060
'2021-01-21'	0.00040
'2021-01-22'	0.00050
'2021-01-25'	0.00060
'2021-01-26'	0.00030
'2021-01-27'	0.00030
'2021-01-28'	0.00040
'2021-01-29'	0.00060
'2021-02-01'	0.00070
'2021-02-02'	0.00070
'2021-02-03'	0.00070
'2021-02-04'	0.00050
'2021-02-05'	0.00020
'2021-02-08'	0.00020
'2021-02-09'	0.00050
'2021-02-10'	0.00060
'2021-02-11'	0.00060
'2021-02-12'	0.00050
'2021-02-16'	0.00060
'2021-02-17'	0.00060
'2021-02-18'	0.00030
'2021-02-19'	0.00020
'2021-02-22'	0.00030
'2021-02-23'	0.00010
'2021-02-24'	0.00020
'2021-02-25'	0.00030
'2021-02-26'	0.00010
'2021-03-01'	0.00020
'2021-03-02'	0.00040
'2021-03-03'	0.00040
'2021-03-04'	0.00020
'2021-03-05'	0.00020
'2021-03-08'	0.00020
'2021-03-09'	0.00020
'2021-03-10'	0.00020
'2021-03-11'	0.00010
'2021-03-12'	0.00010
'2021-03-15'	0.00010
'2021-03-16'	0.00010
'2021-03-17'	0.00010
'2021-03-18'	0.00010
'2021-03-19'	0.00010
'2021-03-22'	0.00010
'2021-03-23'	0.00010
'2021-03-24'	0.00010
'2021-03-25'	0.00010
'2021-03-26'	0.00010
'2021-03-29'	0.00010
'2021-03-30'	0.00010
'2021-03-31'	0.00010
'2021-04-01'	0.00010
'2021-04-05'	0.00010
'2021-04-06'	0.00010
'2021-04-07'	0.00010
'2021-04-08'	0.00010
'2021-04-09'	0.00010
'2021-04-12'	0.00010
'2021-04-13'	0.00010
'2021-04-14'	0.00010
'2021-04-15'	0.00010
'2021-04-16'	0.00010
'2021-04-19'	0.00010
'2021-04-20'	0.00010
'2021-04-21'	0.00010
'2021-04-22'	0.00010
'2021-04-23'	0.00010
'2021-04-26'	0.00010
'2021-04-27'	0.00010
'2021-04-28'	0.00010
'2021-04-29'	0.00010
'2021-04-30'	0.00010
'2021-05-03'	0.00010
'2021-05-04'	0.00010
'2021-05-05'	0.00010
'2021-05-06'	0.00010
'2021-05-07'	0.00010
'2021-05-10'	0.00010
'2021-05-11'	0.00010
'2021-05-12'	0.00010
'2021-05-13'	0.00010
'2021-05-14'	0.00010
'2021-05-17'	0.00010
'2021-05-18'	0.00010
'2021-05-19'	0.00010
'2021-05-20'	0.00010
'2021-05-21'	0.00010
'2021-05-24'	0.00010
'2021-05-25'	0.00010
'2021-05-26'	0.00010
'2021-05-27'	0.00010
'2021-05-28'	0.00010
'2021-06-01'	0.00010
'2021-06-02'	0.00010
'2021-06-03'	0.00010
'2021-06-04'	0.00010
'2021-06-07'	0.00010
'2021-06-08'	0.00010
'2021-06-09'	0.00010
'2021-06-10'	0.00010
'2021-06-11'	0.00010
'2021-06-14'	0.00010
'2021-06-15'	0.00010
'2021-06-16'	0.00010
'2021-06-17'	0.00050
'2021-06-18'	0.00050
'2021-06-21'	0.00050
'2021-06-22'	0.00050
'2021-06-23'	0.00050
'2021-06-24'	0.00050
'2021-06-25'	0.00050
'2021-06-28'	0.00050
'2021-06-29'	0.00050
'2021-06-30'	0.00050
'2021-07-01'	0.00050
'2021-07-02'	0.00050
'2021-07-06'	0.00050
'2021-07-07'	0.00050
'2021-07-08'	0.00050
'2021-07-09'	0.00050
'2021-07-12'	0.00050
'2021-07-13'	0.00050
'2021-07-14'	0.00050
'2021-07-15'	0.00050
'2021-07-16'	0.00050
'2021-07-19'	0.00050
'2021-07-20'	0.00050
'2021-07-21'	0.00050
'2021-07-22'	0.00050
'2021-07-23'	0.00050
'2021-07-26'	0.00050
'2021-07-27'	0.00050
'2021-07-28'	0.00050
'2021-07-29'	0.00050
'2021-07-30'	0.00050
'2021-08-02'	0.00050
'2021-08-03'	0.00050
'2021-08-04'	0.00050
'2021-08-05'	0.00050
'2021-08-06'	0.00050
'2021-08-09'	0.00050
'2021-08-10'	0.00050
'2021-08-11'	0.00050
'2021-08-12'	0.00050
'2021-08-13'	0.00050
'2021-08-16'	0.00050
'2021-08-17'	0.00050
'2021-08-18'	0.00050
'2021-08-19'	0.00050
'2021-08-20'	0.00050
'2021-08-23'	0.00050
'2021-08-24'	0.00050
'2021-08-25'	0.00050
'2021-08-26'	0.00050
'2021-08-27'	0.00050
'2021-08-30'	0.00050
'2021-08-31'	0.00050
'2021-09-01'	0.00050
'2021-09-02'	0.00050
'2021-09-03'	0.00050
'2021-09-07'	0.00050
'2021-09-08'	0.00050
'2021-09-09'	0.00050
'2021-09-10'	0.00050
'2021-09-13'	0.00050
'2021-09-14'	0.00050
'2021-09-15'	0.00050
'2021-09-16'	0.00050
'2021-09-17'	0.00050
'2021-09-20'	0.00050
'2021-09-21'	0.00050
'2021-09-22'	0.00050
'2021-09-23'	0.00050
'2021-09-24'	0.00050
'2021-09-27'	0.00050
'2021-09-28'	0.00050
'2021-09-29'	0.00050
'2021-09-30'	0.00050
'2021-10-01'	0.00050
'2021-10-04'	0.00050
'2021-10-05'	0.00050
'2021-10-06'	0.00050
'2021-10-07'	0.00050
'2021-10-08'	0.00050
'2021-10-12'	0.00050
'2021-10-13'	0.00050
'2021-10-14'	0.00050
'2021-10-15'	0.00050
'2021-10-18'	0.00050
'2021-10-19'	0.00030
'2021-10-20'	0.00030
'2021-10-21'	0.00030
'2021-10-22'	0.00050
'2021-10-25'	0.00040
'2021-10-26'	0.00050
'2021-10-27'	0.00050
'2021-10-28'	0.00050
'2021-10-29'	0.00050
'2021-11-01'	0.00050
'2021-11-02'	0.00050
'2021-11-03'	0.00050
'2021-11-04'	0.00050
'2021-11-05'	0.00050
'2021-11-08'	0.00050
'2021-11-09'	0.00050
'2021-11-10'	0.00050
'2021-11-12'	0.00050
'2021-11-15'	0.00050
'2021-11-16'	0.00050
'2021-11-17'	0.00050
'2021-11-18'	0.00050
'2021-11-19'	0.00050
'2021-11-22'	0.00050
'2021-11-23'	0.00050
'2021-11-24'	0.00050
'2021-11-26'	0.00050
'2021-11-29'	0.00050
'2021-11-30'	0.00050
'2021-12-01'	0.00050
'2021-12-02'	0.00050
'2021-12-03'	0.00050
'2021-12-06'	0.00050
'2021-12-07'	0.00050
'2021-12-08'	0.00050
'2021-12-09'	0.00050
'2021-12-10'	0.00050
'2021-12-13'	0.00050
'2021-12-14'	0.00050
'2021-12-15'	0.00050
'2021-12-16'	0.00050
'2021-12-17'	0.00050
'2021-12-20'	0.00040
'2021-12-21'	0.00050
'2021-12-22'	0.00050
'2021-12-23'	0.00050
'2021-12-27'	0.00050
'2021-12-28'	0.00050
'2021-12-29'	0.00050
'2021-12-30'	0.00050
'2021-12-31'	0.00050
'2022-01-03'	0.00050
'2022-01-04'	0.00050
'2022-01-05'	0.00050
'2022-01-06'	0.00050
'2022-01-07'	0.00050
'2022-01-10'	0.00050
'2022-01-11'	0.00050
'2022-01-12'	0.00050
'2022-01-13'	0.00050
'2022-01-14'	0.00050
'2022-01-18'	0.00050
'2022-01-19'	0.00050
'2022-01-20'	0.00040
'2022-01-21'	0.00050
'2022-01-24'	0.00040
'2022-01-25'	0.00050
'2022-01-26'	0.00040
'2022-01-27'	0.00040
'2022-01-28'	0.00050
'2022-01-31'	0.00050
'2022-02-01'	0.00050
'2022-02-02'	0.00050
'2022-02-03'	0.00050
'2022-02-04'	0.00050
'2022-02-07'	0.00050
'2022-02-08'	0.00050
'2022-02-09'	0.00040
'2022-02-10'	0.00050
'2022-02-11'	0.00050
'2022-02-14'	0.00050
'2022-02-15'	0.00050
'2022-02-16'	0.00050
'2022-02-17'	0.00050
'2022-02-18'	0.00050
'2022-02-22'	0.00050
'2022-02-23'	0.00050
'2022-02-24'	0.00050
'2022-02-25'	0.00050
'2022-02-28'	0.00050
'2022-03-01'	0.00050
'2022-03-02'	0.00050
'2022-03-03'	0.00050
'2022-03-04'	0.00050
'2022-03-07'	0.00050
'2022-03-08'	0.00050
'2022-03-09'	0.00050
'2022-03-10'	0.00050
'2022-03-11'	0.00050
'2022-03-14'	0.00050
'2022-03-15'	0.00050
'2022-03-16'	0.00050
'2022-03-17'	0.00300
'2022-03-18'	0.00300
'2022-03-21'	0.00290
'2022-03-22'	0.00280
'2022-03-23'	0.00270
'2022-03-24'	0.00270
'2022-03-25'	0.00280
'2022-03-28'	0.00280
'2022-03-29'	0.00280
'2022-03-30'	0.00270
'2022-03-31'	0.00290
'2022-04-01'	0.00300
'2022-04-04'	0.00300
'2022-04-05'	0.00300
'2022-04-06'	0.00300
'2022-04-07'	0.00300
'2022-04-08'	0.00300
'2022-04-11'	0.00300
'2022-04-12'	0.00290
'2022-04-13'	0.00290
'2022-04-14'	0.00290
'2022-04-18'	0.00290
'2022-04-19'	0.00280
'2022-04-20'	0.00270
'2022-04-21'	0.00260
'2022-04-22'	0.00270
'2022-04-25'	0.00270
'2022-04-26'	0.00270
'2022-04-27'	0.00280
'2022-04-28'	0.00280
'2022-04-29'	0.00280
'2022-05-02'	0.00300
'2022-05-03'	0.00300
'2022-05-04'	0.00300
'2022-05-05'	0.00790
'2022-05-06'	0.00780
'2022-05-09'	0.00780
'2022-05-10'	0.00780
'2022-05-11'	0.00780
'2022-05-12'	0.00790
'2022-05-13'	0.00790
'2022-05-16'	0.00800
'2022-05-17'	0.00800
'2022-05-18'	0.00790
'2022-05-19'	0.00790
'2022-05-20'	0.00780
'2022-05-23'	0.00780
'2022-05-24'	0.00780
'2022-05-25'	0.00780
'2022-05-26'	0.00780
'2022-05-27'	0.00780
'2022-05-31'	0.00790
'2022-06-01'	0.00800
'2022-06-02'	0.00790
'2022-06-03'	0.00780
'2022-06-06'	0.00780
'2022-06-07'	0.00770
'2022-06-08'	0.00760
'2022-06-09'	0.00750
'2022-06-10'	0.00750
'2022-06-13'	0.00730
'2022-06-14'	0.00690
'2022-06-15'	0.00700
'2022-06-16'	0.01450
'2022-06-17'	0.01450
'2022-06-21'	0.01450
'2022-06-22'	0.01450
'2022-06-23'	0.01440
'2022-06-24'	0.01460
'2022-06-27'	0.01500
'2022-06-28'	0.01520
'2022-06-29'	0.01510
'2022-06-30'	0.01500
'2022-07-01'	0.01520
'2022-07-05'	0.01540
'2022-07-06'	0.01540
'2022-07-07'	0.01540
'2022-07-08'	0.01530
'2022-07-11'	0.01530
'2022-07-12'	0.01540
'2022-07-13'	0.01530
'2022-07-14'	0.01530
'2022-07-15'	0.01540
'2022-07-18'	0.01540
'2022-07-19'	0.01540
'2022-07-20'	0.01530
'2022-07-21'	0.01530
'2022-07-22'	0.01520
'2022-07-25'	0.01530
'2022-07-26'	0.01530
'2022-07-27'	0.01530
'2022-07-28'	0.02280
'2022-07-29'	0.02270
'2022-08-01'	0.02280
'2022-08-02'	0.02300
'2022-08-03'	0.02290
'2022-08-04'	0.02290
'2022-08-05'	0.02280
'2022-08-08'	0.02280
'2022-08-09'	0.02290
'2022-08-10'	0.02280
'2022-08-11'	0.02280
'2022-08-12'	0.02280
'2022-08-15'	0.02290
'2022-08-16'	0.02290
'2022-08-17'	0.02290
'2022-08-18'	0.02280
'2022-08-19'	0.02280
'2022-08-22'	0.02280
'2022-08-23'	0.02270
'2022-08-24'	0.02270
'2022-08-25'	0.02280
'2022-08-26'	0.02280
'2022-08-29'	0.02280
'2022-08-30'	0.02290
'2022-08-31'	0.02290
'2022-09-01'	0.02290
'2022-09-02'	0.02290
'2022-09-06'	0.02290
'2022-09-07'	0.02280
'2022-09-08'	0.02280
'2022-09-09'	0.02280
'2022-09-12'	0.02280
'2022-09-13'	0.02280
'2022-09-14'	0.02270
'2022-09-15'	0.02280
'2022-09-16'	0.02270
'2022-09-19'	0.02270
'2022-09-20'	0.02260
'2022-09-21'	0.02250
'2022-09-22'	0.02990
'2022-09-23'	0.02990
'2022-09-26'	0.02990
'2022-09-27'	0.02980
'2022-09-28'	0.02980
'2022-09-29'	0.02960
'2022-09-30'	0.02980
'2022-10-03'	0.03000
'2022-10-04'	0.03040
'2022-10-05'	0.03040
'2022-10-06'	0.03050
'2022-10-07'	0.03050
'2022-10-11'	0.03050
'2022-10-12'	0.03040
'2022-10-13'	0.03040
'2022-10-14'	0.03040
'2022-10-17'	0.03050
'2022-10-18'	0.03040
'2022-10-19'	0.03040
'2022-10-20'	0.03030
'2022-10-21'	0.03020
'2022-10-24'	0.03010
'2022-10-25'	0.03020
'2022-10-26'	0.03030
'2022-10-27'	0.03040
'2022-10-28'	0.03050
'2022-10-31'	0.03050
'2022-11-01'	0.03050
'2022-11-02'	0.03050
'2022-11-03'	0.03800
'2022-11-04'	0.03800
'2022-11-07'	0.03780
'2022-11-08'	0.03780
'2022-11-09'	0.03780
'2022-11-10'	0.03780
'2022-11-14'	0.03790
'2022-11-15'	0.03800
'2022-11-16'	0.03810
'2022-11-17'	0.03800
'2022-11-18'	0.03800
'2022-11-21'	0.03800
'2022-11-22'	0.03800
'2022-11-23'	0.03790
'2022-11-25'	0.03800
'2022-11-28'	0.03800
'2022-11-29'	0.03810
'2022-11-30'	0.03820
'2022-12-01'	0.03820
'2022-12-02'	0.03810
'2022-12-05'	0.03810
'2022-12-06'	0.03800
'2022-12-07'	0.03800
'2022-12-08'	0.03800
'2022-12-09'	0.03800
'2022-12-12'	0.03800
'2022-12-13'	0.03800
'2022-12-14'	0.03800
'2022-12-15'	0.04320
'2022-12-16'	0.04320
'2022-12-19'	0.04300
'2022-12-20'	0.04300
'2022-12-21'	0.04300
'2022-12-22'	0.04300
'2022-12-23'	0.04300
'2022-12-27'	0.04300
'2022-12-28'	0.04300
'2022-12-29'	0.04300
'2022-12-30'	0.04300
'2023-01-03'	0.04310
'2023-01-04'	0.04300
'2023-01-05'	0.04310
'2023-01-06'	0.04310
'2023-01-09'	0.04310
'2023-01-10'	0.04310
'2023-01-11'	0.04300
'2023-01-12'	0.04300
'2023-01-13'	0.04300
'2023-01-17'	0.04310
'2023-01-18'	0.04300
'2023-01-19'	0.04310
'2023-01-20'	0.04300
'2023-01-23'	0.04300
'2023-01-24'	0.04300
'2023-01-25'	0.04310
};

irIndexFixing_SOFR = containers.Map({dummySOFRFixing{1,1}},{dummySOFRFixing{1,2}});
for idxhong=2:size(dummySOFRFixing,1)
    irIndexFixing_SOFR(dummySOFRFixing{idxhong,1}) = dummySOFRFixing{idxhong,2};
end

aaa = 1.0;

% tempDate = H_Date('2021-05-04');
% irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
% for idxhong=1:10000
%     tempDate = AddDate(tempDate,1,'day');
%     irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
% end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.0441471});
    for idxhong=1:1
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.0441471;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

%Samsung Front System  weird??
% rawData_SOFR = [
% 0.00273972603	0.04359461870
% 0.01917808219	0.04370201200
% 0.03835616438	0.04432644440
% 0.05753424658	0.04497470510
% 0.08333333333	0.04540878960
% 0.16666666667	0.04590952010
% 0.25000000000	0.04675887430
% 0.50000000000	0.04799382350
% 0.75000000000	0.04820477850
% 1.00000000000	0.04769690700
% 2.00000000000	0.04083968150
% 3.00000000000	0.03604939100
% 4.00000000000	0.03353264000
% 5.00000000000	0.03214107290
% 6.00000000000	0.03132646730
% 7.00000000000	0.03083560170
% 8.00000000000	0.03054767440
% 9.00000000000	0.03043123480
% 10.00000000000	0.03041694650
% 12.00000000000	0.03053828620
% 15.00000000000	0.03078507670
% 20.00000000000	0.03046476800
% 25.00000000000	0.02921365170
% 30.00000000000	0.02795405990
% 40.00000000000	0.02500625100
% 50.00000000000	0.02197491160
% ];

%Bloomberg
rawData_SOFR = [
0.01369863	0.043667058
0.030136986	0.043762482
0.049315068	0.04435542
0.068493151	0.044978813
0.095890411	0.045400395
0.17260274	0.045910712
0.260273973	0.046757468
0.509589041	0.048002924
0.75890411	0.048213335
1.010958904	0.047669389
2.01369863	0.040868784
3.01369863	0.036047358
4.019178082	0.033549637
5.016438356	0.03215024
6.016438356	0.031343975
7.016438356	0.030849127
8.016438356	0.03055894
9.016438356	0.030449211
10.02191781	0.030429693
12.01917808	0.030560795
15.02739726	0.030796081
20.02465753	0.030487161
25.02739726	0.029234585
30.03287671	0.027956203
40.03835616	0.02512122
50.04657534	0.022007343
];

rawData_Libor3M_DC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];

rawData_Libor3M_SC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
% referenceDate = H_Date('2015-04-28');
% 

% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '6Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 0;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2022-06-30');

% liborTermDate = H_Date('2033-06-30');

% startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
%                                     fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);
                                
startDate = H_Date('2019-04-30');


useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth);
 


            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);

                    
% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;

fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedLegConvention,payLag);



floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

floatingLeg = FloatingRateLeg(floatingSchedule,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                   
numOfFloatingCoupons = length(floatingLeg.couponRates);

floatingCouponFallbackStartIdx = 1;
for idx=1:numOfFloatingCoupons
    tempDateDiff = DateDiff(floatingLeg.couponRates{idx}.fixingDate,liborTermDate);
    if tempDateDiff <=0
        floatingCouponFallbackStartIdx = floatingCouponFallbackStartIdx +1;
    else
        break;
    end
end

% we construct Libor Floating Leg(floatingLegLIbor) & 
% Libor Fallback FloatingLeg(overnightLeg_ARR)
% separately without redundancy

% constructing floatingLegLibor start
floatingLegLiborSize = floatingCouponFallbackStartIdx-1;
floatingLegLIbor = FloatingRateLegWithUsedScheduleSize(floatingSchedule,floatingLegLiborSize,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                                  
% constructing floatingLegLibor end

% constructing overnightLeg_ARR start

numOfFallbackCoupons = numOfFloatingCoupons - floatingCouponFallbackStartIdx + 1;

if numOfFallbackCoupons <=0
    disp('no fallback coupons !');
    aaa = 1.0;
    error('no fallback coupon');
end

% numOfFallbackRates = 1000;
numOfFallbackRates = 694;


rateRecordDate = cell(numOfFallbackRates,1);
accrualSpotDate = cell(numOfFallbackRates,1);
accrualStartDate = cell(numOfFallbackRates,1);
accrualEndDate = cell(numOfFallbackRates,1);
paymentDate = cell(numOfFallbackRates,1);
 
rateRecordDateStr = cell(numOfFallbackRates,1);
accrualSpotDateStr = cell(numOfFallbackRates,1);
accrualStartDateStr = cell(numOfFallbackRates,1);
accrualEndDateStr = cell(numOfFallbackRates,1);
paymentDateStr = cell(numOfFallbackRates,1);

% fallbackCouponRates = cell(numOfFallbackRates,1);
ARRRates = zeros(numOfFallbackRates,1);
computedFallbackRates = zeros(numOfFallbackRates,1);

% tempDateHong = H_Date('2022-01-31');
tempDateHong = H_Date('2021-03-05');
% tempDateHong = H_Date('2021-09-30');


rateRecordDate{1} = tempDateHong;
adjustingFactor = 1.0;

spread = 0.0026161;

for idx=1:numOfFallbackRates
    
    if idx >1
        rateRecordDate{idx} = GBENCalendar.Advance(rateRecordDate{idx-1},'Following',...
                                    1,'day',false);
    end
    
    
    accrualSpotDate{idx} = USGTCalendar.Advance(rateRecordDate{idx},'Following',...
                                    fallBackSpotLag,'day',false);

    accrualStartDate{idx} = USGTCalendar.Advance(accrualSpotDate{idx},'Preceding',...
                                    -1*fallBackOffsetLag,'day',false);
                                
%     accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},'ModifiedFollowing',...=
%                                     3,'month',true);
%                                 
    accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},'ModifiedFollowing',...=
                                    3,'month',false);
    
                                
    paymentDate{idx}    = USGTCalendar.Advance(accrualEndDate{idx},'Following', 2,'day',false);
    
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    paymentDateStr{idx} = StrDate(paymentDate{idx},'str');
                                
    startDate   = accrualStartDate{idx};
    endDate     = accrualEndDate{idx};
    payDate = paymentDate{idx}; 

    refPeriodStartDate = accrualStartDate{idx};
    refPeriodEndDate   = accrualEndDate{idx};

    if idx == 11
        aaa = 1.0;
    end
    fallbackCouponRates{idx} = OvernightIndexedCouponARR(payDate,nominal,overnightIndex,1.0,spread,'Act360','Act360',...
                                     startDate,endDate,refPeriodStartDate,refPeriodEndDate,adjustingFactor);
    
        

    try
        ARRRates(idx) = fallbackCouponRates{idx}.Rate();
    catch ME
        disp('error!')
        aaaa = 0;
    end
    computedFallbackRates(idx) = ARRRates(idx) +  spread;

end

aaa = 1.0;

for idx=1:numOfFallbackRates
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    paymentDateStr{idx} = StrDate(paymentDate{idx},'str');
end

bbb = 1.0;





In [ ]:
test_LiborFallback_CrossCheck_20230119_SOFRCurveFront_0125.m

clc
clear
tic

% valueDate = '2023-01-19';
valueDate = '2023-01-27';


% we do not apply 2BD Rule

apply2BDFallbackRuleFlag = false;


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;


% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);

%% Calendars Start

%% KRNT Calendar
KRNT.name = 'KRNT';
KRNT.holidaysList = {'2020-01-01'
'2020-01-24'
'2020-01-27'
'2020-04-15'
'2020-04-30'
'2020-05-01'
'2020-05-05'
'2020-08-17'
'2020-09-30'
'2020-10-01'
'2020-10-02'
'2020-10-09'
'2020-12-25'
'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-04'
'2021-10-11'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-10'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2024-01-01'
'2024-02-09'
'2024-02-12'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-03-03'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-05-01'
'2026-05-05'
'2026-09-24'
'2026-09-25'
'2026-10-09'
'2026-12-25'
    };

KRNTCalendar = Calendar();
KRNTCalendar.name = KRNT.name;
for i=1:length(KRNT.holidaysList)
    KRNTCalendar.AddHoliday(KRNT.holidaysList{i})
end

%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2020-01-01'
'2020-04-10'
'2020-04-13'
'2020-05-08'
'2020-05-25'
'2020-08-31'
'2020-12-25'
'2020-12-28'
'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-09-19'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-08'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end


%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2020-01-01'
'2020-01-20'
'2020-02-17'
'2020-04-10'
'2020-05-25'
'2020-07-03'
'2020-09-07'
'2020-10-12'
'2020-11-11'
'2020-11-26'
'2020-12-25'
'2021-01-01'
'2021-01-18'
'2021-02-15'
'2021-05-31'
'2021-07-05'
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-06-19'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-06-18'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-06-19'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-06-19'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-06-19'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-06-19'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-06-18'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-06-20'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-06-19'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-06-19'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-06-19'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-06-19'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-06-18'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-06-20'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-06-19'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-06-19'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-06-19'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-06-19'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-06-20'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-06-19'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-06-19'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-06-19'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-06-19'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-06-18'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-06-20'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-06-19'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-06-19'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-06-19'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-06-19'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-06-18'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-06-19'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-06-19'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-06-19'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-06-19'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-06-18'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-21'
'2061-04-08'
'2061-05-30'
'2061-06-20'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-06-19'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-06-19'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-06-19'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-06-19'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-06-18'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-06-20'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-06-19'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-06-19'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-06-19'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-06-19'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2020-01-01'
'2020-01-20'
'2020-02-17'
'2020-05-25'
'2020-09-07'
'2020-10-12'
'2020-11-11'
'2020-11-26'
'2020-12-25'
'2021-01-01'
'2021-01-18'
'2021-02-15'
'2021-05-31'
'2021-07-05'
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-06-19'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-06-19'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-06-19'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-06-19'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-06-19'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-06-20'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-06-19'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-06-19'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-06-19'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-06-19'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-06-20'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-06-19'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-06-19'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-06-19'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-06-19'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-06-20'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-06-19'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-06-19'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-06-19'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-06-19'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-06-20'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-06-19'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-06-19'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-06-19'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-06-19'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-06-19'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-06-19'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-06-19'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-06-19'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-06-20'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-06-19'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-06-19'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-06-19'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-06-19'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-06-20'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-06-19'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-06-19'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-06-19'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-06-19'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDIRS = JointCalendar({KRNTCalendar,GBENCalendar,USFDCalendar});

    
%% index fixing for SOFR Index start
dummySOFRFixing = {
'2020-05-01'	0.00030
'2020-05-04'	0.00050
'2020-05-05'	0.00050
'2020-05-06'	0.00050
'2020-05-07'	0.00050
'2020-05-08'	0.00060
'2020-05-11'	0.00060
'2020-05-12'	0.00060
'2020-05-13'	0.00040
'2020-05-14'	0.00020
'2020-05-15'	0.00050
'2020-05-18'	0.00040
'2020-05-19'	0.00040
'2020-05-20'	0.00010
'2020-05-21'	0.00020
'2020-05-22'	0.00040
'2020-05-26'	0.00060
'2020-05-27'	0.00060
'2020-05-28'	0.00060
'2020-05-29'	0.00060
'2020-06-01'	0.00060
'2020-06-02'	0.00070
'2020-06-03'	0.00060
'2020-06-04'	0.00070
'2020-06-05'	0.00070
'2020-06-08'	0.00070
'2020-06-09'	0.00080
'2020-06-10'	0.00070
'2020-06-11'	0.00080
'2020-06-12'	0.00080
'2020-06-15'	0.00090
'2020-06-16'	0.00090
'2020-06-17'	0.00090
'2020-06-18'	0.00090
'2020-06-19'	0.00090
'2020-06-22'	0.00080
'2020-06-23'	0.00070
'2020-06-24'	0.00080
'2020-06-25'	0.00090
'2020-06-26'	0.00080
'2020-06-29'	0.00080
'2020-06-30'	0.00100
'2020-07-01'	0.00110
'2020-07-02'	0.00110
'2020-07-06'	0.00100
'2020-07-07'	0.00100
'2020-07-08'	0.00100
'2020-07-09'	0.00100
'2020-07-10'	0.00100
'2020-07-13'	0.00100
'2020-07-14'	0.00110
'2020-07-15'	0.00130
'2020-07-16'	0.00120
'2020-07-17'	0.00120
'2020-07-20'	0.00120
'2020-07-21'	0.00110
'2020-07-22'	0.00100
'2020-07-23'	0.00100
'2020-07-24'	0.00100
'2020-07-27'	0.00100
'2020-07-28'	0.00100
'2020-07-29'	0.00090
'2020-07-30'	0.00100
'2020-07-31'	0.00100
'2020-08-03'	0.00100
'2020-08-04'	0.00090
'2020-08-05'	0.00090
'2020-08-06'	0.00090
'2020-08-07'	0.00090
'2020-08-10'	0.00090
'2020-08-11'	0.00100
'2020-08-12'	0.00090
'2020-08-13'	0.00090
'2020-08-14'	0.00090
'2020-08-17'	0.00100
'2020-08-18'	0.00090
'2020-08-19'	0.00090
'2020-08-20'	0.00070
'2020-08-21'	0.00070
'2020-08-24'	0.00080
'2020-08-25'	0.00080
'2020-08-26'	0.00070
'2020-08-27'	0.00070
'2020-08-28'	0.00070
'2020-08-31'	0.00090
'2020-09-01'	0.00090
'2020-09-02'	0.00090
'2020-09-03'	0.00100
'2020-09-04'	0.00090
'2020-09-08'	0.00090
'2020-09-09'	0.00090
'2020-09-10'	0.00090
'2020-09-11'	0.00090
'2020-09-14'	0.00090
'2020-09-15'	0.00100
'2020-09-16'	0.00100
'2020-09-17'	0.00100
'2020-09-18'	0.00090
'2020-09-21'	0.00080
'2020-09-22'	0.00070
'2020-09-23'	0.00060
'2020-09-24'	0.00070
'2020-09-25'	0.00080
'2020-09-28'	0.00080
'2020-09-29'	0.00070
'2020-09-30'	0.00080
'2020-10-01'	0.00080
'2020-10-02'	0.00100
'2020-10-05'	0.00090
'2020-10-06'	0.00100
'2020-10-07'	0.00090
'2020-10-08'	0.00080
'2020-10-09'	0.00090
'2020-10-13'	0.00090
'2020-10-14'	0.00090
'2020-10-15'	0.00100
'2020-10-16'	0.00090
'2020-10-19'	0.00090
'2020-10-20'	0.00080
'2020-10-21'	0.00070
'2020-10-22'	0.00070
'2020-10-23'	0.00080
'2020-10-26'	0.00090
'2020-10-27'	0.00090
'2020-10-28'	0.00080
'2020-10-29'	0.00090
'2020-10-30'	0.00090
'2020-11-02'	0.00110
'2020-11-03'	0.00100
'2020-11-04'	0.00100
'2020-11-05'	0.00110
'2020-11-06'	0.00100
'2020-11-09'	0.00100
'2020-11-10'	0.00100
'2020-11-12'	0.00090
'2020-11-13'	0.00090
'2020-11-16'	0.00100
'2020-11-17'	0.00090
'2020-11-18'	0.00070
'2020-11-19'	0.00060
'2020-11-20'	0.00050
'2020-11-23'	0.00050
'2020-11-24'	0.00070
'2020-11-25'	0.00080
'2020-11-27'	0.00080
'2020-11-30'	0.00090
'2020-12-01'	0.00080
'2020-12-02'	0.00080
'2020-12-03'	0.00080
'2020-12-04'	0.00090
'2020-12-07'	0.00080
'2020-12-08'	0.00070
'2020-12-09'	0.00080
'2020-12-10'	0.00080
'2020-12-11'	0.00080
'2020-12-14'	0.00080
'2020-12-15'	0.00090
'2020-12-16'	0.00090
'2020-12-17'	0.00090
'2020-12-18'	0.00090
'2020-12-21'	0.00090
'2020-12-22'	0.00070
'2020-12-23'	0.00060
'2020-12-24'	0.00080
'2020-12-28'	0.00090
'2020-12-29'	0.00100
'2020-12-30'	0.00090
'2020-12-31'	0.00070
'2021-01-04'	0.00100
'2021-01-05'	0.00110
'2021-01-06'	0.00100
'2021-01-07'	0.00100
'2021-01-08'	0.00090
'2021-01-11'	0.00090
'2021-01-12'	0.00080
'2021-01-13'	0.00080
'2021-01-14'	0.00080
'2021-01-15'	0.00080
'2021-01-19'	0.00070
'2021-01-20'	0.00060
'2021-01-21'	0.00040
'2021-01-22'	0.00050
'2021-01-25'	0.00060
'2021-01-26'	0.00030
'2021-01-27'	0.00030
'2021-01-28'	0.00040
'2021-01-29'	0.00060
'2021-02-01'	0.00070
'2021-02-02'	0.00070
'2021-02-03'	0.00070
'2021-02-04'	0.00050
'2021-02-05'	0.00020
'2021-02-08'	0.00020
'2021-02-09'	0.00050
'2021-02-10'	0.00060
'2021-02-11'	0.00060
'2021-02-12'	0.00050
'2021-02-16'	0.00060
'2021-02-17'	0.00060
'2021-02-18'	0.00030
'2021-02-19'	0.00020
'2021-02-22'	0.00030
'2021-02-23'	0.00010
'2021-02-24'	0.00020
'2021-02-25'	0.00030
'2021-02-26'	0.00010
'2021-03-01'	0.00020
'2021-03-02'	0.00040
'2021-03-03'	0.00040
'2021-03-04'	0.00020
'2021-03-05'	0.00020
'2021-03-08'	0.00020
'2021-03-09'	0.00020
'2021-03-10'	0.00020
'2021-03-11'	0.00010
'2021-03-12'	0.00010
'2021-03-15'	0.00010
'2021-03-16'	0.00010
'2021-03-17'	0.00010
'2021-03-18'	0.00010
'2021-03-19'	0.00010
'2021-03-22'	0.00010
'2021-03-23'	0.00010
'2021-03-24'	0.00010
'2021-03-25'	0.00010
'2021-03-26'	0.00010
'2021-03-29'	0.00010
'2021-03-30'	0.00010
'2021-03-31'	0.00010
'2021-04-01'	0.00010
'2021-04-05'	0.00010
'2021-04-06'	0.00010
'2021-04-07'	0.00010
'2021-04-08'	0.00010
'2021-04-09'	0.00010
'2021-04-12'	0.00010
'2021-04-13'	0.00010
'2021-04-14'	0.00010
'2021-04-15'	0.00010
'2021-04-16'	0.00010
'2021-04-19'	0.00010
'2021-04-20'	0.00010
'2021-04-21'	0.00010
'2021-04-22'	0.00010
'2021-04-23'	0.00010
'2021-04-26'	0.00010
'2021-04-27'	0.00010
'2021-04-28'	0.00010
'2021-04-29'	0.00010
'2021-04-30'	0.00010
'2021-05-03'	0.00010
'2021-05-04'	0.00010
'2021-05-05'	0.00010
'2021-05-06'	0.00010
'2021-05-07'	0.00010
'2021-05-10'	0.00010
'2021-05-11'	0.00010
'2021-05-12'	0.00010
'2021-05-13'	0.00010
'2021-05-14'	0.00010
'2021-05-17'	0.00010
'2021-05-18'	0.00010
'2021-05-19'	0.00010
'2021-05-20'	0.00010
'2021-05-21'	0.00010
'2021-05-24'	0.00010
'2021-05-25'	0.00010
'2021-05-26'	0.00010
'2021-05-27'	0.00010
'2021-05-28'	0.00010
'2021-06-01'	0.00010
'2021-06-02'	0.00010
'2021-06-03'	0.00010
'2021-06-04'	0.00010
'2021-06-07'	0.00010
'2021-06-08'	0.00010
'2021-06-09'	0.00010
'2021-06-10'	0.00010
'2021-06-11'	0.00010
'2021-06-14'	0.00010
'2021-06-15'	0.00010
'2021-06-16'	0.00010
'2021-06-17'	0.00050
'2021-06-18'	0.00050
'2021-06-21'	0.00050
'2021-06-22'	0.00050
'2021-06-23'	0.00050
'2021-06-24'	0.00050
'2021-06-25'	0.00050
'2021-06-28'	0.00050
'2021-06-29'	0.00050
'2021-06-30'	0.00050
'2021-07-01'	0.00050
'2021-07-02'	0.00050
'2021-07-06'	0.00050
'2021-07-07'	0.00050
'2021-07-08'	0.00050
'2021-07-09'	0.00050
'2021-07-12'	0.00050
'2021-07-13'	0.00050
'2021-07-14'	0.00050
'2021-07-15'	0.00050
'2021-07-16'	0.00050
'2021-07-19'	0.00050
'2021-07-20'	0.00050
'2021-07-21'	0.00050
'2021-07-22'	0.00050
'2021-07-23'	0.00050
'2021-07-26'	0.00050
'2021-07-27'	0.00050
'2021-07-28'	0.00050
'2021-07-29'	0.00050
'2021-07-30'	0.00050
'2021-08-02'	0.00050
'2021-08-03'	0.00050
'2021-08-04'	0.00050
'2021-08-05'	0.00050
'2021-08-06'	0.00050
'2021-08-09'	0.00050
'2021-08-10'	0.00050
'2021-08-11'	0.00050
'2021-08-12'	0.00050
'2021-08-13'	0.00050
'2021-08-16'	0.00050
'2021-08-17'	0.00050
'2021-08-18'	0.00050
'2021-08-19'	0.00050
'2021-08-20'	0.00050
'2021-08-23'	0.00050
'2021-08-24'	0.00050
'2021-08-25'	0.00050
'2021-08-26'	0.00050
'2021-08-27'	0.00050
'2021-08-30'	0.00050
'2021-08-31'	0.00050
'2021-09-01'	0.00050
'2021-09-02'	0.00050
'2021-09-03'	0.00050
'2021-09-07'	0.00050
'2021-09-08'	0.00050
'2021-09-09'	0.00050
'2021-09-10'	0.00050
'2021-09-13'	0.00050
'2021-09-14'	0.00050
'2021-09-15'	0.00050
'2021-09-16'	0.00050
'2021-09-17'	0.00050
'2021-09-20'	0.00050
'2021-09-21'	0.00050
'2021-09-22'	0.00050
'2021-09-23'	0.00050
'2021-09-24'	0.00050
'2021-09-27'	0.00050
'2021-09-28'	0.00050
'2021-09-29'	0.00050
'2021-09-30'	0.00050
'2021-10-01'	0.00050
'2021-10-04'	0.00050
'2021-10-05'	0.00050
'2021-10-06'	0.00050
'2021-10-07'	0.00050
'2021-10-08'	0.00050
'2021-10-12'	0.00050
'2021-10-13'	0.00050
'2021-10-14'	0.00050
'2021-10-15'	0.00050
'2021-10-18'	0.00050
'2021-10-19'	0.00030
'2021-10-20'	0.00030
'2021-10-21'	0.00030
'2021-10-22'	0.00050
'2021-10-25'	0.00040
'2021-10-26'	0.00050
'2021-10-27'	0.00050
'2021-10-28'	0.00050
'2021-10-29'	0.00050
'2021-11-01'	0.00050
'2021-11-02'	0.00050
'2021-11-03'	0.00050
'2021-11-04'	0.00050
'2021-11-05'	0.00050
'2021-11-08'	0.00050
'2021-11-09'	0.00050
'2021-11-10'	0.00050
'2021-11-12'	0.00050
'2021-11-15'	0.00050
'2021-11-16'	0.00050
'2021-11-17'	0.00050
'2021-11-18'	0.00050
'2021-11-19'	0.00050
'2021-11-22'	0.00050
'2021-11-23'	0.00050
'2021-11-24'	0.00050
'2021-11-26'	0.00050
'2021-11-29'	0.00050
'2021-11-30'	0.00050
'2021-12-01'	0.00050
'2021-12-02'	0.00050
'2021-12-03'	0.00050
'2021-12-06'	0.00050
'2021-12-07'	0.00050
'2021-12-08'	0.00050
'2021-12-09'	0.00050
'2021-12-10'	0.00050
'2021-12-13'	0.00050
'2021-12-14'	0.00050
'2021-12-15'	0.00050
'2021-12-16'	0.00050
'2021-12-17'	0.00050
'2021-12-20'	0.00040
'2021-12-21'	0.00050
'2021-12-22'	0.00050
'2021-12-23'	0.00050
'2021-12-27'	0.00050
'2021-12-28'	0.00050
'2021-12-29'	0.00050
'2021-12-30'	0.00050
'2021-12-31'	0.00050
'2022-01-03'	0.00050
'2022-01-04'	0.00050
'2022-01-05'	0.00050
'2022-01-06'	0.00050
'2022-01-07'	0.00050
'2022-01-10'	0.00050
'2022-01-11'	0.00050
'2022-01-12'	0.00050
'2022-01-13'	0.00050
'2022-01-14'	0.00050
'2022-01-18'	0.00050
'2022-01-19'	0.00050
'2022-01-20'	0.00040
'2022-01-21'	0.00050
'2022-01-24'	0.00040
'2022-01-25'	0.00050
'2022-01-26'	0.00040
'2022-01-27'	0.00040
'2022-01-28'	0.00050
'2022-01-31'	0.00050
'2022-02-01'	0.00050
'2022-02-02'	0.00050
'2022-02-03'	0.00050
'2022-02-04'	0.00050
'2022-02-07'	0.00050
'2022-02-08'	0.00050
'2022-02-09'	0.00040
'2022-02-10'	0.00050
'2022-02-11'	0.00050
'2022-02-14'	0.00050
'2022-02-15'	0.00050
'2022-02-16'	0.00050
'2022-02-17'	0.00050
'2022-02-18'	0.00050
'2022-02-22'	0.00050
'2022-02-23'	0.00050
'2022-02-24'	0.00050
'2022-02-25'	0.00050
'2022-02-28'	0.00050
'2022-03-01'	0.00050
'2022-03-02'	0.00050
'2022-03-03'	0.00050
'2022-03-04'	0.00050
'2022-03-07'	0.00050
'2022-03-08'	0.00050
'2022-03-09'	0.00050
'2022-03-10'	0.00050
'2022-03-11'	0.00050
'2022-03-14'	0.00050
'2022-03-15'	0.00050
'2022-03-16'	0.00050
'2022-03-17'	0.00300
'2022-03-18'	0.00300
'2022-03-21'	0.00290
'2022-03-22'	0.00280
'2022-03-23'	0.00270
'2022-03-24'	0.00270
'2022-03-25'	0.00280
'2022-03-28'	0.00280
'2022-03-29'	0.00280
'2022-03-30'	0.00270
'2022-03-31'	0.00290
'2022-04-01'	0.00300
'2022-04-04'	0.00300
'2022-04-05'	0.00300
'2022-04-06'	0.00300
'2022-04-07'	0.00300
'2022-04-08'	0.00300
'2022-04-11'	0.00300
'2022-04-12'	0.00290
'2022-04-13'	0.00290
'2022-04-14'	0.00290
'2022-04-18'	0.00290
'2022-04-19'	0.00280
'2022-04-20'	0.00270
'2022-04-21'	0.00260
'2022-04-22'	0.00270
'2022-04-25'	0.00270
'2022-04-26'	0.00270
'2022-04-27'	0.00280
'2022-04-28'	0.00280
'2022-04-29'	0.00280
'2022-05-02'	0.00300
'2022-05-03'	0.00300
'2022-05-04'	0.00300
'2022-05-05'	0.00790
'2022-05-06'	0.00780
'2022-05-09'	0.00780
'2022-05-10'	0.00780
'2022-05-11'	0.00780
'2022-05-12'	0.00790
'2022-05-13'	0.00790
'2022-05-16'	0.00800
'2022-05-17'	0.00800
'2022-05-18'	0.00790
'2022-05-19'	0.00790
'2022-05-20'	0.00780
'2022-05-23'	0.00780
'2022-05-24'	0.00780
'2022-05-25'	0.00780
'2022-05-26'	0.00780
'2022-05-27'	0.00780
'2022-05-31'	0.00790
'2022-06-01'	0.00800
'2022-06-02'	0.00790
'2022-06-03'	0.00780
'2022-06-06'	0.00780
'2022-06-07'	0.00770
'2022-06-08'	0.00760
'2022-06-09'	0.00750
'2022-06-10'	0.00750
'2022-06-13'	0.00730
'2022-06-14'	0.00690
'2022-06-15'	0.00700
'2022-06-16'	0.01450
'2022-06-17'	0.01450
'2022-06-21'	0.01450
'2022-06-22'	0.01450
'2022-06-23'	0.01440
'2022-06-24'	0.01460
'2022-06-27'	0.01500
'2022-06-28'	0.01520
'2022-06-29'	0.01510
'2022-06-30'	0.01500
'2022-07-01'	0.01520
'2022-07-05'	0.01540
'2022-07-06'	0.01540
'2022-07-07'	0.01540
'2022-07-08'	0.01530
'2022-07-11'	0.01530
'2022-07-12'	0.01540
'2022-07-13'	0.01530
'2022-07-14'	0.01530
'2022-07-15'	0.01540
'2022-07-18'	0.01540
'2022-07-19'	0.01540
'2022-07-20'	0.01530
'2022-07-21'	0.01530
'2022-07-22'	0.01520
'2022-07-25'	0.01530
'2022-07-26'	0.01530
'2022-07-27'	0.01530
'2022-07-28'	0.02280
'2022-07-29'	0.02270
'2022-08-01'	0.02280
'2022-08-02'	0.02300
'2022-08-03'	0.02290
'2022-08-04'	0.02290
'2022-08-05'	0.02280
'2022-08-08'	0.02280
'2022-08-09'	0.02290
'2022-08-10'	0.02280
'2022-08-11'	0.02280
'2022-08-12'	0.02280
'2022-08-15'	0.02290
'2022-08-16'	0.02290
'2022-08-17'	0.02290
'2022-08-18'	0.02280
'2022-08-19'	0.02280
'2022-08-22'	0.02280
'2022-08-23'	0.02270
'2022-08-24'	0.02270
'2022-08-25'	0.02280
'2022-08-26'	0.02280
'2022-08-29'	0.02280
'2022-08-30'	0.02290
'2022-08-31'	0.02290
'2022-09-01'	0.02290
'2022-09-02'	0.02290
'2022-09-06'	0.02290
'2022-09-07'	0.02280
'2022-09-08'	0.02280
'2022-09-09'	0.02280
'2022-09-12'	0.02280
'2022-09-13'	0.02280
'2022-09-14'	0.02270
'2022-09-15'	0.02280
'2022-09-16'	0.02270
'2022-09-19'	0.02270
'2022-09-20'	0.02260
'2022-09-21'	0.02250
'2022-09-22'	0.02990
'2022-09-23'	0.02990
'2022-09-26'	0.02990
'2022-09-27'	0.02980
'2022-09-28'	0.02980
'2022-09-29'	0.02960
'2022-09-30'	0.02980
'2022-10-03'	0.03000
'2022-10-04'	0.03040
'2022-10-05'	0.03040
'2022-10-06'	0.03050
'2022-10-07'	0.03050
'2022-10-11'	0.03050
'2022-10-12'	0.03040
'2022-10-13'	0.03040
'2022-10-14'	0.03040
'2022-10-17'	0.03050
'2022-10-18'	0.03040
'2022-10-19'	0.03040
'2022-10-20'	0.03030
'2022-10-21'	0.03020
'2022-10-24'	0.03010
'2022-10-25'	0.03020
'2022-10-26'	0.03030
'2022-10-27'	0.03040
'2022-10-28'	0.03050
'2022-10-31'	0.03050
'2022-11-01'	0.03050
'2022-11-02'	0.03050
'2022-11-03'	0.03800
'2022-11-04'	0.03800
'2022-11-07'	0.03780
'2022-11-08'	0.03780
'2022-11-09'	0.03780
'2022-11-10'	0.03780
'2022-11-14'	0.03790
'2022-11-15'	0.03800
'2022-11-16'	0.03810
'2022-11-17'	0.03800
'2022-11-18'	0.03800
'2022-11-21'	0.03800
'2022-11-22'	0.03800
'2022-11-23'	0.03790
'2022-11-25'	0.03800
'2022-11-28'	0.03800
'2022-11-29'	0.03810
'2022-11-30'	0.03820
'2022-12-01'	0.03820
'2022-12-02'	0.03810
'2022-12-05'	0.03810
'2022-12-06'	0.03800
'2022-12-07'	0.03800
'2022-12-08'	0.03800
'2022-12-09'	0.03800
'2022-12-12'	0.03800
'2022-12-13'	0.03800
'2022-12-14'	0.03800
'2022-12-15'	0.04320
'2022-12-16'	0.04320
'2022-12-19'	0.04300
'2022-12-20'	0.04300
'2022-12-21'	0.04300
'2022-12-22'	0.04300
'2022-12-23'	0.04300
'2022-12-27'	0.04300
'2022-12-28'	0.04300
'2022-12-29'	0.04300
'2022-12-30'	0.04300
'2023-01-03'	0.04310
'2023-01-04'	0.04300
'2023-01-05'	0.04310
'2023-01-06'	0.04310
'2023-01-09'	0.04310
'2023-01-10'	0.04310
'2023-01-11'	0.04300
'2023-01-12'	0.04300
'2023-01-13'	0.04300
'2023-01-17'	0.04310
'2023-01-18'	0.04300
'2023-01-19'	0.04310
'2023-01-20'	0.04300
'2023-01-23'	0.04300
'2023-01-24'	0.04300
'2023-01-25'	0.04310
};

irIndexFixing_SOFR = containers.Map({dummySOFRFixing{1,1}},{dummySOFRFixing{1,2}});
for idxhong=2:size(dummySOFRFixing,1)
    irIndexFixing_SOFR(dummySOFRFixing{idxhong,1}) = dummySOFRFixing{idxhong,2};
end

aaa = 1.0;

% tempDate = H_Date('2021-05-04');
% irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0001});
% for idxhong=1:10000
%     tempDate = AddDate(tempDate,1,'day');
%     irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0001;
% end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-05-04');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.0441471});
    for idxhong=1:1
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.0441471;
    end
%% index fixing for SOFR_index end

simulValue_NPV = zeros(100,1);
simulValue_fairRate = zeros(100,1);

%     valueDateH = H_Date('2022-04-17');
%     valueDateH = H_Date('2022-04-16');
% valueDateH = H_Date('2021-05-06');

% SOFR vs Libor3M floating - floating basis swap 
% pricing start

% forward & discount zeroCurve
forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M_SC = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

%Samsung Front System  weird??
% rawData_SOFR = [
% 0.00273972603	0.04359461870
% 0.01917808219	0.04370201200
% 0.03835616438	0.04432644440
% 0.05753424658	0.04497470510
% 0.08333333333	0.04540878960
% 0.16666666667	0.04590952010
% 0.25000000000	0.04675887430
% 0.50000000000	0.04799382350
% 0.75000000000	0.04820477850
% 1.00000000000	0.04769690700
% 2.00000000000	0.04083968150
% 3.00000000000	0.03604939100
% 4.00000000000	0.03353264000
% 5.00000000000	0.03214107290
% 6.00000000000	0.03132646730
% 7.00000000000	0.03083560170
% 8.00000000000	0.03054767440
% 9.00000000000	0.03043123480
% 10.00000000000	0.03041694650
% 12.00000000000	0.03053828620
% 15.00000000000	0.03078507670
% 20.00000000000	0.03046476800
% 25.00000000000	0.02921365170
% 30.00000000000	0.02795405990
% 40.00000000000	0.02500625100
% 50.00000000000	0.02197491160
% ];

%Bloomberg
rawData_SOFR = [
0.01369863	0.043667058
0.030136986	0.043762482
0.049315068	0.04435542
0.068493151	0.044978813
0.095890411	0.045400395
0.17260274	0.045910712
0.260273973	0.046757468
0.509589041	0.048002924
0.75890411	0.048213335
1.010958904	0.047669389
2.01369863	0.040868784
3.01369863	0.036047358
4.019178082	0.033549637
5.016438356	0.03215024
6.016438356	0.031343975
7.016438356	0.030849127
8.016438356	0.03055894
9.016438356	0.030449211
10.02191781	0.030429693
12.01917808	0.030560795
15.02739726	0.030796081
20.02465753	0.030487161
25.02739726	0.029234585
30.03287671	0.027956203
40.03835616	0.02512122
50.04657534	0.022007343
];

rawData_Libor3M_DC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];

rawData_Libor3M_SC = [
0.00273972603	0.04369021700
0.08333333333	0.04513979820
0.16666666667	0.04665932500
0.25000000000	0.04814066790
0.50000000000	0.05061773090
0.75000000000	0.05024632310
1.00000000000	0.04976466350
1.50000000000	0.04654570280
2.00000000000	0.04327619550
2.50000000000	0.04089081400
3.00000000000	0.03853033750
3.50000000000	0.03727012090
4.00000000000	0.03604089760
4.50000000000	0.03535489870
5.00000000000	0.03469056690
5.50000000000	0.03436953290
6.00000000000	0.03406016910
6.50000000000	0.03375080530
7.00000000000	0.03344144150
7.50000000000	0.03336907610
8.00000000000	0.03330476730
8.50000000000	0.03324045850
9.00000000000	0.03317614970
9.50000000000	0.03311184080
10.00000000000	0.03304753200
10.50000000000	0.03308159170
11.00000000000	0.03312016100
11.50000000000	0.03315873040
12.00000000000	0.03319729980
12.50000000000	0.03323672970
13.00000000000	0.03327619390
13.50000000000	0.03331565810
14.00000000000	0.03335512230
14.50000000000	0.03339458650
15.00000000000	0.03343405080
15.50000000000	0.03340845130
16.00000000000	0.03337908000
16.50000000000	0.03334970870
17.00000000000	0.03332033740
17.50000000000	0.03329096610
18.00000000000	0.03326159480
18.50000000000	0.03323222360
19.00000000000	0.03320285230
19.50000000000	0.03317348100
20.00000000000	0.03314410970
20.50000000000	0.03302164030
21.00000000000	0.03289434160
21.50000000000	0.03276704300
22.00000000000	0.03263974430
22.50000000000	0.03251244560
23.00000000000	0.03238514690
23.50000000000	0.03225784830
24.00000000000	0.03213054960
24.50000000000	0.03200325090
25.00000000000	0.03187595220
25.50000000000	0.03174463720
26.00000000000	0.03161308930
26.50000000000	0.03148154140
27.00000000000	0.03134999350
27.50000000000	0.03121844560
28.00000000000	0.03108689770
28.50000000000	0.03095534980
29.00000000000	0.03082380190
29.50000000000	0.03069225400
30.00000000000	0.03056070610
30.50000000000	0.03040396930
31.00000000000	0.03024545960
31.50000000000	0.03008695000
32.00000000000	0.02992844030
32.50000000000	0.02976993070
33.00000000000	0.02961142100
33.50000000000	0.02945291140
34.00000000000	0.02929440170
34.50000000000	0.02913589200
35.00000000000	0.02897738240
35.50000000000	0.02881887270
36.00000000000	0.02866036310
36.50000000000	0.02850185340
37.00000000000	0.02834334380
37.50000000000	0.02818483410
38.00000000000	0.02802632450
38.50000000000	0.02786781480
39.00000000000	0.02770930510
39.50000000000	0.02755079550
40.00000000000	0.02739228580
40.50000000000	0.02723377620
41.00000000000	0.02707526650
41.50000000000	0.02691675690
42.00000000000	0.02675824720
42.50000000000	0.02659973750
43.00000000000	0.02644122790
43.50000000000	0.02628271820
44.00000000000	0.02612420860
44.50000000000	0.02596569890
45.00000000000	0.02580718930
45.50000000000	0.02564867960
46.00000000000	0.02549017000
46.50000000000	0.02533166030
47.00000000000	0.02517315060
47.50000000000	0.02501464100
];


% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M_SC.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = false;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

%% Libor3M Index
IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});


IborIndexParams_fwd('forecastTodaysFixing') = true;
IborIndexParams_fwd('useRounding') = useRounding;
IborIndexParams_fwd('useSimpleTeleScoping') = false;
IborIndexParams_fwd('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
IborIndexParams_fwd('historicalFixings') = irIndexFixing_Libor3M;

libor3M_Index_fwd = IborIndex(IborIndexParams_fwd);

%% Index Definition End

%% Vanilla Libor3M Swap(LiborFallback) Pricing Start

% referenceDate = H_Date('2021-05-06');
% referenceDate = valueDateH;
% referenceDate = H_Date('2015-04-28');
% 

% VanillaSwapLiborTerm's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

calcCalendar = jointCalendar_USDIRS;

rateTenor = '6Y';
% rateData = 0.00460200;
rateData = 0.019875;



% FixedLeg
fixedLegTenor = '6M';
fixedLegConvention = 'ModifiedFollowing';
fixedLegDayCounter = '30360';

% FloatingLeg(Libor3M)
floatingLegTenor = '3M';
floatingLegConvention = 'ModifiedFollowing';
floatingLegDayCounter = 'Act360';
        
fallBackSpotLag = 2;
fallBackOffsetLag = 2;

spotLag = 0;

payLag  = 0;

% RFR FloatingLeg
floatingLeg_RFR_Tenor = '3M';
floatingLeg_RFR_Convention = 'ModifiedFollowing';
floatingLeg_RFR_DayCounter = 'Act360';
spotLag_RFR = 0;
payLag_RFR  = 0;

%Libor Termination Date
% liborTermDate = H_Date('2023-06-30');
liborTermDate = H_Date('2022-06-30');

% liborTermDate = H_Date('2033-06-30');

% startDate = calcCalendar.Advance(referenceDate,libor3M_Index_fwd.convention,...
%                                     fallBackSpotLag,'day',libor3M_Index_fwd.endOfMonth);
                                
startDate = H_Date('2019-04-30');


useEndOfMonth = false;
if IsEndOfMonth(startDate)
 useEndOfMonth = true;
end

endDate = AddTenor(startDate,rateTenor);

if strcmp(rateTenor(end),'Y')
%     swapTenorYearly = AddDate(h_Date,str2double(rateTenor(1:end-1)),'year');
    swapTenorYearly = AddDate(startDate,str2double(rateTenor(1:end-1)),'year');
    
end
               
 % fixed & floating schedule
fixedSchedule = ScheduleWTermConvention(startDate,endDate,fixedLegTenor,calcCalendar,...
                                fixedLegConvention,fixedLegConvention,'Backward',useEndOfMonth);
 


            
floatingSchedule = ScheduleWTermConvention(startDate,endDate,floatingLegTenor,calcCalendar,...
                        floatingLegConvention,floatingLegConvention,'Backward',useEndOfMonth);

                    
% RFR leg
type = 'Payer';
nominal = 10000.0;

fixedRate = rateData;
fixedDayCounter = fixedLegDayCounter;

fixedLeg = FixedRateLeg(fixedSchedule,nominal,fixedRate,fixedDayCounter,fixedLegConvention,payLag);



floatingSpread = 0.0;
floatingDayCounter = floatingLegDayCounter;

% Libor3M Fallback Spread 
floatingLeg_RFR_Spread = 0.0026161;

floatingLeg = FloatingRateLeg(floatingSchedule,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                   
numOfFloatingCoupons = length(floatingLeg.couponRates);

floatingCouponFallbackStartIdx = 1;
for idx=1:numOfFloatingCoupons
    tempDateDiff = DateDiff(floatingLeg.couponRates{idx}.fixingDate,liborTermDate);
    if tempDateDiff <=0
        floatingCouponFallbackStartIdx = floatingCouponFallbackStartIdx +1;
    else
        break;
    end
end

% we construct Libor Floating Leg(floatingLegLIbor) & 
% Libor Fallback FloatingLeg(overnightLeg_ARR)
% separately without redundancy

% constructing floatingLegLibor start
floatingLegLiborSize = floatingCouponFallbackStartIdx-1;
floatingLegLIbor = FloatingRateLegWithUsedScheduleSize(floatingSchedule,floatingLegLiborSize,libor3M_Index_fwd,1.0,floatingSpread,nominal,...
                                      floatingDayCounter,floatingSchedule.convention,payLag);
                                  
% constructing floatingLegLibor end

% constructing overnightLeg_ARR start

numOfFallbackCoupons = numOfFloatingCoupons - floatingCouponFallbackStartIdx + 1;

if numOfFallbackCoupons <=0
    disp('no fallback coupons !');
    aaa = 1.0;
    error('no fallback coupon');
end

% numOfFallbackRates = 1000;
numOfFallbackRates = 694;


rateRecordDate = cell(numOfFallbackRates,1);
accrualSpotDate = cell(numOfFallbackRates,1);
accrualStartDate = cell(numOfFallbackRates,1);
accrualEndDate = cell(numOfFallbackRates,1);
paymentDate = cell(numOfFallbackRates,1);
 
rateRecordDateStr = cell(numOfFallbackRates,1);
accrualSpotDateStr = cell(numOfFallbackRates,1);
accrualStartDateStr = cell(numOfFallbackRates,1);
accrualEndDateStr = cell(numOfFallbackRates,1);
paymentDateStr = cell(numOfFallbackRates,1);

% fallbackCouponRates = cell(numOfFallbackRates,1);
ARRRates = zeros(numOfFallbackRates,1);
computedFallbackRates = zeros(numOfFallbackRates,1);

% tempDateHong = H_Date('2022-01-31');
tempDateHong = H_Date('2021-03-05');
% tempDateHong = H_Date('2021-09-30');


rateRecordDate{1} = tempDateHong;
adjustingFactor = 1.0;

spread = 0.0026161;

for idx=1:numOfFallbackRates
    
    if idx >1
        rateRecordDate{idx} = GBENCalendar.Advance(rateRecordDate{idx-1},'Following',...
                                    1,'day',false);
    end
    
    
    accrualSpotDate{idx} = USGTCalendar.Advance(rateRecordDate{idx},'Following',...
                                    fallBackSpotLag,'day',false);

    accrualStartDate{idx} = USGTCalendar.Advance(accrualSpotDate{idx},'Preceding',...
                                    -1*fallBackOffsetLag,'day',false);
                                
%     accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},'ModifiedFollowing',...=
%                                     3,'month',true);
%                                 
    accrualEndDate{idx} = USGTCalendar.Advance(accrualStartDate{idx},'ModifiedFollowing',...=
                                    3,'month',false);
    
                                
    paymentDate{idx}    = USGTCalendar.Advance(accrualEndDate{idx},'Following', 2,'day',false);
    
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    paymentDateStr{idx} = StrDate(paymentDate{idx},'str');
                                
    startDate   = accrualStartDate{idx};
    endDate     = accrualEndDate{idx};
    payDate = paymentDate{idx}; 

    refPeriodStartDate = accrualStartDate{idx};
    refPeriodEndDate   = accrualEndDate{idx};

    if idx == 11
        aaa = 1.0;
    end
    fallbackCouponRates{idx} = OvernightIndexedCouponARR(payDate,nominal,overnightIndex,1.0,spread,'Act360','Act360',...
                                     startDate,endDate,refPeriodStartDate,refPeriodEndDate,adjustingFactor);
    
        

    try
        ARRRates(idx) = fallbackCouponRates{idx}.Rate();
    catch ME
        disp('error!')
        aaaa = 0;
    end
    computedFallbackRates(idx) = ARRRates(idx) +  spread;

end

aaa = 1.0;

for idx=1:numOfFallbackRates
    rateRecordDateStr{idx} = StrDate(rateRecordDate{idx},'str');
    accrualSpotDateStr{idx} = StrDate(accrualSpotDate{idx},'str');
    accrualStartDateStr{idx} = StrDate(accrualStartDate{idx},'str');
    accrualEndDateStr{idx} = StrDate(accrualEndDate{idx},'str');
    paymentDateStr{idx} = StrDate(paymentDate{idx},'str');
end

bbb = 1.0;

manualSchedule = cell(numOfFallbackCoupons,3);
liborFloatingSchedule = cell(numOfFallbackCoupons,2);

for idx=1:numOfFallbackCoupons
    manualSchedule{idx,1} = new_accrualStartDate{idx};
    manualSchedule{idx,2} = new_accrualEndDate{idx};
    manualSchedule{idx,3} = new_floatingRatePaymentDate{idx};
    
    liborFloatingSchedule{idx,1} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.accrualStartDate;
    liborFloatingSchedule{idx,2} = floatingLeg.couponRates{idx + floatingCouponFallbackStartIdx -1}.accrualEndDate;
    
end


% adjustingFactor = DayCount_Libor/DayCount_RFR
% for usd adjustingFactor = 1.0



floatingLeg_RFR = OvernightLegARRWithManualSchedule(manualSchedule,liborFloatingSchedule,overnightIndex,1.0,...
                                      floatingLeg_RFR_Spread,nominal,...
                                      floatingLegDayCounter,floatingLeg_RFR_DayCounter,floatingLeg_RFR_Convention,calcCalendar,adjustingFactor);

% constructing overnightLeg_ARR end

aaa = 1.0;



% vanillaSwap = VanillaSwapLiborTerm(type,nominal,swapTenorYearly,liborTermDate,...
%                           fixedSchedule,fixedRate,fixedDayCounter,...  
%                           floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,spotLag,payLag, ...
%                           floatingLeg_RFR_Schedule,overnightIndex,floatingLeg_RFR_Spread,floatingLeg_RFR_DayCounter,spotLag_RFR,payLag_RFR);


vanillaSwap = VanillaSwapLiborFallback(type,nominal,liborTermDate,...
                                  fixedLeg,...  
                                  floatingLegLIbor,libor3M_Index_fwd,...
                                  floatingLeg_RFR,overnightIndex);

out = vanillaSwap.Calculate(discountCurve_SOFR);
out3 = vanillaSwap.Calculate3(discountCurve_SOFR,discountCurve_SOFR,discountCurve_SOFR);


fairRate = out.fairRate;
ddd = 1.0;

% vanillaSwap = VanillaSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                       floatingSchedule,libor3M_Index_fwd,floatingSpread,floatingDayCounter,payLag);
% 
% % DualCurve
% out = vanillaSwap.Calculate(discountCurve_SOFR);
% SingleCurve
% out = vanillaSwap.Calculate(discountCurve_Libor3M_SC);


% fairRate = out.fairRate;
ddd = 1.0;

% vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                       floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                       floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;


%% Vanilla Libor3M Swap Pricing End

ddd = 1.0;
%         simulValue_NPV(idxhh) = out.NPV;
%         simulValue_fairRate(idxhh) = out.fairRate;


ddd = 1.0;





In [ ]:
test_krw_cd_swap_lgm2f_rm_test_20220831.m


clc
clear
% valueDate = '2017-02-15';
% valueDate = '2031-02-06';
valueDate = '2022-08-31';

%% import and construct a zeroCurve
rawData = [0.002739726	0.999966029
0.25	0.996511226
0.5	0.992280124
0.75	0.987596845
1	0.982621043
2	0.961939787
3	0.940929045
4	0.919913231
5	0.899189379
7	0.859227318
10	0.800719469
12	0.763469385
15	0.713252364
20	0.63604178
25	0.568313762
30	0.507859638
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve

rawData = [0.002739726	0.999966029
0.25	0.996629394
0.5	0.992558462
0.75	0.987746177
1	0.982587748
2	0.960172439
3	0.938671715
5	0.889347269
10	0.777120526
20	0.605931559
30	0.474709431
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
AllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.215065	0.220121	0.218143	0.217268	0.217763	0.220674	0.229047	0.23246	0.23246
2	0.228085	0.22313	0.215734	0.211543	0.206662	0.206834	0.211551	0.216567	0.216567
3	0.226522	0.220726	0.214253	0.208748	0.204926	0.203038	0.206839	0.21189	0.21189
4	0.22624	0.219786	0.212459	0.20725	0.203165	0.200556	0.204231	0.209077	0.209077
5	0.230919	0.220344	0.212208	0.20498	0.201186	0.195794	0.195845	0.200385	0.200385
7	0.225944	0.215296	0.210457	0.203122	0.197162	0.195517	0.194331	0.201949	0.201949
10	0.222322	0.213988	0.211738	0.207052	0.201912	0.196892	0.192926	0.205056	0.205056
15	0.228371	0.2337	0.22742	0.220403	0.216991	0.211288	0.203854	0.207489	0.207489
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
swaptionVolMktData = MktData(mktData);
AllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a LGM2FModel with initial modelParams
% rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

%swaption only initial parameters
% dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

%Samsung
% dHTest = [0.058507682	0.500210296
% 0.06145709	0.515096302
% 0.057841075	0.632558982
% 0.069442883	0.632845436
% 0.081752868	0.607322125
% 0.087574376	0.586095839
% 0.103284643	0.52688632
% 0.113936628	0.552598585
% 0.131046999	0.498409755
% 0.153553385	0.437472215
% 0.173381122	0.423805445
% 0.180703372	0.380344606
% 0.207484315	0.305927652
% 0.218329282	0.268192317
% 0.224885432	0.227191192
% 0.221244544	0.231204789
% 0.262502203	0.212209604
% 0.264579505	0.135257044
% 0.318821555	0.118179799
% 0.271515922	0.107957745
% 0.358024755	0.088834433
% 0.33143161	0.069384916
% 0.457059078	0.033745097
% 0.374067262	0.006573072
% ];

%SamsungRM
dHTest = [0.058199137	0.439812944
0.057647013	0.443886142
0.063354364	0.518082554
0.065269399	0.579740969
0.077128685	0.59114195
0.085528918	0.591340263
0.104393792	0.538750513
0.11291416	0.536902885
0.134194697	0.474805599
0.153678835	0.471481867
0.172052117	0.450723586
0.182453554	0.40951887
0.201728283	0.359690541
0.215075942	0.294697316
0.257820609	0.239127832
0.2740087	0.210367886
0.311975091	0.181279314
0.300261587	0.148976635
0.308961448	0.126759944
0.305067986	0.111096128
0.334786524	0.100394003
0.353049206	0.071752188
0.309407548	0.033515751
0.378161767	0.007321201
];
%Daewoo
% dHTest = [0.037286763	0.525314433
% 0.037286763	0.525314433
% 0.054571525	0.586928199
% 0.069289617	0.618362181
% 0.099111958	0.605543022
% 0.1123501	0.589459841
% 0.139006516	0.523225356
% 0.175987354	0.505500158
% 0.184616726	0.463242812
% 0.204164537	0.412151488
% 0.263444475	0.364947643
% 0.251615512	0.358337407
% 0.259708458	0.315375986
% 0.269854863	0.270026782
% 0.337623406	0.253701114
% 0.258270519	0.179914465
% 0.321985902	0.191343978
% 0.347503907	0.154371663
% 0.36490148	0.127052474
% 0.36706888	0.095165077
% 0.480469769	0.05084228
% 0.423787026	0.033262976
% 0.531944266	0.006463726
% 0.333342463	0.000100001
% ];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

%Samsung
% AlphaTest = [0.0289581	0.006633509
% 0.021585088	0.007003076
% 0.019812596	0.007056061
% 0.015014178	0.007801684
% 0.008683651	0.008360616
% 0.011075137	0.008248663
% 0.008794173	0.011226664
% 0.000329846	0.023982837
% ];

%SamsungRM
AlphaTest = [0.028261166	0.007593916
0.023089658	0.00702423
0.017644546	0.007148734
0.013868725	0.007556361
0.011165254	0.008303304
0.012029721	0.009072243
0.01016322	0.010637579
0.008594788	0.019827028
];
%Daewoo
% AlphaTest = [0.029142908	0.007983918
% 0.016018045	0.007219568
% 0.00912537	0.007233927
% 0.008101129	0.007774553
% 0.012850209	0.008916743
% 0.011065789	0.009477842
% 0.008930619	0.011214209
% 0.0001	0.032096513
% ];

Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% import irModels.*;
% import targetFunctions.*;
                    
KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));


tic
out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
toc


dH1Calib = KRWLGM.dH1;
dH2Calib = KRWLGM.dH2;
Alpha1Calib = KRWLGM.Alpha1;
Alpha2Calib = KRWLGM.Alpha2;


modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
%% LGM model building end
%% CMS CMSRangeAccrual valuation start
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2022-08-31');
rangeAccrualParams.maturity = 3;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0356;
couponFreq = 4;
tenorL = 10;
tenorS = 2;
lower = 0.0;
upper = 1;

tenorCD = 0.25;
lowerCD = -1;
upperCD = 1;

rangeDays = zeros(15,1);


%resetDate startDate endDate paymentDate
couponScheduleInt = [20170217    20170217    20180217    20180219
20180214    20180217    20190217    20190218
20190215    20190217    20200217    20200217
20200217    20200217    20210217    20210217
20210217    20210217    20220217    20220217
20220217    20220217    20230217    20230217
20230217    20230217    20240217    20240219
20240216    20240217    20250217    20250217
20250217    20250217    20260217    20260219
20260213    20260217    20270217    20270217
20270217    20270217    20280217    20280217
20280217    20280217    20290217    20290219
20290216    20290217    20300217    20300218
20300215    20300217    20310217    20310217
20310217    20310217    20320217    20320217
];

% for i=1:length(rangeDays)
%     if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
%         && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
%         rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
%     elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
%         rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
%     end
% end

callYN = ones(15,1);
callYN(1) = 0;
callYN(2) = 0;
callYN(3) = 0;
%resetDate startDate endDate paymentDate
callScheduleInt = [20170217    20170217    20180217    20180219
20180214    20180217    20190217    20190218
20190215    20190217    20200217    20200217
20200210    20200217    20210217    20200217
20210208    20210217    20220217    20210217
20220210    20220217    20230217    20220217
20230210    20230217    20240217    20230217
20240208    20240217    20250217    20240219
20250210    20250217    20260217    20250217
20260209    20260217    20270217    20260219
20270210    20270217    20280217    20270217
20280210    20280217    20290217    20280217
20290207    20290217    20300217    20290219
20300211    20300217    20310217    20300218
20310210    20310217    20320217    20310219
];


%resetDate startDate endDate paymentDate
floatingScheduleInt = [20170217	20170217	20170517	20170517
20170517	20170517	20170817	20170817
20170817	20170817	20171117	20171117
20171117	20171117	20180219	20180219
20180219	20180219	20180517	20180517
20180517	20180517	20180817	20180817
20180817	20180817	20181119	20181119
20181119	20181119	20190218	20190218
20190218	20190218	20190517	20190517
20190517	20190517	20190819	20190819
20190819	20190819	20191118	20191118
20191118	20191118	20200217	20200217
20200217	20200217	20200518	20200518
20200518	20200518	20200817	20200817
20200817	20200817	20201117	20201117
20201117	20201117	20210217	20210217
20210217	20210217	20210517	20210517
20210517	20210517	20210817	20210817
20210817	20210817	20211117	20211117
20211117	20211117	20220217	20220217
20220217	20220217	20220517	20220517
20220517	20220517	20220817	20220817
20220817	20220817	20221117	20221117
20221117	20221117	20230217	20230217
20230217	20230217	20230517	20230517
20230517	20230517	20230817	20230817
20230817	20230817	20231117	20231117
20231117	20231117	20240219	20240219
20240219	20240219	20240517	20240517
20240517	20240517	20240819	20240819
20240819	20240819	20241118	20241118
20241118	20241118	20250217	20250217
20250217	20250217	20250519	20250519
20250519	20250519	20250818	20250818
20250818	20250818	20251117	20251117
20251117	20251117	20260217	20260217
20260217	20260217	20260518	20260518
20260518	20260518	20260817	20260817
20260817	20260817	20261117	20261117
20261117	20261117	20270217	20270217
20270217	20270217	20270517	20270517
20270517	20270517	20270817	20270817
20270817	20270817	20271117	20271117
20271117	20271117	20280217	20280217
20280217	20280217	20280517	20280517
20280517	20280517	20280817	20280817
20280817	20280817	20281117	20281117
20281117	20281117	20290219	20290219
20290219	20290219	20290517	20290517
20290517	20290517	20290817	20290817
20290817	20290817	20291119	20291119
20291119	20291119	20300218	20300218
20300218	20300218	20300517	20300517
20300517	20300517	20300819	20300819
20300819	20300819	20301118	20301118
20301118	20301118	20310217	20310217
20310217	20310217	20310519	20310519
20310519	20310519	20310818	20310818
20310818	20310818	20311117	20311117
20311117	20311117	20320217	20320217
];


manualScheduleYN = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
        && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
    elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
        rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
    end
end

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','tenorCD','lowerCD','upperCD',...
                                'rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                        upper,tenorCD,lowerCD,upperCD,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;              

floatingSpread = 0.0;

floatingIndex = 'Libor3M';

% Libor3MFloatingFixing = containers.Map({'2016-07-15'},{0.0068785});
% 
% Libor3MFloatingFixing = containers.Map({'2017-02-17'},{0.0137});
% Libor3MFloatingFixing('2017-02-18') = 0.0137;

Libor3MFloatingFixing = containers.Map({'2017-02-17'},{0.0149});
Libor3MFloatingFixing('2017-02-18') = 0.0149;

tempDate = H_Date('2017-02-17');
for i=1:6000
%     toDate = AddDate(valueDate,currentTime,'day');
    Libor3MFloatingFixing(StrDate(tempDate,'str')) = 0.0149;
    tempDate = AddDate(tempDate,1,'day');
end

allFixings = containers.Map({'KRWLibor3M'},{Libor3MFloatingFixing});

swapInfo = containers.Map({'floatingScheduleInt','floatingSpread','floatingIndex','allFixings'}, ...
                          {floatingScheduleInt,floatingSpread,floatingIndex,allFixings});

swapParams.params = swapInfo;                       

rangeAccrualSwap = CDCMSSpreadRangeAccrualSwap(rangeAccrualParams,swapParams);

numOfRandom = 40000;
% numOfRandom = 30000;
%numOfRandom = 100;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                        {'AMC',7,numOfRandom});

numMethodAMC = NumMethod(numMethodInfo);

% weekly step calculator
pricer = DailyWeeklyRiskyPricer_CDCMSSpreadRASwapSimple(rangeAccrualSwap,KRWLGMRisky,numMethodAMC,false);

simulOut.nonCall = zeros(1000,1);
simulOut.callable = zeros(1000,1);
simulOut.nonCallClosed = zeros(1000,1);

simulOut.floatingLegMC = zeros(1000,1);
simulOut.floatingLegClosed = zeros(1000,1);

tempValueDateH = valueDateH;
for i=1:2000


for idx=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(idx,2))) >= 0 ...
        && DateDiff(valueDateH,H_Date(couponScheduleInt(idx,3))) < 0
        rangeDays(idx) = DateDiff(valueDateH,H_Date(couponScheduleInt(idx,2))) + 1;
    elseif DateDiff(valueDateH,H_Date(couponScheduleInt(idx,3))) >= 0
        rangeDays(idx) = DateDiff(H_Date(couponScheduleInt(idx,3)),H_Date(couponScheduleInt(idx,2)));
    end
end

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','tenorCD','lowerCD','upperCD',...
                                'rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                        upper,tenorCD,lowerCD,upperCD,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;      

rangeAccrualSwap = CDCMSSpreadRangeAccrualSwap(rangeAccrualParams,swapParams);
% pricer = DailyWeeklyRiskyPricer_CDCMSSpreadRASwapSimple(rangeAccrualSwap,KRWLGMRisky,numMethodAMC,false);
pricer = DailyWeeklyRiskyPricer_CDCMSSpreadRASwapSimple(rangeAccrualSwap,KRWLGMRisky,numMethodAMC,true);

pricingOutMC = pricer.computePriceMCCallableGeneric(valueDateH);
simulOut.nonCall(i)  = pricingOutMC.nonCall.npv;
simulOut.callable(i) = pricingOutMC.callable.npv; 
simulOut.floatingLegMC(i) = pricingOutMC.floatingLeg.npv;
pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);
simulOut.nonCallClosed(i) = pricingOutClosed.nonCall.npv;
simulOut.floatingLegClosed(i) = pricingOutClosed.floatingLeg.npv;

valueDateH = AddDate(tempValueDateH,i,'day');

% simulOut.closed(i) = pricingOutClosed.
end
% daily closed form pricer
%pricingOutLGM =  rangeAccrual.computePrice(valueDate,KRWLGM);

%pricingOut = rangeAccrual.computePriceMCForward(valueDate,KRWLGM);

% out =1;


aaaa =1.0;


In [ ]:
test_krw_cd_ktb3m_spread_dualra_bs1fck1flgm2f_20210612.m

clc
clear
% valueDate = '2018-01-09';
% valueDate = '2030-02-17';
% valueDate = '2029-02-17';

valueDate = '2017-02-17';
simulVolRTenor = zeros(10000,100);
simulVolRQuote = zeros(10000,100);
simulRmse = zeros(10000,1);
simulNoncallMC = zeros(10000,1);
simulNoncallClosed = zeros(10000,1);
simulNoncallMCCashflowNPV = zeros(10000,1);
simulNoncallClosedCashflowNPV = zeros(10000,1);

idxMax = 10;
idxMax = 2;

tempValueDate = valueDate;
tempValueDateH = H_Date(tempValueDate);
for idxSimul=2:idxMax
%     tempValueDateH2 = tempValueDateH.AddDate(+1*(idxSimul-1),'day');
%     valueDate = StrDate(tempValueDateH2,'str');
%     valueDate = '2018-02-16';
%% USDCMSRA KRWCMSSpreadRA Pricing start
%% CMS CMSRangeAccrual valuation start
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2017-02-17');
rangeAccrualParams.maturity = 15;

%% added for 1F Calibration input Generation
rangeAccrualParams.endDate = rangeAccrualParams.startDate.AddDate(rangeAccrualParams.maturity,'year');

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0362;
couponFreq = 1;
tenorL = 10;
tenorS = 2;
lowerS = 0.0;
upperS = 1;

% lowerS = -1.0;
% upperS = 1.0;

tenor = 10;
lower = 0.0;
upper = 0.06;

rangeDays = zeros(15,1);


%resetDate startDate endDate paymentDate
couponScheduleInt = [20170217    20170217    20180217    20180219
20180214    20180217    20190217    20190218
20190215    20190217    20200217    20200217
20200217    20200217    20210217    20210217
20210217    20210217    20220217    20220217
20220217    20220217    20230217    20230217
20230217    20230217    20240217    20240219
20240216    20240217    20250217    20250217
20250217    20250217    20260217    20260219
20260213    20260217    20270217    20270217
20270217    20270217    20280217    20280217
20280217    20280217    20290217    20290219
20290216    20290217    20300217    20300218
20300215    20300217    20310217    20310217
20310217    20310217    20320217    20320217
];

%% CK1F model param matu setting start
% matu = 15 + 1.0/365.0;
matu = DateDiff(H_Date(couponScheduleInt(end,4)),H_Date(valueDate))/365.0;
% matu = 14.0082;
% matu = 13.9918;

%% end

callYN = ones(15,1);
callYN(1) = 0;
callYN(2) = 0;
callYN(3) = 0;
%resetDate startDate endDate paymentDate
% callScheduleInt = [20170217    20170217    20180217    20180219
% 20180214    20180217    20190217    20190218
% 20190215    20190217    20200217    20200217
% 20200210    20200217    20210217    20200217
% 20210208    20210217    20220217    20210217
% 20220210    20220217    20230217    20220217
% 20230210    20230217    20240217    20230217
% 20240208    20240217    20250217    20240219
% 20250210    20250217    20260217    20250217
% 20260209    20260217    20270217    20260219
% 20270210    20270217    20280217    20270217
% 20280210    20280217    20290217    20280217
% 20290207    20290217    20300217    20290219
% 20300211    20300217    20310217    20300218
% 20310210    20310217    20320217    20310219
% ];

callScheduleInt = [20200210	20200217
20210208	20210217
20220210	20220217
20230210	20230217
20240208	20240219
20250210	20250217
20260209	20260219
20270210	20270217
20280210	20280217
20290207	20290219
20300211	20300218
20310210	20310219
];

manualScheduleYN = 1;

for i=1:length(rangeDays)
    if DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) >= 0 ...
        && DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) < 0
        rangeDays(i) = DateDiff(valueDateH,H_Date(couponScheduleInt(i,2))) + 1;
    elseif DateDiff(valueDateH,H_Date(couponScheduleInt(i,3))) >= 0
        rangeDays(i) = DateDiff(H_Date(couponScheduleInt(i,3)),H_Date(couponScheduleInt(i,2)));
    end
end

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lowerS','upperS','tenor','lower','upper',...
                                'rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lowerS, ...
                        upperS,tenor,lower,upper,...
                        rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;

rangeAccrual = CMSCMSSpreadDualRangeAccrual(rangeAccrualParams);

%% KRW Model building

%% import and construct a zeroCurve
rawData = [0.0027397	0.999959728
0.25	0.995871845
0.5	0.99154544
0.75	0.986866679
1	0.981892589
2	0.960911374
3	0.93966943
4	0.918499662
5	0.897462069
7	0.856996516
10	0.797414396
12	0.758158208
15	0.706176858
20	0.628858172
25	0.560374492
30	0.499419054
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'KRW','KRWZero'});

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDateH,'KRW','KRWZero'});
                    
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);

KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve
rawData = [0.002739726	0.999959728
0.25	0.996247006
0.5	0.992160837
0.75	0.98726598
1	0.982295684
2	0.959567666
3	0.93844269
5	0.889131313
10	0.775743705
20	0.60332667
30	0.469009914
    ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDateH,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);

KRWAllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.180918	0.187412	0.191522	0.193809	0.197145	0.197977	0.202251	0.204123	0.204123
2	0.201024	0.201271	0.197475	0.196981	0.195931	0.194313	0.195361	0.199344	0.199344
3	0.206411	0.20229	0.197703	0.195155	0.193529	0.191295	0.19266	0.196655	0.196655
4	0.204366	0.201238	0.195861	0.193106	0.191647	0.188532	0.191248	0.195619	0.195619
5	0.208548	0.202396	0.19528	0.190753	0.189714	0.184224	0.18525	0.189847	0.189847
7	0.202382	0.195093	0.189328	0.18233	0.177407	0.176157	0.177346	0.191365	0.191365
10	0.189532	0.183121	0.178702	0.177781	0.174801	0.175461	0.177286	0.193966	0.193966
15	0.20237	0.207678	0.1942	0.190125	0.188303	0.187428	0.18773	0.198563	0.198563
    ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
swaptionVolMktData = MktData(mktData);

 KRWAllMktData('swaptionVol') = swaptionVolMktData;
%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a LGM2FModel with initial modelParams
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

%swaption only initial parameters
% dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

%Samsung
% dHTest = [0.058507682	0.500210296
% 0.06145709	0.515096302
% 0.057841075	0.632558982
% 0.069442883	0.632845436
% 0.081752868	0.607322125
% 0.087574376	0.586095839
% 0.103284643	0.52688632
% 0.113936628	0.552598585
% 0.131046999	0.498409755
% 0.153553385	0.437472215
% 0.173381122	0.423805445
% 0.180703372	0.380344606
% 0.207484315	0.305927652
% 0.218329282	0.268192317
% 0.224885432	0.227191192
% 0.221244544	0.231204789
% 0.262502203	0.212209604
% 0.264579505	0.135257044
% 0.318821555	0.118179799
% 0.271515922	0.107957745
% 0.358024755	0.088834433
% 0.33143161	0.069384916
% 0.457059078	0.033745097
% 0.374067262	0.006573072
% ];

%Daewoo
% dHTest = [0.037286763	0.525314433
% 0.037286763	0.525314433
% 0.054571525	0.586928199
% 0.069289617	0.618362181
% 0.099111958	0.605543022
% 0.1123501	0.589459841
% 0.139006516	0.523225356
% 0.175987354	0.505500158
% 0.184616726	0.463242812
% 0.204164537	0.412151488
% 0.263444475	0.364947643
% 0.251615512	0.358337407
% 0.259708458	0.315375986
% 0.269854863	0.270026782
% 0.337623406	0.253701114
% 0.258270519	0.179914465
% 0.321985902	0.191343978
% 0.347503907	0.154371663
% 0.36490148	0.127052474
% 0.36706888	0.095165077
% 0.480469769	0.05084228
% 0.423787026	0.033262976
% 0.531944266	0.006463726
% 0.333342463	0.000100001
% ];

% %Sungwoo
% dHTest = [0.060654929	0.585843811
% 0.065181321	0.621357434
% 0.059213061	0.705433024
% 0.070356009	0.697274699
% 0.084854535	0.641491331
% 0.078986949	0.60204605
% 0.103979263	0.504724505
% 0.128905871	0.522635178
% 0.142868097	0.436948669
% 0.167927649	0.341253642
% 0.171414266	0.384418469
% 0.173137707	0.332946905
% 0.205465936	0.252922264
% 0.190811162	0.233849244
% 0.212134317	0.235149902
% 0.208305697	0.261176412
% 0.267225845	0.213982892
% 0.244169602	0.131637885
% 0.293167102	0.110338057
% 0.302235653	0.107810085
% 0.341199539	0.095697121
% 0.353976	0.072468098
% 0.421517994	0.033547027
% 0.357938773	0.006568033
% ];

%JaeYeol
dHTest = [0.060438773	0.585467789
0.064928789	0.621766986
0.059045192	0.705355899
0.070178721	0.6972879
0.084746775	0.641430112
0.079080439	0.601700141
0.1039792	0.504405369
0.129262749	0.522031261
0.14309303	0.436403222
0.167840108	0.341211636
0.171134327	0.384814515
0.17295042	0.333229804
0.205446544	0.253016302
0.190785578	0.233852044
0.212042588	0.235092969
0.208563343	0.261106368
0.26724335	0.213909029
0.24410813	0.131731127
0.292886806	0.110481688
0.302159283	0.107800708
0.341084552	0.095566454
0.353944447	0.072422244
0.421105867	0.033562964
0.357823112	0.006568462
];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

%Samsung
% AlphaTest = [0.0289581	0.006633509
% 0.021585088	0.007003076
% 0.019812596	0.007056061
% 0.015014178	0.007801684
% 0.008683651	0.008360616
% 0.011075137	0.008248663
% 0.008794173	0.011226664
% 0.000329846	0.023982837
% ];

%Daewoo
% AlphaTest = [0.029142908	0.007983918
% 0.016018045	0.007219568
% 0.00912537	0.007233927
% 0.008101129	0.007774553
% 0.012850209	0.008916743
% 0.011065789	0.009477842
% 0.008930619	0.011214209
% 0.0001	0.032096513
% ];

% %sungwoo
% AlphaTest = [0.032526561	0.005901286
% 0.021888486	0.006159185
% 0.015500653	0.006183779
% 0.011483817	0.007702683
% 0.008112119	0.008910715
% 0.010323471	0.008310996
% 0.006966815	0.012340985
% 0.000302754	0.020086129
% ];

%jaeweol
AlphaTest = [0.032529518	0.005904921
0.021882297	0.0061619
0.015498414	0.006184972
0.011471555	0.007705108
0.008105853	0.008915352
0.010349202	0.008306695
0.006993193	0.012332999
0.000309834	0.020072884
];


Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});

KRWLGM2F = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));


tic
out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% out = KRWLGM2F.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
toc


dH1Calib = KRWLGM2F.dH1;
dH2Calib = KRWLGM2F.dH2;
Alpha1Calib = KRWLGM2F.Alpha1;
Alpha2Calib = KRWLGM2F.Alpha2;

modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
%% LGM model building end

%% KRW model building end

%% USD model building start
%% import and construct a zeroCurve
% rawData = [0.002739726	0.999939642532736
% 0.083333333	0.998079845818571
% 0.166666667	0.995617822
% 0.25	0.993222896
% 0.5	0.985718537
% 0.75	0.978326529
% 1	0.970948408
% 2	0.939217438
% 3	0.907604118
% 4	0.87691966
% 5	0.847097877
% 7	0.789249595
% 10	0.707038201
% 12	0.655827342
% 15	0.585716079
% 20	0.485573763
% 25	0.404443236
% 30	0.338376001
%     ];

%20181120 test(15Y 0bp)
% rawData = [0.002739726000000 	0.999939642532736 
% 0.083333333000000 	0.998079845818571 
% 0.166666667000000 	0.996017172709282 
% 0.250000000000000 	0.993681782812906 
% 0.500000000000000 	0.986127635282528 
% 0.750000000000000 	0.978883385727447 
% 1.000000000000000 	0.971735256532315 
% 2.000000000000000 	0.941019604108658 
% 3.000000000000000 	0.911607831104224 
% 4.000000000000000 	0.883515696365466 
% 5.000000000000000 	0.856299172379924 
% 7.000000000000000 	0.803171551260153 
% 10.000000000000000 	0.726677130452903 
% 12.000000000000000 	0.678514036638640 
% 15.000000000000000 	0.612652100084208 
% 20.000000000000000 	0.518466538040656 
% 25.000000000000000 	0.441278924133659 
% 30.000000000000000 	0.376384618090949 
% ];
% 
% %20181120 test(15Y -50bp)
rawData = [0.002739726000000 	0.999939642532736 
0.083333333000000 	0.998079845818571 
0.166666667000000 	0.996017172709282 
0.250000000000000 	0.993681782812906 
0.500000000000000 	0.986127635282528 
0.750000000000000 	0.978883385727447 
1.000000000000000 	0.971735256532315 
2.000000000000000 	0.941019604108658 
3.000000000000000 	0.911607831104224 
4.000000000000000 	0.883515696365466 
5.000000000000000 	0.856299172379924 
7.000000000000000 	0.803171551260153 
10.000000000000000 	0.726677130452903 
12.000000000000000 	0.678514036638640 
15.000000000000000 	0.668571167311167 
20.000000000000000 	0.511950505202417 
25.000000000000000 	0.435472745934597 
30.000000000000000 	0.371467333985270 
];


% rawData = [0.002739726	0.999960071
% 0.083333333	0.998705285
% 0.166666667	0.997278569
% 0.25	0.99570906
% 0.5	0.990659472
% 0.75	0.985691562
% 1	0.980706602
% 2	0.958190889
% 3	0.935244778
% 4	0.91270743
% 5	0.890529514
% 7	0.846476648
% 10	0.781398058
% 12	0.739443263
% 15	0.680504997
% 20	0.593081133
% 25	0.519315395
% 30	0.456759825
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDateH,'USD','USDZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
USDAllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve
% rawData = [0.002739726	0.999932675
% 0.083333333	0.997873377
% 0.166666667	0.995617822
% 0.25	0.993222896
% 0.5	0.985718537
% 0.75	0.978326529
% 1	0.970948408
% 2	0.939217438
% 3	0.907604118
% 4	0.87691966
% 5	0.847097877
% 7	0.789249595
% 10	0.707038201
% 12	0.655827342
% 15	0.585716079
% 20	0.485573763
% 25	0.404443236
% 30	0.338376001
%     ];
%20181120 test(15Y 0bp)
% rawData = [0.002739726000000 	0.999939642532736 
% 0.083333333000000 	0.998079845818571 
% 0.166666667000000 	0.996017172709282 
% 0.250000000000000 	0.993681782812906 
% 0.500000000000000 	0.986127635282528 
% 0.750000000000000 	0.978883385727447 
% 1.000000000000000 	0.971735256532315 
% 2.000000000000000 	0.941019604108658 
% 3.000000000000000 	0.911607831104224 
% 4.000000000000000 	0.883515696365466 
% 5.000000000000000 	0.856299172379924 
% 7.000000000000000 	0.803171551260153 
% 10.000000000000000 	0.726677130452903 
% 12.000000000000000 	0.678514036638640 
% 15.000000000000000 	0.612652100084208 
% 20.000000000000000 	0.518466538040656 
% 25.000000000000000 	0.441278924133659 
% 30.000000000000000 	0.376384618090949 
% ];

%20181120 test(15Y -50bp)
rawData = [0.002739726000000 	0.999939642532736 
0.083333333000000 	0.998079845818571 
0.166666667000000 	0.996017172709282 
0.250000000000000 	0.993681782812906 
0.500000000000000 	0.986127635282528 
0.750000000000000 	0.978883385727447 
1.000000000000000 	0.971735256532315 
2.000000000000000 	0.941019604108658 
3.000000000000000 	0.911607831104224 
4.000000000000000 	0.883515696365466 
5.000000000000000 	0.856299172379924 
7.000000000000000 	0.803171551260153 
10.000000000000000 	0.726677130452903 
12.000000000000000 	0.678514036638640 
15.000000000000000 	0.668571167311167 
20.000000000000000 	0.511950505202417 
25.000000000000000 	0.435472745934597 
30.000000000000000 	0.371467333985270 
];

% rawData = [0.002739726	0.999960071
% 0.083333333	0.998705285
% 0.166666667	0.997278569
% 0.25	0.99570906
% 0.5	0.990659472
% 0.75	0.985691562
% 1	0.980706602
% 2	0.958190889
% 3	0.935244778
% 4	0.91270743
% 5	0.890529514
% 7	0.846476648
% 10	0.781398058
% 12	0.739443263
% 15	0.680504997
% 20	0.593081133
% 25	0.519315395
% 30	0.456759825
%     ];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDateH,'USD','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
USDAllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% 1	0.18785	0.21535	0.228	0.23405	0.23705	0.23735	0.2341	0.23	0.2259
% 2	0.22675	0.24245	0.25075	0.25335	0.25235	0.2504	0.2464	0.23945	0.2325
% 3	0.25885	0.26305	0.2647	0.263	0.26135	0.2569	0.2502	0.243325	0.23645
% 4	0.2736	0.27245	0.2692	0.266	0.263	0.25715	0.2503	0.243125	0.23595
% 5	0.27605	0.272	0.26905	0.26565	0.2627	0.2561	0.24885	0.24125	0.23365
% 7	0.2656	0.26325	0.25995	0.25655	0.25355	0.24835	0.24125	0.233575	0.2259
% 10	0.24575	0.2439	0.24245	0.23995	0.2385	0.23425	0.22875	0.2217	0.21465
% 15	0.2192	0.21675	0.2156	0.21345	0.21175	0.20865	0.20585	0.2007	0.19555
%     ];

%20181120 test
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.186400	0.215150	0.222300	0.222650	0.222400	0.217500	0.210450	0.206325	0.202200
2	0.239000	0.250650	0.248850	0.244300	0.238350	0.229700	0.220900	0.215750	0.210600
3	0.261700	0.264550	0.257200	0.249350	0.242750	0.233600	0.223450	0.218025	0.212600
4	0.268400	0.263150	0.255750	0.248800	0.240700	0.231850	0.222550	0.217200	0.211850
5	0.261400	0.256900	0.250650	0.243250	0.237200	0.228700	0.220600	0.215025	0.209450
7	0.244800	0.240650	0.235800	0.231150	0.226900	0.220200	0.213000	0.207375	0.201750
10	0.220550	0.217350	0.215400	0.213450	0.212400	0.207450	0.200850	0.195400	0.189950
15	0.199900	0.197550	0.195750	0.194300	0.192700	0.187450	0.182350	0.177775	0.173200
    ];

% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% 1	0.180918	0.187412	0.191522	0.193809	0.197145	0.197977	0.202251	0.204123	0.204123
% 2	0.201024	0.201271	0.197475	0.196981	0.195931	0.194313	0.195361	0.199344	0.199344
% 3	0.206411	0.20229	0.197703	0.195155	0.193529	0.191295	0.19266	0.196655	0.196655
% 4	0.204366	0.201238	0.195861	0.193106	0.191647	0.188532	0.191248	0.195619	0.195619
% 5	0.208548	0.202396	0.19528	0.190753	0.189714	0.184224	0.18525	0.189847	0.189847
% 7	0.202382	0.195093	0.189328	0.18233	0.177407	0.176157	0.177346	0.191365	0.191365
% 10	0.189532	0.183121	0.178702	0.177781	0.174801	0.175461	0.177286	0.193966	0.193966
% 15	0.20237	0.207678	0.1942	0.190125	0.188303	0.187428	0.18773	0.198563	0.198563
%     ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDateH,'USD','USDSwaptionVol'});
swaptionVolMktData = MktData(mktData);
USDAllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
USDBlack = IRBlack(IRModel(Model(USDAllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        swaptionPriceSurf.surface(i,j)= USDBlack.BlackATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = USDBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a CK1FModel with initial modelParams

% vol_r.tenor = [1;2;3;4;5;7;10;15];
% vol_r.quote = ones(1,length(vol_r.tenor))*0.01;
% 
% meanR_r = 0.03;
% 
% joint_swap_tenor = 1;
% 
% targetSize = 15;
% 
% freq = 4;
% tic
% modelParams= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
%                         {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
% USDCK1F = CK1F(IRModel(Model(USDAllMktData,modelParams)));
% 
% out = USDCK1F.CalibrateToATMDiagSwaption(USDBlack,'global');
% aaa =1.0;


%% We change instrument specific local Calibration Setting
%% instead of matu and targetSize
%% we have instrument startDate, instrument endDate, valueDate
%% and calibration tenor as a new Input
%% create a CK1FModel with initial modelParams

%vol_r.tenor = [1;2;3;4;5;7;10;15];

%20181120 test
vol_r.tenor = [1;2;3;4;5;7;10;15];
vol_r.quote = ones(1,length(vol_r.tenor))*0.01;

meanR_r = 0.03;

joint_swap_tenor = 1;

targetSize = 15;

freq = 4;

tic


modelParams= containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'}, ...
                        {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});
                    
% added for instrument specific calibration input
instrumentStartDate = rangeAccrual.startDate;
instrumentEndDate = rangeAccrual.endDate;
% same name as current quantlib hw1f calibration input

% calibrationTargetTenor == swaption option expiry from the instrumentStartDate

%1y,2y,...,14y
calibrationTargetTenor = 1:1:14;
%20181120 test
%calibrationTargetTenor = [1;2;3;4;5;7;10;12];

modelParams('instrumentStartDate') = instrumentStartDate;
modelParams('instrumentEndDate') = instrumentEndDate;
modelParams('calibrationTargetTenor') = calibrationTargetTenor;

USDCK1F = CK1F(IRModel(Model(USDAllMktData,modelParams)));

out = USDCK1F.CalibrateToATMDiagSwaption(USDBlack,'global');
% out = USDCK1F.CalibrateToATMDiagSwaptionForward(USDBlack,'global');
% out = USDCK1F.CalibrateToATMDiagSwaptionBackward(USDBlack,'global');

% targetSwapTenor = 10;
targetSwapTenor = rangeAccrual.tenor;
% out = USDCK1F.CalibrateToATMSwaptionTenorStrip(USDBlack,targetSwapTenor,'global');

% out = USDCK1F.CalibrateToATMSwaptionTenorDiagSwaption(USDBlack,targetSwapTenor,'global');

for idxTemp=1:length(out.vol_r.tenor)
    simulVolRTenor(idxSimul,idxTemp) = out.vol_r.tenor(idxTemp);
    simulVolRQuote(idxSimul,idxTemp) = out.vol_r.quote(idxTemp);
end

simulRmse(idxSimul) = out.rmseTotal;

aaa =1.0;

%% USD model building end

%% USDKRW fx model buidling start
   rawData = [0.083333333	0.0907
0.25	0.0995
0.5	0.0997
1	0.1023
2	0.1073
3	0.1092
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','USDKRWATMTermVol'});
impliedTermVolMktData = MktData(mktData);
%     impliedTermVole = FXVolSurface(impliedTermVolMktData);
%     impliedVolSurface.params('moneyNessType') = 'deltaFwdMoneyNess';
USDKRWAllMktData =containers.Map('impliedTermVol',impliedTermVolMktData);
%     AllMktData('impliedVolSurface') = impliedTermVolMktData;

%% create  an FXBlackModel to calculate market PlainVanilla Options
%     modelParams= containers.Map({'modelName','forwardType'},{'Black','forwardCurve'});

%     USDKRWBlack = FXBlack(FXModel(Model(USDKRWAllMktData,modelParams))); 

%% USDKRW fx model building end

%% MultiAssetModel building start
multiAssetMktData = containers.Map({'KRW','USD','USDKRW'}, ...
                        {KRWAllMktData,USDAllMktData,USDKRWAllMktData});

%KRW,USD & USDKRW
correlationMatrix = ones(3,3);
% correlationMatrix(1,2) = 0.3;
correlationMatrix(1,2) = 0.85;
% correlationMatrix(1,2) = 0.99;
% correlationMatrix(1,2) = -0.98 + 0.2*(idxSimul) - 0.03;


correlationMatrix(1,3) = -0.0;
correlationMatrix(2,3) = 0.0;
correlationMatrix(2,1) = correlationMatrix(1,2);
correlationMatrix(3,1) = correlationMatrix(1,3);
correlationMatrix(3,2) = correlationMatrix(2,3);


multiAssetMktData('correlationMatrix') = correlationMatrix;

modelNameMap = containers.Map({'KRW','USD'},{KRWLGM2FRisky,USDCK1F});
modelParams= containers.Map({'modelNameMap'},{modelNameMap});
% correlationNames = {'KRW','USD'};
modelParams('stochasticModelNames') = {'KRW','USD'};
modelParams('correlationMatrix') = correlationMatrix;

modelParams('refModelName') = 'KRW';

% modelParams('useQuasiRandomYN') = 'true';
modelParams('useQuasiRandomYN') = 'false';

modelParams('useBrigoCorrYN') = 'simple_adjust';
% modelParams('useBrigoCorrYN') = 'NO';
% modelParams('useBrigoCorrYN') = 'YES';


KRWLGM2FRiskyUSDCK1F = MultiAssetModelLGM2FCK1F(multiAssetMktData,modelParams);

%% MultiAssetModel building end

%% NumMethod Init Start

numOfRandom = 40000;

numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                        {'AMC',7,numOfRandom});

numMethodAMC = NumMethod(numMethodInfo);

%% NumMethod Init End
% pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRA(rangeAccrual,KRWLGM2FRiskyUSDCK1F,numMethodAMC,false);
pricer = DailyWeeklyRiskyPricer_USDCMS_KRWCMSSpreadDualRASimpleCall(rangeAccrual,KRWLGM2FRiskyUSDCK1F,numMethodAMC,false);

% pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSANo');
pricingOutMC1 = pricer.computePriceMCCallableGeneric(valueDateH,'PSA');

pricingOutClosed = pricer.computePriceClosedSimple(valueDateH);

simulNoncallMC(idxSimul) = pricingOutMC1.nonCall.npv;
simulNoncallClosed(idxSimul) = pricingOutClosed.pricerInfo.nonCall.npv;

for indexj=1:length(pricingOutMC1.nonCall.cashflow_npv_e)
    simulNoncallMCCashflowNPV(idxSimul,indexj) = pricingOutMC1.nonCall.cashflow_npv_e(indexj);
    simulNoncallClosedCashflowNPV(idxSimul,indexj) = pricingOutClosed.pricerInfo.nonCall.cashflow_npv_e(indexj);
end

end

aaaa =1.0;


In [ ]:
test_InverseFloater_parallelShift_dealStart20201221_CK1F_KSA2.m


clc
clear

valueDate = '2020-12-18';

    

zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;
deltaShiftAmount = 0.0001;

callablePrice = zeros(10000,1);
nonCallPrice = zeros(10000,1);
nonCallBPrice = zeros(10000,1);
eventTimeStep = zeros(10000,10000);
closedPrice = zeros(100,1);

delta_callablePrice = zeros(10000,2);
delta_nonCallPrice = zeros(10000,2);
delta_nonCallBPrice = zeros(10000,2);

simulIRSZcv = zeros(63,10000);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CMSAVGSteepener Pricing start
idxMax = 365 + 10;
maxIdx = 3;

idxMax = 11;

simulationType = 'null';
% 
% % simulationType = 'ParallelDelta';
% 
% if strcmp(simulationType,'PointByPointDelta')
%     numberOfTenor = 17;
% elseif strcmp(simulationType,'ParallelDelta')
%     numberOfTenor = 1;
% else
%     numberOfTenor = 1;
% end
% idxMax = 2*numberOfTenor + 1;

idxMax = 11;
maxIdx = 3;
for idx2=1:idxMax
    
% tempDateHong = AddDate(H_Date('2020-02-27'),idx2-1,'day');
% valueDateH = tempDateHong;
valueDateH = H_Date('2020-12-18');
%% security info start
% valueDateH = H_Date(valueDate);
steepenerParams.payCcy = 'KRW';
steepenerParams.startDate = H_Date('2020-12-18');
steepenerParams.endDate = H_Date('2040-12-18');

nominal = 10000;
basePrice = 100.0;

%% Inverse Floater
%structuredCoupon Info

structuredCouponInfo = struct;
%dummy coupon
% structuredCouponInfo.coupon = 0.03;
structuredCouponInfo.coupon = 0.056;

% local floor
% local cap
% multipler
structuredCouponInfo.localFloor = -0.03;
structuredCouponInfo.localCap = 10000.0;
structuredCouponInfo.multiplier = 2.0;
structuredCouponInfo.tenorInverseFloater = 0.25;
structuredCouponInfo.rateType_tenorInverseFloater = 'LIBOR';

% structuredCouponInfo.tenorS = 2;
% structuredCouponInfo.rateType_tenorS = 'CMS';
structuredCouponInfo.spread = 0.0;

% structuredCouponInfo.eventType = 'RESET';
structuredCouponInfo.eventType = 'END';


%couponSchedule  reset start end pay (quarterly schedule)
structuredCouponInfo.structuredCouponScheduleInt=[20201218	20201218	20210318	20210318
20210318	20210318	20210618	20210618
20210618	20210618	20210918	20210918
20210918	20210918	20211218	20211218
20211218	20211218	20220318	20220318
20220318	20220318	20220618	20220618
20220618	20220618	20220918	20220918
20220918	20220918	20221218	20221218
20221218	20221218	20230318	20230318
20230318	20230318	20230618	20230618
20230618	20230618	20230918	20230918
20230918	20230918	20231218	20231218
20231218	20231218	20240318	20240318
20240318	20240318	20240618	20240618
20240618	20240618	20240918	20240918
20240918	20240918	20241218	20241218
20241218	20241218	20250318	20250318
20250318	20250318	20250618	20250618
20250618	20250618	20250918	20250918
20250918	20250918	20251218	20251218
20251218	20251218	20260318	20260318
20260318	20260318	20260618	20260618
20260618	20260618	20260918	20260918
20260918	20260918	20261218	20261218
20261218	20261218	20270318	20270318
20270318	20270318	20270618	20270618
20270618	20270618	20270918	20270918
20270918	20270918	20271218	20271218
20271218	20271218	20280318	20280318
20280318	20280318	20280618	20280618
20280618	20280618	20280918	20280918
20280918	20280918	20281218	20281218
20281218	20281218	20290318	20290318
20290318	20290318	20290618	20290618
20290618	20290618	20290918	20290918
20290918	20290918	20291218	20291218
20291218	20291218	20300318	20300318
20300318	20300318	20300618	20300618
20300618	20300618	20300918	20300918
20300918	20300918	20301218	20301218
20301218	20301218	20310318	20310318
20310318	20310318	20310618	20310618
20310618	20310618	20310918	20310918
20310918	20310918	20311218	20311218
20311218	20311218	20320318	20320318
20320318	20320318	20320618	20320618
20320618	20320618	20320918	20320918
20320918	20320918	20321218	20321218
20321218	20321218	20330318	20330318
20330318	20330318	20330618	20330618
20330618	20330618	20330918	20330918
20330918	20330918	20331218	20331218
20331218	20331218	20340318	20340318
20340318	20340318	20340618	20340618
20340618	20340618	20340918	20340918
20340918	20340918	20341218	20341218
20341218	20341218	20350318	20350318
20350318	20350318	20350618	20350618
20350618	20350618	20350918	20350918
20350918	20350918	20351218	20351218
20351218	20351218	20360318	20360318
20360318	20360318	20360618	20360618
20360618	20360618	20360918	20360918
20360918	20360918	20361218	20361218
20361218	20361218	20370318	20370318
20370318	20370318	20370618	20370618
20370618	20370618	20370918	20370918
20370918	20370918	20371218	20371218
20371218	20371218	20380318	20380318
20380318	20380318	20380618	20380618
20380618	20380618	20380918	20380918
20380918	20380918	20381218	20381218
20381218	20381218	20390318	20390318
20390318	20390318	20390618	20390618
20390618	20390618	20390918	20390918
20390918	20390918	20391218	20391218
20391218	20391218	20400318	20400318
20400318	20400318	20400618	20400618
20400618	20400618	20400918	20400918
20400918	20400918	20401218	20401218
];


% callable schedule

callableInfo = struct;

% callPayment is the amount that should be paid at callPayDate

% callEventDate callPayDate callAmount(of notional)
callableInfo.callableScheduleInt = [20211218	20211218	0
20220318	20220318	0
20220618	20220618	0
20220918	20220918	0
20221218	20221218	0
20230318	20230318	0
20230618	20230618	0
20230918	20230918	0
20231218	20231218	0
20240318	20240318	0
20240618	20240618	0
20240918	20240918	0
20241218	20241218	0
20250318	20250318	0
20250618	20250618	0
20250918	20250918	0
20251218	20251218	0
20260318	20260318	0
20260618	20260618	0
20260918	20260918	0
20261218	20261218	0
20270318	20270318	0
20270618	20270618	0
20270918	20270918	0
20271218	20271218	0
20280318	20280318	0
20280618	20280618	0
20280918	20280918	0
20281218	20281218	0
20290318	20290318	0
20290618	20290618	0
20290918	20290918	0
20291218	20291218	0
20300318	20300318	0
20300618	20300618	0
20300918	20300918	0
20301218	20301218	0
20310318	20310318	0
20310618	20310618	0
20310918	20310918	0
20311218	20311218	0
20320318	20320318	0
20320618	20320618	0
20320918	20320918	0
20321218	20321218	0
20330318	20330318	0
20330618	20330618	0
20330918	20330918	0
20331218	20331218	0
20340318	20340318	0
20340618	20340618	0
20340918	20340918	0
20341218	20341218	0
20350318	20350318	0
20350618	20350618	0
20350918	20350918	0
20351218	20351218	0
20360318	20360318	0
20360618	20360618	0
20360918	20360918	0
20361218	20361218	0
20370318	20370318	0
20370618	20370618	0
20370918	20370918	0
20371218	20371218	0
20380318	20380318	0
20380618	20380618	0
20380918	20380918	0
20381218	20381218	0
20390318	20390318	0
20390618	20390618	0
20390918	20390918	0
20391218	20391218	0
20400318	20400318	0
20400618	20400618	0
20400918	20400918	0
];

% if isExercised then callable option has been exercised
% we will value it as accordingly

% callableInfo.isExercised = true;
callableInfo.isExercised = false;

steepenerInfo = struct('nominal',nominal,'basePrice',basePrice,'structuredCouponInfo',structuredCouponInfo, ...
                        'callableInfo',callableInfo);

%% swapFloatingCouponInfo start
swapFloatingCouponInfo = struct;
% multipler
% spread
% tenorFloat
swapFloatingCouponInfo.multiplier = 1.0;
% swapFloatingCouponInfo.multiplier = 0.0;

swapFloatingCouponInfo.tenorFloat = 0.25;
swapFloatingCouponInfo.rateType_tenorFloat = 'LIBOR';

swapFloatingCouponInfo.spread = 0.0005;
% swapFloatingCouponInfo.spread = 0.0;

% swapFloatingCouponInfo.eventType = 'RESET';
swapFloatingCouponInfo.eventType = 'END';
%couponSchedule  reset start end pay (annual schedule)

swapFloatingCouponInfo.swapFloatingCouponScheduleInt=[20201218	20201218	20210318	20210318
20210318	20210318	20210618	20210618
20210618	20210618	20210918	20210918
20210918	20210918	20211218	20211218
20211218	20211218	20220318	20220318
20220318	20220318	20220618	20220618
20220618	20220618	20220918	20220918
20220918	20220918	20221218	20221218
20221218	20221218	20230318	20230318
20230318	20230318	20230618	20230618
20230618	20230618	20230918	20230918
20230918	20230918	20231218	20231218
20231218	20231218	20240318	20240318
20240318	20240318	20240618	20240618
20240618	20240618	20240918	20240918
20240918	20240918	20241218	20241218
20241218	20241218	20250318	20250318
20250318	20250318	20250618	20250618
20250618	20250618	20250918	20250918
20250918	20250918	20251218	20251218
20251218	20251218	20260318	20260318
20260318	20260318	20260618	20260618
20260618	20260618	20260918	20260918
20260918	20260918	20261218	20261218
20261218	20261218	20270318	20270318
20270318	20270318	20270618	20270618
20270618	20270618	20270918	20270918
20270918	20270918	20271218	20271218
20271218	20271218	20280318	20280318
20280318	20280318	20280618	20280618
20280618	20280618	20280918	20280918
20280918	20280918	20281218	20281218
20281218	20281218	20290318	20290318
20290318	20290318	20290618	20290618
20290618	20290618	20290918	20290918
20290918	20290918	20291218	20291218
20291218	20291218	20300318	20300318
20300318	20300318	20300618	20300618
20300618	20300618	20300918	20300918
20300918	20300918	20301218	20301218
20301218	20301218	20310318	20310318
20310318	20310318	20310618	20310618
20310618	20310618	20310918	20310918
20310918	20310918	20311218	20311218
20311218	20311218	20320318	20320318
20320318	20320318	20320618	20320618
20320618	20320618	20320918	20320918
20320918	20320918	20321218	20321218
20321218	20321218	20330318	20330318
20330318	20330318	20330618	20330618
20330618	20330618	20330918	20330918
20330918	20330918	20331218	20331218
20331218	20331218	20340318	20340318
20340318	20340318	20340618	20340618
20340618	20340618	20340918	20340918
20340918	20340918	20341218	20341218
20341218	20341218	20350318	20350318
20350318	20350318	20350618	20350618
20350618	20350618	20350918	20350918
20350918	20350918	20351218	20351218
20351218	20351218	20360318	20360318
20360318	20360318	20360618	20360618
20360618	20360618	20360918	20360918
20360918	20360918	20361218	20361218
20361218	20361218	20370318	20370318
20370318	20370318	20370618	20370618
20370618	20370618	20370918	20370918
20370918	20370918	20371218	20371218
20371218	20371218	20380318	20380318
20380318	20380318	20380618	20380618
20380618	20380618	20380918	20380918
20380918	20380918	20381218	20381218
20381218	20381218	20390318	20390318
20390318	20390318	20390618	20390618
20390618	20390618	20390918	20390918
20390918	20390918	20391218	20391218
20391218	20391218	20400318	20400318
20400318	20400318	20400618	20400618
20400618	20400618	20400918	20400918
20400918	20400918	20401218	20401218
];

steepenerInfo.swapFloatingCouponInfo = swapFloatingCouponInfo;
%% swapFloatingCouponInfo End

%EODFlag
% if EODflag (endOfDate) is true then we treat that event has already(past event) happened
%  and not included in the valuation
% if EODFlag is false then we should include today's event
% or we are valuing intraday.

steepenerInfo.EODFlag_Cashflow = false;
% steepenerInfo.EODFlag_Cashflow = true;


steepenerInfo.EODFlag_Exercise = true;

steepenerParams.params = steepenerInfo;  

% maxIdx = 3;

% rmseSimul = zeros(100,1);

    %% KRW Model building
        
    %% import and construct a zeroCurve
    rawData = [0.00274	0.004799968
0.25	0.006484216
0.5	0.006779794
0.75	0.007053792
1	0.007339112
1.5	0.00792028
2	0.008522218
3	0.009532352
4	0.010461786
5	0.011103165
7	0.011846731
10	0.012644148
12	0.013115807
15	0.012750885
20	0.012217231
25	0.011825582
30	0.01166089
    ];

% rawDataSim = rawData;
% 
%     if strcmp(simulationType,'PointByPointDelta')
%         if idx2==1
%             aaa = 1;
%         else
%             if mod(idx2,2)==0
%                 tempIdx = (idx2)/2;
%     %             rawDataSim(tempIdx,2) = rawData(tempIdx,2) +rawData(tempIdx,2)*0.01;
%                 rawDataSim(tempIdx,2) = rawData(tempIdx,2) + 1.0*deltaShiftAmount;
% 
%             else
%                 tempIdx = (idx2-1)/2;
%     %             rawDataSim(tempIdx,2) = rawData(tempIdx,2) - rawData(tempIdx,2)*0.01;
%                 rawDataSim(tempIdx,2) = rawData(tempIdx,2) - 1.0*deltaShiftAmount;
%             end
%         end
%     elseif strcmp(simulationType,'ParallelDelta')   
% 
%     %% forward curve simulation by parallel translation for delta
% 
%         if idx2==1
%             aaa = 1;
%         else
%             if mod(idx2,2)==0
% 
%                 for idx3= 1: size(rawData,1)
%                     rawDataSim(idx3,2) = rawData(idx3,2) + 1*deltaShiftAmount;
%                 end
% 
%             else
%                 for idx3= 1: size(rawData,1)
%                     rawDataSim(idx3,2) = rawData(idx3,2) - 1*deltaShiftAmount;
%                 end
%             end
%         end
% 
%     end
%     
%     rawData = rawDataSim;

    
%     %simulation testCase
%     for idx=1:size(rawData,1)
%         rawData(idx,2) = rawData(idx,2) + rawData(idx,1)*(- maxIdx*zcStep + (idx2-1)*zcStep)/10;
%         simulIRSZcv(idx,idx2) = rawData(idx,2);
%     end
% 
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
        rawData = [0.00274	0.004799968
0.25	0.004019196
0.5	0.004772386
0.75	0.005368417
1	0.005819943
2	0.00885494
3	0.009626873
5	0.013470451
10	0.017206717
20	0.018218138
30	0.018261571
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



% % GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0035	0.0038	0.0041	0.0044	0.0047	0.0049	0.0052	0.0053	0.0050	0.0044
% 2	0.0042	0.0043	0.0044	0.0045	0.0046	0.0047	0.0049	0.0049	0.0046	0.0040
% 3	0.0044	0.0044	0.0044	0.0044	0.0044	0.0046	0.0048	0.0049	0.0044	0.0039
% 4	0.0045	0.0045	0.0045	0.0045	0.0044	0.0046	0.0048	0.0048	0.0042	0.0039
% 5	0.0046	0.0046	0.0045	0.0045	0.0045	0.0046	0.0047	0.0047	0.0041	0.0039
% 7	0.0046	0.0046	0.0046	0.0046	0.0045	0.0046	0.0048	0.0048	0.0044	0.0043
% 10	0.0047	0.0047	0.0047	0.0047	0.0046	0.0047	0.0042	0.0043	0.0042	0.0043
% 15	0.0032	0.0031	0.0029	0.0028	0.0026	0.0031	0.0033	0.0035	0.0037	0.0040
% 20	0.0037	0.0037	0.0037	0.0037	0.0037	0.0038	0.0038	0.0039	0.0040	0.0041
%     ];

% SwaptionVol From FrontSystem 
% 20 x 20
rawDataSwaption = [0	1	2	3	4	5	7		10	12	15	20
1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026		0.00475148	0.00489589	0.00489589	0.00489589
2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983		0.00469545	0.00476574	0.00476574	0.00476574
3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097		0.00459433	0.00464883	0.00464883	0.00464883
4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359		0.00449447	0.00453408	0.00453408	0.00453408
5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589		0.0043945	0.0044123	0.0044123	0.0044123
7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624		0.00446172	0.00448601	0.00448601	0.00448601
10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166		0.00461303	0.00463415	0.00463415	0.00463415
15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
20	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022		0.00463187	0.00464424	0.00464424	0.00464424
    ];

% % 15 x 15
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
% 1	0.00313868	0.00345381	0.00376428	0.00408151	0.00440039	0.00454026	0.00475148	0.00489589	0.00489589
% 2	0.00372804	0.00387327	0.00400155	0.00414319	0.00428737	0.00444983	0.00469545	0.00476574	0.00476574
% 3	0.00390354	0.00401313	0.00409429	0.00418996	0.0042849	0.0044097	0.00459433	0.00464883	0.00464883
% 4	0.00408523	0.00411002	0.00416392	0.00421762	0.00426983	0.00436359	0.00449447	0.00453408	0.00453408
% 5	0.00418943	0.00421751	0.00423044	0.00424642	0.0042546	0.00430589	0.0043945	0.0044123	0.0044123
% 7	0.00421476	0.00425097	0.00425793	0.00428002	0.00430127	0.00436624	0.00446172	0.00448601	0.00448601
% 10	0.0042714	0.0042711	0.00431105	0.00434579	0.00438057	0.00447166	0.00461303	0.00463415	0.00463415
% 15	0.00428332	0.00429442	0.00433276	0.00437088	0.00440288	0.00450022	0.00463187	0.00464424	0.00464424
%     ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 
    aaa = 1.0;

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
   
%% create a CK1FRisky Model with initial modelParams
%     vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    vol_r.tenor = [1;2;3;4;5;7;10;15;20];
    
    vol_r.quote = ones(1,length(vol_r.tenor)) * 0.01;
    meanR_r = 0.03;
%     meanR_r = 0.0;
    
    
    joint_swap_tenor = 1;
%     targetSize = 10;

    targetSize = 20;
    
    freq = 4;
    
    
    matu = DateDiff(steepenerParams.endDate,valueDateH)/365.0;
    
    modelParams = containers.Map({'vol_r','meanR_r','matu','joint_swap_tenor','modelName','targetSize','freq'},...
                            {vol_r, meanR_r ,matu,joint_swap_tenor, 'CK1F',targetSize,freq});

    %% allFixings
    %% CMS10Y & CMS2Y
    tempDate = steepenerParams.startDate;

    %CMS10Y
    irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});

    %CMS2Y
    irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});

    %LIBOR3M
    irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.006466038});

    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
        irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
        irIndexFixing3(StrDate(tempDate,'str')) = 0.006466038;
    end

    %% fixing info -> let it be the modelParams !

    allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
    allFixings('CMS2Y') = irIndexFixing2;
    allFixings('LIBOR3M') = irIndexFixing3;

    modelParams('allFixings') = allFixings;
    
    KRWCK1FRisky = CK1FRisky(IRModelRisky(Model(KRWAllMktData, modelParams)));
    
%     tic

    out = KRWCK1FRisky.CalibrateToATMDiagSwaptionNormal(KRWBlack,'global');
    
    for idx3 = 1:length(out.vol_r.quote)
        KRWVol_r(1,idx3) = out.vol_r.quote(idx3);
        KRWRe(1,idx3) = out.re(idx3);
    end
    KRWRMSE(1) = out.rmseTotal;
    
%     toc
    %% CK1FRisky model building end
    %% KRW model building end
    
%   %% create a LGM2FModel with initial modelParams
% % rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
% tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
% dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
% dH2.tenor = dH1.tenor;
% 
% %swaption only initial parameters
% % dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
% 
% %Samsung
% % dHTest = [0.058507682	0.500210296
% % 0.06145709	0.515096302
% % 0.057841075	0.632558982
% % 0.069442883	0.632845436
% % 0.081752868	0.607322125
% % 0.087574376	0.586095839
% % 0.103284643	0.52688632
% % 0.113936628	0.552598585
% % 0.131046999	0.498409755
% % 0.153553385	0.437472215
% % 0.173381122	0.423805445
% % 0.180703372	0.380344606
% % 0.207484315	0.305927652
% % 0.218329282	0.268192317
% % 0.224885432	0.227191192
% % 0.221244544	0.231204789
% % 0.262502203	0.212209604
% % 0.264579505	0.135257044
% % 0.318821555	0.118179799
% % 0.271515922	0.107957745
% % 0.358024755	0.088834433
% % 0.33143161	0.069384916
% % 0.457059078	0.033745097
% % 0.374067262	0.006573072
% % ];
% 
% %Daewoo
% % dHTest = [0.037286763	0.525314433
% % 0.037286763	0.525314433
% % 0.054571525	0.586928199
% % 0.069289617	0.618362181
% % 0.099111958	0.605543022
% % 0.1123501	0.589459841
% % 0.139006516	0.523225356
% % 0.175987354	0.505500158
% % 0.184616726	0.463242812
% % 0.204164537	0.412151488
% % 0.263444475	0.364947643
% % 0.251615512	0.358337407
% % 0.259708458	0.315375986
% % 0.269854863	0.270026782
% % 0.337623406	0.253701114
% % 0.258270519	0.179914465
% % 0.321985902	0.191343978
% % 0.347503907	0.154371663
% % 0.36490148	0.127052474
% % 0.36706888	0.095165077
% % 0.480469769	0.05084228
% % 0.423787026	0.033262976
% % 0.531944266	0.006463726
% % 0.333342463	0.000100001
% % ];
% 
% % %Sungwoo
% % dHTest = [0.060654929	0.585843811
% % 0.065181321	0.621357434
% % 0.059213061	0.705433024
% % 0.070356009	0.697274699
% % 0.084854535	0.641491331
% % 0.078986949	0.60204605
% % 0.103979263	0.504724505
% % 0.128905871	0.522635178
% % 0.142868097	0.436948669
% % 0.167927649	0.341253642
% % 0.171414266	0.384418469
% % 0.173137707	0.332946905
% % 0.205465936	0.252922264
% % 0.190811162	0.233849244
% % 0.212134317	0.235149902
% % 0.208305697	0.261176412
% % 0.267225845	0.213982892
% % 0.244169602	0.131637885
% % 0.293167102	0.110338057
% % 0.302235653	0.107810085
% % 0.341199539	0.095697121
% % 0.353976	0.072468098
% % 0.421517994	0.033547027
% % 0.357938773	0.006568033
% % ];
% 
% %JaeYeol
% dHTest = [0.058982775	0.543001377
% 0.061234997	0.56742444
% 0.062961086	0.622386628
% 0.067490351	0.649674014
% 0.075410597	0.63934757
% 0.087663903	0.593591285
% 0.112669112	0.501377655
% 0.117220704	0.537782529
% 0.140692951	0.444873509
% 0.168027009	0.390384507
% 0.182609655	0.410559905
% 0.184911175	0.327958872
% 0.214621342	0.297359459
% 0.213082379	0.268233279
% 0.228595305	0.243594185
% 0.220204578	0.241085585
% 0.27611566	0.202850389
% 0.244581841	0.134958255
% 0.314853582	0.120816977
% 0.317684576	0.109773131
% 0.325368543	0.100229783
% 0.348723657	0.073457815
% 0.31747133	0.033017077
% 0.359049924	0.006573421
% ];
% 
% dH1.quote = dHTest(:,1);
% dH2.quote = dHTest(:,2);
% 
% 
% dH1Orig = dH1;
% dH2Orig = dH2;
% 
% % Alpha1.tenor =  [1;2;3;4;5;7;10;15];
% Alpha1.tenor =  expiry;
% Alpha2.tenor =  Alpha1.tenor;
% correl.tenor = Alpha1.tenor;
% correl.quote = zeros(length(expiry),1);
% 
% %swaption only initial parameters
% 
% %Samsung
% % AlphaTest = [0.0289581	0.006633509
% % 0.021585088	0.007003076
% % 0.019812596	0.007056061
% % 0.015014178	0.007801684
% % 0.008683651	0.008360616
% % 0.011075137	0.008248663
% % 0.008794173	0.011226664
% % 0.000329846	0.023982837
% % ];
% 
% %Daewoo
% % AlphaTest = [0.029142908	0.007983918
% % 0.016018045	0.007219568
% % 0.00912537	0.007233927
% % 0.008101129	0.007774553
% % 0.012850209	0.008916743
% % 0.011065789	0.009477842
% % 0.008930619	0.011214209
% % 0.0001	0.032096513
% % ];
% 
% % %sungwoo
% % AlphaTest = [0.032526561	0.005901286
% % 0.021888486	0.006159185
% % 0.015500653	0.006183779
% % 0.011483817	0.007702683
% % 0.008112119	0.008910715
% % 0.010323471	0.008310996
% % 0.006966815	0.012340985
% % 0.000302754	0.020086129
% % ];
% 
% %jaeweol
% AlphaTest = [0.027941111	0.006258382
% 0.021327126	0.006691243
% 0.015009253	0.006502172
% 0.012188678	0.007718057
% 0.010231971	0.009065476
% 0.006437447	0.009477921
% 0.004646654	0.012890028
% 0.004203719	0.024248438
% ];
% 
% 
% Alpha1.quote = AlphaTest(:,1);
% Alpha2.quote = AlphaTest(:,2);
% 
% smoothdH = 1.0;
% smoothAlpha = 1.0;
% smoothTermCorr =0.1;
% targetTermCorr = 0.92;
% 
% freq = 4;
% 
% modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
%                         {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% % import irModels.*;
% % import targetFunctions.*;
%                     
% KRWLGM = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));
% 
% 
% tic
% % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% out = KRWLGM.CalibrateToATMSwaptionSurfaceNormal(KRWBlack,'global',200);
% % out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');
% 
% dH1Calib = KRWLGM.dH1;
% dH2Calib = KRWLGM.dH2;
% Alpha1Calib = KRWLGM.Alpha1;
% Alpha2Calib = KRWLGM.Alpha2;
% 
% modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
%                         {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});
%                     
%                     
% %% allFixings
% %% CMS10Y & CMS2Y
% tempDate = steepenerParams.startDate;
% 
% %CMS10Y
% irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});
% 
% %CMS2Y
% irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});
% 
% %LIBOR3M
% irIndexFixing3 = containers.Map({StrDate(tempDate,'str')},{0.0063});
% 
% for idxhong=1:10000
%     tempDate = AddDate(tempDate,1,'day');
%     irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
%     irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
%     irIndexFixing3(StrDate(tempDate,'str')) = 0.0063;
% end
% 
% %% fixing info -> let it be the modelParams !
% 
% allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
% allFixings('CMS2Y') = irIndexFixing2;
% allFixings('LIBOR3M') = irIndexFixing3;
% 
% modelParamsCalib('allFixings') = allFixings;
% 
% KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
%     
% 
% toc
%     %% LGM2FRisky model building end
%     %% KRW model building end

    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    tic
    pricingOutMC1 = KRWCK1FRisky.computeInverseFloaterSwapGeneric2(valueDateH,steepenerParams,numMethodAMC);
    toc
    aaa = 1.0;

    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
    nonCallBPrice(idx2) = pricingOutMC1.nonCallB.npv;
    eventTimeStepT= pricingOutMC1.eventTimeStep;
    eventTimeStepTSize = length(eventTimeStepT);
    eventTimeStep(1:eventTimeStepTSize,idx2) = pricingOutMC1.eventTimeStep(:);
end

% if strcmp(simulationType,'ParallelDelta')
% 
%     delta_callablePrice(1,1) = (callablePrice(2) - callablePrice(3))/2.0;
%     delta_callablePrice(1,2) = (callablePrice(2) + callablePrice(3) -2*callablePrice(1));
%     
%     delta_nonCallPrice(1,1) = (nonCallPrice(2) - nonCallPrice(3))/2.0;
%     delta_nonCallPrice(1,2) = (nonCallPrice(2) + nonCallPrice(3)-2*nonCallPrice(1));
%     
%     delta_nonCallBPrice(1,1) = (nonCallBPrice(2) - nonCallBPrice(3))/2.0; 
% 
% elseif strcmp(simulationType,'PointByPointDelta')
% 
%     for idxZ =1: numberOfTenor
%         delta_callablePrice(idxZ,1) = (callablePrice(2*idxZ) - callablePrice(2*idxZ+1))/2.0;
%         delta_callablePrice(idxZ,2) = (callablePrice(2*idxZ) + callablePrice(2*idxZ+1)-2*callablePrice(1));
%         
%         delta_nonCallPrice(idxZ,1) = (nonCallPrice(2*idxZ) - nonCallPrice(2*idxZ+1))/2.0;
%         delta_nonCallPrice(idxZ,2) = (nonCallPrice(2*idxZ) + nonCallPrice(2*idxZ+1)-2*nonCallPrice(1));
%         
%         delta_nonCallBPrice(idxZ,1) = (nonCallBPrice(2*idxZ) - nonCallBPrice(2*idxZ+1))/2.0;
%     end
% end

aaa =1.0;


In [ ]:
test_HestonSLVModel_20220530_SX5E_vanilla_atmskew.m


clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 1;
priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

valueDateH = H_Date('2022-05-30');
valueDate =  StrDate(valueDateH,'str'); 

% targetMaturities = 7:7:49;

targetMaturities = 1:30;

% [81
% 109
% 200
% 291
% 382
% 564
% 753
% 935
% ];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:3
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2022-05-30');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
%     vanillaParams.startDate = valueDateH.AddDate(365,'day');
    vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.99 + 0.01*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;


    %% import and construct a zeroCurve
    rawData = [0.049315068	-0.005899975
0.126027397	-0.005766491
0.221917808	-0.004972071
0.298630137	-0.004056075
0.547945205	-0.001342706
0.797260274	0.001031455
1.046575342	0.003166984
1.545205479	0.006295056
2.063013699	0.008251063
2.561643836	0.009282908
3.55890411	0.010598925
4.556164384	0.011541199
5.553424658	0.012268496
6.550684932	0.012896201
7.567123288	0.013542971
8.564383562	0.014200644
9.561643836	0.014866947
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For KOSPI200
    rawData = 3841.62;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor KOSPI200
   rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
81	0.591573	0.503071	0.417725	0.336232	0.270457	0.211908	0.168043	0.170101	0.187082	0.207676	0.229034
109	0.562976	0.477712	0.39804	0.327322	0.268269	0.21295	0.169561	0.162384	0.174212	0.191371	0.209742
200	0.48943	0.420322	0.357842	0.305338	0.259318	0.214615	0.175982	0.157034	0.157095	0.164936	0.17618
291	0.441731	0.382138	0.331833	0.289186	0.250839	0.212886	0.179106	0.158509	0.153481	0.155438	0.162007
382	0.405318	0.353314	0.310619	0.273361	0.239282	0.205502	0.176355	0.157992	0.150039	0.150269	0.15517
564	0.377121	0.333803	0.293922	0.259843	0.22989	0.202559	0.179758	0.163916	0.155801	0.153783	0.156323
753	0.350113	0.311544	0.277642	0.247328	0.220153	0.196325	0.176888	0.163524	0.157063	0.156224	0.158833
935	0.337495	0.299169	0.265051	0.23653	0.21305	0.194026	0.178288	0.167614	0.160976	0.158949	0.160193
    ];


    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','KOSPI200VolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor KOSPI200
    rawData = [0	0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200DividendCurve'});
    dividendCurveMktData = MktData(mktData);
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For KOSPI200

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200epoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;

    %% import and construct a forward Curve For KOSPI200
    rawData = [18	3838.68645
46	3831.65174
81	3826.12109
109	3823.79008
200	3813.23232
291	3806.47266
382	3741.53513
564	3746.43949
753	3693.16727
935	3701.94908
1299	3670.68008
1663	3651.6736
2027	3643.27925
2391	3644.91866
2762	3658.49518
3126	3682.48471
3490	3712.2747
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200ForwardCurve'});

    mktData('useInterpolatedForward') = true;

    forwardCurveMktData = MktData(mktData);
    AllMktData('forwardCurve') = forwardCurveMktData;


    %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParams= containers.Map({'modelName'},{'Black'});
    KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
    maturities = KOSPI200Black.volSurface.maturities;
    strikes =    KOSPI200Black.volSurface.strikes;
    impVolSurface = KOSPI200Black.volSurface.vol;
    spot = KOSPI200Black.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));


    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;

    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {KOSPI200ImpliedVolSurface,'Dupire'});


    %% vol calibration grid setting
    modelParamsDupire('Kmin') = 0.1;
    modelParamsDupire('Kmax') = 2.5;
    modelParamsDupire('Ns') = 241;

    dK = (modelParamsDupire('Kmax') - modelParamsDupire('Kmin'))/(modelParamsDupire('Ns')-1);
    modelParamsDupire('dK') = dK;

    Ks = zeros(modelParamsDupire('Ns'),1);
    for i=1:modelParamsDupire('Ns')
        Ks(i) = modelParamsDupire('Kmin') + modelParamsDupire('dK')*(i-1);
    end

    modelParamsDupire('Ks') = Ks;

    % restricted interval for calibration target

    modelParamsDupire('numOfCutoffTenor') = length(maturities);

    modelParamsDupire('lowerCutoff') = 0.0;
    modelParamsDupire('upperCutoff') = 2.0;

    lowerCutoffVector = modelParamsDupire('lowerCutoff')*ones(length(maturities),1);
    upperCutoffVector = modelParamsDupire('upperCutoff')*ones(length(maturities),1);

    modelParamsDupire('lowerCutoffVector') = lowerCutoffVector;
    modelParamsDupire('upperCutoffVector') = upperCutoffVector;

    GFType = -1;
    %     backGFType = 14;
    backGFType = -21;

    % GFType = 4;
    % backGFType = 4;

    modelParamsDupire('GFType') = GFType;
    modelParamsDupire('backGFType') = backGFType;


    % mc time steps in days 
    % modelParamsDupire('mcOneTimeStep') = 1095;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;

    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    %pricingSchemeType = 1 for PDE oneTimesStep
    %pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
    %pricingSchemeType = 4 for one time markov chain monte carlo

    modelParamsDupire('pricingSchemeType') = 2;

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    %     modelParamsDupire('maxIter') = 1000;
    modelParamsDupire('maxIter') = 100;

    %    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    outLV = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = KOSPI200Dupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = KOSPI200Dupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
    localVolSurface.volProxy = outLV.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    %% Heston modelparams

    modelParams= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurface,'Heston'});

    % we use fixed kappa
    modelParams('kappa') = 2.0;
    modelParams('theta') = 0.1;
    modelParams('rho')   = -0.5;
    modelParams('sigma') = 0.5;
    modelParams('v0')    = 0.1;
    
    modelParams('tol') = 1e-10;
    modelParams('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParams('quadrature_nodesAndWeights') = laguerre192;


    modelParams('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParams('quadrature_NumOfWeights') = 32;

    modelParams('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

    modelParams('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
    % modelParams('modelPriceFormula') = 'GatheralR';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParams('strikeUseBound') = strikeUseBound;

    modelParams('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParams('useQuasiRandomYN') = 'NO';
    modelParams('NMC') = 1e5;

    modelParams('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParams('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParams('smoothWeightKappa') = 0.001;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];
    % 
    % 
    % modelParams('kappa') = modelParamsDummy(1);
    % modelParams('theta') = modelParamsDummy(2);
    % modelParams('rho')    = modelParamsDummy(3);
    % modelParams('sigma') = modelParamsDummy(4);
    % modelParams('v0')   = modelParamsDummy(5);


    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

%     out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'global');
    outHeston = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
    % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParams('kappa') = modelParamsDummy(1);
    modelParams('theta') = modelParamsDummy(2);
    modelParams('rho')    = modelParamsDummy(3);
    modelParams('sigma') = modelParamsDummy(4);
    modelParams('v0')   = modelParamsDummy(5);

    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

    tic
    pricingOutHeston = KOSPI200Heston.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = KOSPI200Heston.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

    modelParams('sigma') = KOSPI200Heston.sigma*mixingWeight;
    
    KOSPI200HestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParams)));


    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = KOSPI200Heston.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = KOSPI200HestonSLV.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = KOSPI200HestonSLV.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = KOSPI200Heston.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end
aaa = 1.0;
% 
% ccc = 1.0;
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalGA');
% 
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaConstrained');
% bbb = 1.0;
% 
% priceSimul = zeros(length(expiry),101);
% impVolSimul = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% % price simulation
% bbb =  1.0;
% spot =  KOSPI200HestonSLV.spot.params('rawData');
% 
% for j=1:length(expiry)
%     fwd = KOSPI200HestonSLV.Fwd(spot,expiry(j));
%     df = KOSPI200HestonSLV.zeroCurve.DF(expiry(j)/365.0);
%     for k=1:length(strikes)
%         tic
%         relStrike = strikes(k);
%         if relStrike <= 1.0
%             vanillaParams.params('callPutFlag') = 'P';
%             callputStringhsw = 'Put';
%         else
%             vanillaParams.params('callPutFlag') = 'C';
%             callputStringhsw = 'Call';
%         end
%         
%         priceSimul(j,k)  = KOSPI200Heston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
%         impVolSimul(j,k) = KOSPI200Heston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         KOSPI200HestonSLV.modelParams('MCScheme') = 'QE';
%         KOSPI200Heston.modelParams('MCScheme') = 'QE';
% 
%         vanillaParams.params('strike') = fwd*relStrike;
%         vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
%         
%         pricingOutMCHestonSLV = KOSPI200HestonSLV.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
%         impVolSimulQESLV(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         pricingOutMCHeston = KOSPI200Heston.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
%         impVolSimulQE(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         toc
%     end
% end


fff = 1.0;

In [ ]:

test_HestonSLVModel_20220530_SX5E_forwardStarting_vanilla2_1Y_atmskew.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 1;
priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

valueDateH = H_Date('2022-05-30');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = 7:7:49;
% [81
% 109
% 200
% 291
% 382
% 564
% 753
% 935
% ];
%% Pricing Forward Starting Vanilla 1S
for idxhh=1: length(targetMaturities)
    for idxSim = 1:3
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2022-05-30');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.99 + 0.01*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;


    %% import and construct a zeroCurve
    rawData = [0.049315068	-0.005899975
0.126027397	-0.005766491
0.221917808	-0.004972071
0.298630137	-0.004056075
0.547945205	-0.001342706
0.797260274	0.001031455
1.046575342	0.003166984
1.545205479	0.006295056
2.063013699	0.008251063
2.561643836	0.009282908
3.55890411	0.010598925
4.556164384	0.011541199
5.553424658	0.012268496
6.550684932	0.012896201
7.567123288	0.013542971
8.564383562	0.014200644
9.561643836	0.014866947
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For KOSPI200
    rawData = 3841.62;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor KOSPI200
   rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
81	0.591573	0.503071	0.417725	0.336232	0.270457	0.211908	0.168043	0.170101	0.187082	0.207676	0.229034
109	0.562976	0.477712	0.39804	0.327322	0.268269	0.21295	0.169561	0.162384	0.174212	0.191371	0.209742
200	0.48943	0.420322	0.357842	0.305338	0.259318	0.214615	0.175982	0.157034	0.157095	0.164936	0.17618
291	0.441731	0.382138	0.331833	0.289186	0.250839	0.212886	0.179106	0.158509	0.153481	0.155438	0.162007
382	0.405318	0.353314	0.310619	0.273361	0.239282	0.205502	0.176355	0.157992	0.150039	0.150269	0.15517
564	0.377121	0.333803	0.293922	0.259843	0.22989	0.202559	0.179758	0.163916	0.155801	0.153783	0.156323
753	0.350113	0.311544	0.277642	0.247328	0.220153	0.196325	0.176888	0.163524	0.157063	0.156224	0.158833
935	0.337495	0.299169	0.265051	0.23653	0.21305	0.194026	0.178288	0.167614	0.160976	0.158949	0.160193
    ];


    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','KOSPI200VolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor KOSPI200
    rawData = [0	0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200DividendCurve'});
    dividendCurveMktData = MktData(mktData);
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For KOSPI200

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200epoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;

    %% import and construct a forward Curve For KOSPI200
    rawData = [18	3838.68645
46	3831.65174
81	3826.12109
109	3823.79008
200	3813.23232
291	3806.47266
382	3741.53513
564	3746.43949
753	3693.16727
935	3701.94908
1299	3670.68008
1663	3651.6736
2027	3643.27925
2391	3644.91866
2762	3658.49518
3126	3682.48471
3490	3712.2747
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200ForwardCurve'});

    mktData('useInterpolatedForward') = true;

    forwardCurveMktData = MktData(mktData);
    AllMktData('forwardCurve') = forwardCurveMktData;


    %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParams= containers.Map({'modelName'},{'Black'});
    KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
    maturities = KOSPI200Black.volSurface.maturities;
    strikes =    KOSPI200Black.volSurface.strikes;
    impVolSurface = KOSPI200Black.volSurface.vol;
    spot = KOSPI200Black.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));


    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;

    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {KOSPI200ImpliedVolSurface,'Dupire'});


    %% vol calibration grid setting
    modelParamsDupire('Kmin') = 0.1;
    modelParamsDupire('Kmax') = 2.5;
    modelParamsDupire('Ns') = 241;

    dK = (modelParamsDupire('Kmax') - modelParamsDupire('Kmin'))/(modelParamsDupire('Ns')-1);
    modelParamsDupire('dK') = dK;

    Ks = zeros(modelParamsDupire('Ns'),1);
    for i=1:modelParamsDupire('Ns')
        Ks(i) = modelParamsDupire('Kmin') + modelParamsDupire('dK')*(i-1);
    end

    modelParamsDupire('Ks') = Ks;

    % restricted interval for calibration target

    modelParamsDupire('numOfCutoffTenor') = length(maturities);

    modelParamsDupire('lowerCutoff') = 0.0;
    modelParamsDupire('upperCutoff') = 2.0;

    lowerCutoffVector = modelParamsDupire('lowerCutoff')*ones(length(maturities),1);
    upperCutoffVector = modelParamsDupire('upperCutoff')*ones(length(maturities),1);

    modelParamsDupire('lowerCutoffVector') = lowerCutoffVector;
    modelParamsDupire('upperCutoffVector') = upperCutoffVector;

    GFType = -1;
    %     backGFType = 14;
    backGFType = -21;

    % GFType = 4;
    % backGFType = 4;

    modelParamsDupire('GFType') = GFType;
    modelParamsDupire('backGFType') = backGFType;


    % mc time steps in days 
    % modelParamsDupire('mcOneTimeStep') = 1095;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;

    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    %pricingSchemeType = 1 for PDE oneTimesStep
    %pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
    %pricingSchemeType = 4 for one time markov chain monte carlo

    modelParamsDupire('pricingSchemeType') = 2;

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    %     modelParamsDupire('maxIter') = 1000;
    modelParamsDupire('maxIter') = 100;

    %    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    outLV = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = KOSPI200Dupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = KOSPI200Dupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
    localVolSurface.volProxy = outLV.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    %% Heston modelparams

    modelParams= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurface,'Heston'});

    % we use fixed kappa
    modelParams('kappa') = 2.0;
    modelParams('theta') = 0.1;
    modelParams('rho')   = -0.5;
    modelParams('sigma') = 0.5;
    modelParams('v0')    = 0.1;
    
    modelParams('tol') = 1e-10;
    modelParams('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParams('quadrature_nodesAndWeights') = laguerre192;


    modelParams('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParams('quadrature_NumOfWeights') = 32;

    modelParams('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

    modelParams('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
    % modelParams('modelPriceFormula') = 'GatheralR';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParams('strikeUseBound') = strikeUseBound;

    modelParams('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParams('useQuasiRandomYN') = 'NO';
    modelParams('NMC') = 1e5;

    modelParams('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParams('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParams('smoothWeightKappa') = 0.001;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];
    % 
    % 
    % modelParams('kappa') = modelParamsDummy(1);
    % modelParams('theta') = modelParamsDummy(2);
    % modelParams('rho')    = modelParamsDummy(3);
    % modelParams('sigma') = modelParamsDummy(4);
    % modelParams('v0')   = modelParamsDummy(5);


    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

%     out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'global');
    outHeston = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
    % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParams('kappa') = modelParamsDummy(1);
    modelParams('theta') = modelParamsDummy(2);
    modelParams('rho')    = modelParamsDummy(3);
    modelParams('sigma') = modelParamsDummy(4);
    modelParams('v0')   = modelParamsDummy(5);

    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

    tic
    pricingOutHeston = KOSPI200Heston.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = KOSPI200Heston.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

    modelParams('sigma') = KOSPI200Heston.sigma*mixingWeight;
    
    KOSPI200HestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParams)));


    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = KOSPI200Heston.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = KOSPI200HestonSLV.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = KOSPI200HestonSLV.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = KOSPI200Heston.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end
aaa = 1.0;
% 
% ccc = 1.0;
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalGA');
% 
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaConstrained');
% bbb = 1.0;
% 
% priceSimul = zeros(length(expiry),101);
% impVolSimul = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% % price simulation
% bbb =  1.0;
% spot =  KOSPI200HestonSLV.spot.params('rawData');
% 
% for j=1:length(expiry)
%     fwd = KOSPI200HestonSLV.Fwd(spot,expiry(j));
%     df = KOSPI200HestonSLV.zeroCurve.DF(expiry(j)/365.0);
%     for k=1:length(strikes)
%         tic
%         relStrike = strikes(k);
%         if relStrike <= 1.0
%             vanillaParams.params('callPutFlag') = 'P';
%             callputStringhsw = 'Put';
%         else
%             vanillaParams.params('callPutFlag') = 'C';
%             callputStringhsw = 'Call';
%         end
%         
%         priceSimul(j,k)  = KOSPI200Heston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
%         impVolSimul(j,k) = KOSPI200Heston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         KOSPI200HestonSLV.modelParams('MCScheme') = 'QE';
%         KOSPI200Heston.modelParams('MCScheme') = 'QE';
% 
%         vanillaParams.params('strike') = fwd*relStrike;
%         vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
%         
%         pricingOutMCHestonSLV = KOSPI200HestonSLV.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
%         impVolSimulQESLV(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         pricingOutMCHeston = KOSPI200Heston.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
%         impVolSimulQE(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         toc
%     end
% end


fff = 1.0;

In [ ]:
test_HestonSLVModel_20220530_SX5E_forwardStarting_vanilla2_1Y.m


clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 1;
priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

valueDateH = H_Date('2022-05-30');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [81
109
200
291
382
564
753
935
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=5: 5
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2022-05-30');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;


    %% import and construct a zeroCurve
    rawData = [0.049315068	-0.005899975
0.126027397	-0.005766491
0.221917808	-0.004972071
0.298630137	-0.004056075
0.547945205	-0.001342706
0.797260274	0.001031455
1.046575342	0.003166984
1.545205479	0.006295056
2.063013699	0.008251063
2.561643836	0.009282908
3.55890411	0.010598925
4.556164384	0.011541199
5.553424658	0.012268496
6.550684932	0.012896201
7.567123288	0.013542971
8.564383562	0.014200644
9.561643836	0.014866947
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For KOSPI200
    rawData = 3841.62;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor KOSPI200
   rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
81	0.591573	0.503071	0.417725	0.336232	0.270457	0.211908	0.168043	0.170101	0.187082	0.207676	0.229034
109	0.562976	0.477712	0.39804	0.327322	0.268269	0.21295	0.169561	0.162384	0.174212	0.191371	0.209742
200	0.48943	0.420322	0.357842	0.305338	0.259318	0.214615	0.175982	0.157034	0.157095	0.164936	0.17618
291	0.441731	0.382138	0.331833	0.289186	0.250839	0.212886	0.179106	0.158509	0.153481	0.155438	0.162007
382	0.405318	0.353314	0.310619	0.273361	0.239282	0.205502	0.176355	0.157992	0.150039	0.150269	0.15517
564	0.377121	0.333803	0.293922	0.259843	0.22989	0.202559	0.179758	0.163916	0.155801	0.153783	0.156323
753	0.350113	0.311544	0.277642	0.247328	0.220153	0.196325	0.176888	0.163524	0.157063	0.156224	0.158833
935	0.337495	0.299169	0.265051	0.23653	0.21305	0.194026	0.178288	0.167614	0.160976	0.158949	0.160193
    ];


    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','KOSPI200VolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor KOSPI200
    rawData = [0	0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200DividendCurve'});
    dividendCurveMktData = MktData(mktData);
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For KOSPI200

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200epoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;

    %% import and construct a forward Curve For KOSPI200
    rawData = [18	3838.68645
46	3831.65174
81	3826.12109
109	3823.79008
200	3813.23232
291	3806.47266
382	3741.53513
564	3746.43949
753	3693.16727
935	3701.94908
1299	3670.68008
1663	3651.6736
2027	3643.27925
2391	3644.91866
2762	3658.49518
3126	3682.48471
3490	3712.2747
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200ForwardCurve'});

    mktData('useInterpolatedForward') = true;

    forwardCurveMktData = MktData(mktData);
    AllMktData('forwardCurve') = forwardCurveMktData;


    %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParams= containers.Map({'modelName'},{'Black'});
    KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
    maturities = KOSPI200Black.volSurface.maturities;
    strikes =    KOSPI200Black.volSurface.strikes;
    impVolSurface = KOSPI200Black.volSurface.vol;
    spot = KOSPI200Black.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));


    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;

    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {KOSPI200ImpliedVolSurface,'Dupire'});


    %% vol calibration grid setting
    modelParamsDupire('Kmin') = 0.1;
    modelParamsDupire('Kmax') = 2.5;
    modelParamsDupire('Ns') = 241;

    dK = (modelParamsDupire('Kmax') - modelParamsDupire('Kmin'))/(modelParamsDupire('Ns')-1);
    modelParamsDupire('dK') = dK;

    Ks = zeros(modelParamsDupire('Ns'),1);
    for i=1:modelParamsDupire('Ns')
        Ks(i) = modelParamsDupire('Kmin') + modelParamsDupire('dK')*(i-1);
    end

    modelParamsDupire('Ks') = Ks;

    % restricted interval for calibration target

    modelParamsDupire('numOfCutoffTenor') = length(maturities);

    modelParamsDupire('lowerCutoff') = 0.0;
    modelParamsDupire('upperCutoff') = 2.0;

    lowerCutoffVector = modelParamsDupire('lowerCutoff')*ones(length(maturities),1);
    upperCutoffVector = modelParamsDupire('upperCutoff')*ones(length(maturities),1);

    modelParamsDupire('lowerCutoffVector') = lowerCutoffVector;
    modelParamsDupire('upperCutoffVector') = upperCutoffVector;

    GFType = -1;
    %     backGFType = 14;
    backGFType = -21;

    % GFType = 4;
    % backGFType = 4;

    modelParamsDupire('GFType') = GFType;
    modelParamsDupire('backGFType') = backGFType;


    % mc time steps in days 
    % modelParamsDupire('mcOneTimeStep') = 1095;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;

    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    %pricingSchemeType = 1 for PDE oneTimesStep
    %pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
    %pricingSchemeType = 4 for one time markov chain monte carlo

    modelParamsDupire('pricingSchemeType') = 2;

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    %     modelParamsDupire('maxIter') = 1000;
    modelParamsDupire('maxIter') = 100;

    %    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    outLV = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = KOSPI200Dupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = KOSPI200Dupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

       impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
    localVolSurface.volProxy = outLV.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    %% Heston modelparams

    modelParams= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurface,'Heston'});

    % we use fixed kappa
    modelParams('kappa') = 2.0;
    modelParams('theta') = 0.1;
    modelParams('rho')   = -0.5;
    modelParams('sigma') = 0.5;
    modelParams('v0')    = 0.1;
    
    modelParams('tol') = 1e-10;
    modelParams('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParams('quadrature_nodesAndWeights') = laguerre192;


    modelParams('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParams('quadrature_NumOfWeights') = 32;

    modelParams('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

    modelParams('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
    % modelParams('modelPriceFormula') = 'GatheralR';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParams('strikeUseBound') = strikeUseBound;

    modelParams('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParams('useQuasiRandomYN') = 'NO';
    modelParams('NMC') = 1e5;

    modelParams('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParams('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParams('smoothWeightKappa') = 0.001;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];
    % 
    % 
    % modelParams('kappa') = modelParamsDummy(1);
    % modelParams('theta') = modelParamsDummy(2);
    % modelParams('rho')    = modelParamsDummy(3);
    % modelParams('sigma') = modelParamsDummy(4);
    % modelParams('v0')   = modelParamsDummy(5);


    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

%     out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'global');
    outHeston = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
    % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParams('kappa') = modelParamsDummy(1);
    modelParams('theta') = modelParamsDummy(2);
    modelParams('rho')    = modelParamsDummy(3);
    modelParams('sigma') = modelParamsDummy(4);
    modelParams('v0')   = modelParamsDummy(5);

    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

    tic
    pricingOutHeston = KOSPI200Heston.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = KOSPI200Heston.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

    modelParams('sigma') = KOSPI200Heston.sigma*mixingWeight;
    
    KOSPI200HestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParams)));


    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = KOSPI200Heston.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = KOSPI200HestonSLV.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = KOSPI200HestonSLV.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = KOSPI200Heston.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end
aaa = 1.0;
% 
% ccc = 1.0;
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalGA');
% 
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaConstrained');
% bbb = 1.0;
% 
% priceSimul = zeros(length(expiry),101);
% impVolSimul = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% % price simulation
% bbb =  1.0;
% spot =  KOSPI200HestonSLV.spot.params('rawData');
% 
% for j=1:length(expiry)
%     fwd = KOSPI200HestonSLV.Fwd(spot,expiry(j));
%     df = KOSPI200HestonSLV.zeroCurve.DF(expiry(j)/365.0);
%     for k=1:length(strikes)
%         tic
%         relStrike = strikes(k);
%         if relStrike <= 1.0
%             vanillaParams.params('callPutFlag') = 'P';
%             callputStringhsw = 'Put';
%         else
%             vanillaParams.params('callPutFlag') = 'C';
%             callputStringhsw = 'Call';
%         end
%         
%         priceSimul(j,k)  = KOSPI200Heston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
%         impVolSimul(j,k) = KOSPI200Heston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         KOSPI200HestonSLV.modelParams('MCScheme') = 'QE';
%         KOSPI200Heston.modelParams('MCScheme') = 'QE';
% 
%         vanillaParams.params('strike') = fwd*relStrike;
%         vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
%         
%         pricingOutMCHestonSLV = KOSPI200HestonSLV.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
%         impVolSimulQESLV(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         pricingOutMCHeston = KOSPI200Heston.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
%         impVolSimulQE(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         toc
%     end
% end


fff = 1.0;

In [ ]:
test_HestonSLVModel_20220530_SX5E_forwardStarting_vanilla2.m

clc
clear

mixingWeight = 0.5;
mcOneTimeStepD = 1;
priceSimulMC1 = zeros(200,1);

priceSimulMC2 = zeros(200,1);
priceSimulClosed = zeros(200,1);

hestonParamsSimul= zeros(200,100);

localVolVolMSESimul= zeros(200,1);
hestonVolMSESimul = zeros(200,1);

levFunctionSimul = zeros(200,100);

priceSimulMCLV = zeros(200,1);
priceSimulMCHeston= zeros(200,1);
priceSimulMCHestonSLV = zeros(200,1);


impVolLVSim = zeros(200,100);
impVolHestonSim = zeros(200,100);
impVolHestonSLVSim = zeros(200,100);

valueDateH = H_Date('2022-05-30');
valueDate =  StrDate(valueDateH,'str'); 

targetMaturities = [81
109
200
291
382
564
753
935
];
%% Pricing Forward Starting Vanilla 1S
for idxhh=5: 5
    for idxSim = 1:11
    %vanilla info
    vanillaParams.payCcy = 'KRW';
    valueDateH = H_Date('2022-05-30');
    valueDate =  StrDate(valueDateH,'str'); 
    %forward starting date
    vanillaParams.startDate = valueDateH.AddDate(365,'day');
%     vanillaParams.startDate = valueDateH.AddDate(0,'day');

    vanillaParams.endDate = vanillaParams.startDate.AddDate(targetMaturities(idxhh),'day');

    % relative strike of forward 1Y spot

    relativeStrike = 0.5 + 0.1*(idxSim-1);

    callPutFlag = 'P';

    if relativeStrike <= 1.0
        callPutFlag = 'P';
        callputStringhsw = 'Put';
    else
        callPutFlag = 'C';
        callputStringhsw = 'Call';
    end

    EuropeanAmericanFlag = 'E';
    SSpayoffYN = 'YES';

    digitalYN = 'NO';
    digitalCoupon = 0.05;


    dailyTimeStepsYN = 'NO';
    vanillaInfo = containers.Map({'relativeStrike','startDate','maturityDate','callPutFlag','EuropeanAmericanFlag','dailyTimeStepsYN'}, ...
                            {relativeStrike,vanillaParams.startDate,vanillaParams.endDate,callPutFlag,EuropeanAmericanFlag,dailyTimeStepsYN});

    vanillaInfo('digitalYN') = digitalYN;
    vanillaInfo('digitalCoupon') = digitalCoupon;

    vanillaParams.params = vanillaInfo;


    %% import and construct a zeroCurve
    rawData = [0.049315068	-0.005899975
0.126027397	-0.005766491
0.221917808	-0.004972071
0.298630137	-0.004056075
0.547945205	-0.001342706
0.797260274	0.001031455
1.046575342	0.003166984
1.545205479	0.006295056
2.063013699	0.008251063
2.561643836	0.009282908
3.55890411	0.010598925
4.556164384	0.011541199
5.553424658	0.012268496
6.550684932	0.012896201
7.567123288	0.013542971
8.564383562	0.014200644
9.561643836	0.014866947
    ];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);
    AllMktData =containers.Map('zeroCurve',zeroCurve);
    %% import and construct a spot For KOSPI200
    rawData = 3841.62;

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KRWSpot'});
    spotMktData = MktData(mktData);
    AllMktData('spot') = spotMktData;
    %% import and construct an impliedVolSurfaceFor KOSPI200
   rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
81	0.591573	0.503071	0.417725	0.336232	0.270457	0.211908	0.168043	0.170101	0.187082	0.207676	0.229034
109	0.562976	0.477712	0.39804	0.327322	0.268269	0.21295	0.169561	0.162384	0.174212	0.191371	0.209742
200	0.48943	0.420322	0.357842	0.305338	0.259318	0.214615	0.175982	0.157034	0.157095	0.164936	0.17618
291	0.441731	0.382138	0.331833	0.289186	0.250839	0.212886	0.179106	0.158509	0.153481	0.155438	0.162007
382	0.405318	0.353314	0.310619	0.273361	0.239282	0.205502	0.176355	0.157992	0.150039	0.150269	0.15517
564	0.377121	0.333803	0.293922	0.259843	0.22989	0.202559	0.179758	0.163916	0.155801	0.153783	0.156323
753	0.350113	0.311544	0.277642	0.247328	0.220153	0.196325	0.176888	0.163524	0.157063	0.156224	0.158833
935	0.337495	0.299169	0.265051	0.23653	0.21305	0.194026	0.178288	0.167614	0.160976	0.158949	0.160193
    ];


    mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                            {rawData,'20190228','fwdMoneyNess','KRW','KOSPI200VolSurface'});
    impliedVolSurfaceMktData = MktData(mktData);
    impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);
%     impliedVolSurface.params('moneyNessType') = 'fwdMoneyNess';
    impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

    AllMktData('impliedVolSurface') = impliedVolSurface;

    %% import and construct a dividend CurveFor KOSPI200
    rawData = [0	0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200DividendCurve'});
    dividendCurveMktData = MktData(mktData);
    AllMktData('dividendCurve') = dividendCurveMktData;

    %% import and construct a repoRate Curve For KOSPI200

    rawData = [0    0.0
    ];
    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200epoRateCurve'});
    repoRateCurveMktData = MktData(mktData);
    AllMktData('repoRateCurve') = repoRateCurveMktData;

    %% import and construct a forward Curve For KOSPI200
    rawData = [18	3838.68645
46	3831.65174
81	3826.12109
109	3823.79008
200	3813.23232
291	3806.47266
382	3741.53513
564	3746.43949
753	3693.16727
935	3701.94908
1299	3670.68008
1663	3651.6736
2027	3643.27925
2391	3644.91866
2762	3658.49518
3126	3682.48471
3490	3712.2747
];


    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,'20190228','KRW','KOSPI200ForwardCurve'});

    mktData('useInterpolatedForward') = true;

    forwardCurveMktData = MktData(mktData);
    AllMktData('forwardCurve') = forwardCurveMktData;


    %% create  an EQBlackModel to calculate market PlainVanilla Options
    modelParams= containers.Map({'modelName'},{'Black'});
    KOSPI200Black = EQBlack(EQModel(Model(AllMktData,modelParams))); 
    maturities = KOSPI200Black.volSurface.maturities;
    strikes =    KOSPI200Black.volSurface.strikes;
    impVolSurface = KOSPI200Black.volSurface.vol;
    spot = KOSPI200Black.spot.params('rawData');
    expiry = maturities;
    fwdFactor = zeros(length(maturities),1);
    fwdValues = zeros(length(maturities),1);
    dfValues = zeros(length(maturities),1);
    fwdMoneyness = zeros(length(maturities),length(strikes));
    blackVolInput = zeros(length(maturities),length(strikes));


    %% create a DupireModel with initial modelParamsDupire
    %Initialize local volatility
    KOSPI200ImpliedVolSurface = KOSPI200Black.volSurface;

    modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                            {KOSPI200ImpliedVolSurface,'Dupire'});


    %% vol calibration grid setting
    modelParamsDupire('Kmin') = 0.1;
    modelParamsDupire('Kmax') = 2.5;
    modelParamsDupire('Ns') = 241;

    dK = (modelParamsDupire('Kmax') - modelParamsDupire('Kmin'))/(modelParamsDupire('Ns')-1);
    modelParamsDupire('dK') = dK;

    Ks = zeros(modelParamsDupire('Ns'),1);
    for i=1:modelParamsDupire('Ns')
        Ks(i) = modelParamsDupire('Kmin') + modelParamsDupire('dK')*(i-1);
    end

    modelParamsDupire('Ks') = Ks;

    % restricted interval for calibration target

    modelParamsDupire('numOfCutoffTenor') = length(maturities);

    modelParamsDupire('lowerCutoff') = 0.0;
    modelParamsDupire('upperCutoff') = 2.0;

    lowerCutoffVector = modelParamsDupire('lowerCutoff')*ones(length(maturities),1);
    upperCutoffVector = modelParamsDupire('upperCutoff')*ones(length(maturities),1);

    modelParamsDupire('lowerCutoffVector') = lowerCutoffVector;
    modelParamsDupire('upperCutoffVector') = upperCutoffVector;

    GFType = -1;
    %     backGFType = 14;
    backGFType = -21;

    % GFType = 4;
    % backGFType = 4;

    modelParamsDupire('GFType') = GFType;
    modelParamsDupire('backGFType') = backGFType;


    % mc time steps in days 
    % modelParamsDupire('mcOneTimeStep') = 1095;
    modelParamsDupire('mcOneTimeStep') = mcOneTimeStepD;

    modelParamsDupire('NMC') = 1e5;

    modelParamsDupire('useQuasiRandomYN') = 'false';

    %pricingSchemeType = 1 for PDE oneTimesStep
    %pricingSchemeType = 3 for mc euler scheme with mcOneTimesStep
    %pricingSchemeType = 4 for one time markov chain monte carlo

    modelParamsDupire('pricingSchemeType') = 2;

    % optimizer setting

    modelParamsDupire('tol') = 0.0001;
    %     modelParamsDupire('maxIter') = 1000;
    modelParamsDupire('maxIter') = 100;

    %    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('UseLetsBeRational') = 'YES';
    modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

    KOSPI200Dupire = EQCOMDupireSpotGF(EQModel(Model(AllMktData,modelParamsDupire)));

    tic
    outLV = KOSPI200Dupire.CalibrateToVolSurface(KOSPI200Black,'bootstrapForwardOnly');
    toc

    tic
    pricingOutLV = KOSPI200Dupire.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolLV = KOSPI200Dupire.ImpliedBlackVol(pricingOutLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

       impVolLVSim(idxhh,idxSim) = impVolLV;

    aaa = 1.0;


    localVolSurface.expiry = maturities;
    localVolSurface.strike = strikes;
    localVolSurface.volProxy = outLV.volProxy;
    localVolSurface.strikeType = 'fwdMoneyness';

    %% Heston modelparams

    modelParams= containers.Map({'localVolSurface','modelName'}, ...
                            {localVolSurface,'Heston'});

    % we use fixed kappa
    modelParams('kappa') = 2.0;
    modelParams('theta') = 0.1;
    modelParams('rho')   = -0.5;
    modelParams('sigma') = 0.5;
    modelParams('v0')    = 0.1;
    
    modelParams('tol') = 1e-10;
    modelParams('maxIter') = 1000;

    load('laguerre64.mat','laguerre64');
    load('laguerre100.mat','laguerre100');
    load('laguerre90.mat','laguerre90');

    load('laguerre128.mat','laguerre128');
    load('laguerre144.mat','laguerre144');

    load('laguerre192.mat','laguerre192');

    % modelParams('quadrature_nodesAndWeights') = laguerre64;
    % modelParams('quadrature_nodesAndWeights') = laguerre90;
    % modelParams('quadrature_nodesAndWeights') = laguerre100;
    % modelParams('quadrature_nodesAndWeights') = laguerre128;
    % modelParams('quadrature_nodesAndWeights') = laguerre144;
    modelParams('quadrature_nodesAndWeights') = laguerre192;


    modelParams('quadrature_generateNodesYN') = 'false';
    % modelParams('quadrature_generateNodesYN') = 'true';

    modelParams('quadrature_NumOfWeights') = 32;

    modelParams('calibrationErrorType') = 'impVolError';
    % modelParams('calibrationErrorType') = 'relativePriceError';
    % modelParams('calibrationErrorType') = 'relativePriceErrorSS';

    % modelParams('calibrationErrorType') = 'priceError';

    modelParams('modelPriceFormula') = 'Quantlib';
    % modelParams('modelPriceFormula') = 'Rouah';
    % modelParams('modelPriceFormula') = 'GatheralR';

    % strikeUseBound = [30	0.8	1.2
    % 91	0.8	1.2
    % 182	0.8	1.2
    % 365	0.5	1.5
    % 730	0.5	1.5
    % 1095	0.5	1.5
    % 1460	0.5	1.5
    % 1825	0.5	1.5
    % ];

    strikeUseBound = [91	0.8	1.2
    182	0.8	1.2
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0.5	1.5
    182	0.5	1.5
    365	0.5	1.5
    730	0.5	1.5
    1095	0.5	1.5
    1460	0.5	1.5
    1825	0.5	1.5
    ];

    strikeUseBound = [91	0	2
    182	0	2
    365	0	2
    730	0	2
    1095	0	2
    1460	0	2
    1825	0	2
    ];

    modelParams('strikeUseBound') = strikeUseBound;

    modelParams('mcOneTimeStep') = mcOneTimeStepD;
    % modelParams('mcOneTimeStep') = 1;

    modelParams('useQuasiRandomYN') = 'NO';
    modelParams('NMC') = 1e5;

    modelParams('MCScheme') = 'QE';
    % modelParams('MCScheme') = 'NonCentralChiSquare';

    modelParams('initialKappa') = 2.0;
    % modelParams('smoothWeightKappa') = 0.01;
    modelParams('smoothWeightKappa') = 0.001;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];
    % 
    % 
    % modelParams('kappa') = modelParamsDummy(1);
    % modelParams('theta') = modelParamsDummy(2);
    % modelParams('rho')    = modelParamsDummy(3);
    % modelParams('sigma') = modelParamsDummy(4);
    % modelParams('v0')   = modelParamsDummy(5);


    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

%     out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'global');
    outHeston = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
    % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'JustCalculateError');

    modelParamsDummy = outHeston.paramsOut;

    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.414265212
    % 0.022341764
    %     ];

    % very small vol of vol
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 1.2
    % 0.022341764
    %     ];
    % 
    % modelParamsDummy = [2.126092506
    % 0.046531369
    % -0.353662845
    % 0.01
    % 0.022341764
    %     ];

    modelParams('kappa') = modelParamsDummy(1);
    modelParams('theta') = modelParamsDummy(2);
    modelParams('rho')    = modelParamsDummy(3);
    modelParams('sigma') = modelParamsDummy(4);
    modelParams('v0')   = modelParamsDummy(5);

    KOSPI200Heston = EQHeston(EQModel(Model(AllMktData,modelParams)));

    tic
    pricingOutHeston = KOSPI200Heston.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHeston.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHeston = KOSPI200Heston.ImpliedBlackVol(pricingOutHeston.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSim(idxhh,idxSim) = impVolHeston;

    toc

    %% HestonSLV model init

    modelParams('sigma') = KOSPI200Heston.sigma*mixingWeight;
    
    KOSPI200HestonSLV = EQHestonSLV(EQModel(Model(AllMktData,modelParams)));


    %% Generate LeverageFunction for Stochastic Local Vol
    leverageFunctionParams.numOfBins = 100;
    % leverageFunctionParams.numOfBins = 20;
    leverageFunctionParams.numOfPath = 40000;
    leverageFunctionParams.numOfPathPerBins = leverageFunctionParams.numOfPath/leverageFunctionParams.numOfBins;
    volTimeStep = localVolSurface.expiry;
    maturity = volTimeStep(end);
    timeStepP = KOSPI200Heston.modelParams('mcOneTimeStep');
    dailyTimeSteps = [maturity:-timeStepP:0]';
    dailyTimeSteps = [dailyTimeSteps;0];
    timeStep = sort(union(timeStepP,dailyTimeSteps),'ascend');
    %add volKnotPoints
    timeStep = sort(union(timeStep,volTimeStep),'ascend');
    % we exclude 1st state which is zero
    timeStepLVF = timeStep(2:end);
    % % 1st element is timeStep = 0, therefore we ignored this
    % % and we add 1D small timeSteps for this
    % timeStep = timeStep(2:end);
    % timeStep = sort(union(timeStep,[1]),'ascend');
    leverageFunctionParams.timeStepLVF = timeStepLVF;
    expiryDate = valueDateH.AddDate(expiry(end),'day');
    leverageFunctionParams.expiryDate = expiryDate;

    outDD = KOSPI200HestonSLV.GenerateLeverageFunction(valueDateH,leverageFunctionParams);

    tic
    pricingOutHestonSLV = KOSPI200HestonSLV.computeForwardStartingVanillaMC(valueDateH,vanillaParams);
    toc

    fwdRatio = pricingOutHestonSLV.fwdRatio;

    fwdOptionMaturity = DateDiff(vanillaParams.endDate,vanillaParams.startDate);

    impVolHestonSLV = KOSPI200Heston.ImpliedBlackVol(pricingOutHestonSLV.europeanValue.npv,fwdRatio,fwdRatio*relativeStrike,fwdOptionMaturity/365.0,callputStringhsw);

    impVolHestonSLVSim(idxhh,idxSim) = impVolHestonSLV;

    toc

    end
end
aaa = 1.0;
% 
% ccc = 1.0;
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalGA');
% 
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaFixed');
% % out = KOSPI200Heston.CalibrateToVolSurface(KOSPI200Black,'globalKappaConstrained');
% bbb = 1.0;
% 
% priceSimul = zeros(length(expiry),101);
% impVolSimul = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% 
% priceSimulQE = zeros(length(expiry),101);
% impVolSimulQE = zeros(length(expiry),101);
% 
% % price simulation
% bbb =  1.0;
% spot =  KOSPI200HestonSLV.spot.params('rawData');
% 
% for j=1:length(expiry)
%     fwd = KOSPI200HestonSLV.Fwd(spot,expiry(j));
%     df = KOSPI200HestonSLV.zeroCurve.DF(expiry(j)/365.0);
%     for k=1:length(strikes)
%         tic
%         relStrike = strikes(k);
%         if relStrike <= 1.0
%             vanillaParams.params('callPutFlag') = 'P';
%             callputStringhsw = 'Put';
%         else
%             vanillaParams.params('callPutFlag') = 'C';
%             callputStringhsw = 'Call';
%         end
%         
%         priceSimul(j,k)  = KOSPI200Heston.modelOTMPrice(spot,fwd*relStrike,expiry(j),callputStringhsw);
%         impVolSimul(j,k) = KOSPI200Heston.ImpliedBlackVol(priceSimul(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         KOSPI200HestonSLV.modelParams('MCScheme') = 'QE';
%         KOSPI200Heston.modelParams('MCScheme') = 'QE';
% 
%         vanillaParams.params('strike') = fwd*relStrike;
%         vanillaParams.params('maturityDate') = AddDate(valueDateH,expiry(j),'day');
%         
%         pricingOutMCHestonSLV = KOSPI200HestonSLV.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQESLV(j,k) = pricingOutMCHestonSLV.europeanValue.npv/df;
%         impVolSimulQESLV(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQESLV(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         pricingOutMCHeston = KOSPI200Heston.computeVanillaMC(valueDateH,vanillaParams);
%         priceSimulQE(j,k) = pricingOutMCHeston.europeanValue.npv/df;
%         impVolSimulQE(j,k) = KOSPI200HestonSLV.ImpliedBlackVol(priceSimulQE(j,k),fwd,fwd*relStrike,expiry(j)/365.0,callputStringhsw);
%         
%         toc
%     end
% end


fff = 1.0;

In [ ]:
test_FileIO_Stepdown3S_SSSABR_20230228_fwdmnOrg_simple.m

clc
clear


keywordsMap = KeywordsMap('D:\KRS000266452_Repo0.input');

valueDate = char(keywordsMap('Today'));

useSSSABRAutoCallScheduleFlag = true;

mcOneTimeStepTest = 3;

NMCTest = str2double(keywordsMap('NPath'));

%% security info start
valueDateH = H_Date(valueDate);
stepdownParams.payCcy = 'KRW';

stepdownParams.startDate = H_Date(char(keywordsMap('DBStartDate')));
stepdownParams.endDate = H_Date(char(keywordsMap('DBEndDate')));

nominal = str2double(keywordsMap('Face'));
coupon = 0.0522;
couponFreq = 0.5;

basePrice = str2double(keywordsMap('S0'));

lowerBarrierOrig = str2double(keywordsMap('DownBarrier'));
barrierShift = str2double(keywordsMap('DownBarrierShift'));

lowerBarrier = lowerBarrierOrig(1) + barrierShift(1);

callStrike0 = str2double(keywordsMap('CallStrike0'));
callValue0 = str2double(keywordsMap('CallValue0'))/nominal;

callStrike1 = str2double(keywordsMap('CallStrike1'));
callValue1 = str2double(keywordsMap('CallValue1'))/nominal;

BarrierHitFlag = str2double(keywordsMap('BarrierHitFlag'));
if BarrierHitFlag < 0
    KIYN = true;
else
    KIYN = false;
end

scheduleDummy = keywordsMap('ExerciseDate');
scheduleInt = zeros(length(scheduleDummy),1);
for i=1:size(scheduleInt,1); scheduleInt(i) = str2num(H_Date(char(scheduleDummy(i))).StrDate('fng'));end

scheduleInt2 = str2double(keywordsMap('DStrike'));
scheduleInt3 = str2double(keywordsMap('Coupon'));

autoCallScheduleInt = [scheduleInt,scheduleInt2,scheduleInt3];

overHedgeShift = 0.0;
overHedgeSpread = 6.0;

overHedgeSpreadVector = str2double(keywordsMap('ohFactor'));

manualScheduleYN = 1;

SSpayoffYN = 'YES';

dailyTimeStepsYN = 'NO';
stepdownInfo = containers.Map({'nominal','coupon','couponFreq','lowerBarrier','barrierShift','lowerBarrierOrig','basePrice', ...
                               'callStrike0','callValue0','callStrike1','callValue1', ...
                                'KIYN', 'SSpayoffYN','dailyTimeStepsYN',...
                                'autoCallScheduleInt','overHedgeShift','overHedgeSpreadVector','overHedgeSpread','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,lowerBarrier,barrierShift, lowerBarrierOrig, basePrice, ...
                        callStrike0, callValue0, callStrike1, callValue1, ...
                        KIYN, SSpayoffYN,dailyTimeStepsYN,...
                        autoCallScheduleInt,overHedgeShift,overHedgeSpreadVector,overHedgeSpread,manualScheduleYN});

stepdownParams.params = stepdownInfo;   

%% security info end
useReducedFutures = 'NNMVolOnly';

slopeSimul.value = zeros(1000,200);
%HSCEI
slopeSimul.HSCEI_rmse = zeros(1000,200);
slopeSimul.HSCEI_fwdVolMseTotal = zeros(1000,200);
slopeSimul.HSCEI_arbitrageCheckSum = zeros(1000,200);
slopeSimul.HSCEI_quantoFwd = zeros(1000,200);
slopeSimul.HSCEI_delta = zeros(1000,200);
slopeSimul.HSCEI_vega = zeros(1000,200);
slopeSimul.HSCEI_volProxy = cell(1000,200);
slopeSimul.HSCEI_volError = cell(1000,200);
slopeSimul.HSCEI_fwdMoneyness = cell(1000,200);
slopeSimul.HSCEI_modelImpVol = cell(1000,200);
slopeSimul.HSCEI_nIter = cell(1000,200);

HSCEI_fwdFactor = zeros(5000,1);
SX5E_fwdFactor  = zeros(5000,1);
asset3_fwdFactor   = zeros(5000,1);

% SX5E 
slopeSimul.SX5E_rmse = zeros(1000,200);
slopeSimul.SX5E_fwdVolMseTotal = zeros(1000,200);
slopeSimul.SX5E_arbitrageCheckSum = zeros(1000,200);
slopeSimul.SX5E_quantoFwd = zeros(1000,200);
slopeSimul.SX5E_delta = zeros(1000,200);
slopeSimul.SX5E_vega = zeros(1000,200);
slopeSimul.SX5E_volProxy = cell(1000,200);
slopeSimul.SX5E_volError = cell(1000,200);
slopeSimul.SX5E_fwdMoneyness = cell(1000,200);
slopeSimul.SX5E_modelImpVol = cell(1000,200);
slopeSimul.SX5E_nIter = cell(1000,200);

%asset3 
slopeSimul.asset3_rmse = zeros(1000,200);
slopeSimul.asset3_fwdVolMseTotal = zeros(1000,200);
slopeSimul.asset3_arbitrageCheckSum = zeros(1000,200);
slopeSimul.asset3_quantoFwd = zeros(1000,200);
slopeSimul.asset3_delta = zeros(1000,200);
slopeSimul.asset3_vega = zeros(1000,200);
slopeSimul.asset3_volProxy = cell(1000,200);
slopeSimul.asset3_volError = cell(1000,200);
slopeSimul.asset3_fwdMoneyness = cell(1000,200);
slopeSimul.asset3_modelImpVol = cell(1000,200);
slopeSimul.asset3_nIter = cell(1000,200);

idxSlopeMax = 15;

simulationAsset = 'SX5E';

simulationType = 'Null';
idxMax = 1;

volShiftAmount = 0.001;

skewShiftAmount = 0.001;

slopeValue = 1.0;

for idxSlope = 3:3

simulOut.rmse = zeros(200,100);

simulOut.european.valueCurve = zeros(200,100);
simulOut.european.valueSnapshot = zeros(241,100);

simulOut.european.impliedVol = zeros(200,100);

simulOut.american.valueCurve = zeros(200,100);
simulOut.american.valueSnapshot = zeros(241,100);


% shiftAmount : future curves parallel shift amount
shiftAmount = 0;

% Theta simulation
% idxMax = 10;

for idx =1: idxMax

% timeShftAmount : time translation for the future date
timeShiftAmount = 0;
tempDate = H_Date(valueDate);
shiftedDate = tempDate.AddDate(timeShiftAmount,'day');
valueDate = StrDate(shiftedDate,'str');
valueDateH = H_Date(valueDate);


%% KRW deterministic forward IR model building start
zcy = str2double(keywordsMap('ZCY'));
rawData = zcy(:,1:2);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

%% create  a dummy IR BlackModel to discount 
modelParams= containers.Map({'modelName'},{'Fwd'});
KRWFwd = IRDetModel(IRModel(Model(KRWAllMktData,modelParams))); 
%% KRW deterministic forward IR model building end

%% multiAssetModel buildingStart
multiAssetMktData = containers.Map();
modelNameMap = containers.Map();

multiAssetMktData('KRW') = KRWAllMktData;
modelNameMap('KRW') = KRWFwd;

% multiAssetMktData = containers.Map({'asset1','asset2','asset3','KRW'}, ...
%                         {asset1AllMktData,asset2AllMktData,asset3AllMktData,KRWAllMktData});
% modelNameMap = containers.Map({'asset1','asset2','asset3','KRW'},{asset1Dupire,asset2Dupire,asset3Dupire,KRWFwd});

for idxAsset = 1:3
assetName = 'asset' + string(idxAsset);

%% asset 1 Model Building Start
%% zeroCurve
rfz = str2double(keywordsMap('RFZ'));

% filter out -1 from RFZ
abscisses = rfz(:,idxAsset*2-1);
maxRfzIdx = max(find(abscisses >=0));
rawData = rfz(1:maxRfzIdx,2*idxAsset-1:2*idxAsset);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName+'_zero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
asset1AllMktData =containers.Map('zeroCurve',zeroCurve);

%% implied VolSurface For asset1

% % marking vol KYJ
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210320	0.715449924	0.619913577	0.534507853	0.45605482	0.382639462	0.314114896	0.256149669	0.226746229	0.228914762	0.244116304	0.262768918
20210520	0.659176657	0.574885105	0.499719908	0.430955777	0.367024435	0.307829901	0.257115044	0.226361114	0.221382002	0.230618384	0.244766197
20210819	0.560947225	0.495542515	0.437547212	0.384942068	0.336633301	0.292504498	0.254155794	0.226011461	0.212573018	0.211655842	0.217498243
20220218	0.503496233	0.448532495	0.399997776	0.356230133	0.316343314	0.280180002	0.248604694	0.223900474	0.2087973	0.203327393	0.204498882
20230218	0.464727218	0.416471036	0.373995844	0.33585701	0.301285223	0.270097858	0.242821279	0.220844254	0.205915624	0.19849488	0.197003744
20240218	0.435775704	0.392425448	0.354372138	0.320322454	0.289582626	0.261948089	0.237742019	0.217865615	0.20349337	0.195127719	0.19193289
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
end


    
mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess',assetName + '_ccy',assetName+'_volSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);

impliedVolSurface.params('moneyNessType') = 'fixedStrike';

asset1AllMktData('impliedVolSurface') = impliedVolSurface;

%% dividend CurveFor asset1


rawData = [20220331	0
];

for idxhong=1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

% rawData = [0	0.0293
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName + '_dividendCurve'});

divYield = str2double(keywordsMap('DivYield'));

mktData('contiDivYld') = divYield(idxAsset);                   

mktData('divdCutoffDays') = 0;

dividendCurveMktData = MktData(mktData);

dividendCurveMktData.params('dividendType') = 'dividendYieldAndDiscreteDivdRatio';

asset1AllMktData('dividendCurve') = dividendCurveMktData;

%% repoRate Curve For asset1
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName + '_repoRateCurve'});
repoRateCurveMktData = MktData(mktData);
asset1AllMktData('repoRateCurve') = repoRateCurveMktData;

%% spot For asset1

S = str2double(keywordsMap('S'));
rawData = S(idxAsset);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'HKD','asset1Spot'});
spotMktData = MktData(mktData);
asset1AllMktData('spot') = spotMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName'},{'Black'});
asset1Black = EQBlack(EQModel(Model(asset1AllMktData,modelParams))); 
maturities = asset1Black.volSurface.maturities;
strikes =    asset1Black.volSurface.strikes;
impVolSurface = asset1Black.volSurface.vol;
spot = asset1Black.spot.params('rawData');
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

%% create a DupireModel with initial modelParamsDupire
%Initialize local volatility
asset1ImpliedVolSurface =asset1Black.volSurface;
localVolSurfaceDupire.expiry = maturities;
localVolSurfaceDupire.strike = strikes;
localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
% localVolSurfaceDupire.strikeType = 'fwdMoneyness';
localVolSurfaceDupire.strikeType = 'spotMoneyness';
% localVolSurfaceDupire.strikeType = 'fixedStrike';
modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurfaceDupire,'Dupire'});
%% SS SABR vol calibration setting

modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;
modelParamsDupire('NMC') = NMCTest;
modelParamsDupire('useQuasiRandomYN') = 'false';

% optimizer setting
modelParamsDupire('tol') = 0.0001;
modelParamsDupire('maxIter') = 1000;

modelParamsDupire('UseLetsBeRational') = 'YES';
modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

slopeParams = str2double(keywordsMap('slope'));
modelParamsDupire('slope') = slopeParams(idxAsset);

%% SS SABR params

modelParamsDupire('alpha_0') = 0.211;
modelParamsDupire('alpha_inf') = 0.219;
modelParamsDupire('rho')   = -0.218;
modelParamsDupire('nu') = 0.868;
modelParamsDupire('beta') = 1;
modelParamsDupire('lambda') = 0.7;
modelParamsDupire('omega')   = 0.6;
modelParamsDupire('time_shift') = 0.1667;

modelParamsDupire('lb') = [1.00E-08
1.00E-08
-0.9999
1.00E-08
% 1.00E-08
% 1.00E-08
% 1.00E-08
];

%alpah_0
%alpha_inf
%rho
%nu
%lambda
%omega

modelParamsDupire('ub') = [100
100
0.9999
100
% 100.0
% 100
% 100
];


calibratedParams = [0.2505
0.2423
-0.4872
0.7511
% 1.034002631
% 0.630720371
% 0.462011155
];

sabr_alpha_0 = str2double(keywordsMap('alpha_0'));
sabr_alpha_inf = str2double(keywordsMap('alpha_inf'));
sabr_rho = str2double(keywordsMap('rho'));
sabr_nu = str2double(keywordsMap('nu'));

sabr_params = [sabr_alpha_0';sabr_alpha_inf';sabr_rho';sabr_nu'];
calibratedParams = sabr_params(:,idxAsset);

modelParamsDupire('alpha_0') = calibratedParams(1);
modelParamsDupire('alpha_inf') = calibratedParams(2);
modelParamsDupire('rho')   = calibratedParams(3);
modelParamsDupire('nu') = calibratedParams(4);

modelParamsDupire('calibrationErrorType') = 'impVolError';
% modelParamsDupire('calibrationErrorType') = 'priceError';

reference = str2double(keywordsMap('reference'));
modelParamsDupire('reference') = reference(idxAsset);
asset1Dupire = EQCOMDupireSSSABRRaw(EQModel(Model(asset1AllMktData,modelParamsDupire)));

% asset1Dupire = EQCOMDupireSSSABR(EQModel(Model(asset1AllMktData,modelParamsDupire)));
% asset1Dupire = EQCOMDupireSSSABRRawNewLV(EQModel(Model(asset1AllMktData,modelParamsDupire)));

%% localVolSurfaceParams
atmVols = [30	0.3013
91	0.2935
182	0.2789
365	0.2706
730	0.2611
1095	0.2559
];

volTenor = keywordsMap('VolTenor');
atmVolsData = str2double(keywordsMap('Vol'));

atmVols(1:length(volTenor),2) = atmVolsData(:,idxAsset);

refExpiryH = localVolSurfaceDupire.expiry;
settleFutures = zeros(length(refExpiryH),1);
settleFuturesRef = zeros(length(refExpiryH),1);

referenceSpot = asset1Dupire.modelParams('reference');

spotRawData = asset1Dupire.spot.params('rawData');
for idxhh=1:length(refExpiryH)
    settleFutures(idxhh) = spotRawData;
    settleFuturesRef(idxhh) = modelParamsDupire('reference');
end

modelParamsDupire('settleFutures') = settleFutures;
modelParamsDupire('settleFuturesRef') = settleFuturesRef;
modelParamsDupire('atmVols') = atmVols;

asset1Dupire.settleFutures = settleFutures;
asset1Dupire.settleFuturesRef = settleFuturesRef;
asset1Dupire.atmVols = atmVols;

% out4 = asset1Dupire.CalibrateToVolSurfaceSSSABR(asset1Black,'justCalculateError');
%% fixed Strike info end
aaa = 1.0;
outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
%% asset1 Model Building End
multiAssetMktData(assetName) = asset1AllMktData;
modelNameMap(assetName) = asset1Dupire;
end



%shortTermCorr

correlationMatrix_short = str2double(keywordsMap('ShortTermCorr'));
correlationMatrix_long = str2double(keywordsMap('LongTermCorr'));

multiAssetMktData('correlationMatrix_short') = correlationMatrix_short;
multiAssetMktData('correlationMatrix_long') = correlationMatrix_long;


modelParams= containers.Map({'modelNameMap'},{modelNameMap});
modelParams('stochasticModelNames') = {'asset1','asset2','asset3'};

%% LocalCorr Start
modelParams('correlationMatrix_short') = correlationMatrix_short;
modelParams('correlationMatrix_long') = correlationMatrix_long;
modelParams('lambdaCorr') = 1.0;
% modelParams('corrAlpha') = 02;
modelParams('corrAlpha') = 0.0;
modelParams('alphaBarrier') = 0.0;

%% LocalCorr End
modelParams('refModel') = KRWFwd;
% modelParams('useQuasiRandomYN') = 'true';
modelParams('useQuasiRandomYN') = 'false';
modelParams('computeRealizedCorrelationYN') = 'false';
% modelParams('computeRealizedCorrelationYN') = 'true';
% modelParams('correlationBrigoGamma') = 0.967;

%% Target Curve Curve Correlation fitting end
SX5EDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ3FLocalCorr(multiAssetMktData,modelParams);
%% TermCorr Generate
outTermCorr = SX5EDupireHSCEIDupireasset3DupireKRWFwd.GenerateTermCorr(valueDateH,stepdownParams);
% pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced_ohVec(valueDateH,stepdownParams);

slopeSimul.value(idx,idxSlope) = pricingOutGF.unhitValue.npv;
aaa = 1.0;


        

end

if strcmp(simulationAsset,'asset3')
    if strcmp(simulationType,'ParallelDelta')

        slopeSimul.asset3_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 

    elseif strcmp(simulationType,'PointByPointDelta')

        for idxZ =1: numberOfTenor
            slopeSimul.asset3_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
        end

    elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
            || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')

        slopeSimul.asset3_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 

    elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
            || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')

        for idxZ =1: numberOfTenor
            slopeSimul.asset3_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')
        for idxZ =1: numberOfTenor
            for idxZZ = 1:numberOfStrikes
                slopeSimul.asset3_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
                 - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
            end
        end
    else
        aaaa = 1.0;
    %     error('unimplemented');
    end
elseif strcmp(simulationAsset,'DAX')
    if strcmp(simulationType,'ParallelDelta')

        slopeSimul.DAX_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 

    elseif strcmp(simulationType,'PointByPointDelta')

        for idxZ =1: numberOfTenor
            slopeSimul.DAX_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
        end

    elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
            || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')

        slopeSimul.DAX_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 

    elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
            || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')

        for idxZ =1: numberOfTenor
            slopeSimul.DAX_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')
        for idxZ =1: numberOfTenor
            for idxZZ = 1:numberOfStrikes
                slopeSimul.DAX_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
                 - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
            end
        end
    else
        aaaa = 1.0;
    %     error('unimplemented');
    end
    
end

end


disp('finished!');

bbb= 1.0;



In [ ]:
%test_FileIO_Stepdown3S_SSSABR_20230228_fwdmn_simple.m

clc
clear


% keywordsMap = KeywordsMap('D:\KRS000266452_Repo0.input');
keywordsMap = KeywordsMap('D:\KRS000266452_0321.dat');


valueDate = char(keywordsMap('Today'));

useSSSABRAutoCallScheduleFlag = true;

mcOneTimeStepTest = 3;

NMCTest = str2double(keywordsMap('NPath'));

%% security info start
valueDateH = H_Date(valueDate);
stepdownParams.payCcy = 'KRW';

stepdownParams.startDate = H_Date(char(keywordsMap('DBStartDate')));
stepdownParams.endDate = H_Date(char(keywordsMap('DBEndDate')));

nominal = str2double(keywordsMap('Face'));
coupon = 0.0522;
couponFreq = 0.5;

basePrice = str2double(keywordsMap('S0'));

lowerBarrierOrig = str2double(keywordsMap('DownBarrier'));
barrierShift = str2double(keywordsMap('DownBarrierShift'));

lowerBarrier = lowerBarrierOrig(1) + barrierShift(1);

callStrike0 = str2double(keywordsMap('CallStrike0'));
callValue0 = str2double(keywordsMap('CallValue0'))/nominal;

callStrike1 = str2double(keywordsMap('CallStrike1'));
callValue1 = str2double(keywordsMap('CallValue1'))/nominal;

BarrierHitFlag = str2double(keywordsMap('BarrierHitFlag'));
if BarrierHitFlag < 0
    KIYN = true;
else
    KIYN = false;
end

scheduleDummy = keywordsMap('ExerciseDate');
scheduleInt = zeros(length(scheduleDummy),1);
for i=1:size(scheduleInt,1); scheduleInt(i) = str2num(H_Date(char(scheduleDummy(i))).StrDate('fng'));end

scheduleInt2 = str2double(keywordsMap('DStrike'));
scheduleInt3 = str2double(keywordsMap('Coupon'));

autoCallScheduleInt = [scheduleInt,scheduleInt2,scheduleInt3];

overHedgeShift = 0.0;
overHedgeSpread = 6.0;

overHedgeSpreadVector = str2double(keywordsMap('ohFactor'));

manualScheduleYN = 1;

SSpayoffYN = 'YES';

dailyTimeStepsYN = 'NO';
stepdownInfo = containers.Map({'nominal','coupon','couponFreq','lowerBarrier','barrierShift','lowerBarrierOrig','basePrice', ...
                               'callStrike0','callValue0','callStrike1','callValue1', ...
                                'KIYN', 'SSpayoffYN','dailyTimeStepsYN',...
                                'autoCallScheduleInt','overHedgeShift','overHedgeSpreadVector','overHedgeSpread','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,lowerBarrier,barrierShift, lowerBarrierOrig, basePrice, ...
                        callStrike0, callValue0, callStrike1, callValue1, ...
                        KIYN, SSpayoffYN,dailyTimeStepsYN,...
                        autoCallScheduleInt,overHedgeShift,overHedgeSpreadVector,overHedgeSpread,manualScheduleYN});

stepdownParams.params = stepdownInfo;   

%% security info end
useReducedFutures = 'NNMVolOnly';

slopeSimul.value = zeros(1000,200);
%HSCEI
slopeSimul.HSCEI_rmse = zeros(1000,200);
slopeSimul.HSCEI_fwdVolMseTotal = zeros(1000,200);
slopeSimul.HSCEI_arbitrageCheckSum = zeros(1000,200);
slopeSimul.HSCEI_quantoFwd = zeros(1000,200);
slopeSimul.HSCEI_delta = zeros(1000,200);
slopeSimul.HSCEI_vega = zeros(1000,200);
slopeSimul.HSCEI_volProxy = cell(1000,200);
slopeSimul.HSCEI_volError = cell(1000,200);
slopeSimul.HSCEI_fwdMoneyness = cell(1000,200);
slopeSimul.HSCEI_modelImpVol = cell(1000,200);
slopeSimul.HSCEI_nIter = cell(1000,200);

HSCEI_fwdFactor = zeros(5000,1);
SX5E_fwdFactor  = zeros(5000,1);
asset3_fwdFactor   = zeros(5000,1);

% SX5E 
slopeSimul.SX5E_rmse = zeros(1000,200);
slopeSimul.SX5E_fwdVolMseTotal = zeros(1000,200);
slopeSimul.SX5E_arbitrageCheckSum = zeros(1000,200);
slopeSimul.SX5E_quantoFwd = zeros(1000,200);
slopeSimul.SX5E_delta = zeros(1000,200);
slopeSimul.SX5E_vega = zeros(1000,200);
slopeSimul.SX5E_volProxy = cell(1000,200);
slopeSimul.SX5E_volError = cell(1000,200);
slopeSimul.SX5E_fwdMoneyness = cell(1000,200);
slopeSimul.SX5E_modelImpVol = cell(1000,200);
slopeSimul.SX5E_nIter = cell(1000,200);

%asset3 
slopeSimul.asset3_rmse = zeros(1000,200);
slopeSimul.asset3_fwdVolMseTotal = zeros(1000,200);
slopeSimul.asset3_arbitrageCheckSum = zeros(1000,200);
slopeSimul.asset3_quantoFwd = zeros(1000,200);
slopeSimul.asset3_delta = zeros(1000,200);
slopeSimul.asset3_vega = zeros(1000,200);
slopeSimul.asset3_volProxy = cell(1000,200);
slopeSimul.asset3_volError = cell(1000,200);
slopeSimul.asset3_fwdMoneyness = cell(1000,200);
slopeSimul.asset3_modelImpVol = cell(1000,200);
slopeSimul.asset3_nIter = cell(1000,200);

idxSlopeMax = 15;

simulationAsset = 'SX5E';

simulationType = 'Null';
idxMax = 1;

volShiftAmount = 0.001;

skewShiftAmount = 0.001;

slopeValue = 1.0;

for idxSlope = 3:3

simulOut.rmse = zeros(200,100);

simulOut.european.valueCurve = zeros(200,100);
simulOut.european.valueSnapshot = zeros(241,100);

simulOut.european.impliedVol = zeros(200,100);

simulOut.american.valueCurve = zeros(200,100);
simulOut.american.valueSnapshot = zeros(241,100);


% shiftAmount : future curves parallel shift amount
shiftAmount = 0;

% Theta simulation
% idxMax = 10;

for idx =1: idxMax

% timeShftAmount : time translation for the future date
timeShiftAmount = 0;
tempDate = H_Date(valueDate);
shiftedDate = tempDate.AddDate(timeShiftAmount,'day');
valueDate = StrDate(shiftedDate,'str');
valueDateH = H_Date(valueDate);


%% KRW deterministic forward IR model building start
zcy = str2double(keywordsMap('ZCY'));
rawData = zcy(:,1:2);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

%% create  a dummy IR BlackModel to discount 
modelParams= containers.Map({'modelName'},{'Fwd'});
KRWFwd = IRDetModel(IRModel(Model(KRWAllMktData,modelParams))); 
%% KRW deterministic forward IR model building end

%% multiAssetModel buildingStart
multiAssetMktData = containers.Map();
modelNameMap = containers.Map();

multiAssetMktData('KRW') = KRWAllMktData;
modelNameMap('KRW') = KRWFwd;

% multiAssetMktData = containers.Map({'asset1','asset2','asset3','KRW'}, ...
%                         {asset1AllMktData,asset2AllMktData,asset3AllMktData,KRWAllMktData});
% modelNameMap = containers.Map({'asset1','asset2','asset3','KRW'},{asset1Dupire,asset2Dupire,asset3Dupire,KRWFwd});

for idxAsset = 1:3
assetName = 'asset' + string(idxAsset);

%% asset 1 Model Building Start
%% zeroCurve
rfz = str2double(keywordsMap('RFZ'));

% filter out -1 from RFZ
abscisses = rfz(:,idxAsset*2-1);
maxRfzIdx = max(find(abscisses >=0));
rawData = rfz(1:maxRfzIdx,2*idxAsset-1:2*idxAsset);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName+'_zero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
asset1AllMktData =containers.Map('zeroCurve',zeroCurve);

%% implied VolSurface For asset1

% % marking vol KYJ
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210320	0.715449924	0.619913577	0.534507853	0.45605482	0.382639462	0.314114896	0.256149669	0.226746229	0.228914762	0.244116304	0.262768918
20210520	0.659176657	0.574885105	0.499719908	0.430955777	0.367024435	0.307829901	0.257115044	0.226361114	0.221382002	0.230618384	0.244766197
20210819	0.560947225	0.495542515	0.437547212	0.384942068	0.336633301	0.292504498	0.254155794	0.226011461	0.212573018	0.211655842	0.217498243
20220218	0.503496233	0.448532495	0.399997776	0.356230133	0.316343314	0.280180002	0.248604694	0.223900474	0.2087973	0.203327393	0.204498882
20230218	0.464727218	0.416471036	0.373995844	0.33585701	0.301285223	0.270097858	0.242821279	0.220844254	0.205915624	0.19849488	0.197003744
20240218	0.435775704	0.392425448	0.354372138	0.320322454	0.289582626	0.261948089	0.237742019	0.217865615	0.20349337	0.195127719	0.19193289
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
end


    
mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess',assetName + '_ccy',assetName+'_volSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);

impliedVolSurface.params('moneyNessType') = 'fixedStrike';

asset1AllMktData('impliedVolSurface') = impliedVolSurface;

%% dividend CurveFor asset1


rawData = [20220331	0
];

for idxhong=1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

% rawData = [0	0.0293
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName + '_dividendCurve'});

divYield = str2double(keywordsMap('DivYield'));

mktData('contiDivYld') = divYield(idxAsset);                   

mktData('divdCutoffDays') = 0;

dividendCurveMktData = MktData(mktData);

dividendCurveMktData.params('dividendType') = 'dividendYieldAndDiscreteDivdRatio';

asset1AllMktData('dividendCurve') = dividendCurveMktData;

%% repoRate Curve For asset1
rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,assetName + '_ccy',assetName + '_repoRateCurve'});
repoRateCurveMktData = MktData(mktData);
asset1AllMktData('repoRateCurve') = repoRateCurveMktData;

%% spot For asset1

S = str2double(keywordsMap('S'));
rawData = S(idxAsset);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'HKD','asset1Spot'});
spotMktData = MktData(mktData);
asset1AllMktData('spot') = spotMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName'},{'Black'});
asset1Black = EQBlack(EQModel(Model(asset1AllMktData,modelParams))); 
maturities = asset1Black.volSurface.maturities;
strikes =    asset1Black.volSurface.strikes;
impVolSurface = asset1Black.volSurface.vol;
spot = asset1Black.spot.params('rawData');
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

%% create a DupireModel with initial modelParamsDupire
%Initialize local volatility
asset1ImpliedVolSurface =asset1Black.volSurface;
localVolSurfaceDupire.expiry = maturities;
localVolSurfaceDupire.strike = strikes;
localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
% localVolSurfaceDupire.strikeType = 'fwdMoneyness';
localVolSurfaceDupire.strikeType = 'spotMoneyness';
% localVolSurfaceDupire.strikeType = 'fixedStrike';
modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurfaceDupire,'Dupire'});
%% SS SABR vol calibration setting

modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;
modelParamsDupire('NMC') = NMCTest;
modelParamsDupire('useQuasiRandomYN') = 'false';

% optimizer setting
modelParamsDupire('tol') = 0.0001;
modelParamsDupire('maxIter') = 1000;

modelParamsDupire('UseLetsBeRational') = 'YES';
modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

slopeParams = str2double(keywordsMap('slope'));
modelParamsDupire('slope') = slopeParams(idxAsset);

%% SS SABR params

modelParamsDupire('alpha_0') = 0.211;
modelParamsDupire('alpha_inf') = 0.219;
modelParamsDupire('rho')   = -0.218;
modelParamsDupire('nu') = 0.868;
modelParamsDupire('beta') = 1;
modelParamsDupire('lambda') = 0.7;
modelParamsDupire('omega')   = 0.6;
modelParamsDupire('time_shift') = 0.1667;

modelParamsDupire('lb') = [1.00E-08
1.00E-08
-0.9999
1.00E-08
% 1.00E-08
% 1.00E-08
% 1.00E-08
];

%alpah_0
%alpha_inf
%rho
%nu
%lambda
%omega

modelParamsDupire('ub') = [100
100
0.9999
100
% 100.0
% 100
% 100
];


calibratedParams = [0.2505
0.2423
-0.4872
0.7511
% 1.034002631
% 0.630720371
% 0.462011155
];

sabr_alpha_0 = str2double(keywordsMap('alpha_0'));
sabr_alpha_inf = str2double(keywordsMap('alpha_inf'));
sabr_rho = str2double(keywordsMap('rho'));
sabr_nu = str2double(keywordsMap('nu'));

sabr_params = [sabr_alpha_0';sabr_alpha_inf';sabr_rho';sabr_nu'];
calibratedParams = sabr_params(:,idxAsset);

modelParamsDupire('alpha_0') = calibratedParams(1);
modelParamsDupire('alpha_inf') = calibratedParams(2);
modelParamsDupire('rho')   = calibratedParams(3);
modelParamsDupire('nu') = calibratedParams(4);

modelParamsDupire('calibrationErrorType') = 'impVolError';
% modelParamsDupire('calibrationErrorType') = 'priceError';

reference = str2double(keywordsMap('reference'));
modelParamsDupire('reference') = reference(idxAsset);
asset1Dupire = EQCOMDupireSSSABRRaw(EQModel(Model(asset1AllMktData,modelParamsDupire)));

% asset1Dupire = EQCOMDupireSSSABR(EQModel(Model(asset1AllMktData,modelParamsDupire)));
% asset1Dupire = EQCOMDupireSSSABRRawNewLV(EQModel(Model(asset1AllMktData,modelParamsDupire)));

%% localVolSurfaceParams
atmVols = [30	0.3013
91	0.2935
182	0.2789
365	0.2706
730	0.2611
1095	0.2559
];

volTenor = keywordsMap('VolTenor');
atmVolsData = str2double(keywordsMap('Vol'));

atmVols(1:length(volTenor),2) = atmVolsData(:,idxAsset);

refExpiryH = localVolSurfaceDupire.expiry;
settleFutures = zeros(length(refExpiryH),1);
settleFuturesRef = zeros(length(refExpiryH),1);

referenceSpot = asset1Dupire.modelParams('reference');

spotRawData = asset1Dupire.spot.params('rawData');
for idxhh=1:length(refExpiryH)
    settleFutures(idxhh) = spotRawData;
    settleFuturesRef(idxhh) = modelParamsDupire('reference');
end

modelParamsDupire('settleFutures') = settleFutures;
modelParamsDupire('settleFuturesRef') = settleFuturesRef;
modelParamsDupire('atmVols') = atmVols;

asset1Dupire.settleFutures = settleFutures;
asset1Dupire.settleFuturesRef = settleFuturesRef;
asset1Dupire.atmVols = atmVols;

% out4 = asset1Dupire.CalibrateToVolSurfaceSSSABR(asset1Black,'justCalculateError');
%% fixed Strike info end
aaa = 1.0;
outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
%% asset1 Model Building End
multiAssetMktData(assetName) = asset1AllMktData;
modelNameMap(assetName) = asset1Dupire;
end



%shortTermCorr

correlationMatrix_short = str2double(keywordsMap('ShortTermCorr'));
correlationMatrix_long = str2double(keywordsMap('LongTermCorr'));

multiAssetMktData('correlationMatrix_short') = correlationMatrix_short;
multiAssetMktData('correlationMatrix_long') = correlationMatrix_long;


modelParams= containers.Map({'modelNameMap'},{modelNameMap});
modelParams('stochasticModelNames') = {'asset1','asset2','asset3'};

%% LocalCorr Start
modelParams('correlationMatrix_short') = correlationMatrix_short;
modelParams('correlationMatrix_long') = correlationMatrix_long;
modelParams('lambdaCorr') = 1.0;
% modelParams('corrAlpha') = 02;
modelParams('corrAlpha') = 0.0;
modelParams('alphaBarrier') = 0.0;

%% LocalCorr End
modelParams('refModel') = KRWFwd;
% modelParams('useQuasiRandomYN') = 'true';
modelParams('useQuasiRandomYN') = 'false';
modelParams('computeRealizedCorrelationYN') = 'false';
% modelParams('computeRealizedCorrelationYN') = 'true';
% modelParams('correlationBrigoGamma') = 0.967;

%% Target Curve Curve Correlation fitting end
SX5EDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ3FLocalCorr(multiAssetMktData,modelParams);
%% TermCorr Generate
outTermCorr = SX5EDupireHSCEIDupireasset3DupireKRWFwd.GenerateTermCorr(valueDateH,stepdownParams);
% pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced_ohVec(valueDateH,stepdownParams);

slopeSimul.value(idx,idxSlope) = pricingOutGF.unhitValue.npv;
aaa = 1.0;


end        

end
aaa =1.0;
% if strcmp(simulationAsset,'asset3')
%     if strcmp(simulationType,'ParallelDelta')
% 
%         slopeSimul.asset3_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 
% 
%     elseif strcmp(simulationType,'PointByPointDelta')
% 
%         for idxZ =1: numberOfTenor
%             slopeSimul.asset3_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
%         end
% 
%     elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
%             || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')
% 
%         slopeSimul.asset3_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 
% 
%     elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
%             || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')
% 
%         for idxZ =1: numberOfTenor
%             slopeSimul.asset3_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
%         end
%     elseif strcmp(vegaSimulationType,'PointByPointVega')
%         for idxZ =1: numberOfTenor
%             for idxZZ = 1:numberOfStrikes
%                 slopeSimul.asset3_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
%                  - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
%             end
%         end
%     else
%         aaaa = 1.0;
%     %     error('unimplemented');
%     end
% elseif strcmp(simulationAsset,'DAX')
%     if strcmp(simulationType,'ParallelDelta')
% 
%         slopeSimul.DAX_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 
% 
%     elseif strcmp(simulationType,'PointByPointDelta')
% 
%         for idxZ =1: numberOfTenor
%             slopeSimul.DAX_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
%         end
% 
%     elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
%             || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')
% 
%         slopeSimul.DAX_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 
% 
%     elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
%             || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')
% 
%         for idxZ =1: numberOfTenor
%             slopeSimul.DAX_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
%         end
%     elseif strcmp(vegaSimulationType,'PointByPointVega')
%         for idxZ =1: numberOfTenor
%             for idxZZ = 1:numberOfStrikes
%                 slopeSimul.DAX_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
%                  - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
%             end
%         end
%     else
%         aaaa = 1.0;
%     %     error('unimplemented');
%     end
%     
% end
% 
% end
% 

disp('finished!');

bbb= 1.0;



In [ ]:
%test_FileIO_Stepdown3S_SSSABR_20230228_fwdmn.m

% clc
% clear


keywordsMap = KeywordsMap('D:\KRS000266452_Repo0.input');

valueDate = '2023-02-28';
valueDate = char(keywordsMap('Today'));

useSSSABRAutoCallScheduleFlag = true;
% useSSSABRAutoCallScheduleFlag = false;

mcOneTimeStepTest = 3;

% mcOneTimeStepTest = 7;

NMCTest = 200000;
NMCTest = str2double(keywordsMap('NPath'));

% NMCTest = 100000;


%% security info start
valueDateH = H_Date(valueDate);
stepdownParams.payCcy = 'KRW';

stepdownParams.startDate = H_Date('2021-10-22');
stepdownParams.endDate = H_Date('2024-10-22');

stepdownParams.startDate = H_Date(char(keywordsMap('DBStartDate')));
stepdownParams.endDate = H_Date(char(keywordsMap('DBEndDate')));


nominal = 10000;
nominal = str2double(keywordsMap('Face'));


coupon = 0.0522;
couponFreq = 0.5;

basePrice = [
4188.81
9356.85
4544.9
];


basePrice = str2double(keywordsMap('S0'));


% basePrice = [
% 0.418881
% 9356.85
% 0.45449
% ];


lowerBarrierOrig = 0.45;
barrierShift = 0.015;

lowerBarrierOrig = str2double(keywordsMap('DownBarrier'));
barrierShift = str2double(keywordsMap('DownBarrierShift'));

% lowerBarrier = lowerBarrierOrig(1) - barrierShift(1);
lowerBarrier = lowerBarrierOrig(1) + barrierShift(1);


callStrike0 = 0.65;
callValue0 = 1.0 + 0.2088;

callStrike0 = str2double(keywordsMap('CallStrike0'));
callValue0 = str2double(keywordsMap('CallValue0'))/nominal;

callStrike1 = 0.65;
callValue1 =  0.65;

callStrike1 = str2double(keywordsMap('CallStrike1'));
callValue1 = str2double(keywordsMap('CallValue1'))/nominal;

KIYN = false;
% KIYN = true;

BarrierHitFlag = str2double(keywordsMap('BarrierHitFlag'));
if BarrierHitFlag < 0
    KIYN = true;
else
    KIYN = false;
end

autoCallScheduleInt =[20220121	0.95	0.0174
20220422	0.95	0.0348
20220722	0.95	0.0522
20221021	0.95	0.0696
20230120	0.9	0.087
20230421	0.9	0.1044
20230721	0.9	0.1218
20231020	0.9	0.1392
20240122	0.85	0.1566
20240422	0.8	0.174
20240722	0.75	0.1914
20241022	0.7	0.2088
];

scheduleDummy = keywordsMap('ExerciseDate');
scheduleInt = zeros(length(scheduleDummy),1);
for i=1:size(scheduleInt,1); scheduleInt(i) = str2num(H_Date(char(scheduleDummy(i))).StrDate('fng'));end

scheduleInt2 = str2double(keywordsMap('DStrike'));
scheduleInt3 = str2double(keywordsMap('Coupon'));

autoCallScheduleInt = [scheduleInt,scheduleInt2,scheduleInt3];



overHedgeShift = 0.0;
overHedgeSpread = 6.0;

overHedgeSpreadVector = [
3.0000000000
3.0000000000
4.0000000000
4.0000000000
5.0000000000
5.0000000000
6.0000000000
6.0000000000
7.0000000000
7.0000000000
8.0000000000
8.0000000000
];

overHedgeSpreadVector = str2double(keywordsMap('ohFactor'));


manualScheduleYN = 1;

SSpayoffYN = 'YES';

dailyTimeStepsYN = 'NO';
stepdownInfo = containers.Map({'nominal','coupon','couponFreq','lowerBarrier','barrierShift','lowerBarrierOrig','basePrice', ...
                               'callStrike0','callValue0','callStrike1','callValue1', ...
                                'KIYN', 'SSpayoffYN','dailyTimeStepsYN',...
                                'autoCallScheduleInt','overHedgeShift','overHedgeSpreadVector','overHedgeSpread','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,lowerBarrier,barrierShift, lowerBarrierOrig, basePrice, ...
                        callStrike0, callValue0, callStrike1, callValue1, ...
                        KIYN, SSpayoffYN,dailyTimeStepsYN,...
                        autoCallScheduleInt,overHedgeShift,overHedgeSpreadVector,overHedgeSpread,manualScheduleYN});

stepdownParams.params = stepdownInfo;   

%% security info end

% useSSRFlag = 'YES';
useSSRFlag = 'NO';

% useReducedFutures = 'NMVolOnly';
useReducedFutures = 'NNMVolOnly';




slopeSimul.value = zeros(1000,200);
%HSCEI
slopeSimul.HSCEI_rmse = zeros(1000,200);
slopeSimul.HSCEI_fwdVolMseTotal = zeros(1000,200);
slopeSimul.HSCEI_arbitrageCheckSum = zeros(1000,200);
slopeSimul.HSCEI_quantoFwd = zeros(1000,200);
slopeSimul.HSCEI_delta = zeros(1000,200);
slopeSimul.HSCEI_vega = zeros(1000,200);
slopeSimul.HSCEI_volProxy = cell(1000,200);
slopeSimul.HSCEI_volError = cell(1000,200);
slopeSimul.HSCEI_fwdMoneyness = cell(1000,200);
slopeSimul.HSCEI_modelImpVol = cell(1000,200);
slopeSimul.HSCEI_nIter = cell(1000,200);

HSCEI_fwdFactor = zeros(5000,1);
SX5E_fwdFactor  = zeros(5000,1);
asset3_fwdFactor   = zeros(5000,1);

% SX5E 
slopeSimul.SX5E_rmse = zeros(1000,200);
slopeSimul.SX5E_fwdVolMseTotal = zeros(1000,200);
slopeSimul.SX5E_arbitrageCheckSum = zeros(1000,200);
slopeSimul.SX5E_quantoFwd = zeros(1000,200);
slopeSimul.SX5E_delta = zeros(1000,200);
slopeSimul.SX5E_vega = zeros(1000,200);
slopeSimul.SX5E_volProxy = cell(1000,200);
slopeSimul.SX5E_volError = cell(1000,200);
slopeSimul.SX5E_fwdMoneyness = cell(1000,200);
slopeSimul.SX5E_modelImpVol = cell(1000,200);
slopeSimul.SX5E_nIter = cell(1000,200);




%asset3 
slopeSimul.asset3_rmse = zeros(1000,200);
slopeSimul.asset3_fwdVolMseTotal = zeros(1000,200);
slopeSimul.asset3_arbitrageCheckSum = zeros(1000,200);
slopeSimul.asset3_quantoFwd = zeros(1000,200);
slopeSimul.asset3_delta = zeros(1000,200);
slopeSimul.asset3_vega = zeros(1000,200);
slopeSimul.asset3_volProxy = cell(1000,200);
slopeSimul.asset3_volError = cell(1000,200);
slopeSimul.asset3_fwdMoneyness = cell(1000,200);
slopeSimul.asset3_modelImpVol = cell(1000,200);
slopeSimul.asset3_nIter = cell(1000,200);


idxSlopeMax = 15;

simulationAsset = 'SX5E';

simulationType = 'Null';
idxMax = 1;

% 
% simulationType = 'ParallelDelta';
% numberOfTenor = 1;
% idxMax = 2*numberOfTenor + 1;
% % 
% simulationType = 'PointByPointDelta';
% numberOfTenor = 7;
% idxMax = 2*numberOfTenor + 1;

vegaSimulationType = 'Null';
 
% % vegaSimulationType = 'Null';
% vegaSimulationType = 'ParallelVega';
% numberOfTenor = 1;
% idxMax = 2*numberOfTenor + 1;

% vegaSimulationType = 'TermVega';
% if removeDAXMay20Flag
%     numberOfTenor = 5;
% else
%     numberOfTenor = 6;
% end
% idxMax = 2*numberOfTenor + 1;

% vegaSimulationType = 'PointByPointVega';
% numberOfTenor = 6;
% numberOfStrikes = 11;
% % numberOfStrikes = 9;
% idxMax = 2*numberOfTenor*numberOfStrikes + 1;


volShiftAmount = 0.001;

skewShiftAmount = 0.001;

slopeValue = 1.0;

for idxSlope = 3:3

simulOut.rmse = zeros(200,100);

simulOut.european.valueCurve = zeros(200,100);
simulOut.european.valueSnapshot = zeros(241,100);

simulOut.european.impliedVol = zeros(200,100);

simulOut.american.valueCurve = zeros(200,100);
simulOut.american.valueSnapshot = zeros(241,100);


% shiftAmount : future curves parallel shift amount
shiftAmount = 0;

% Theta simulation
% idxMax = 10;

for idx =1: idxMax

% timeShftAmount : time translation for the future date

% timeShiftAmount = idx-1;
timeShiftAmount = 0;

tempDate = H_Date(valueDate);

shiftedDate = tempDate.AddDate(timeShiftAmount,'day');

valueDate = StrDate(shiftedDate,'str');

valueDateH = H_Date(valueDate);


%% asset1 Model Building Start
%% zeroCurve
rawData = [0.002739726	0.012295106
0.25	0.016874629
0.5	0.01946514
0.75	0.022582146
1	0.025694238
1.25	0.027254061
1.5	0.028811471
1.75	0.030366467
2	0.031919049
2.25	0.032262136
2.5	0.032563444
2.75	0.032864753
3	0.033166061
3.25	0.033188742
3.5	0.033207498
3.75	0.033226255
4	0.033245011
4.25	0.033240791
4.5	0.033236315
4.75	0.03323184
5	0.033227365
5.25	0.033237831
5.5	0.033248463
5.75	0.033259095
6	0.033269726
6.25	0.033280358
6.5	0.03329099
6.75	0.033301622
7	0.033312254
];

rfz = str2double(keywordsMap('RFZ'));
rawData = rfz(:,1:2);



mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'HKD','HKDZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
asset1AllMktData =containers.Map('zeroCurve',zeroCurve);

%% implied VolSurface For asset1

% % marking vol KYJ
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210320	0.715449924	0.619913577	0.534507853	0.45605482	0.382639462	0.314114896	0.256149669	0.226746229	0.228914762	0.244116304	0.262768918
20210520	0.659176657	0.574885105	0.499719908	0.430955777	0.367024435	0.307829901	0.257115044	0.226361114	0.221382002	0.230618384	0.244766197
20210819	0.560947225	0.495542515	0.437547212	0.384942068	0.336633301	0.292504498	0.254155794	0.226011461	0.212573018	0.211655842	0.217498243
20220218	0.503496233	0.448532495	0.399997776	0.356230133	0.316343314	0.280180002	0.248604694	0.223900474	0.2087973	0.203327393	0.204498882
20230218	0.464727218	0.416471036	0.373995844	0.33585701	0.301285223	0.270097858	0.242821279	0.220844254	0.205915624	0.19849488	0.197003744
20240218	0.435775704	0.392425448	0.354372138	0.320322454	0.289582626	0.261948089	0.237742019	0.217865615	0.20349337	0.195127719	0.19193289
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
end


    
mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess','HKD','asset1VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);

impliedVolSurface.params('moneyNessType') = 'fixedStrike';

asset1AllMktData('impliedVolSurface') = impliedVolSurface;

%% dividend CurveFor asset1


rawData = [20220331	0
];

for idxhong=1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

% rawData = [0	0.0293
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'HKD','asset1DividendCurve'});

mktData('contiDivYld') = 0.0324;                   
divYield = str2double(keywordsMap('DivYield'));

mktData('contiDivYld') = divYield(1);                   



mktData('divdCutoffDays') = 0;

dividendCurveMktData = MktData(mktData);

dividendCurveMktData.params('dividendType') = 'dividendYieldAndDiscreteDivdRatio';

asset1AllMktData('dividendCurve') = dividendCurveMktData;

%% repoRate Curve For asset1

% rawData = [28	-0.003999
% 60	-0.005823
% 90	0.009482
% 182	-0.010925
% 273	-0.009570
% 364	-0.008168
% 455	-0.002457
% 637	-0.003361
% 819	-0.004611
% 1005	-0.004801
% 1370	-0.005454
% 1735	-0.005664
% 2100	-0.006238
% 2464	-0.006391
% 2829	-0.006118
% 3196	-0.006876
% 3561	-0.007049
% ];

rawData = [0	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','asset1RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
asset1AllMktData('repoRateCurve') = repoRateCurveMktData;

%% spot For asset1
rawData = 6581.47;

S = str2double(keywordsMap('S'));
rawData = S(1);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'HKD','asset1Spot'});
spotMktData = MktData(mktData);
asset1AllMktData('spot') = spotMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName'},{'Black'});
asset1Black = EQBlack(EQModel(Model(asset1AllMktData,modelParams))); 
maturities = asset1Black.volSurface.maturities;
strikes =    asset1Black.volSurface.strikes;
impVolSurface = asset1Black.volSurface.vol;
spot = asset1Black.spot.params('rawData');
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

%% create a DupireModel with initial modelParamsDupire
%Initialize local volatility
asset1ImpliedVolSurface =asset1Black.volSurface;

localVolSurfaceDupire.expiry = maturities;
localVolSurfaceDupire.strike = strikes;
localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
% localVolSurfaceDupire.strikeType = 'fwdMoneyness';
localVolSurfaceDupire.strikeType = 'spotMoneyness';

% localVolSurfaceDupire.strikeType = 'fixedStrike';
modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurfaceDupire,'Dupire'});
%% SS SABR vol calibration setting

modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

modelParamsDupire('NMC') = NMCTest;

modelParamsDupire('useQuasiRandomYN') = 'false';

% optimizer setting

modelParamsDupire('tol') = 0.0001;
modelParamsDupire('maxIter') = 1000;

modelParamsDupire('UseLetsBeRational') = 'YES';
modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

% modelParamsDupire('slope') = 1.0;
modelParamsDupire('slope') = 0.5;

slopeParams = str2double(keywordsMap('slope'));
modelParamsDupire('slope') = slopeParams(1);

%% SS SABR params

modelParamsDupire('alpha_0') = 0.211;
modelParamsDupire('alpha_inf') = 0.219;
modelParamsDupire('rho')   = -0.218;
modelParamsDupire('nu') = 0.868;
modelParamsDupire('beta') = 1;
modelParamsDupire('lambda') = 0.7;
modelParamsDupire('omega')   = 0.6;
modelParamsDupire('time_shift') = 0.1667;



modelParamsDupire('lb') = [1.00E-08
1.00E-08
-0.9999
1.00E-08
% 1.00E-08
% 1.00E-08
% 1.00E-08
];

%alpah_0
%alpha_inf
%rho
%nu
%lambda
%omega

modelParamsDupire('ub') = [100
100
0.9999
100
% 100.0
% 100
% 100
];


calibratedParams = [0.2505
0.2423
-0.4872
0.7511
% 1.034002631
% 0.630720371
% 0.462011155
];

sabr_alpha_0 = str2double(keywordsMap('alpha_0'));
sabr_alpha_inf = str2double(keywordsMap('alpha_inf'));
sabr_rho = str2double(keywordsMap('rho'));
sabr_nu = str2double(keywordsMap('nu'));

sabr_params = [sabr_alpha_0';sabr_alpha_inf';sabr_rho';sabr_nu'];
calibratedParams = sabr_params(:,1);

modelParamsDupire('alpha_0') = calibratedParams(1);
modelParamsDupire('alpha_inf') = calibratedParams(2);
modelParamsDupire('rho')   = calibratedParams(3);
modelParamsDupire('nu') = calibratedParams(4);

modelParamsDupire('calibrationErrorType') = 'impVolError';
% modelParamsDupire('calibrationErrorType') = 'priceError';

modelParamsDupire('reference') = 6581.47;

reference = str2double(keywordsMap('reference'));
modelParamsDupire('reference') = reference(1);

% asset1Dupire = EQCOMDupireSSSABR(EQModel(Model(asset1AllMktData,modelParamsDupire)));
asset1Dupire = EQCOMDupireSSSABRRaw(EQModel(Model(asset1AllMktData,modelParamsDupire)));
% asset1Dupire = EQCOMDupireSSSABRRawNewLV(EQModel(Model(asset1AllMktData,modelParamsDupire)));

for idxJong = 1:5000
    tDaysJong = idxJong;
%     tDaysJong = 915;
    asset1_fwdFactor(idxJong) = asset1Dupire.FwdFactor(tDaysJong);
end

%% localVolSurfaceParams
atmVols = [30	0.3013
91	0.2935
182	0.2789
365	0.2706
730	0.2611
1095	0.2559
1460	0.2559
1825	0.2559
];

volTenor = keywordsMap('VolTenor');
atmVolsData = str2double(keywordsMap('Vol'));

atmVols(1:length(volTenor),2) = atmVolsData(:,1);
atmVols(length(volTenor):end,2) = atmVolsData(end,1);

refExpiryH = localVolSurfaceDupire.expiry;
settleFutures = zeros(length(refExpiryH),1);
settleFuturesRef = zeros(length(refExpiryH),1);

referenceSpot = asset1Dupire.modelParams('reference');

spotRawData = asset1Dupire.spot.params('rawData');
for idxhh=1:length(refExpiryH)
%     settleFutures(idxhh) = asset1Dupire.FwdWithoutSpot(refExpiryH(idxhh));
    settleFutures(idxhh) = spotRawData;
%     settleFuturesRef(idxhh) = asset1Dupire.Fwd(referenceSpot,refExpiryH(idxhh));
    settleFuturesRef(idxhh) = modelParamsDupire('reference');
end

modelParamsDupire('settleFutures') = settleFutures;
modelParamsDupire('settleFuturesRef') = settleFuturesRef;
modelParamsDupire('atmVols') = atmVols;

asset1Dupire.settleFutures = settleFutures;
asset1Dupire.settleFuturesRef = settleFuturesRef;
asset1Dupire.atmVols = atmVols;

out4 = asset1Dupire.CalibrateToVolSurfaceSSSABR(asset1Black,'justCalculateError');
%% fixed Strike info end
aaa = 1.0;
% modelParamsDupire('generateLocalVolFlag') = true;


% useSSSABRAutoCallScheduleFlag = true;
% outLocalVolSS_asset1 = asset1Dupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset1_KYJ_FwdMN = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

outLocalVolSS_asset1_KYJ_FwdMN_FwdFactor = asset1Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset1_KYJ = asset1Dupire.GenerateLocalVolSS_KYJ(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset1_Qlib = asset1Dupire.GenerateLocalVolSS_Qlib(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset1_Qlib_FwdMN = asset1Dupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
aaa = 1.0;
%% asset1 Model Building End

%% asset2 Model Building Start
%% zeroCurve
%20220331
rawData = [0.002739726	0.009807984
0.25	0.009630833
0.5	0.010827048
0.75	0.01073329
1	0.010609072
1.5	0.011206315
2	0.011776405
2.5	0.011818089
3	0.011855976
3.5	0.011967666
4	0.012079938
4.5	0.012217165
5	0.012355387
5.5	0.012511665
6	0.012668557
6.5	0.01282545
7	0.012982342
];

rawData = rfz(:,3:4);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'EUR','EURZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
asset2AllMktData =containers.Map('zeroCurve',zeroCurve);

%% implied VolSurface For SX5E

% Markit Vol 20210218 
% Filtering : 1M< expiry < 6Y
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210416	0.641688	0.549477	0.454107	0.36232	0.269162	0.18144	0.137903	0.157325	0.187404	0.217011	0.244745
20210521	0.567319	0.487313	0.409716	0.331614	0.256301	0.187196	0.14707	0.152296	0.173213	0.195728	0.217496
20210618	0.537098	0.463247	0.391713	0.321053	0.253368	0.192513	0.15336	0.150549	0.16649	0.185447	0.20435
20210917	0.475848	0.412135	0.352556	0.297153	0.245435	0.198744	0.163633	0.14926	0.152741	0.16321	0.175651
20211217	0.432648	0.376788	0.328085	0.282189	0.239306	0.200344	0.169841	0.153433	0.150428	0.155179	0.163453
20220617	0.3847	0.338792	0.297844	0.260365	0.225935	0.196823	0.174574	0.160503	0.155717	0.156805	0.161675
20221216	0.358015	0.317298	0.280415	0.247108	0.21888	0.195978	0.178469	0.166305	0.15837	0.159363	0.163357
20230616	0.32651	0.293679	0.263788	0.234858	0.210952	0.19214	0.177977	0.168042	0.161946	0.160355	0.16199
20231215	0.324872	0.288315	0.254661	0.22908	0.208087	0.1921	0.180348	0.172221	0.164672	0.166459	0.170789
20241220	0.290404	0.2634	0.239631	0.218457	0.20209	0.189878	0.181198	0.17494	0.171537	0.171032	0.17231
20251219	0.278316	0.253237	0.232068	0.214239	0.19958	0.188141	0.179986	0.174965	0.172631	0.172356	0.173523
20261218	0.268939	0.246108	0.227041	0.211199	0.198377	0.188523	0.181564	0.177262	0.175196	0.174852	0.175743
];

% Markit Vol 1% up
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210416	0.651688	0.559477	0.464107	0.37232	0.279162	0.19144	0.147903	0.167325	0.197404	0.227011	0.254745
20210521	0.577319	0.497313	0.419716	0.341614	0.266301	0.197196	0.15707	0.162296	0.183213	0.205728	0.227496
20210618	0.547098	0.473247	0.401713	0.331053	0.263368	0.202513	0.16336	0.160549	0.17649	0.195447	0.21435
20210917	0.485848	0.422135	0.362556	0.307153	0.255435	0.208744	0.173633	0.15926	0.162741	0.17321	0.185651
20211217	0.442648	0.386788	0.338085	0.292189	0.249306	0.210344	0.179841	0.163433	0.160428	0.165179	0.173453
20220617	0.3947	0.348792	0.307844	0.270365	0.235935	0.206823	0.184574	0.170503	0.165717	0.166805	0.171675
20221216	0.368015	0.327298	0.290415	0.257108	0.22888	0.205978	0.188469	0.176305	0.16837	0.169363	0.173357
20230616	0.33651	0.303679	0.273788	0.244858	0.220952	0.20214	0.187977	0.178042	0.171946	0.170355	0.17199
20231215	0.334872	0.298315	0.264661	0.23908	0.218087	0.2021	0.190348	0.182221	0.174672	0.176459	0.180789
20241220	0.300404	0.2734	0.249631	0.228457	0.21209	0.199878	0.191198	0.18494	0.181537	0.181032	0.18231
20251219	0.288316	0.263237	0.242068	0.224239	0.20958	0.198141	0.189986	0.184965	0.182631	0.182356	0.183523
20261218	0.278939	0.256108	0.237041	0.221199	0.208377	0.198523	0.191564	0.187262	0.185196	0.184852	0.185743
];

% %dummy vol
% 
% rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
% 20210320	0.715449924	0.619913577	0.534507853	0.45605482	0.382639462	0.314114896	0.256149669	0.226746229	0.228914762	0.244116304	0.262768918
% 20210520	0.659176657	0.574885105	0.499719908	0.430955777	0.367024435	0.307829901	0.257115044	0.226361114	0.221382002	0.230618384	0.244766197
% 20210819	0.560947225	0.495542515	0.437547212	0.384942068	0.336633301	0.292504498	0.254155794	0.226011461	0.212573018	0.211655842	0.217498243
% 20220218	0.503496233	0.448532495	0.399997776	0.356230133	0.316343314	0.280180002	0.248604694	0.223900474	0.2087973	0.203327393	0.204498882
% 20230218	0.464727218	0.416471036	0.373995844	0.33585701	0.301285223	0.270097858	0.242821279	0.220844254	0.205915624	0.19849488	0.197003744
% 20240218	0.435775704	0.392425448	0.354372138	0.320322454	0.289582626	0.261948089	0.237742019	0.217865615	0.20349337	0.195127719	0.19193289
% ];

for idxhong=1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
end
    
mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess','USD','asset2VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);

impliedVolSurface.params('moneyNessType') = 'spotMoneyNess';

asset2AllMktData('impliedVolSurface') = impliedVolSurface;

%% dividend CurveFor SX5E
rawData = [20220331	0
];

for idxhong=1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

% rawData = [0	0.0184
% ];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'EUR','asset2DividendCurve'});


mktData('contiDivYld') = 0.0307;

divYield = str2double(keywordsMap('DivYield'));

mktData('contiDivYld') = divYield(2);    

mktData('divdCutoffDays') = 0;                    
                    
dividendCurveMktData = MktData(mktData);

dividendCurveMktData.params('dividendType') = 'dividendYieldAndDiscreteDivdRatio';

asset2AllMktData('dividendCurve') = dividendCurveMktData;



%% spot For asset2
rawData = 4238.38;

S = str2double(keywordsMap('S'));
rawData = S(2);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'EUR','asset2Spot'});
spotMktData = MktData(mktData);
asset2AllMktData('spot') = spotMktData;

%% forwardCurve For asset2
% % Markit Fwd
% rawData = [20210416	3667.33332
% 20210521	3625.52817
% 20210618	3618.09201
% 20210917	3604.72113
% 20211217	3585.44035
% 20220617	3505.83188
% 20221216	3481.54411
% 20230616	3405.91452
% 20231215	3385.18137
% 20241220	3300.19647
% 20251219	3226.3573
% 20261218	3161.38669
% ];
% 
% % % Trader Fwd
% % 
% % rawData = [20210416	3663.899858
% % 20210521	3661.909627
% % 20210618	3628.404524
% % 20210917	3614.675105
% % 20211217	3598.18784
% % 20220617	3532.411998
% % 20221216	3500.873063
% % 20230616	3443.164984
% % 20231215	3412.413584
% % 20241220	3337.638227
% % 20251219	3288.41536
% % 20261218	3244.171028
% % ];
% % 
% % % Pricing Fwd
% % 
% % rawData = [20210416	3664.251923
% % 20210521	3653.913048
% % 20210618	3645.738563
% % 20210917	3619.435708
% % 20211217	3593.543197
% % 20220617	3542.438139
% % 20221216	3492.143041
% % 20230616	3443.321136
% % 20231215	3395.991806
% % 20241220	3305.239138
% % 20251219	3223.606062
% % 20261218	3148.731289
% % ];
% 
% for idxhong=1:size(rawData,1)
%     rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
% end
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'EUR','asset2ForwardCurve'});
% forwardCurveMktData = MktData(mktData);
% 
% % forwardCurveMktData.params('useInterpolatedForward') = true;
% forwardCurveMktData.params('useInterpolatedForward') = false;
% 
% 
% asset2AllMktData('forwardCurve') = forwardCurveMktData;

%% repoRate Curve For asset2
%% BootStrapping repo Rate from market Fwd Start
% asset2EQ is a dummy model that fit market fwd into and save the result into repoRateCurve
modelParamsDummy= containers.Map({'modelName'},{'EQ'});
asset2EQ = EQModel(Model(asset2AllMktData,modelParamsDummy));

% bootStrapParams = struct;
% bootStrapParams.forwardCurve = asset2AllMktData('forwardCurve');
% outrepoRate = asset2EQ.BootStrapRepoRate(bootStrapParams);
% rawData = outrepoRate.repoRateCurveRawData;

rawData = [8	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'EUR','asset2RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
asset2AllMktData('repoRateCurve') = repoRateCurveMktData;
aaa = 1.0;
%% BootStrapping repo Rate from market Fwd End


% %% repoRate Curve For asset2
% 
% rawData = [0	0.0
% ];
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'EUR','asset2RepoRateCurve'});
% repoRateCurveMktData = MktData(mktData);
% asset2AllMktData('repoRateCurve') = repoRateCurveMktData;

%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName'},{'Black'});
asset2Black = EQBlack(EQModel(Model(asset2AllMktData,modelParams))); 
maturities = asset2Black.volSurface.maturities;
strikes =    asset2Black.volSurface.strikes;
impVolSurface = asset2Black.volSurface.vol;
spot = asset2Black.spot.params('rawData');
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

for i=1:length(maturities)
    fwd(i) = asset2Black.Fwd(spot,expiry(i));
    fwdValues(i) = asset2Black.Fwd(spot,expiry(i));
    df(i) = asset2Black.zeroCurve.DF(expiry(i)/365.0);
    dfValues(i) = asset2Black.zeroCurve.DF(expiry(i)/365.0);
    fwdFactor(i) = asset2Black.FwdFactor(expiry(i));
    for j=1:length(strikes)
        fwdMoneyness(i,j) = strikes(j)/fwdFactor(i);
%         fwdMoneyness(i,j) = strikes(j);
        blackVolInput(i,j) = impVolSurface(expiry(i),strikes(j));
        if fwdMoneyness(i,j) <= 1
%   blackOTMPrices2 : the usual black formula
%   blackOTMPrice   : Peter Jackel's Let's be rational
            blackOTMPrices2(i,j) = asset2Black.BlackVanilla(expiry(i),expiry(i),spot,fwd(i)*strikes(j),impVolSurface(expiry(i),strikes(j)),'P')/fwd(i)/df(i);
            blackOTMPrices(i,j) = blackPrice(1,fwd(i)*strikes(j)/fwd(i),impVolSurface(expiry(i),strikes(j)),expiry(i)/365.0,-1);
        else
            blackOTMPrices2(i,j) = asset2Black.BlackVanilla(expiry(i),expiry(i),spot,fwd(i)*strikes(j),impVolSurface(expiry(i),strikes(j)),'C')/fwd(i)/df(i);
            blackOTMPrices(i,j) = blackPrice(1,fwd(i)*strikes(j)/fwd(i),impVolSurface(expiry(i),strikes(j)),expiry(i)/365.0,1);
        end
    end
 end


%% create a DupireModel with initial modelParamsDupire
%Initialize local volatility
asset2ImpliedVolSurface =asset2Black.volSurface;

localVolSurfaceDupire.expiry = maturities;
localVolSurfaceDupire.strike = strikes;
localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
% localVolSurfaceDupire.strikeType = 'fwdMoneyness';
localVolSurfaceDupire.strikeType = 'spotMoneyness';

% localVolSurfaceDupire.strikeType = 'fixedStrike';
modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurfaceDupire,'Dupire'});
%% SS SABR vol calibration setting

modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

modelParamsDupire('NMC') = NMCTest;

modelParamsDupire('useQuasiRandomYN') = 'false';

% optimizer setting

modelParamsDupire('tol') = 0.0001;
modelParamsDupire('maxIter') = 1000;

modelParamsDupire('UseLetsBeRational') = 'YES';
modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

% modelParamsDupire('slope') = 1.0;
modelParamsDupire('slope') = 0.5;

slopeParams = str2double(keywordsMap('slope'));
modelParamsDupire('slope') = slopeParams(2);
%% SS SABR params

modelParamsDupire('alpha_0') = 0.199;
modelParamsDupire('alpha_inf') = 0.224;
modelParamsDupire('rho')   = -0.755;
modelParamsDupire('nu') = 1.009;
modelParamsDupire('beta') = 1;
modelParamsDupire('lambda') = 0.7;
modelParamsDupire('omega')   = 0.6;
modelParamsDupire('time_shift') = 0.1667;



modelParamsDupire('lb') = [1.00E-08
1.00E-08
-0.9999
1.00E-08
% 1.00E-08
% 1.00E-08
% 1.00E-08
];

%alpah_0
%alpha_inf
%rho
%nu
%lambda
%omega

modelParamsDupire('ub') = [100
100
0.9999
100
% 100.0
% 100
% 100
];


calibratedParams = [0.1542
0.1939
-0.7155
1.0173
];

modelParamsDupire('alpha_0') = 0.25;
modelParamsDupire('alpha_inf') = 0.25;
modelParamsDupire('rho')   = -0.6;
modelParamsDupire('nu') = 0.9;


sabr_alpha_0 = str2double(keywordsMap('alpha_0'));
sabr_alpha_inf = str2double(keywordsMap('alpha_inf'));
sabr_rho = str2double(keywordsMap('rho'));
sabr_nu = str2double(keywordsMap('nu'));

sabr_params = [sabr_alpha_0';sabr_alpha_inf';sabr_rho';sabr_nu'];
calibratedParams = sabr_params(:,2);

modelParamsDupire('alpha_0') = calibratedParams(1);
modelParamsDupire('alpha_inf') = calibratedParams(2);
modelParamsDupire('rho')   = calibratedParams(3);
modelParamsDupire('nu') = calibratedParams(4);


modelParamsDupire('calibrationErrorType') = 'impVolError';
% modelParamsDupire('calibrationErrorType') = 'priceError';

modelParamsDupire('reference') = 4238.38;

reference = str2double(keywordsMap('reference'));
modelParamsDupire('reference') = reference(2);

% modelParamsDupire('reference') = 3681.04
% asset2Dupire = EQCOMDupireSSSABR(EQModel(Model(asset2AllMktData,modelParamsDupire)));
asset2Dupire = EQCOMDupireSSSABRRaw(EQModel(Model(asset2AllMktData,modelParamsDupire)));
% asset2Dupire = EQCOMDupireSSSABRRawNewLV(EQModel(Model(asset2AllMktData,modelParamsDupire)));

% out4 = asset2Dupire.CalibrateToVolSurfaceSSSABR(asset2Black,'global4');
out4 = asset2Dupire.CalibrateToVolSurfaceSSSABR(asset2Black,'justCalculateError');

aaa = 1.0;
%% localVolSurfaceParams
atmVols = [30	0.2376
91	0.2197
182	0.212
365	0.2073
730	0.2049
1095	0.2037
];

volTenor = keywordsMap('VolTenor');
atmVolsData = str2double(keywordsMap('Vol'));

atmVols(1:length(volTenor),2) = atmVolsData(:,2);
atmVols(length(volTenor):end,2) = atmVolsData(end,2);


refExpiryH = localVolSurfaceDupire.expiry;
settleFutures = zeros(length(refExpiryH),1);
settleFuturesRef = zeros(length(refExpiryH),1);

referenceSpot = asset2Dupire.modelParams('reference');

spotRawData = asset2Dupire.spot.params('rawData');
for idxhh=1:length(refExpiryH)
%     settleFutures(idxhh) = asset2Dupire.FwdWithoutSpot(refExpiryH(idxhh));
    settleFutures(idxhh) = spotRawData;
%     settleFuturesRef(idxhh) = asset2Dupire.Fwd(referenceSpot,refExpiryH(idxhh));
    settleFuturesRef(idxhh) = modelParamsDupire('reference');
end

modelParamsDupire('settleFutures') = settleFutures;
modelParamsDupire('settleFuturesRef') = settleFuturesRef;
modelParamsDupire('atmVols') = atmVols;

asset2Dupire.settleFutures = settleFutures;
asset2Dupire.settleFuturesRef = settleFuturesRef;
asset2Dupire.atmVols = atmVols;

%% fixed Strike info end
% modelParamsDupire('generateLocalVolFlag') = true;


% useSSSABRAutoCallScheduleFlag = true;
% outLocalVolSS_asset2 = asset2Dupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset2_KYJ_FwdMN = asset2Dupire.GenerateLocalVolSS_KYJ_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset2_KYJ_FwdMN_FwdFactor = asset2Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

outLocalVolSS_asset2_KYJ_FwdMN_FwdFactor = asset2Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset2_KYJ = asset2Dupire.GenerateLocalVolSS_KYJ(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset2_Qlib = asset2Dupire.GenerateLocalVolSS_Qlib(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset2_Qlib_FwdMN = asset2Dupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);


aaa = 1.0;
%% asset2 Model Building End

%% asset3 Model Building Start
%% zeroCurve

%20220331
rawData = [0.002739726	0.010220617
0.083333333	0.011163116
0.166666667	0.013374114
0.25	0.015740807
0.5	0.020736881
0.75	0.023127766
1	0.025376856
1.5	0.02838266
2	0.031353355
2.5	0.031897301
3	0.032372263
3.5	0.032112565
4	0.031831291
4.5	0.031461729
5	0.031088646
5.5	0.030897884
6	0.030713321
6.5	0.030528757
7	0.030344194
];

rfz = str2double(keywordsMap('RFZ'));
rawData = rfz(:,5:6);


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','USDZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
asset3AllMktData =containers.Map('zeroCurve',zeroCurve);

%% implied VolSurface For asset3

% % marking vol KYJ
rawData = [0	0.5	0.6	0.7	0.8	0.9	1	1.1	1.2	1.3	1.4	1.5
20210320	0.715449924	0.619913577	0.534507853	0.45605482	0.382639462	0.314114896	0.256149669	0.226746229	0.228914762	0.244116304	0.262768918
20210520	0.659176657	0.574885105	0.499719908	0.430955777	0.367024435	0.307829901	0.257115044	0.226361114	0.221382002	0.230618384	0.244766197
20210819	0.560947225	0.495542515	0.437547212	0.384942068	0.336633301	0.292504498	0.254155794	0.226011461	0.212573018	0.211655842	0.217498243
20220218	0.503496233	0.448532495	0.399997776	0.356230133	0.316343314	0.280180002	0.248604694	0.223900474	0.2087973	0.203327393	0.204498882
20230218	0.464727218	0.416471036	0.373995844	0.33585701	0.301285223	0.270097858	0.242821279	0.220844254	0.205915624	0.19849488	0.197003744
20240218	0.435775704	0.392425448	0.354372138	0.320322454	0.289582626	0.261948089	0.237742019	0.217865615	0.20349337	0.195127719	0.19193289
];

for idxhong=2:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),shiftedDate);
end


    
mktData = containers.Map({'rawData','asOfDate','moneyNessType','ccy','name'}, ...
                        {rawData,valueDate,'deltaFwdMoneyNess','USD','asset3VolSurface'});
impliedVolSurfaceMktData = MktData(mktData);
impliedVolSurface = EQVolSurface(impliedVolSurfaceMktData);

impliedVolSurface.params('moneyNessType') = 'fixedStrike';

asset3AllMktData('impliedVolSurface') = impliedVolSurface;

%% dividend CurveFor asset3
rawData = [20220331	0.0
];

for idxhong= 1:size(rawData,1)
    rawData(idxhong,1) = DateDiff(H_Date(rawData(idxhong,1)),valueDateH);
end

% rawData = [0	0.0144
% ];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','asset3DividendCurve'});
                    
mktData('contiDivYld') = 0.0166;

divYield = str2double(keywordsMap('DivYield'));

mktData('contiDivYld') = divYield(3);

mktData('divdCutoffDays') = 0;



dividendCurveMktData = MktData(mktData);

dividendCurveMktData.params('dividendType') = 'dividendYieldAndDiscreteDivdRatio';

% dividendCurveMktData.params('dividendType') = 'dividendYield';

asset3AllMktData('dividendCurve') = dividendCurveMktData;

%% repoRate Curve For asset3

rawData = [14	0.0
];
mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','asset3RepoRateCurve'});
repoRateCurveMktData = MktData(mktData);
asset3AllMktData('repoRateCurve') = repoRateCurveMktData;

%% spot For asset3
rawData = 3970.15;

S = str2double(keywordsMap('S'));
rawData = S(3);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'USD','asset3Spot'});
spotMktData = MktData(mktData);
asset3AllMktData('spot') = spotMktData;


%% create  an EQBlackModel to calculate market PlainVanilla Options
modelParams= containers.Map({'modelName'},{'Black'});
asset3Black = EQBlack(EQModel(Model(asset3AllMktData,modelParams))); 
maturities = asset3Black.volSurface.maturities;
strikes =    asset3Black.volSurface.strikes;
impVolSurface = asset3Black.volSurface.vol;
spot = asset3Black.spot.params('rawData');
expiry = maturities;
fwdFactor = zeros(length(maturities),1);
fwdValues = zeros(length(maturities),1);
dfValues = zeros(length(maturities),1);
fwdMoneyness = zeros(length(maturities),length(strikes));
blackVolInput = zeros(length(maturities),length(strikes));

%% create a DupireModel with initial modelParamsDupire
%Initialize local volatility
asset3ImpliedVolSurface =asset3Black.volSurface;

localVolSurfaceDupire.expiry = maturities;
localVolSurfaceDupire.strike = strikes;
localVolSurfaceDupire.volProxy = zeros(length(maturities),length(strikes));
% localVolSurfaceDupire.strikeType = 'fwdMoneyness';
localVolSurfaceDupire.strikeType = 'spotMoneyness';

% localVolSurfaceDupire.strikeType = 'fixedStrike';
modelParamsDupire= containers.Map({'localVolSurface','modelName'}, ...
                        {localVolSurfaceDupire,'Dupire'});
%% SS SABR vol calibration setting

modelParamsDupire('mcOneTimeStep') = mcOneTimeStepTest;

modelParamsDupire('NMC') = NMCTest;

modelParamsDupire('useQuasiRandomYN') = 'false';

% optimizer setting

modelParamsDupire('tol') = 0.0001;
modelParamsDupire('maxIter') = 1000;

modelParamsDupire('UseLetsBeRational') = 'YES';
modelParamsDupire('LocalVolInterpol') = 'matlabLinearMarketFwdMoneyness';

% modelParamsDupire('slope') = 1.0;
modelParamsDupire('slope') = 1.2;

slopeParams = str2double(keywordsMap('slope'));
modelParamsDupire('slope') = slopeParams(3);

%% SS SABR params

modelParamsDupire('alpha_0') = 0.203;
modelParamsDupire('alpha_inf') = 0.249;
modelParamsDupire('rho')   =-0.742;
modelParamsDupire('nu') = 1.083;
modelParamsDupire('beta') = 1;
modelParamsDupire('lambda') = 0.7;
modelParamsDupire('omega')   = 0.6;
modelParamsDupire('time_shift') = 0.1667;



modelParamsDupire('lb') = [1.00E-08
1.00E-08
-0.9999
1.00E-08
% 1.00E-08
% 1.00E-08
% 1.00E-08
];

%alpah_0
%alpha_inf
%rho
%nu
%lambda
%omega

modelParamsDupire('ub') = [100
100
0.9999
100
% 100.0
% 100
% 100
];


calibratedParams = [0.1677
0.2006
-0.7375
0.9245
% 1.034002631
% 0.630720371
% 0.462011155
];

sabr_alpha_0 = str2double(keywordsMap('alpha_0'));
sabr_alpha_inf = str2double(keywordsMap('alpha_inf'));
sabr_rho = str2double(keywordsMap('rho'));
sabr_nu = str2double(keywordsMap('nu'));

sabr_params = [sabr_alpha_0';sabr_alpha_inf';sabr_rho';sabr_nu'];
calibratedParams = sabr_params(:,3);

modelParamsDupire('alpha_0') = calibratedParams(1);
modelParamsDupire('alpha_inf') = calibratedParams(2);
modelParamsDupire('rho')   = calibratedParams(3);
modelParamsDupire('nu') = calibratedParams(4);


modelParamsDupire('calibrationErrorType') = 'impVolError';
% modelParamsDupire('calibrationErrorType') = 'priceError';

modelParamsDupire('reference') = 3970.15;

reference = str2double(keywordsMap('reference'));
modelParamsDupire('reference') = reference(3);

% asset3Dupire = EQCOMDupireSSSABR(EQModel(Model(asset3AllMktData,modelParamsDupire)));
asset3Dupire = EQCOMDupireSSSABRRaw(EQModel(Model(asset3AllMktData,modelParamsDupire)));
% asset3Dupire = EQCOMDupireSSSABRRawNewLV(EQModel(Model(asset3AllMktData,modelParamsDupire)));



%% localVolSurfaceParams
atmVols = [30	0.2468
91	0.2387
182	0.2304
365	0.226
730	0.2239
1095	0.2222
];

volTenor = keywordsMap('VolTenor');
atmVolsData = str2double(keywordsMap('Vol'));

atmVols(1:length(volTenor),2) = atmVolsData(:,3);
atmVols(length(volTenor):end,2) = atmVolsData(end,3);


refExpiryH = localVolSurfaceDupire.expiry;
settleFutures = zeros(length(refExpiryH),1);
settleFuturesRef = zeros(length(refExpiryH),1);

referenceSpot = asset3Dupire.modelParams('reference');

spotRawData = asset3Dupire.spot.params('rawData');
for idxhh=1:length(refExpiryH)
%     settleFutures(idxhh) = asset3Dupire.FwdWithoutSpot(refExpiryH(idxhh));
    settleFutures(idxhh) = spotRawData;
%     settleFuturesRef(idxhh) = asset3Dupire.Fwd(referenceSpot,refExpiryH(idxhh));
    settleFuturesRef(idxhh) = modelParamsDupire('reference');
end

modelParamsDupire('settleFutures') = settleFutures;
modelParamsDupire('settleFuturesRef') = settleFuturesRef;
modelParamsDupire('atmVols') = atmVols;

asset3Dupire.settleFutures = settleFutures;
asset3Dupire.settleFuturesRef = settleFuturesRef;
asset3Dupire.atmVols = atmVols;

out4 = asset3Dupire.CalibrateToVolSurfaceSSSABR(asset3Black,'justCalculateError');
%% fixed Strike info end
% modelParamsDupire('generateLocalVolFlag') = true;


% useSSSABRAutoCallScheduleFlag = true;
% outLocalVolSS_asset3 = asset3Dupire.GenerateLocalVolSS(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset3_KYJ_FwdMN = asset3Dupire.GenerateLocalVolSS_KYJ_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset3_KYJ_FwdMN_FwdFactor = asset3Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactorOrg(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
outLocalVolSS_asset3_KYJ_FwdMN_FwdFactor = asset3Dupire.GenerateLocalVolSS_KYJ_FwdMN_FwdFactor(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset3_KYJ = asset3Dupire.GenerateLocalVolSS_KYJ(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
% outLocalVolSS_asset3_Qlib = asset3Dupire.GenerateLocalVolSS_Qlib(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);

% outLocalVolSS_asset3_Qlib_FwdMN = asset3Dupire.GenerateLocalVolSS_Qlib_FwdMN(valueDateH,stepdownParams,useSSSABRAutoCallScheduleFlag);
aaa = 1.0;
%% asset3 Model Building End
%% 111111111111111111111111111111111111111111111

%% KRW deterministic forward IR model building start
rawData = [0.002739726	0.03427839
0.019178082	0.034336757
0.083333333	0.034564547
0.166666667	0.034860432
0.25	0.035156317
0.5	0.035585383
0.75	0.036087616
1	0.036388111
1.25	0.036903274
1.5	0.037418437
1.75	0.037940003
2	0.03846157
2.25	0.038273618
2.5	0.038085666
2.75	0.037877727
3	0.037669789
3.25	0.037694695
3.5	0.0377196
3.75	0.037749724
4	0.037779847
4.25	0.037809971
4.5	0.037840094
4.75	0.037870218
5	0.037900341
];

zcy = str2double(keywordsMap('ZCY'));
rawData = zcy(:,1:2);

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

%% create  a dummy IR BlackModel to discount 
modelParams= containers.Map({'modelName'},{'Fwd'});
KRWFwd = IRDetModel(IRModel(Model(KRWAllMktData,modelParams))); 
%% KRW deterministic forward IR model building end

%% multiAssetModel buildingStart
multiAssetMktData = containers.Map({'asset1','asset2','asset3','KRW'}, ...
                        {asset1AllMktData,asset2AllMktData,asset3AllMktData,KRWAllMktData});
                    
% multiAssetMktData = containers.Map({'asset3','KRW'}, ...
%                         {asset3AllMktData,KRWAllMktData});

% correlationMatrix = ones(1,1);
% correlationMatrix_short = 1.0;
% correlationMatrix_long = 1.0;

%shortTermCorr

correlationMatrix_short=[
1	0.357	0.5838
0.357	1	0.2898
0.5838	0.2898	1
];

correlationMatrix_short = str2double(keywordsMap('ShortTermCorr'));


%longTermCorr

correlationMatrix_long=[
1	0.4212	0.6594
0.4212	1	0.2742
0.6594	0.2742	1
];

correlationMatrix_long = str2double(keywordsMap('LongTermCorr'));

multiAssetMktData('correlationMatrix_short') = correlationMatrix_short;
multiAssetMktData('correlationMatrix_long') = correlationMatrix_long;

modelNameMap = containers.Map({'asset1','asset2','asset3','KRW'},{asset1Dupire,asset2Dupire,asset3Dupire,KRWFwd});

% modelNameMap = containers.Map({'asset3','KRW'},{asset3Dupire,KRWFwd});

modelParams= containers.Map({'modelNameMap'},{modelNameMap});

% modelParams('stochasticModelNames') = {'asset1','asset3','DAX'};
% modelParams('stochasticModelNames') = {'asset1','asset2','asset3'};
% modelParams('stochasticModelNames') = {'asset2','asset1','asset3'};
modelParams('stochasticModelNames') = {'asset1','asset2','asset3'};

% modelParams('stochasticModelNames') = {'asset3'};

%% LocalCorr Start
modelParams('correlationMatrix_short') = correlationMatrix_short;
modelParams('correlationMatrix_long') = correlationMatrix_long;
modelParams('lambdaCorr') = 1.0;
% modelParams('corrAlpha') = 02;
modelParams('corrAlpha') = 0.0;
% modelParams('alphaBarrier') = stepdownParams.params('lowerBarrierOrig');
modelParams('alphaBarrier') = 0.0;

%% LocalCorr End


modelParams('refModel') = KRWFwd;

% modelParams('useQuasiRandomYN') = 'true';
modelParams('useQuasiRandomYN') = 'false';
modelParams('computeRealizedCorrelationYN') = 'false';
% modelParams('computeRealizedCorrelationYN') = 'true';
% modelParams('correlationBrigoGamma') = 0.967;

%% Target Curve Curve Correlation fitting end
SX5EDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ3FLocalCorr(multiAssetMktData,modelParams);
%% TermCorr Generate
outTermCorr = SX5EDupireHSCEIDupireasset3DupireKRWFwd.GenerateTermCorr(valueDateH,stepdownParams);
% pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
pricingOutGF = SX5EDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced_ohVec(valueDateH,stepdownParams);

slopeSimul.value(idx,idxSlope) = pricingOutGF.unhitValue.npv;

% % % asset3DupireDAXDupireKRWFwd = MultiAssetModel(multiAssetMktData,modelParams);
% % if strcmp(use2FFlag,'YES')
% % %     HSCEIasset3DupireDAXDupireKRWFwd = MultiAssetModelEQ1FSSCom2FCom2F(multiAssetMktData,modelParams);
% % %     HSCEIasset3DupireDAXDupireKRWFwd = MultiAssetModelEQ1FSSCom2FCom2FQuanto(multiAssetMktData,modelParams);
% % %     HSCEIasset3DupireDAXDupireKRWFwd = MultiAssetModelEQ1FSSCom2FCom2FQuantoTermCorr(multiAssetMktData,modelParams);
% % %     DAXDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ1FSSCom2FCom2FQuantoLocalCorr(multiAssetMktData,modelParams);
% %     DAXDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ3FLocalCorr(multiAssetMktData,modelParams);
% %     
% % else
% %     DAXDupireHSCEIDupireasset3DupireKRWFwd = MultiAssetModelEQ3FLocalCorr(multiAssetMktData,modelParams);
% % end
% 
% %% TermCorr Generate
% 
% outTermCorr = DAXDupireHSCEIDupireasset3DupireKRWFwd.GenerateTermCorr(valueDateH,stepdownParams);
% %% Pricing Stepdown 3S
% 
% tic
% 
% if strcmp(use2FFlag,'YES')
%     if strcmp(useReducedFutures,'NO')
%         pricingOutGF = DAXDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
%     else
%         pricingOutGF = DAXDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
% %         pricingOutGF = HSCEIasset3DupireDAXDupireKRWFwd.computeStepdownNSMCReducedSerial(valueDateH,stepdownParams);
% %         pricingOutGF = DAXDupire.computeStepdown1SMCReduced_Square(valueDateH,stepdownParams);
% %         computeStepdown1SMCReduced_Square
%         
% %         pricingOutGF = DAXDupire.computeStepdown1SMCReducedSimpleQuantoAdj(valueDateH,stepdownParams);
% %         pricingOutGF = DAXDupire.computeStepdown1SMCReduced(valueDateH,stepdownParams);
%     end
%     
% else
%     if strcmp(useReducedFutures,'NO')
%         pricingOutGF = DAXDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
%     else
%         pricingOutGF = DAXDupireHSCEIDupireasset3DupireKRWFwd.computeStepdownNSMCReduced(valueDateH,stepdownParams);
% %         pricingOutGF = DAXDupire.computeStepdown1SMCReduced(valueDateH,stepdownParams);
%     end
% end
% 
% toc
% 
% disp([num2str(idx),'pricing done'])
% 
% slopeSimul.value(idx,idxSlope) = pricingOutGF.unhitValue.npv;
% 
% %asset3
% slopeSimul.asset3_rmse(idx,idxSlope) = out_asset3.rmseTotal;
% slopeSimul.asset3_fwdVolMseTotal(idx,idxSlope) = out_asset3.fwdVolMseTotal;
% slopeSimul.asset3_arbitrageCheckSum(idx,idxSlope) = out_asset3.arbitrageCheckSum;
% slopeSimul.asset3_volProxy{idx,idxSlope} = out_asset3.volProxy;
% slopeSimul.asset3_volError{idx,idxSlope} = out_asset3.volError;
% slopeSimul.asset3_fwdMoneyness{idx,idxSlope} = out_asset3.fwdMoneyness;
% slopeSimul.asset3_modelImpVol{idx,idxSlope} = out_asset3.modelImpVol;
% slopeSimul.asset3_nIter{idx,idxSlope} = out_asset3.nIter;
% 
% %DAX
% slopeSimul.DAX_rmse(idx,idxSlope) = out_DAX.rmseTotal;
% slopeSimul.DAX_fwdVolMseTotal(idx,idxSlope) = out_DAX.fwdVolMseTotal;
% slopeSimul.DAX_arbitrageCheckSum(idx,idxSlope) = out_DAX.arbitrageCheckSum;
% slopeSimul.DAX_volProxy{idx,idxSlope} = out_DAX.volProxy;
% slopeSimul.DAX_volError{idx,idxSlope} = out_DAX.volError;
% slopeSimul.DAX_fwdMoneyness{idx,idxSlope} = out_DAX.fwdMoneyness;
% slopeSimul.DAX_modelImpVol{idx,idxSlope} = out_DAX.modelImpVol;
% slopeSimul.DAX_nIter{idx,idxSlope} = out_DAX.nIter;

        

end

if strcmp(simulationAsset,'asset3')
    if strcmp(simulationType,'ParallelDelta')

        slopeSimul.asset3_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 

    elseif strcmp(simulationType,'PointByPointDelta')

        for idxZ =1: numberOfTenor
            slopeSimul.asset3_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
        end

    elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
            || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')

        slopeSimul.asset3_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 

    elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
            || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')

        for idxZ =1: numberOfTenor
            slopeSimul.asset3_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')
        for idxZ =1: numberOfTenor
            for idxZZ = 1:numberOfStrikes
                slopeSimul.asset3_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
                 - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
            end
        end
    else
        aaaa = 1.0;
    %     error('unimplemented');
    end
elseif strcmp(simulationAsset,'DAX')
    if strcmp(simulationType,'ParallelDelta')

        slopeSimul.DAX_delta(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0; 

    elseif strcmp(simulationType,'PointByPointDelta')

        for idxZ =1: numberOfTenor
            slopeSimul.DAX_delta(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0;
        end

    elseif strcmp(vegaSimulationType,'ParallelVega') || strcmp(vegaSimulationType,'ParallelVegaATMSkew') || strcmp(vegaSimulationType,'ParallelVegaATMVariance') || strcmp(vegaSimulationType,'ParallelVegaATMVol') ...
            || strcmp(vegaSimulationType,'ParallelVegaP') || strcmp(vegaSimulationType,'ParallelVegaC') || strcmp(vegaSimulationType,'ParallelVegaVTilda')

        slopeSimul.DAX_vega(1,idxSlope) = (slopeSimul.value(2,idxSlope) - slopeSimul.value(3,idxSlope))/2.0*10.0; 

    elseif strcmp(vegaSimulationType,'TermVega') || strcmp(vegaSimulationType,'TermVegaATMVariance') || strcmp(vegaSimulationType,'TermVegaATMVol') || strcmp(vegaSimulationType,'TermVegaATMSkew') ...
            || strcmp(vegaSimulationType,'TermVegaP') || strcmp(vegaSimulationType,'TermVegaC') || strcmp(vegaSimulationType,'TermVegaVTilda')

        for idxZ =1: numberOfTenor
            slopeSimul.DAX_vega(idxZ,idxSlope) = (slopeSimul.value(2*idxZ,idxSlope) - slopeSimul.value(2*idxZ+1,idxSlope))/2.0*10.0;
        end
    elseif strcmp(vegaSimulationType,'PointByPointVega')
        for idxZ =1: numberOfTenor
            for idxZZ = 1:numberOfStrikes
                slopeSimul.DAX_vega(numberOfStrikes*(idxZ-1)+idxZZ,idxSlope) = (slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+2,idxSlope) ...
                 - slopeSimul.value(2*numberOfStrikes*(idxZ-1)+2*(idxZZ-1)+3,idxSlope))/2.0;
            end
        end
    else
        aaaa = 1.0;
    %     error('unimplemented');
    end
    
end

end


disp('finished!');

bbb= 1.0;



In [ ]:
%test_file_io_20210222_1.m

clc;
clear;
% keywordsList = cell(100,1);
% keywordsValList = cell(100,1);

keywordsMap = containers.Map;

% fileName = 'input.dat';
fileName = 'D:\25572_20210218.dat';
fid = fopen(fileName);

block = 1;
while ~feof(fid)
    tline = fgetl(fid);
    
    % check whether it is #END
    if strcmp(tline,'#END')
        fclose(fid);
        break;
    end
    
    tlineStr = strsplit(tline);
    
    if strcmp(tlineStr,'')
        continue;
    end
    keyword = tlineStr{1,1};
    
    if keyword(1) ~= '#'
       fprintf('%s is not keyword. \n',keyword);
       fclose(fid);
       break;
    end
    
    try
        rowSizeStr = tlineStr{1,2};
        colSizeStr = tlineStr{1,3};
    catch ME
        if strcmp(tlineStr{1,1},'#END')
            fclose(fid);
            break;
        else
            rethrow(ME)
        end
    end
    rowSizeErr = false;
    % check whether rowSizeStr is a string format
    for i=1:length(rowSizeStr)
        if ~isstrprop(rowSizeStr(i),'digit')
            rowSizeErr = true;
            break;
        end
    end
    
    if rowSizeErr
        fprintf('%s has no row size. \n',keyword);
        fclose(fid);
        break;
    end
    
    rowSize = sscanf(rowSizeStr,'%d');
    
    colSizeErr = false;
    % check whether rowSizeStr is a string format
    for i=1:length(colSizeStr)
        if ~isstrprop(colSizeStr(i),'digit')
            colSizeErr = true;
            break;
        end
    end
    
    if colSizeErr
        fprintf('%s has no col size. \n',keyword);
        fclose(fid);
        break;
    end
    colSize = sscanf(colSizeStr,'%d');
    
    if rowSize == 0 || colSize == 0
        fprintf('%s has zero row size or col size. \n',keyword);
        fclose(fid);
        break;
    end
    
%     curMat = cell(rowSize,colSize);
    curMat = cell(colSize,rowSize);
    
    % row & col is wrong definition
    % row means col & col means rows
    % therefore we swap
    
%    for i=1:rowSize
%        for j=1:colSize
%            curMat{i,j} = fgetl(fid);
%        end
%    end
   
    for i=1:rowSize
       for j=1:colSize
           curMat{j,i} = fgetl(fid);
       end
   end
   
   keyword1 = keyword(2:end);
   keywordsMap(keyword1) = curMat;
%    if block ==1
%        keywordsMap = containers.Map(keyword1,curMat);
%    else
%        keywordsMap(keyword1) = curMat;
%    end
   
   block = block +1;
    
end
aaa = 0.0;

In [ ]:
%test_CRSRFRCurveBootStrap_USDKRW_20211029_20221121.m

clc
clear
tic

% valueDate = '2021-10-29';
valueDate = '2021-11-02';


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;

% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);
%% Calendars Start
%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end

%% KRSK Calendar
KRSK.name = 'KRSK';
KRSK.holidaysList = {'2021-01-01'
'2021-02-11'
'2021-02-12'
'2021-03-01'
'2021-05-05'
'2021-05-19'
'2021-08-16'
'2021-09-20'
'2021-09-21'
'2021-09-22'
'2021-10-04'
'2021-10-11'
'2021-12-31'
'2022-01-31'
'2022-02-01'
'2022-02-02'
'2022-03-01'
'2022-03-09'
'2022-05-05'
'2022-06-01'
'2022-06-06'
'2022-08-15'
'2022-09-09'
'2022-09-12'
'2022-10-03'
'2022-10-10'
'2022-12-30'
'2023-01-23'
'2023-01-24'
'2023-03-01'
'2023-05-01'
'2023-05-05'
'2023-06-06'
'2023-08-15'
'2023-09-28'
'2023-09-29'
'2023-10-03'
'2023-10-09'
'2023-12-25'
'2023-12-29'
'2024-01-01'
'2024-02-09'
'2024-03-01'
'2024-04-10'
'2024-05-01'
'2024-05-06'
'2024-05-15'
'2024-06-06'
'2024-08-15'
'2024-09-16'
'2024-09-17'
'2024-09-18'
'2024-10-03'
'2024-10-09'
'2024-12-25'
'2024-12-31'
'2025-01-01'
'2025-01-28'
'2025-01-29'
'2025-01-30'
'2025-05-01'
'2025-05-05'
'2025-05-06'
'2025-06-06'
'2025-08-15'
'2025-10-03'
'2025-10-06'
'2025-10-07'
'2025-10-08'
'2025-10-09'
'2025-12-25'
'2025-12-31'
'2026-01-01'
'2026-02-16'
'2026-02-17'
'2026-02-18'
'2026-05-01'
'2026-05-05'
'2026-09-24'
'2026-09-25'
'2026-10-09'
'2026-12-25'
'2026-12-31'
'2027-01-01'
'2027-02-08'
'2027-02-09'
'2027-03-01'
'2027-03-03'
'2027-05-05'
'2027-05-13'
'2027-09-14'
'2027-09-15'
'2027-09-16'
'2027-12-31'
'2028-01-26'
'2028-01-27'
'2028-01-28'
'2028-03-01'
'2028-05-01'
'2028-05-02'
'2028-05-05'
'2028-06-06'
'2028-08-15'
'2028-10-02'
'2028-10-03'
'2028-10-04'
'2028-10-05'
'2028-10-09'
'2028-12-25'
'2028-12-29'
'2029-01-01'
'2029-02-12'
'2029-02-13'
'2029-02-14'
'2029-03-01'
'2029-05-01'
'2029-05-07'
'2029-06-06'
'2029-08-15'
'2029-09-21'
'2029-10-03'
'2029-10-09'
'2029-12-25'
'2029-12-31'
'2030-01-01'
'2030-02-04'
'2030-03-01'
'2030-05-01'
'2030-05-06'
'2030-05-09'
'2030-06-06'
'2030-08-15'
'2030-09-11'
'2030-09-12'
'2030-09-13'
'2030-10-03'
'2030-10-09'
'2030-12-25'
'2030-12-31'
'2031-01-01'
'2031-01-22'
'2031-01-23'
'2031-01-24'
'2031-05-01'
'2031-05-05'
'2031-05-28'
'2031-06-06'
'2031-08-15'
'2031-09-30'
'2031-10-01'
'2031-10-02'
'2031-10-03'
'2031-10-09'
'2031-12-25'
'2031-12-31'
'2032-01-01'
'2032-02-10'
'2032-02-11'
'2032-02-12'
'2032-03-01'
'2032-05-05'
'2032-05-28'
'2032-09-20'
'2032-12-31'
'2033-01-31'
'2033-02-01'
'2033-03-01'
'2033-05-05'
'2033-05-06'
'2033-06-06'
'2033-08-15'
'2033-09-07'
'2033-09-08'
'2033-09-09'
'2033-10-03'
'2033-12-30'
'2034-02-20'
'2034-03-01'
'2034-05-01'
'2034-05-05'
'2034-05-25'
'2034-06-06'
'2034-08-15'
'2034-09-26'
'2034-09-27'
'2034-09-28'
'2034-10-03'
'2034-10-09'
'2034-12-25'
'2034-12-29'
'2035-01-01'
'2035-02-07'
'2035-02-08'
'2035-02-09'
'2035-03-01'
'2035-05-01'
'2035-05-07'
'2035-05-15'
'2035-06-06'
'2035-08-15'
'2035-09-17'
'2035-09-18'
'2035-10-03'
'2035-10-09'
'2035-12-25'
'2035-12-31'
'2036-01-01'
'2036-01-28'
'2036-01-29'
'2036-01-30'
'2036-05-01'
'2036-05-05'
'2036-06-06'
'2036-08-15'
'2036-10-03'
'2036-10-06'
'2036-10-07'
'2036-10-09'
'2036-12-25'
'2036-12-31'
'2037-01-01'
'2037-02-16'
'2037-02-17'
'2037-05-01'
'2037-05-05'
'2037-05-22'
'2037-09-23'
'2037-09-24'
'2037-09-25'
'2037-10-09'
'2037-12-25'
'2037-12-31'
'2038-01-01'
'2038-02-03'
'2038-02-04'
'2038-02-05'
'2038-03-01'
'2038-05-05'
'2038-05-11'
'2038-09-13'
'2038-09-14'
'2038-09-15'
'2038-12-31'
'2039-01-24'
'2039-01-25'
'2039-01-26'
'2039-03-01'
'2039-05-05'
'2039-06-06'
'2039-08-15'
'2039-10-03'
'2039-10-04'
'2039-10-05'
'2039-12-30'
'2040-02-13'
'2040-02-14'
'2040-03-01'
'2040-05-01'
'2040-05-07'
'2040-05-18'
'2040-06-06'
'2040-08-15'
'2040-09-19'
'2040-09-20'
'2040-09-21'
'2040-10-03'
'2040-10-09'
'2040-12-25'
'2040-12-31'
'2041-01-01'
'2041-01-31'
'2041-02-01'
'2041-03-01'
'2041-05-01'
'2041-05-06'
'2041-05-07'
'2041-06-06'
'2041-08-15'
'2041-09-09'
'2041-09-10'
'2041-09-11'
'2041-10-03'
'2041-10-09'
'2041-12-25'
'2041-12-31'
'2042-01-01'
'2042-01-21'
'2042-01-22'
'2042-01-23'
'2042-05-01'
'2042-05-05'
'2042-05-26'
'2042-06-06'
'2042-08-15'
'2042-09-29'
'2042-09-30'
'2042-10-03'
'2042-10-09'
'2042-12-25'
'2042-12-31'
'2043-01-01'
'2043-02-09'
'2043-02-10'
'2043-02-11'
'2043-05-01'
'2043-05-05'
'2043-09-16'
'2043-09-17'
'2043-09-18'
'2043-10-09'
'2043-12-25'
'2043-12-31'
'2044-01-01'
'2044-01-29'
'2044-02-01'
'2044-03-01'
'2044-05-05'
'2044-05-06'
'2044-06-06'
'2044-08-15'
'2044-10-03'
'2044-10-04'
'2044-10-05'
'2044-10-06'
'2044-12-30'
'2045-02-16'
'2045-02-17'
'2045-03-01'
'2045-05-01'
'2045-05-05'
'2045-05-24'
'2045-06-06'
'2045-08-15'
'2045-09-25'
'2045-09-26'
'2045-09-27'
'2045-10-03'
'2045-10-09'
'2045-12-25'
'2045-12-29'
'2046-01-01'
'2046-02-05'
'2046-02-06'
'2046-02-07'
'2046-03-01'
'2046-05-01'
'2046-05-07'
'2046-06-06'
'2046-08-15'
'2046-09-14'
'2046-09-17'
'2046-10-03'
'2046-10-09'
'2046-12-25'
'2046-12-31'
'2047-01-01'
'2047-01-25'
'2047-01-28'
'2047-03-01'
'2047-05-01'
'2047-05-02'
'2047-05-06'
'2047-06-06'
'2047-08-15'
'2047-10-03'
'2047-10-04'
'2047-10-07'
'2047-10-09'
'2047-12-25'
'2047-12-31'
    };

KRSKCalendar = Calendar();
KRSKCalendar.name = KRSK.name;
for i=1:length(KRSK.holidaysList)
    KRSKCalendar.AddHoliday(KRSK.holidaysList{i})
end

%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD IRS Swap Schedule Generation
jointCalendar_USDKRWCRS = JointCalendar({GBENCalendar,USFDCalendar,KRSKCalendar});

    
%% index fixing for SOFR Index start
    tempDate = H_Date('2021-10-29');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.0005});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.0005;
    end
%% index fixing for SOFR_index end

%% index fixing for Libor3M start
    tempDate = H_Date('2021-10-29');
    irIndexFixing_Libor3M = containers.Map({StrDate(tempDate,'str')},{0.00132250});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor3M(StrDate(tempDate,'str')) = 0.001322500;
    end
%% index fixing for Libor3M end

%% index fixing for Libor6M start
    tempDate = H_Date('2021-10-29');
    irIndexFixing_Libor6M = containers.Map({StrDate(tempDate,'str')},{0.002010});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Libor6M(StrDate(tempDate,'str')) = 0.002010;
    end
%% index fixing for Libor6M end

%% import and construct an CRSCurve

dataType= {
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'CrossCurrencySwap'
'CrossCurrencySwap'
'CrossCurrencySwap'
'CrossCurrencySwap'
'CrossCurrencySwap'
'CrossCurrencySwap'
'CrossCurrencySwap'
};

dataTenor = {
'1W'
'1M'
'2M'
'3M'
'6M'
'1Y'
'2Y'
'3Y'
'4Y'
'5Y'
'7Y'
'10Y'
'20Y'
};

% dataRate = {1066.3
% 1065
% 1063.5
% 1059.4
% 0.0101233
% 0.0112578
% 0.01225
% 0.012875
% 0.01345
% 0.0147875
% 0.016775
% };

% dataRate = {
% 0.164999999105930
% 0.412499979138374
% 0.950000077486038
% 1.367499977350230
% 2.132500067353250
% 5.027500033378600
% 0.006700000166893
% 0.007649999856949
% 0.008600000143051
% 0.009500000178814
% 0.010699999928474
% 0.012300000190735
% 0.013299999833107
% };

% dataRate = {
% 0.165000 
% 0.412500 
% 0.950000 
% 1.367500 
% 2.132500 
% 5.027500 
% 0.006700000166893
% 0.007649999856949
% 0.008600000143051
% 0.009500000178814
% 0.010699999928474
% 0.012300000190735
% 0.013299999833107
% };

% exchangeRate = 1176.66497802734;

dataRate = {
0.3125000111758710000
0.8250000029802320000
1.4500000067055200000
2.1149999499321000000
4.4000000953674300000
7.7325000762939500000
0.0114499998092651000
0.0126999998092651000
0.0141000002622604000
0.0151499998569489000
0.0163499999046326000
0.0171499997377396000
0.0181499999761581000
};

exchangeRate = 1168.402465820310000;
% exchangeRate = 1176.7;

%% Construct USDIRS_Libor3M_Curve
%% Construct USDIRS_Libor6M_Curve
%% Construct USDSOFR_Curve

% forward & discount zeroCurve
forwardCurve_Libor6M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_Libor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_Libor3M = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

rawData_SOFR = [
0.00273972602739726000	0.00050692240811557300
0.01917808219178080000	0.00042684542009732000
0.03835616438356160000	0.00042684372058727700
0.05753424657534250000	0.00045117468867678800
0.08219178082191780000	0.00044762371452863000
0.16986301369863000000	0.00046789113611163900
0.25205479452054800000	0.00049424002880757500
0.49589041095890400000	0.00067969796459307900
0.74794520547945200000	0.00114165826034183000
1.00000000000000000000	0.00179297492802996000
2.00000000000000000000	0.00522718328207199000
3.00821917808219000000	0.00767013654656890000
4.00547945205479000000	0.00910672623726694000
5.00273972602740000000	0.01011244928700360000
6.00273972602740000000	0.01103862443946350000
7.00547945205479000000	0.01180604854094490000
8.00547945205479000000	0.01239991781962910000
9.01095890410959000000	0.01287282287143140000
10.00821917808220000000	0.01328245599875750000
12.00821917808220000000	0.01390103300787300000
15.01369863013700000000	0.01449807070118240000
20.01917808219180000000	0.01498834208910890000
25.01643835616440000000	0.01493461540811120000
30.01917808219180000000	0.01471013374153880000
40.02739726027400000000	0.01375248133744370000
50.03287671232880000000	0.01257980355671750000
];

rawData_Libor3M_DC = [
0.2520547945205480	0.00134064151849348000
0.3671232876712330	0.00183950816832919000
0.6164383561643840	0.00215659652533920000
0.8849315068493150	0.00284708939740920000
1.1342465753424700	0.00363664572854453000
1.3643835616438400	0.00449856518146918000
1.6328767123287700	0.00550820156405681000
2.0000000000000000	0.00693124786617120000
3.0082191780821900	0.00967993170426422000
4.0054794520547900	0.01126482193509440000
5.0027397260274000	0.01236610089651140000
6.0027397260274000	0.01334842027599820000
7.0054794520547900	0.01415782682210360000
8.0054794520547900	0.01477877650098960000
9.0109589041095900	0.01528350085890360000
10.0082191780822000	0.01571700684813580000
11.0082191780822000	0.01608591555245660000
12.0082191780822000	0.01636266957896630000
15.0136986301370000	0.01699579858132950000
20.0191780821918000	0.01750077613049380000
25.0164383561644000	0.01746743467492110000
30.0191780821918000	0.01726814261829030000
40.0273972602740000	0.01636892708021210000
50.0328767123288000	0.01523031575161490000
];

rawData_Libor3M_SC = [
0.2520547945205480	0.00134064151849348000
0.3671232876712330	0.00183950816832919000
0.6164383561643840	0.00215659652533920000
0.8849315068493150	0.00284708939740920000
1.1342465753424700	0.00363664572854453000
1.3643835616438400	0.00449856518146918000
1.6328767123287700	0.00550820156405681000
2.0000000000000000	0.00693124786617120000
3.0082191780821900	0.00967993170426422000
4.0054794520547900	0.01126482193509440000
5.0027397260274000	0.01236610089651140000
6.0027397260274000	0.01334842027599820000
7.0054794520547900	0.01415782682210360000
8.0054794520547900	0.01477877650098960000
9.0109589041095900	0.01528350085890360000
10.0082191780822000	0.01571700684813580000
11.0082191780822000	0.01608591555245660000
12.0082191780822000	0.01636266957896630000
15.0136986301370000	0.01699579858132950000
20.0191780821918000	0.01750077613049380000
25.0164383561644000	0.01746743467492110000
30.0191780821918000	0.01726814261829030000
40.0273972602740000	0.01636892708021210000
50.0328767123288000	0.01523031575161490000
];

%new Libor6M Forward Curve zeros DualCurveStripping

rawData_Libor6M_DC = [
0.4986301369863010000	0.0020368819360600100
1.0000000000000000000	0.0035661701502091800
1.4958904109589000000	0.0053347538391876600
2.0000000000000000000	0.0071964749090331500
3.0082191780821900000	0.0103950949672581000
4.0054794520547900000	0.0121888624887890000
5.0027397260274000000	0.0134332016749882000
7.0054794520547900000	0.0153912679119363000
10.0082191780822000000	0.0170658429832061000
12.0082191780822000000	0.0177633966082776000
15.0136986301370000000	0.0184430513618314000
20.0191780821918000000	0.0189994073074511000
25.0164383561644000000	0.0190062071397650000
30.0191780821918000000	0.0188341043359082000
];

% %new Libor6M Forward Curve zeros
% rawData_Libor6M_SC = [
% 0.4986301369863010000	0.0020368819360600100
% 1.0000000000000000000	0.0035661701502091800
% 1.4958904109589000000	0.0053347538391876600
% 2.0000000000000000000	0.0071964749090331500
% 3.0082191780821900000	0.0103950949672581000
% 4.0054794520547900000	0.0121888624887890000
% 5.0027397260274000000	0.0134332016749882000
% 7.0054794520547900000	0.0153912679119363000
% 10.0082191780822000000	0.0170658429832061000
% 12.0082191780822000000	0.0177633966082776000
% 15.0136986301370000000	0.0184430513618314000
% 20.0191780821918000000	0.0189994073074511000
% 25.0164383561644000000	0.0190062071397650000
% 30.0191780821918000000	0.0188341043359082000
% ];

forwardCurve_Libor6M.params('rawData') = rawData_Libor6M_DC;
forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;
% forwardCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

discountCurve_Libor3M.params('rawData') = rawData_Libor3M_SC;

forwardCurve_SOFR.params('rawData') = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;




% dummyCurve2 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

libor6MIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'6M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor6M});

% IborIndexParams_fwd= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','USD',2,GBENCalendar,'Following','Act360',false,forwardCurve_Libor3M});

libor6MIndexParams('forecastTodaysFixing') = true;
libor6MIndexParams('historicalFixings') = irIndexFixing_Libor6M;
        
USD_Libor_6M = IborIndex(libor6MIndexParams);

rateHelpers = {};
        
for i=1:length(dataRate)
    if strcmp(dataType{i},'SwapPoint')
        % we are valuing on settleDate
        % so settlegap(here fixingDays) is equal to 0
        swapPointHelperParams = containers.Map({'exchangeRate','fixingDays','rate','rateTenor'}, ...
                            {exchangeRate,0,dataRate{i},dataTenor{i}});
        
        swapPointHelperParams('exchangeRate') = exchangeRate;
        swapPointHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        swapPointHelperParams('calendar') = jointCalendar_USDKRWCRS;
        swapPointHelperParams('convention') = 'ModifiedFollowing';
%         swapPointHelperParams('foreignDCurve') = discountCurve_Libor3M;
        swapPointHelperParams('foreignDCurve') = discountCurve_SOFR;
        
        swapPointHelper = SwapPointHelper(swapPointHelperParams);
    
        rateHelpers{end+1} = swapPointHelper;
        
    elseif strcmp(dataType{i},'CrossCurrencySwap')
        
        domesticFixedLegTenor = '6M';
        domesticFixedLegConvention = 'ModifiedFollowing';
        domesticFixedLegDayCounter = 'Act365';
        domesticFixedLegCalendar = jointCalendar_USDKRWCRS;
        
        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        spotLag = 0;
        
        crossCurrencySwapHelperParams = containers.Map({'domesticFixedLegTenor','domesticFixedLegConvention','domesticFixedLegDayCounter',...
                                                        'domesticFixedLegCalendar','foreignIndex','foreignDCurve','spotLag','discountCurve'}, ...
                                {domesticFixedLegTenor,domesticFixedLegConvention,domesticFixedLegDayCounter,...,
                                domesticFixedLegCalendar,USD_Libor_6M,discountCurve_SOFR,spotLag,dummyCurve});

        crossCurrencySwapHelperParams('exchangeRate') = exchangeRate;
        crossCurrencySwapHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        crossCurrencySwapHelperParams('rate') = dataRate{i};
        crossCurrencySwapHelperParams('rateTenor') = dataTenor{i};
        crossCurrencySwapHelper = CrossCurrencySwapHelper(crossCurrencySwapHelperParams);
        rateHelpers{end+1} = crossCurrencySwapHelper;
    end
end

% forward & discount zeroCurve
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','discountCurve'},{rateHelpers,dummyCurve1});

tic
yieldCurve = YieldCurveCC(yieldCurveParams);
out = yieldCurve.BootStrap();
toc

KRWDiscountCurve = out.discountCurve;

%% Pricing a new CRS Deal (KRW Fixed vs USD SOFR compounding 3M + Spread) Start

% construct overnightIndex

IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('useRounding') = useRounding;

%% daily ovservation schedule

IborIndexParams('useSimpleTeleScoping') = true;
IborIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;



IborIndexParams('historicalFixings') = irIndexFixing_SOFR;
%% SOFR Rate Index
overnightIndex = IborIndex(IborIndexParams);

referenceDate = valueDateH;
    
% basis swap's swap schedule is generated by
% jointCalendar_USDIRS(EN,FD)

targetTenor={
'1Y'
'2Y'
'3Y'
'4Y'
'5Y'
'7Y'
'10Y'
'20Y'
};

% SOFR 3M
defaultRate = {
0.00839775687498969000
0.00945058455818486000
0.00993716375569923000
0.01097441584765900000
0.01178155014850230000
0.01271357724487280000
0.01331943707266010000
0.01411657818866770000
};

% SOFR 3M + FallbackSpread(0.0026161)
defaultRate = {
0.0110619444340615000
0.0121196962550378000
0.0126075582399920000
0.0136451848485427000
0.0144529722768291000
0.0153852358534958000
0.0159882453404093000
0.0167666988459102000
};


simulNPV = zeros(100,1);
simulFairRate = zeros(100,1);

simulNPV2 = zeros(100,1);
simulFairRate2 = zeros(100,1);

for idxHH = 1:length(targetTenor)
    calcCalendar = jointCalendar_USDKRWCRS;
    rateTenor = targetTenor{idxHH};
    rateData = defaultRate{idxHH};
%     rateData = 0.0;

    % KRW FixedLeg
    domesticFixedLegTenor = '3M';
    domesticFixedLegConvention = 'ModifiedFollowing';
    domesticFixedLegDayCounter = 'Act365';
    domesticFixedLegCalendar = jointCalendar_USDKRWCRS;

    %USD FloatingLeg (SOFR3M + Spread)
    foreignFloatingLegTenor = '3M';
    foreignFloatingLegConvention = 'ModifiedFollowing';
    foreignFloatingLegDayCounter = 'Act360';
    foreignFloatingLegCalendar = jointCalendar_USDKRWCRS;
    foreignIndex = overnightIndex;

    spotLag = 0;
    payLag  = 2;

    payLagNotionalMaturity = 0;


    startDate = calcCalendar.Advance(referenceDate,overnightIndex.convention,...
                                        spotLag,'day',overnightIndex.endOfMonth);

     useEndOfMonth = false;
     if IsEndOfMonth(startDate)
         useEndOfMonth = true;
     end

     endDate = AddTenor(startDate,rateTenor);
     % fixed & floating schedule
     domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFixedLegTenor,calcCalendar,...
                                domesticFixedLegConvention,domesticFixedLegConvention, 'Backward',useEndOfMonth);

     foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignFloatingLegTenor,calcCalendar,...
                                foreignFloatingLegConvention,foreignFloatingLegConvention, 'Backward',useEndOfMonth);

     type = 'Payer';
     domesticNominal = exchangeRate;
     domesticFixedRate = rateData;
     domesticDayCounter = domesticFixedLegDayCounter;

     foreignNominal = 1.0;
     foreignGearing = 1.0;
%      foreignSpread = 0.0;
     foreignSpread = 0.0026161;
     
     foreignDayCounter = foreignFloatingLegDayCounter;

     notionalExchangeType = 'Both';

     crossCurrencySwapRFR = CrossCurrencySwapRFR(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
                                       foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                       foreignIndex,...
                                       notionalExchangeType,spotLag,payLag,payLagNotionalMaturity);

    %  out = crossCurrencySwapNew;

    outInfo = crossCurrencySwapRFR.Calculate(KRWDiscountCurve,discountCurve_SOFR);
    simulNPV(idxHH) = outInfo.NPV;
%     simulFairRate(idxHH) = outInfo.fairRate;
end

bbb = 1.0;

for idxHH = 1:length(targetTenor)
    calcCalendar = jointCalendar_USDKRWCRS;
    rateTenor = targetTenor{idxHH};
%     rateData = defaultRate{idxHH};
    rateData = 0.0;

    % KRW FixedLeg
    domesticFixedLegTenor = '3M';
    domesticFixedLegConvention = 'ModifiedFollowing';
    domesticFixedLegDayCounter = 'Act365';
    domesticFixedLegCalendar = jointCalendar_USDKRWCRS;

    %USD FloatingLeg (SOFR3M + Spread)
    foreignFloatingLegTenor = '3M';
    foreignFloatingLegConvention = 'ModifiedFollowing';
    foreignFloatingLegDayCounter = 'Act360';
    foreignFloatingLegCalendar = jointCalendar_USDKRWCRS;
    foreignIndex = overnightIndex;

    spotLag = 0;
%     payLag  = 2;
    payLag  = 2;

    payLagNotionalMaturity = 0;


    startDate = calcCalendar.Advance(referenceDate,overnightIndex.convention,...
                                        spotLag,'day',overnightIndex.endOfMonth);

     useEndOfMonth = false;
     if IsEndOfMonth(startDate)
         useEndOfMonth = true;
     end

     endDate = AddTenor(startDate,rateTenor);
     % fixed & floating schedule
     domesticSchedule = ScheduleWTermConvention(startDate,endDate,domesticFixedLegTenor,calcCalendar,...
                                domesticFixedLegConvention,domesticFixedLegConvention, 'Backward',useEndOfMonth);

     foreignSchedule = ScheduleWTermConvention(startDate,endDate,foreignFloatingLegTenor,calcCalendar,...
                                foreignFloatingLegConvention,foreignFloatingLegConvention, 'Backward',useEndOfMonth);

     type = 'Payer';
     domesticNominal = exchangeRate;
     domesticFixedRate = rateData;
     domesticDayCounter = domesticFixedLegDayCounter;

     foreignNominal = 1.0;
     foreignGearing = 1.0;
     foreignSpread = 0.0;
%      foreignSpread = 0.0026161;
     
     foreignDayCounter = foreignFloatingLegDayCounter;

     notionalExchangeType = 'Both';

     crossCurrencySwapRFR = CrossCurrencySwapRFR(type,domesticNominal,domesticSchedule,domesticFixedRate,domesticDayCounter, ...
                                       foreignNominal,foreignSchedule,foreignGearing,foreignSpread,foreignDayCounter,...
                                       foreignIndex,...
                                       notionalExchangeType,spotLag,payLag,payLagNotionalMaturity);

    %  out = crossCurrencySwapNew;

    outInfo = crossCurrencySwapRFR.Calculate(KRWDiscountCurve,discountCurve_SOFR);
%     simulNPV(idxHH) = outInfo.NPV;
    simulFairRate2(idxHH) = outInfo.fairRate;
end
aaa = 1.0;


%  vanilla_Floating_OvernightIndex_BasisSwap = Vanilla_Floating_OvernightIndex_BasisSwap(type,nominal,...
%                           floatingLeg1_Schedule,libor3M_Index_fwd,floatingLeg1_Spread,floatingLeg1_DayCounter,spotLag_1,payLag_1, ...
%                           floatingLeg2_Schedule,overnightIndex,floatingLeg2_Spread,floatingLeg2_DayCounter,spotLag_2,payLag_2);

%      vanillaSwap = VanillaOvernightIndexSwap(type,nominal,fixedSchedule,fixedRate,fixedDayCounter, ...
%                               floatingSchedule,overnightIndex,floatingSpread,floatingDayCounter,payLag);
%                           

% out = vanilla_Floating_OvernightIndex_BasisSwap.Calculate(discountCurve_SOFR,discountCurve_SOFR);
% 
% fairSpread = out.fairSpread;
    
%% Pricing a new CRS Deal (KRW Fixed vs USD SOFR compounding 3M + Spread) End   


outInfo = struct;
outInfo.impliedRate = zeros(length(yieldCurve.rateHelpers),1);

outInfo.diff = zeros(length(yieldCurve.rateHelpers),1);
outInfo.NPV = zeros(length(yieldCurve.rateHelpers),1);

for i=1:length(yieldCurve.rateHelpers)
    
    if strcmp(yieldCurve.rateHelpers{i}.rateType,'SwapPoint')
        outInfo.impliedRate(i) = yieldCurve.rateHelpers{i}.ImpliedQuote();
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    elseif strcmp(yieldCurve.rateHelpers{i}.rateType,'CrossCurrencySwap')
        for j=1:length(yieldCurve.rateHelpers{i}.crossCurrencySwap.domesticLeg.couponRates)
            if strcmp(class(yieldCurve.rateHelpers{i}.crossCurrencySwap.domesticLeg.couponRates{j}),'FixedRateCoupon')
                yieldCurve.rateHelpers{i}.crossCurrencySwap.domesticLeg.couponRates{j}.fixedRate = dataRate{i};
            end
        end
        
        swapOut = yieldCurve.rateHelpers{i}.crossCurrencySwap.Calculate(yieldCurve.rateHelpers{i}.discountCurve,yieldCurve.rateHelpers{i}.foreignDCurve);
        outInfo.NPV(i) = swapOut.NPV;
        outInfo.impliedRate(i) = swapOut.fairRate;
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    end
end
aaa =1;

% 
% %% import and construct a zeroCurve
% rawData = [0.002739726	0.999965207
% 0.25	0.996569895
% 0.5	0.992648282
% 0.75	0.988507257
% 1	0.984021343
% 2	0.965105185
% 3	0.945510165
% 4	0.925938679
% 5	0.906326256
% 7	0.867854044
% 10	0.809835931
% 12	0.773180526
% 15	0.723742908
% 20	0.649044875
% 25	0.582936139
% 30	0.523634856
% ];
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,H_Date(valueDate),'KRW','KRWZero'});
% zeroCurveMktData = MktData(mktData);
% zeroCurveMktData.params('convention') = {};
% zeroCurve =  ZeroCurve(zeroCurveMktData);
% AllMktData =containers.Map('zeroCurve',zeroCurve);
% 
% %% import and construct a discountCurve
% 
% rawData = [0.0	0.0
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'KRW','discountZero'});
% discountCurveMktData = MktData(mktData);
% discountCurveMktData.params('convention') = {};
% discountCurve =  ZeroCurve(discountCurveMktData);
% AllMktData('discountCurve') = discountCurve;
% 
% freq = 4;
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',1,Calendar(),'ModifiedFollowing','Act365',true,dummyCurve});
% 
% KRW_Libor_3M = IborIndex(IborIndexParams);
% 
% depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
%                         {IborIndexParams});
% 
% depositRateHelperParams('rate') = 0.03;
% depositRateHelperParams('rateTenor') = '1D';
% depositRateHelper_KRW_Libor_3M = DepositRateHelper(depositRateHelperParams);
% 
% %swapRateHelper
% % swapRateHeperParams =  containers.Map();
% % fixedLegParams = containers.Map();
% fixedLegTenor = '3M';
% fixedLegConvention = 'ModifiedFollowing';
% fixedLegDayCounter = 'Act365';
% 
% 
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% 
% swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
%                         {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});
%                     
%                     
% swapRateHelperParams('rate') = 0.03;
% swapRateHelperParams('rateTenor') = '6M';
% swapRateHelperParams_KRW_SwapRate_6M = SwapRateHelper(swapRateHelperParams);
% 
% aaa = 1.0;

In [ ]:
%test_cmt_spread_ra_cheyette4f_20220831_newRebonatoCorr3_Normal.m

clc
clear
valueDate = '2022-08-31';
%% import and construct a zeroCurve
rawData = [0.002739726	0.999933484
0.25	0.992775672
0.5	0.984145686
0.75	0.974484655
1	0.964322091
1.25	0.954840044
1.5	0.945168372
1.75	0.936021865
2	0.926910159
2.25	0.918397511
2.5	0.910037825
2.75	0.901810897
3	0.893714497
3.25	0.885781127
3.5	0.877979465
3.75	0.870306562
4	0.862760239
4.25	0.855364609
4.5	0.848094566
4.75	0.840947437
5	0.833921151
5.25	0.826990016
5.5	0.820173317
5.75	0.813469666
6	0.806877205
6.25	0.800394116
6.5	0.794018617
6.75	0.787748964
7	0.781583447
7.25	0.775157087
7.5	0.768790858
7.75	0.762504142
8	0.756295843
8.25	0.750164881
8.5	0.744110191
8.75	0.738130729
9	0.732225465
9.25	0.726393383
9.5	0.720633488
9.75	0.714944796
10	0.709326341
10.25	0.703381751
10.5	0.697474357
10.75	0.691621444
11	0.685822475
11.25	0.680076914
11.5	0.674384234
11.75	0.668743912
12	0.663155432
12.25	0.65785742
12.5	0.652627112
12.75	0.647452993
13	0.642334386
13.25	0.637270621
13.5	0.632261038
13.75	0.627304988
14	0.622401826
14.25	0.61755092
14.5	0.612751643
14.75	0.60800338
15	0.603305522
15.25	0.599365823
15.5	0.595529865
15.75	0.59175608
16	0.588043594
16.25	0.584391552
16.5	0.580799117
16.75	0.577265465
17	0.573789793
17.25	0.570371309
17.5	0.56700924
17.75	0.563702828
18	0.560451327
18.25	0.55725401
18.5	0.554110161
18.75	0.551019081
19	0.547980083
19.25	0.544992494
19.5	0.542055656
19.75	0.539168922
20	0.536331659
20.25	0.533358621
20.5	0.530418173
20.75	0.527522649
21	0.524671489
21.25	0.521864145
21.5	0.519100076
21.75	0.516378753
22	0.513699658
22.25	0.511062279
22.5	0.508466116
22.75	0.505910678
23	0.503395484
23.25	0.500920058
23.5	0.498483938
23.75	0.496086668
24	0.493727799
24.25	0.491406895
24.5	0.489123523
24.75	0.486877261
25	0.484667696
25.25	0.481172535
25.5	0.47757546
25.75	0.474002852
26	0.470454562
26.25	0.466930446
26.5	0.463430358
26.75	0.459954154
27	0.456501691
27.25	0.453072824
27.5	0.449667412
27.75	0.446285314
28	0.442926387
28.25	0.439590493
28.5	0.436277492
28.75	0.432987244
29	0.429719613
29.25	0.42647446
29.5	0.423251648
29.75	0.420051043
30	0.416872508
30.25	0.413716748
30.5	0.410584864
30.75	0.407476689
31	0.404392044
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve
rawData = [0.002739726	0.999933484
0.25	0.99362716
0.5	0.98616041
0.75	0.976958583
1	0.968084324
1.5	0.949552945
2	0.929453742
2.5	0.912686044
3	0.896345657
3.5	0.879390988
4	0.862533461
4.5	0.8457887
5	0.829162762
5.5	0.814093476
6	0.799372129
6.5	0.784975806
7	0.770896515
7.5	0.757126478
8	0.743658126
8.5	0.730484092
9	0.717597206
9.5	0.704990487
10	0.692657141
10.5	0.680411837
11	0.668412259
11.5	0.656656712
12	0.645139755
12.5	0.633856076
13	0.62280049
13.5	0.611967938
14	0.601353479
14.5	0.590952294
15	0.580759674
15.5	0.570771025
16	0.560981862
16.5	0.551387805
17	0.541984577
17.5	0.532768006
18	0.523734013
18.5	0.51487862
19	0.506197939
19.5	0.497688175
20	0.489345622
20.5	0.481083917
21	0.472978238
21.5	0.465028049
22	0.457230094
22.5	0.449581191
23	0.442078229
23.5	0.434718167
24	0.427498033
24.5	0.42041492
25	0.413465986
25.5	0.406648452
26	0.399959601
26.5	0.393396778
27	0.386957385
27.5	0.380638881
28	0.374438782
28.5	0.368354661
29	0.36238414
29.5	0.356524897
30	0.350774661
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
KRWAllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
 rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.01229236	0.01159588	0.01086647	0.01016157	0.00943588	0.00956537	0.00975864	0.01001241	0.01001241
2	0.01024166	0.00977958	0.00933128	0.00886517	0.00841642	0.00847103	0.00849227	0.00871772	0.00871772
3	0.00964502	0.00926191	0.00887582	0.00846261	0.0080798	0.00802154	0.00788601	0.00808606	0.00808606
4	0.00861066	0.00844779	0.00824759	0.00808397	0.00791678	0.00779303	0.00759142	0.00773357	0.00773357
5	0.00794063	0.00785398	0.00782268	0.00779042	0.00773372	0.00749903	0.00712158	0.00721904	0.00721904
7	0.00729079	0.00738116	0.00746297	0.00751825	0.00759991	0.00740403	0.00708813	0.00715302	0.00715302
10	0.00681849	0.00702018	0.00717538	0.00734175	0.00751103	0.00727976	0.00688309	0.00690678	0.00690678
15	0.00724832	0.0074211	0.00755901	0.00770422	0.00784287	0.00749232	0.00695141	0.00695678	0.00695678
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end

%% create a Cheyette4FModel with initial modelParams
numOfFactor = 4;
lambda= cell(numOfFactor,1);
kappa = zeros(numOfFactor,1);
tau = zeros(numOfFactor,1);


lambdaTest = [0.002912578	0.004171663	0.005293117	0.006527589
0.003594776	0.004301173	0.004639124	0.006290859
0.004559247	0.004431009	0.004237566	0.005984846
0.005190577	0.004786825	0.003911099	0.005546121
0.005738932	0.005274986	0.003722272	0.004959664
0.005984895	0.00593834	0.003671681	0.004225094
0.006115824	0.006786952	0.003769711	0.003343236
0.006160184	0.007689104	0.004035461	0.002390556
];


for i=1:numOfFactor
    lambda{i,1}.tenor = [1;2;3;4;5;7;10;15];
    lambda{i,1}.quote = ones(length(lambda{i,1}.tenor),1)*0.01;
%     lambda{i,1}.quote = lambdaTest(:,i);
end

kappa(1) = 0.03;
kappa(2) = 0.1;
kappa(3) = 0.3;
kappa(4) = 1.2;

tau(1) = 1.0;
tau(2) = 2.0;
tau(3) = 5.0;
tau(4) = 15.0;

% %Rebonato 3 parametric correlation
% corr_fwd = [1	0.891700106	0.69316567	0.535560843
% 0.891700106	1	0.926895723	0.754912104
% 0.69316567	0.926895723	1	0.997366354
% 0.535560843	0.754912104	0.997366354	1
% ];

% Rebonato 0.4 & 0.15

% corr_fwd = [1	0.916424786	0.729286982	0.473473857
% 0.916424786	1	0.782576891	0.485364443
% 0.729286982	0.782576891	1	0.533878096
% 0.473473857	0.485364443	0.533878096	1
% ];

%Rebonato 0.35 & 0.15

corr_fwd = [1	0.909460185	0.706727563	0.429596678
0.909460185	1	0.764458299	0.442478147
0.706727563	0.764458299	1	0.495034604
0.429596678	0.442478147	0.495034604	1
];


% swaption calibration yn
swaptionCalibrationYN = [1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
];

% smoothAlpha = 0.01;
% smoothAlpha = 10.0;
smoothAlpha = 1.0;

modelParams= containers.Map({'smoothAlpha','numOfFactor', 'lambda','kappa','tau','corr_fwd','swaptionCalibrationYN','modelName'}, ...
                        {smoothAlpha,numOfFactor,lambda ,kappa, tau ,corr_fwd,swaptionCalibrationYN,'Cheyette4F'});
KRWCheyette = Cheyette(IRModel(Model(KRWAllMktData,modelParams)));

%% Calibrate 
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'bootstrapLevMar');
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'dummy');
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'globalLevMar');
tic
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'globalVegaWeight');
out = KRWCheyette.CalibrateToATMSwaptionNormal(KRWBlack,'globalVegaWeight');



% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'global');
toc

%% file io
fileNameInsFwdCorr = strcat('KRWCheyette_','InsFwdCorr','_',valueDate,'.txt');
fileIDInsFwdCorr = fopen(fileNameInsFwdCorr,'w');

fileNameInsSwapFwdCorr = strcat('KRWCheyette_','InsSwapFwdCorr','_',valueDate,'.txt');
fileIDInsSwapFwdCorr = fopen(fileNameInsSwapFwdCorr,'w');

fileNameImpCorrel = strcat('KRWCheyette_','ImpCorrel ','_',valueDate,'.txt');
fileIDImpCorrel  = fopen(fileNameImpCorrel,'w');

% investigaion on the implied correlation
newexpiry = [1,2,3,4,5,7,10,15];

%newexpiry = [1,2,3,4,5,7,10];

newtenor  = newexpiry;
calTimes = [0:1:15]';
testLGMKRW.InsFwdCorr = cell(length(calTimes),1);
testLGMKRW.InsSwapFwdCorr = cell(length(calTimes),1);
testLGMKRW.ImpCorrel = cell(length(calTimes),1);

testLGMKRW.InsFwdCovar = cell(length(calTimes),1);
testLGMKRW.InsSwapFwdCovar = cell(length(calTimes),1);
testLGMKRW.ImpCovar = cell(length(calTimes),1);

for i=1:length(calTimes)
    for j=1:length(newexpiry)
        for k=1:length(newtenor)
            testLGMKRW.InsFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWCheyette.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
            
            testLGMKRW.InsFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWCheyette.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
            
            fprintf(fileIDInsFwdCorr,'%f\t',testLGMKRW.InsFwdCorr{i,1}.quote(j,k));
            fprintf(fileIDInsSwapFwdCorr,'%f\t',testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k));
            fprintf(fileIDImpCorrel,'%f\t',testLGMKRW.ImpCorrel{i,1}.quote(j,k));

        end
        fprintf(fileIDInsFwdCorr,'\n');
        fprintf(fileIDInsSwapFwdCorr,'\n');
        fprintf(fileIDImpCorrel,'\n');
    end
    fprintf(fileIDInsFwdCorr,'\n');
    fprintf(fileIDInsFwdCorr,'\n');
    
    fprintf(fileIDInsSwapFwdCorr,'\n');
    fprintf(fileIDInsSwapFwdCorr,'\n');
    
    fprintf(fileIDImpCorrel,'\n');
    fprintf(fileIDImpCorrel,'\n');
    
end

fclose(fileIDInsFwdCorr);
fclose(fileIDInsSwapFwdCorr);
fclose(fileIDImpCorrel);
%% file io end

lambdaCalib= cell(numOfFactor,1);

for i=1:numOfFactor
    lambdaCalib{i,1}.tenor = KRWCheyette.lambda{i,1}.tenor;
    lambdaCalib{i,1}.quote = KRWCheyette.lambda{i,1}.quote;
end

modelParamsCalib= containers.Map({'smoothAlpha','numOfFactor', 'lambda','kappa','tau','corr_fwd','modelName'}, ...
                        {smoothAlpha,numOfFactor,lambdaCalib ,kappa, tau ,corr_fwd,'Cheyette4F'});

KRWCheyetteRisky = CheyetteRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
%% Pricing Spread Range Accrual
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2018-05-31');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0432;
couponFreq = 1;
tenorL = 10;
tenorS = 5;
lower = 0.0;
upper = 999;
rangeDays = zeros(15,1);
%rangeDays(1) = 1;
rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


callYN = ones(15,1);
callYN(1) = 0;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20180531	20180531	20190531	20190531
20190531	20190531	20200529	20200529
20200529	20200529	20210531	20210531
20210531	20210531	20220531	20220531
20220531	20220531	20230531	20230531
20230531	20230531	20240531	20240531
20240531	20240531	20250530	20250530
20250530	20250530	20260529	20260529
20260529	20260529	20270531	20270531
20270531	20270531	20280531	20280531
20280531	20280531	20290531	20290531
20290531	20290531	20300531	20300531
20300531	20300531	20310530	20310530
20310530	20310530	20320531	20320531
20320531	20320531	20330531	20330531
];

%resetDate startDate endDate paymentDate
callScheduleInt = [20180531	20180531	20180531	20180531
20190531	20190531	20190531	20190531
20200529	20200529	20200529	20200529
20210531	20210531	20210531	20210531
20220531	20220531	20220531	20220531
20230531	20230531	20230531	20230531
20240531	20240531	20240531	20240531
20250530	20250530	20250530	20250530
20260529	20260529	20260529	20260529
20270531	20270531	20270531	20270531
20280531	20280531	20280531	20280531
20290531	20290531	20290531	20290531
20300531	20300531	20300531	20300531
20310530	20310530	20310530	20310530
20320531	20320531	20320531	20320531
];

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', 'tenorS', ...
                                'lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                        upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;              

rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);


%computePriceMCForward
numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                        {'AMC',7,40000});

numMethodAMC = NumMethod(numMethodInfo);

% daily weekly step calculator
numFactor = 4;
pricer = DailyWeeklyRiskyPricerNF(rangeAccrual,KRWCheyetteRisky,numMethodAMC,false,numFactor);
% pricer = DailyWeeklyRiskyPricerNFOneTime(rangeAccrual,KRWCheyetteRisky,numMethodAMC,true,numFactor);

pricingOutMC = pricer.computePriceMCCallable(valueDateH);

pricingOutClosed = pricer.computePriceClosedTRZ(valueDateH);

% daily closed form pricer
%pricingOutLGM =  rangeAccrual.computePrice(valueDate,KRWCheyette);

%pricingOut = rangeAccrual.computePriceMCForward(valueDate,KRWCheyette);

newexpiry = [1;2;3;4;5;7;10;15];


testCheyette.impCorrel = zeros(size(newexpiry,1),size(newexpiry,1));
testCK.insSwapFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));
testCK.insFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));

testCheyette.impFwdLiborCorr = KRWCheyette.InsFwdCorrTest2(0,newexpiry,0.25);
for i=1:size(newexpiry,1)
    for j=1:size(newexpiry,1)
        testCheyette.impCorrel(i,j)= KRWCheyette.ImpCorrel(5,newexpiry(i),newexpiry(j));
        testCK.insSwapFwdCorr(i,j) = KRWCheyette.InsSwapFwdCorrTest(5,newexpiry(i),newexpiry(j));
        testCK.insFwdCorr(i,j) = KRWCheyette.InsFwdCorrTest(5,newexpiry(i),newexpiry(j));
    end
end

newexpiryFwd = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
testCheyette.impFwdLiborCorrNew = KRWCheyette.InsFwdCorrTest2(0,newexpiryFwd,0.25);


newexpiry2 = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];

for i=1:size(newexpiry2,1)
    testCheyette.impCorrel10_5(i)= KRWCheyette.ImpCorrel(newexpiry2(i),10,2);
    testCheyette.impCovarl10_5(i)= sqrt(KRWCheyette.ImpCovar(newexpiry2(i),10,2)/newexpiry2(i));
    testCheyette.spreadInfo10_5(i) =  KRWCheyette.SpreadInfo(newexpiry2(i),10,2);
    testCheyette.fwdSpread(i) = testCheyette.spreadInfo10_5(i).fwdSpread';
    testCheyette.muSpread(i) = testCheyette.spreadInfo10_5(i).mu';
    testCheyette.sigma(i) = testCheyette.spreadInfo10_5(i).sigma';
    testCheyette.corr(i) = testCheyette.spreadInfo10_5(i).corr;
    testCheyette.mu1(i) =testCheyette.spreadInfo10_5(i).mu1;
    testCheyette.mu2(i) =testCheyette.spreadInfo10_5(i).mu2;
    testCheyette.sig1(i) =testCheyette.spreadInfo10_5(i).sig1;
    testCheyette.sig2(i) =testCheyette.spreadInfo10_5(i).sig2;
    
end

aaaa =1.0;
 






In [ ]:
%test_cmt_spread_ra_cheyette4f_20220831_newRebonatoCorr3.m


clc
clear
valueDate = '2022-08-31';
%% import and construct a zeroCurve
rawData = [0.002739726	0.999960 
0.25	0.995905 
0.5	0.991478 
0.75	0.986803 
1	0.982018 
2	0.961895 
3	0.941440 
4	0.921094 
5	0.900744 
7	0.862017 
10	0.803538 
12	0.765150 
15	0.712785 
20	0.634851 
25	0.567052 
30	0.506500 
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve


rawData = [0.002739726	0.999960 
0.25	0.996071 
0.5	0.991810 
0.75	0.986904 
1	0.982002 
2	0.961193 
3	0.939654 
5	0.890808 
10	0.776170 
20	0.603720 
30	0.471696 
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
AllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
 rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.195085	0.197082	0.199887	0.205007	0.207273	0.209988	0.215711	0.215051	0.215051
2	0.20827	0.210154	0.206121	0.205622	0.205393	0.204251	0.206121	0.206984	0.206984
3	0.211711	0.206756	0.202049	0.199717	0.197153	0.193716	0.195065	0.197963	0.197963
4	0.214233	0.210822	0.203227	0.19763	0.193533	0.189172	0.191003	0.193957	0.193957
5	0.222268	0.212571	0.202898	0.195732	0.192448	0.188655	0.191581	0.194774	0.194774
7	0.212873	0.200717	0.195521	0.18569	0.178918	0.178066	0.185034	0.187173	0.187173
10	0.201468	0.188597	0.183828	0.181607	0.177301	0.177381	0.182212	0.184714	0.184714
15	0.212008	0.209388	0.192599	0.188335	0.185841	0.186096	0.190715	0.183324	0.183324
];


mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
swaptionVolMktData = MktData(mktData);
AllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end

%% create a Cheyette4FModel with initial modelParams
numOfFactor = 4;
lambda= cell(numOfFactor,1);
kappa = zeros(numOfFactor,1);
tau = zeros(numOfFactor,1);


lambdaTest = [0.002912578	0.004171663	0.005293117	0.006527589
0.003594776	0.004301173	0.004639124	0.006290859
0.004559247	0.004431009	0.004237566	0.005984846
0.005190577	0.004786825	0.003911099	0.005546121
0.005738932	0.005274986	0.003722272	0.004959664
0.005984895	0.00593834	0.003671681	0.004225094
0.006115824	0.006786952	0.003769711	0.003343236
0.006160184	0.007689104	0.004035461	0.002390556
];


for i=1:numOfFactor
    lambda{i,1}.tenor = [1;2;3;4;5;7;10;15];
    lambda{i,1}.quote = ones(length(lambda{i,1}.tenor),1)*0.01;
%     lambda{i,1}.quote = lambdaTest(:,i);
end

kappa(1) = 0.03;
kappa(2) = 0.1;
kappa(3) = 0.3;
kappa(4) = 1.2;

tau(1) = 1.0;
tau(2) = 2.0;
tau(3) = 5.0;
tau(4) = 15.0;

%Rebonato 3 parametric correlation
corr_fwd = [1	0.891700106	0.69316567	0.535560843
0.891700106	1	0.926895723	0.754912104
0.69316567	0.926895723	1	0.997366354
0.535560843	0.754912104	0.997366354	1
];

% Rebonato 0.4 & 0.15

% corr_fwd = [1	0.916424786	0.729286982	0.473473857
% 0.916424786	1	0.782576891	0.485364443
% 0.729286982	0.782576891	1	0.533878096
% 0.473473857	0.485364443	0.533878096	1
% ];


% swaption calibration yn
swaptionCalibrationYN = [1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
1	1	1	1	1	1	1	1	1
];

% smoothAlpha = 0.01;
smoothAlpha = 10.0;
% smoothAlpha = 1.0;

modelParams= containers.Map({'smoothAlpha','numOfFactor', 'lambda','kappa','tau','corr_fwd','swaptionCalibrationYN','modelName'}, ...
                        {smoothAlpha,numOfFactor,lambda ,kappa, tau ,corr_fwd,swaptionCalibrationYN,'Cheyette4F'});
KRWCheyette = Cheyette(IRModel(Model(AllMktData,modelParams)));

%% Calibrate 
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'bootstrapLevMar');
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'dummy');
% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'globalLevMar');
tic
out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'globalVegaWeight');



% out = KRWCheyette.CalibrateToATMSwaption(KRWBlack,'global');
toc

%% file io
fileNameInsFwdCorr = strcat('KRWCheyette_','InsFwdCorr','_',valueDate,'.txt');
fileIDInsFwdCorr = fopen(fileNameInsFwdCorr,'w');

fileNameInsSwapFwdCorr = strcat('KRWCheyette_','InsSwapFwdCorr','_',valueDate,'.txt');
fileIDInsSwapFwdCorr = fopen(fileNameInsSwapFwdCorr,'w');

fileNameImpCorrel = strcat('KRWCheyette_','ImpCorrel ','_',valueDate,'.txt');
fileIDImpCorrel  = fopen(fileNameImpCorrel,'w');

% investigaion on the implied correlation
newexpiry = [1,2,3,4,5,7,10,15];

%newexpiry = [1,2,3,4,5,7,10];

newtenor  = newexpiry;
calTimes = [0:1:15]';
testLGMKRW.InsFwdCorr = cell(length(calTimes),1);
testLGMKRW.InsSwapFwdCorr = cell(length(calTimes),1);
testLGMKRW.ImpCorrel = cell(length(calTimes),1);

testLGMKRW.InsFwdCovar = cell(length(calTimes),1);
testLGMKRW.InsSwapFwdCovar = cell(length(calTimes),1);
testLGMKRW.ImpCovar = cell(length(calTimes),1);

for i=1:length(calTimes)
    for j=1:length(newexpiry)
        for k=1:length(newtenor)
            testLGMKRW.InsFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWCheyette.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
            
            testLGMKRW.InsFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWCheyette.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
            testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWCheyette.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
            
            fprintf(fileIDInsFwdCorr,'%f\t',testLGMKRW.InsFwdCorr{i,1}.quote(j,k));
            fprintf(fileIDInsSwapFwdCorr,'%f\t',testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k));
            fprintf(fileIDImpCorrel,'%f\t',testLGMKRW.ImpCorrel{i,1}.quote(j,k));

        end
        fprintf(fileIDInsFwdCorr,'\n');
        fprintf(fileIDInsSwapFwdCorr,'\n');
        fprintf(fileIDImpCorrel,'\n');
    end
    fprintf(fileIDInsFwdCorr,'\n');
    fprintf(fileIDInsFwdCorr,'\n');
    
    fprintf(fileIDInsSwapFwdCorr,'\n');
    fprintf(fileIDInsSwapFwdCorr,'\n');
    
    fprintf(fileIDImpCorrel,'\n');
    fprintf(fileIDImpCorrel,'\n');
    
end

fclose(fileIDInsFwdCorr);
fclose(fileIDInsSwapFwdCorr);
fclose(fileIDImpCorrel);
%% file io end

lambdaCalib= cell(numOfFactor,1);

for i=1:numOfFactor
    lambdaCalib{i,1}.tenor = KRWCheyette.lambda{i,1}.tenor;
    lambdaCalib{i,1}.quote = KRWCheyette.lambda{i,1}.quote;
end

modelParamsCalib= containers.Map({'smoothAlpha','numOfFactor', 'lambda','kappa','tau','corr_fwd','modelName'}, ...
                        {smoothAlpha,numOfFactor,lambdaCalib ,kappa, tau ,corr_fwd,'Cheyette4F'});

KRWCheyetteRisky = CheyetteRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
%% Pricing Spread Range Accrual
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2018-05-31');
rangeAccrualParams.maturity = 15;

valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0432;
couponFreq = 1;
tenorL = 10;
tenorS = 5;
lower = 0.0;
upper = 999;
rangeDays = zeros(15,1);
%rangeDays(1) = 1;
rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;


callYN = ones(15,1);
callYN(1) = 0;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20180531	20180531	20190531	20190531
20190531	20190531	20200529	20200529
20200529	20200529	20210531	20210531
20210531	20210531	20220531	20220531
20220531	20220531	20230531	20230531
20230531	20230531	20240531	20240531
20240531	20240531	20250530	20250530
20250530	20250530	20260529	20260529
20260529	20260529	20270531	20270531
20270531	20270531	20280531	20280531
20280531	20280531	20290531	20290531
20290531	20290531	20300531	20300531
20300531	20300531	20310530	20310530
20310530	20310530	20320531	20320531
20320531	20320531	20330531	20330531
];

%resetDate startDate endDate paymentDate
callScheduleInt = [20180531	20180531	20180531	20180531
20190531	20190531	20190531	20190531
20200529	20200529	20200529	20200529
20210531	20210531	20210531	20210531
20220531	20220531	20220531	20220531
20230531	20230531	20230531	20230531
20240531	20240531	20240531	20240531
20250530	20250530	20250530	20250530
20260529	20260529	20260529	20260529
20270531	20270531	20270531	20270531
20280531	20280531	20280531	20280531
20290531	20290531	20290531	20290531
20300531	20300531	20300531	20300531
20310530	20310530	20310530	20310530
20320531	20320531	20320531	20320531
];

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', 'tenorS', ...
                                'lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                        upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;              

rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);


%computePriceMCForward
numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                        {'AMC',7,40000});

numMethodAMC = NumMethod(numMethodInfo);

% daily weekly step calculator
numFactor = 4;
pricer = DailyWeeklyRiskyPricerNF(rangeAccrual,KRWCheyetteRisky,numMethodAMC,false,numFactor);
% pricer = DailyWeeklyRiskyPricerNFOneTime(rangeAccrual,KRWCheyetteRisky,numMethodAMC,true,numFactor);

pricingOutMC = pricer.computePriceMCCallable(valueDateH);

pricingOutClosed = pricer.computePriceClosedTRZ(valueDateH);

% daily closed form pricer
%pricingOutLGM =  rangeAccrual.computePrice(valueDate,KRWCheyette);

%pricingOut = rangeAccrual.computePriceMCForward(valueDate,KRWCheyette);

newexpiry = [1;2;3;4;5;7;10;15];


testCheyette.impCorrel = zeros(size(newexpiry,1),size(newexpiry,1));
testCK.insSwapFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));
testCK.insFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));

testCheyette.impFwdLiborCorr = KRWCheyette.InsFwdCorrTest2(0,newexpiry,0.25);
for i=1:size(newexpiry,1)
    for j=1:size(newexpiry,1)
        testCheyette.impCorrel(i,j)= KRWCheyette.ImpCorrel(5,newexpiry(i),newexpiry(j));
        testCK.insSwapFwdCorr(i,j) = KRWCheyette.InsSwapFwdCorrTest(5,newexpiry(i),newexpiry(j));
        testCK.insFwdCorr(i,j) = KRWCheyette.InsFwdCorrTest(5,newexpiry(i),newexpiry(j));
    end
end

newexpiryFwd = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
testCheyette.impFwdLiborCorrNew = KRWCheyette.InsFwdCorrTest2(0,newexpiryFwd,0.25);


newexpiry2 = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];

for i=1:size(newexpiry2,1)
    testCheyette.impCorrel10_5(i)= KRWCheyette.ImpCorrel(newexpiry2(i),10,2);
    testCheyette.impCovarl10_5(i)= sqrt(KRWCheyette.ImpCovar(newexpiry2(i),10,2)/newexpiry2(i));
    testCheyette.spreadInfo10_5(i) =  KRWCheyette.SpreadInfo(newexpiry2(i),10,2);
    testCheyette.fwdSpread(i) = testCheyette.spreadInfo10_5(i).fwdSpread';
    testCheyette.muSpread(i) = testCheyette.spreadInfo10_5(i).mu';
    testCheyette.sigma(i) = testCheyette.spreadInfo10_5(i).sigma';
    testCheyette.corr(i) = testCheyette.spreadInfo10_5(i).corr;
    testCheyette.mu1(i) =testCheyette.spreadInfo10_5(i).mu1;
    testCheyette.mu2(i) =testCheyette.spreadInfo10_5(i).mu2;
    testCheyette.sig1(i) =testCheyette.spreadInfo10_5(i).sig1;
    testCheyette.sig2(i) =testCheyette.spreadInfo10_5(i).sig2;
    
end

aaaa =1.0;
 






In [ ]:
%test_CMSAVGSteepener_20201123_20230519.m

clc
clear
valueDate = '2020-10-13';

maxIdx = 3;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
nonCallBPrice = zeros(100,1);

closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CMSAVGSteepener Pricing start

%% security info start
valueDateH = H_Date(valueDate);
steepenerParams.payCcy = 'KRW';
steepenerParams.startDate = H_Date('2020-02-27');
steepenerParams.endDate = H_Date('2030-02-27');

nominal = 10000;
basePrice = 100.0;

%coupon Info

couponInfo = struct;
%dummy coupon
couponInfo.coupon = 0.02;
% local floor
% local cap
% multipler
couponInfo.localFloor = 0.0;
couponInfo.localCap = 100.0;
couponInfo.multiplier = 1.0;
couponInfo.tenorL = 10;
couponInfo.tenorS = 2;
couponInfo.spread = 0.01;

couponInfo.eventType = 'PeriodEnd';

%couponSchedule  reset start end pay (annual schedule)
couponInfo.couponScheduleInt=[20200227	20200227	20210227	20210227
20210227	20210227	20220227	20220227
20220227	20220227	20230227	20230227
20230227	20230227	20240227	20240227
20240227	20240227	20250227	20250227
20250227	20250227	20260227	20260227
20260227	20260227	20270227	20270227
20270227	20270227	20280227	20280227
20280227	20280227	20290227	20290227
20290227	20290227	20300227	20300227
];

% callable schedule

callableInfo = struct;

% callPayment is the amount that should be paid at callPayDate
callableInfo.isCallableYN = 1;

% callEventDate callPayDate callAmount(of notional)
callableInfo.callableScheduleInt = [20210227	20210227	1
20220227	20220227	1
20230227	20230227	1
20240227	20240227	1
20250227	20250227	1
20260227	20260227	1
20270227	20270227	1
20280227	20280227	1
20290227	20290227	1
];

% callableInfo.EODFlag_Cashflow = 'YES';
callableInfo.EODFlag_Cashflow = true;

% callableInfo.EODFlag_Cashflow = 'NO';
% callableInfo.EODFlag_Exercise = 'YES';
callableInfo.EODFlag_Exercise = true;

% callableInfo.EODFlag_Exercise = 'NO';

averageInfo = struct;

steepenerInfo = struct('nominal',nominal,'basePrice',basePrice,'couponInfo',couponInfo, ...
                        'callableInfo',callableInfo,'floatingAverageInfo',averageInfo);

steepenerParams.params = steepenerInfo;  

maxIdx = 3;

rmseSimul = zeros(100,1);
for idx2=1:11
    %% KRW Model building
        
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999985206
0.25	0.998434139
0.5	0.996588992
0.75	0.994828128
1	0.993035132
1.25	0.991023149
1.5	0.988901694
1.75	0.98676087
2	0.984538831
2.25	0.982181264
2.5	0.979730755
2.75	0.977188564
3	0.974555437
3.25	0.971950686
3.5	0.969278409
3.75	0.966535505
4	0.963722613
4.25	0.961068253
4.5	0.958376016
4.75	0.955641483
5	0.952865058
5.25	0.950270281
5.5	0.947659567
5.75	0.945028351
6	0.942376844
6.25	0.939705257
6.5	0.937013803
6.75	0.934302698
7	0.931572156
7.25	0.928877783
7.5	0.926169169
7.75	0.923445295
8	0.920706339
8.25	0.917952486
8.5	0.915183917
8.75	0.912400815
9	0.909603364
9.25	0.906791747
9.5	0.90396615
9.75	0.901126758
10	0.898273755
10.25	0.895464659
10.5	0.892646737
10.75	0.88981824
11	0.886979325
11.25	0.884130149
11.5	0.881270868
11.75	0.878401641
12	0.875522625
12.25	0.873602529
12.5	0.871752303
12.75	0.869928181
13	0.868130016
13.25	0.866357662
13.5	0.864610977
13.75	0.862889819
14	0.86119405
14.25	0.859523534
14.5	0.857878137
14.75	0.856257727
15	0.854662173
15.25	0.853010285
15.5	0.851375882
15.75	0.849763481
16	0.848172968
16.25	0.846604233
16.5	0.845057165
16.75	0.843531656
17	0.842027599
17.25	0.84054489
17.5	0.839083425
17.75	0.837643102
18	0.836223821
18.25	0.834825483
18.5	0.833447992
18.75	0.832091252
19	0.830755168
19.25	0.829439648
19.5	0.828144602
19.75	0.82686994
20	0.825615574
20.25	0.824002891
20.5	0.822370525
20.75	0.820749539
21	0.819139878
21.25	0.817541487
21.5	0.815954313
21.75	0.814378303
22	0.812813403
22.25	0.811259562
22.5	0.809716728
22.75	0.808184848
23	0.806663873
23.25	0.80515375
23.5	0.803654431
23.75	0.802165865
24	0.800688002
24.25	0.799220794
24.5	0.797764192
24.75	0.796318147
25	0.794882613
25.25	0.79321226
25.5	0.791521382
25.75	0.789836583
26	0.788157839
26.25	0.786485127
26.5	0.784818423
26.75	0.783157704
27	0.781502947
27.25	0.77985413
27.5	0.778211228
27.75	0.77657422
28	0.774943083
28.25	0.773317795
28.5	0.771698332
28.75	0.770084673
29	0.768476795
29.25	0.766874677
29.5	0.765278295
29.75	0.763687629
30	0.762102657
30.25	0.760522364
30.5	0.758945358
30.75	0.757371622
31	0.755801149
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end
    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999985206
0.25	0.998821159
0.5	0.997240293
0.75	0.995522659
1	0.993714564
1.5	0.989185872
2	0.98374605
2.5	0.978529211
3	0.972872405
3.5	0.965963711
4	0.958392048
4.5	0.950185292
5	0.941360816
5.5	0.934008574
6	0.926435713
6.5	0.918628004
7	0.91059244
7.5	0.90233617
8	0.893866497
8.5	0.885190857
9	0.876316818
9.5	0.867252061
10	0.858004374
10.5	0.850986315
11	0.844022798
11.5	0.83707572
12	0.830145614
12.5	0.82323301
13	0.816338427
13.5	0.80946238
14	0.802605377
14.5	0.795767918
15	0.788950494
15.5	0.782153593
16	0.775377693
16.5	0.768623265
17	0.761890774
17.5	0.755180677
18	0.748493424
18.5	0.741829458
19	0.735189213
19.5	0.728573118
20	0.721981594
20.5	0.716100959
21	0.710294835
21.5	0.704535912
22	0.698823805
22.5	0.693158131
23	0.687538514
23.5	0.681964577
24	0.676435948
24.5	0.670952259
25	0.665513141
25.5	0.660118234
26	0.654767175
26.5	0.649459608
27	0.644195179
27.5	0.638973535
28	0.633794329
28.5	0.628657214
29	0.623561848
29.5	0.618507889
30	0.613495002
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



% % GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0035	0.0038	0.0041	0.0044	0.0047	0.0049	0.0052	0.0053	0.0050	0.0044
% 2	0.0042	0.0043	0.0044	0.0045	0.0046	0.0047	0.0049	0.0049	0.0046	0.0040
% 3	0.0044	0.0044	0.0044	0.0044	0.0044	0.0046	0.0048	0.0049	0.0044	0.0039
% 4	0.0045	0.0045	0.0045	0.0045	0.0044	0.0046	0.0048	0.0048	0.0042	0.0039
% 5	0.0046	0.0046	0.0045	0.0045	0.0045	0.0046	0.0047	0.0047	0.0041	0.0039
% 7	0.0046	0.0046	0.0046	0.0046	0.0045	0.0046	0.0048	0.0048	0.0044	0.0043
% 10	0.0047	0.0047	0.0047	0.0047	0.0046	0.0047	0.0042	0.0043	0.0042	0.0043
% 15	0.0032	0.0031	0.0029	0.0028	0.0026	0.0031	0.0033	0.0035	0.0037	0.0040
% 20	0.0037	0.0037	0.0037	0.0037	0.0037	0.0038	0.0038	0.0039	0.0040	0.0041
%     ];

% SwaptionVol From FrontSystem 
% % 20 x 20
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.00321772	0.00354476	0.00389697	0.00424674	0.00459727	0.00480495	0.00512276	0.00527918	0.00527918	0.00527918
% 2	0.00380361	0.00393397	0.0040487	0.0041681	0.00427948	0.00447705	0.00478293	0.00485168	0.00485168	0.00485168
% 3	0.00396916	0.00404699	0.00413234	0.00419804	0.00427601	0.00445133	0.00471047	0.00476705	0.00476705	0.00476705
% 4	0.00406143	0.00411377	0.00417249	0.00421175	0.00426608	0.0044168	0.00464154	0.00468129	0.00468129	0.00468129
% 5	0.00414985	0.0041761	0.0041968	0.00422444	0.00425391	0.00438098	0.00457183	0.00459557	0.00459557	0.00459557
% 7	0.00415456	0.00420399	0.00425019	0.00429059	0.00432913	0.00446055	0.00466101	0.00468814	0.00468814	0.00468814
% 10	0.00420754	0.00427513	0.00431915	0.00438068	0.0044432	0.00460262	0.00484237	0.00486215	0.00486215	0.00486215
% 15	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664	0.00488664
% 20	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664	0.00488664
%     ];

% 15 x 15
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00321772	0.00354476	0.00389697	0.00424674	0.00459727	0.00480495	0.00512276	0.00527918	0.00527918
2	0.00380361	0.00393397	0.0040487	0.0041681	0.00427948	0.00447705	0.00478293	0.00485168	0.00485168
3	0.00396916	0.00404699	0.00413234	0.00419804	0.00427601	0.00445133	0.00471047	0.00476705	0.00476705
4	0.00406143	0.00411377	0.00417249	0.00421175	0.00426608	0.0044168	0.00464154	0.00468129	0.00468129
5	0.00414985	0.0041761	0.0041968	0.00422444	0.00425391	0.00438098	0.00457183	0.00459557	0.00459557
7	0.00415456	0.00420399	0.00425019	0.00429059	0.00432913	0.00446055	0.00466101	0.00468814	0.00468814
10	0.00420754	0.00427513	0.00431915	0.00438068	0.0044432	0.00460262	0.00484237	0.00486215	0.00486215
15	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
  %% create a LGM2FModel with initial modelParams
% rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

%swaption only initial parameters
% dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

%Samsung
% dHTest = [0.058507682	0.500210296
% 0.06145709	0.515096302
% 0.057841075	0.632558982
% 0.069442883	0.632845436
% 0.081752868	0.607322125
% 0.087574376	0.586095839
% 0.103284643	0.52688632
% 0.113936628	0.552598585
% 0.131046999	0.498409755
% 0.153553385	0.437472215
% 0.173381122	0.423805445
% 0.180703372	0.380344606
% 0.207484315	0.305927652
% 0.218329282	0.268192317
% 0.224885432	0.227191192
% 0.221244544	0.231204789
% 0.262502203	0.212209604
% 0.264579505	0.135257044
% 0.318821555	0.118179799
% 0.271515922	0.107957745
% 0.358024755	0.088834433
% 0.33143161	0.069384916
% 0.457059078	0.033745097
% 0.374067262	0.006573072
% ];

%Daewoo
% dHTest = [0.037286763	0.525314433
% 0.037286763	0.525314433
% 0.054571525	0.586928199
% 0.069289617	0.618362181
% 0.099111958	0.605543022
% 0.1123501	0.589459841
% 0.139006516	0.523225356
% 0.175987354	0.505500158
% 0.184616726	0.463242812
% 0.204164537	0.412151488
% 0.263444475	0.364947643
% 0.251615512	0.358337407
% 0.259708458	0.315375986
% 0.269854863	0.270026782
% 0.337623406	0.253701114
% 0.258270519	0.179914465
% 0.321985902	0.191343978
% 0.347503907	0.154371663
% 0.36490148	0.127052474
% 0.36706888	0.095165077
% 0.480469769	0.05084228
% 0.423787026	0.033262976
% 0.531944266	0.006463726
% 0.333342463	0.000100001
% ];

% %Sungwoo
% dHTest = [0.060654929	0.585843811
% 0.065181321	0.621357434
% 0.059213061	0.705433024
% 0.070356009	0.697274699
% 0.084854535	0.641491331
% 0.078986949	0.60204605
% 0.103979263	0.504724505
% 0.128905871	0.522635178
% 0.142868097	0.436948669
% 0.167927649	0.341253642
% 0.171414266	0.384418469
% 0.173137707	0.332946905
% 0.205465936	0.252922264
% 0.190811162	0.233849244
% 0.212134317	0.235149902
% 0.208305697	0.261176412
% 0.267225845	0.213982892
% 0.244169602	0.131637885
% 0.293167102	0.110338057
% 0.302235653	0.107810085
% 0.341199539	0.095697121
% 0.353976	0.072468098
% 0.421517994	0.033547027
% 0.357938773	0.006568033
% ];

%JaeYeol
dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

%Samsung
% AlphaTest = [0.0289581	0.006633509
% 0.021585088	0.007003076
% 0.019812596	0.007056061
% 0.015014178	0.007801684
% 0.008683651	0.008360616
% 0.011075137	0.008248663
% 0.008794173	0.011226664
% 0.000329846	0.023982837
% ];

%Daewoo
% AlphaTest = [0.029142908	0.007983918
% 0.016018045	0.007219568
% 0.00912537	0.007233927
% 0.008101129	0.007774553
% 0.012850209	0.008916743
% 0.011065789	0.009477842
% 0.008930619	0.011214209
% 0.0001	0.032096513
% ];

% %sungwoo
% AlphaTest = [0.032526561	0.005901286
% 0.021888486	0.006159185
% 0.015500653	0.006183779
% 0.011483817	0.007702683
% 0.008112119	0.008910715
% 0.010323471	0.008310996
% 0.006966815	0.012340985
% 0.000302754	0.020086129
% ];

%jaeweol
AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
];


Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% import irModels.*;
% import targetFunctions.*;
                    
KRWLGM = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));


tic
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
out = KRWLGM.CalibrateToATMSwaptionSurfaceNormal(KRWBlack,'global',200);
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');

dH1Calib = KRWLGM.dH1;
dH2Calib = KRWLGM.dH2;
Alpha1Calib = KRWLGM.Alpha1;
Alpha2Calib = KRWLGM.Alpha2;

modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});
                    
                    
%% allFixings
%% CMS10Y & CMS2Y
tempDate = steepenerParams.startDate;

%CMS10Y
irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});

%CMS2Y
irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});

for idxhong=1:2000
    tempDate = AddDate(tempDate,1,'day');
    irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
    irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
end

%% fixing info -> let it be the modelParams !

allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
allFixings('CMS2Y') = irIndexFixing2;

modelParamsCalib('allFixings') = allFixings;

KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
    

toc
    %% LGM2FRisky model building end
    %% KRW model building end
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    pricingOutMC1 = KRWLGM2FRisky.computeAVGSteepener(valueDateH,steepenerParams,numMethodAMC);
    aaa = 1.0;

    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
    nonCallBPrice(idx2) = pricingOutMC1.nonCallB.npv;
end

aaa =1.0;


In [ ]:
%test_CMSAVGSteepener_20201123.m

clc
clear
valueDate = '2020-10-13';

maxIdx = 3;
zcStep = 0.001;
%shift size 10bp
shiftAmount = 0.001;

callablePrice = zeros(100,1);
nonCallPrice = zeros(100,1);
nonCallBPrice = zeros(100,1);

closedPrice = zeros(100,1);

callablePrice_cashflowNPV = zeros(100,100);
nonCallPrice_cashflowNPV = zeros(100,100);
closedPrice_cashflowNPV = zeros(100,100);

closedPrice_fwdCD = zeros(100,100);
closedPrice_fwdKTB = zeros(100,100);
closedPrice_fwdSpread = zeros(100,100);
closedPrice_fwdMu = zeros(100,100);
closedPrice_fwdSigma = zeros(100,100);

KRWVol_r = zeros(100,100);
KRWRe = zeros(100,100);
KRWRMSE = zeros(100,1);

USDVol_r = zeros(100,100);
USDRe = zeros(100,100);
USDRMSE = zeros(100,1);
%% CMSAVGSteepener Pricing start

%% security info start
valueDateH = H_Date(valueDate);
steepenerParams.payCcy = 'KRW';
steepenerParams.startDate = H_Date('2020-02-27');
steepenerParams.endDate = H_Date('2030-02-27');

nominal = 10000;
basePrice = 100.0;

%coupon Info

couponInfo = struct;
%dummy coupon
couponInfo.coupon = 0.02;
% local floor
% local cap
% multipler
couponInfo.localFloor = 0.0;
couponInfo.localCap = 100.0;
couponInfo.multiplier = 1.0;
couponInfo.tenorL = 10;
couponInfo.tenorS = 2;
couponInfo.spread = 0.01;

couponInfo.eventType = 'PeriodEnd';

%couponSchedule  reset start end pay (annual schedule)
couponInfo.couponScheduleInt=[20200227	20200227	20210227	20210227
20210227	20210227	20220227	20220227
20220227	20220227	20230227	20230227
20230227	20230227	20240227	20240227
20240227	20240227	20250227	20250227
20250227	20250227	20260227	20260227
20260227	20260227	20270227	20270227
20270227	20270227	20280227	20280227
20280227	20280227	20290227	20290227
20290227	20290227	20300227	20300227
];

% callable schedule

callableInfo = struct;

% callPayment is the amount that should be paid at callPayDate
callableInfo.isCallableYN = 1;

% callEventDate callPayDate callAmount(of notional)
callableInfo.callableScheduleInt = [20210227	20210227	1
20220227	20220227	1
20230227	20230227	1
20240227	20240227	1
20250227	20250227	1
20260227	20260227	1
20270227	20270227	1
20280227	20280227	1
20290227	20290227	1
];

% callableInfo.EODFlag_Cashflow = 'YES';
callableInfo.EODFlag_Cashflow = true;

% callableInfo.EODFlag_Cashflow = 'NO';
% callableInfo.EODFlag_Exercise = 'YES';
callableInfo.EODFlag_Exercise = true;

% callableInfo.EODFlag_Exercise = 'NO';

averageInfo = struct;

steepenerInfo = struct('nominal',nominal,'basePrice',basePrice,'couponInfo',couponInfo, ...
                        'callableInfo',callableInfo,'floatingAverageInfo',averageInfo);

steepenerParams.params = steepenerInfo;  

maxIdx = 3;

rmseSimul = zeros(100,1);
for idx2=1:11
    %% KRW Model building
        
    %% import and construct a zeroCurve
    rawData = [0.002739726	0.999985206
0.25	0.998434139
0.5	0.996588992
0.75	0.994828128
1	0.993035132
1.25	0.991023149
1.5	0.988901694
1.75	0.98676087
2	0.984538831
2.25	0.982181264
2.5	0.979730755
2.75	0.977188564
3	0.974555437
3.25	0.971950686
3.5	0.969278409
3.75	0.966535505
4	0.963722613
4.25	0.961068253
4.5	0.958376016
4.75	0.955641483
5	0.952865058
5.25	0.950270281
5.5	0.947659567
5.75	0.945028351
6	0.942376844
6.25	0.939705257
6.5	0.937013803
6.75	0.934302698
7	0.931572156
7.25	0.928877783
7.5	0.926169169
7.75	0.923445295
8	0.920706339
8.25	0.917952486
8.5	0.915183917
8.75	0.912400815
9	0.909603364
9.25	0.906791747
9.5	0.90396615
9.75	0.901126758
10	0.898273755
10.25	0.895464659
10.5	0.892646737
10.75	0.88981824
11	0.886979325
11.25	0.884130149
11.5	0.881270868
11.75	0.878401641
12	0.875522625
12.25	0.873602529
12.5	0.871752303
12.75	0.869928181
13	0.868130016
13.25	0.866357662
13.5	0.864610977
13.75	0.862889819
14	0.86119405
14.25	0.859523534
14.5	0.857878137
14.75	0.856257727
15	0.854662173
15.25	0.853010285
15.5	0.851375882
15.75	0.849763481
16	0.848172968
16.25	0.846604233
16.5	0.845057165
16.75	0.843531656
17	0.842027599
17.25	0.84054489
17.5	0.839083425
17.75	0.837643102
18	0.836223821
18.25	0.834825483
18.5	0.833447992
18.75	0.832091252
19	0.830755168
19.25	0.829439648
19.5	0.828144602
19.75	0.82686994
20	0.825615574
20.25	0.824002891
20.5	0.822370525
20.75	0.820749539
21	0.819139878
21.25	0.817541487
21.5	0.815954313
21.75	0.814378303
22	0.812813403
22.25	0.811259562
22.5	0.809716728
22.75	0.808184848
23	0.806663873
23.25	0.80515375
23.5	0.803654431
23.75	0.802165865
24	0.800688002
24.25	0.799220794
24.5	0.797764192
24.75	0.796318147
25	0.794882613
25.25	0.79321226
25.5	0.791521382
25.75	0.789836583
26	0.788157839
26.25	0.786485127
26.5	0.784818423
26.75	0.783157704
27	0.781502947
27.25	0.77985413
27.5	0.778211228
27.75	0.77657422
28	0.774943083
28.25	0.773317795
28.5	0.771698332
28.75	0.770084673
29	0.768476795
29.25	0.766874677
29.5	0.765278295
29.75	0.763687629
30	0.762102657
30.25	0.760522364
30.5	0.758945358
30.75	0.757371622
31	0.755801149
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end
    
    %simulation testCase
    for idx=1:size(rawData,1)
        rawData(idx,2) = rawData(idx,2) - maxIdx*zcStep + (idx2-1)*zcStep;
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','KRWZero'});
    zeroCurveMktData = MktData(mktData);
    zeroCurveMktData.params('convention') = {};
    zeroCurve =  ZeroCurve(zeroCurveMktData);

    KRWAllMktData =containers.Map('zeroCurve',zeroCurve);

    %% import and construct a discountCurve
    rawData = [0.002739726	0.999985206
0.25	0.998821159
0.5	0.997240293
0.75	0.995522659
1	0.993714564
1.5	0.989185872
2	0.98374605
2.5	0.978529211
3	0.972872405
3.5	0.965963711
4	0.958392048
4.5	0.950185292
5	0.941360816
5.5	0.934008574
6	0.926435713
6.5	0.918628004
7	0.91059244
7.5	0.90233617
8	0.893866497
8.5	0.885190857
9	0.876316818
9.5	0.867252061
10	0.858004374
10.5	0.850986315
11	0.844022798
11.5	0.83707572
12	0.830145614
12.5	0.82323301
13	0.816338427
13.5	0.80946238
14	0.802605377
14.5	0.795767918
15	0.788950494
15.5	0.782153593
16	0.775377693
16.5	0.768623265
17	0.761890774
17.5	0.755180677
18	0.748493424
18.5	0.741829458
19	0.735189213
19.5	0.728573118
20	0.721981594
20.5	0.716100959
21	0.710294835
21.5	0.704535912
22	0.698823805
22.5	0.693158131
23	0.687538514
23.5	0.681964577
24	0.676435948
24.5	0.670952259
25	0.665513141
25.5	0.660118234
26	0.654767175
26.5	0.649459608
27	0.644195179
27.5	0.638973535
28	0.633794329
28.5	0.628657214
29	0.623561848
29.5	0.618507889
30	0.613495002
    ];

    for idx=1:size(rawData,1)
        rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
    end

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawData,valueDate,'KRW','discountZero'});
    discountCurveMktData = MktData(mktData);
    discountCurveMktData.params('convention') = {};
    discountCurve =  ZeroCurve(discountCurveMktData);

    KRWAllMktData('discountCurve') = discountCurve;

    %% import and construct a swaptionVolSurfaceNormal



% % GIROPremiumToNormalVol From LIB
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.0035	0.0038	0.0041	0.0044	0.0047	0.0049	0.0052	0.0053	0.0050	0.0044
% 2	0.0042	0.0043	0.0044	0.0045	0.0046	0.0047	0.0049	0.0049	0.0046	0.0040
% 3	0.0044	0.0044	0.0044	0.0044	0.0044	0.0046	0.0048	0.0049	0.0044	0.0039
% 4	0.0045	0.0045	0.0045	0.0045	0.0044	0.0046	0.0048	0.0048	0.0042	0.0039
% 5	0.0046	0.0046	0.0045	0.0045	0.0045	0.0046	0.0047	0.0047	0.0041	0.0039
% 7	0.0046	0.0046	0.0046	0.0046	0.0045	0.0046	0.0048	0.0048	0.0044	0.0043
% 10	0.0047	0.0047	0.0047	0.0047	0.0046	0.0047	0.0042	0.0043	0.0042	0.0043
% 15	0.0032	0.0031	0.0029	0.0028	0.0026	0.0031	0.0033	0.0035	0.0037	0.0040
% 20	0.0037	0.0037	0.0037	0.0037	0.0037	0.0038	0.0038	0.0039	0.0040	0.0041
%     ];

% SwaptionVol From FrontSystem 
% % 20 x 20
% rawDataSwaption = [0	1	2	3	4	5	7	10	12	15	20
% 1	0.00321772	0.00354476	0.00389697	0.00424674	0.00459727	0.00480495	0.00512276	0.00527918	0.00527918	0.00527918
% 2	0.00380361	0.00393397	0.0040487	0.0041681	0.00427948	0.00447705	0.00478293	0.00485168	0.00485168	0.00485168
% 3	0.00396916	0.00404699	0.00413234	0.00419804	0.00427601	0.00445133	0.00471047	0.00476705	0.00476705	0.00476705
% 4	0.00406143	0.00411377	0.00417249	0.00421175	0.00426608	0.0044168	0.00464154	0.00468129	0.00468129	0.00468129
% 5	0.00414985	0.0041761	0.0041968	0.00422444	0.00425391	0.00438098	0.00457183	0.00459557	0.00459557	0.00459557
% 7	0.00415456	0.00420399	0.00425019	0.00429059	0.00432913	0.00446055	0.00466101	0.00468814	0.00468814	0.00468814
% 10	0.00420754	0.00427513	0.00431915	0.00438068	0.0044432	0.00460262	0.00484237	0.00486215	0.00486215	0.00486215
% 15	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664	0.00488664
% 20	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664	0.00488664
%     ];

% 15 x 15
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.00321772	0.00354476	0.00389697	0.00424674	0.00459727	0.00480495	0.00512276	0.00527918	0.00527918
2	0.00380361	0.00393397	0.0040487	0.0041681	0.00427948	0.00447705	0.00478293	0.00485168	0.00485168
3	0.00396916	0.00404699	0.00413234	0.00419804	0.00427601	0.00445133	0.00471047	0.00476705	0.00476705
4	0.00406143	0.00411377	0.00417249	0.00421175	0.00426608	0.0044168	0.00464154	0.00468129	0.00468129
5	0.00414985	0.0041761	0.0041968	0.00422444	0.00425391	0.00438098	0.00457183	0.00459557	0.00459557
7	0.00415456	0.00420399	0.00425019	0.00429059	0.00432913	0.00446055	0.00466101	0.00468814	0.00468814
10	0.00420754	0.00427513	0.00431915	0.00438068	0.0044432	0.00460262	0.00484237	0.00486215	0.00486215
15	0.00422073	0.00429626	0.00434327	0.0044046	0.00446601	0.00462814	0.00486878	0.00488664	0.00488664
    ];

    mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                            {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
    swaptionVolMktData = MktData(mktData);
    KRWAllMktData('swaptionVol') = swaptionVolMktData;

    %% create  a KRWBlackModel to calculate market Swaption
    modelParams= containers.Map({'modelName'},{'Black'});
    KRWBlack = IRBlack(IRModel(Model(KRWAllMktData,modelParams))); 

    expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
    tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
    swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
    swaptionPriceSurf.expiry= expiry;
    swaptionPriceSurf.tenor = tenor;
    
    fwdSwapRate = zeros(size(rawDataSwaption,1)-1,size(rawDataSwaption,2)-1);

    tempdHTenor = zeros(size(expiry,1),size(tenor,1));
    for i=1:size(expiry,1)
        for j=1:size(tenor,1)
            fwdSwapRate(i,j) = KRWBlack.Fsr(expiry(i),expiry(i)+tenor(j),0.25);
            swaptionPriceSurf.surface(i,j)= KRWBlack.NormalATMSwaption(expiry(i),tenor(j));
            swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
            tempdHTenor(i,j) = expiry(i) + tenor(j);
        end
    end
    
  %% create a LGM2FModel with initial modelParams
% rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

%swaption only initial parameters
% dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

%Samsung
% dHTest = [0.058507682	0.500210296
% 0.06145709	0.515096302
% 0.057841075	0.632558982
% 0.069442883	0.632845436
% 0.081752868	0.607322125
% 0.087574376	0.586095839
% 0.103284643	0.52688632
% 0.113936628	0.552598585
% 0.131046999	0.498409755
% 0.153553385	0.437472215
% 0.173381122	0.423805445
% 0.180703372	0.380344606
% 0.207484315	0.305927652
% 0.218329282	0.268192317
% 0.224885432	0.227191192
% 0.221244544	0.231204789
% 0.262502203	0.212209604
% 0.264579505	0.135257044
% 0.318821555	0.118179799
% 0.271515922	0.107957745
% 0.358024755	0.088834433
% 0.33143161	0.069384916
% 0.457059078	0.033745097
% 0.374067262	0.006573072
% ];

%Daewoo
% dHTest = [0.037286763	0.525314433
% 0.037286763	0.525314433
% 0.054571525	0.586928199
% 0.069289617	0.618362181
% 0.099111958	0.605543022
% 0.1123501	0.589459841
% 0.139006516	0.523225356
% 0.175987354	0.505500158
% 0.184616726	0.463242812
% 0.204164537	0.412151488
% 0.263444475	0.364947643
% 0.251615512	0.358337407
% 0.259708458	0.315375986
% 0.269854863	0.270026782
% 0.337623406	0.253701114
% 0.258270519	0.179914465
% 0.321985902	0.191343978
% 0.347503907	0.154371663
% 0.36490148	0.127052474
% 0.36706888	0.095165077
% 0.480469769	0.05084228
% 0.423787026	0.033262976
% 0.531944266	0.006463726
% 0.333342463	0.000100001
% ];

% %Sungwoo
% dHTest = [0.060654929	0.585843811
% 0.065181321	0.621357434
% 0.059213061	0.705433024
% 0.070356009	0.697274699
% 0.084854535	0.641491331
% 0.078986949	0.60204605
% 0.103979263	0.504724505
% 0.128905871	0.522635178
% 0.142868097	0.436948669
% 0.167927649	0.341253642
% 0.171414266	0.384418469
% 0.173137707	0.332946905
% 0.205465936	0.252922264
% 0.190811162	0.233849244
% 0.212134317	0.235149902
% 0.208305697	0.261176412
% 0.267225845	0.213982892
% 0.244169602	0.131637885
% 0.293167102	0.110338057
% 0.302235653	0.107810085
% 0.341199539	0.095697121
% 0.353976	0.072468098
% 0.421517994	0.033547027
% 0.357938773	0.006568033
% ];

%JaeYeol
dHTest = [0.058982775	0.543001377
0.061234997	0.56742444
0.062961086	0.622386628
0.067490351	0.649674014
0.075410597	0.63934757
0.087663903	0.593591285
0.112669112	0.501377655
0.117220704	0.537782529
0.140692951	0.444873509
0.168027009	0.390384507
0.182609655	0.410559905
0.184911175	0.327958872
0.214621342	0.297359459
0.213082379	0.268233279
0.228595305	0.243594185
0.220204578	0.241085585
0.27611566	0.202850389
0.244581841	0.134958255
0.314853582	0.120816977
0.317684576	0.109773131
0.325368543	0.100229783
0.348723657	0.073457815
0.31747133	0.033017077
0.359049924	0.006573421
];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

%Samsung
% AlphaTest = [0.0289581	0.006633509
% 0.021585088	0.007003076
% 0.019812596	0.007056061
% 0.015014178	0.007801684
% 0.008683651	0.008360616
% 0.011075137	0.008248663
% 0.008794173	0.011226664
% 0.000329846	0.023982837
% ];

%Daewoo
% AlphaTest = [0.029142908	0.007983918
% 0.016018045	0.007219568
% 0.00912537	0.007233927
% 0.008101129	0.007774553
% 0.012850209	0.008916743
% 0.011065789	0.009477842
% 0.008930619	0.011214209
% 0.0001	0.032096513
% ];

% %sungwoo
% AlphaTest = [0.032526561	0.005901286
% 0.021888486	0.006159185
% 0.015500653	0.006183779
% 0.011483817	0.007702683
% 0.008112119	0.008910715
% 0.010323471	0.008310996
% 0.006966815	0.012340985
% 0.000302754	0.020086129
% ];

%jaeweol
AlphaTest = [0.027941111	0.006258382
0.021327126	0.006691243
0.015009253	0.006502172
0.012188678	0.007718057
0.010231971	0.009065476
0.006437447	0.009477921
0.004646654	0.012890028
0.004203719	0.024248438
];


Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% import irModels.*;
% import targetFunctions.*;
                    
KRWLGM = LGM2F(IRModel(Model(KRWAllMktData,modelParams)));


tic
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
out = KRWLGM.CalibrateToATMSwaptionSurfaceNormal(KRWBlack,'global',200);
% out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'dummy');

dH1Calib = KRWLGM.dH1;
dH2Calib = KRWLGM.dH2;
Alpha1Calib = KRWLGM.Alpha1;
Alpha2Calib = KRWLGM.Alpha2;

modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});
                    
                    
%% allFixings
%% CMS10Y & CMS2Y
tempDate = steepenerParams.startDate;

%CMS10Y
irIndexFixing1 = containers.Map({StrDate(tempDate,'str')},{0.0107});

%CMS2Y
irIndexFixing2 = containers.Map({StrDate(tempDate,'str')},{0.0078});

for idxhong=1:2000
    tempDate = AddDate(tempDate,1,'day');
    irIndexFixing1(StrDate(tempDate,'str')) = 0.0107;
    irIndexFixing2(StrDate(tempDate,'str')) = 0.0078;
end

%% fixing info -> let it be the modelParams !

allFixings = containers.Map({'CMS10Y'},{irIndexFixing1});
allFixings('CMS2Y') = irIndexFixing2;

modelParamsCalib('allFixings') = allFixings;

KRWLGM2FRisky = LGM2FRisky(IRModelRisky(Model(KRWAllMktData,modelParamsCalib)));
    

toc
    %% LGM2FRisky model building end
    %% KRW model building end
    %% NumMethod Init Start
    numOfRandom = 40000;
    
    numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'},...
                            {'AMC', 7, numOfRandom});

    numMethodAMC = NumMethod(numMethodInfo);
    %% NumMethod Init End
    
    pricingOutMC1 = KRWLGM2FRisky.computeAVGSteepener(valueDateH,steepenerParams,numMethodAMC);
    aaa = 1.0;

    
    callablePrice(idx2) = pricingOutMC1.callable.npv;
    nonCallPrice(idx2) = pricingOutMC1.nonCall.npv;
    nonCallBPrice(idx2) = pricingOutMC1.nonCallB.npv;
end

aaa =1.0;


In [ ]:
%test_cms_spread_ra_lgm2f_DLS508_exFlag.m

clc
clear
% valueDateOrg = '2020-01-21';
valueDateOrg = '2017-02-16';

outCallable = zeros(14, 1);
outNonCallable = zeros(14, 1);

for iterNo = 1:14
    
tic

targetDate = H_Date(valueDateOrg).AddDate(iterNo, 'day').StrDate('str');
valueDate = targetDate;

%% import and construct a zeroCurve
rawData = [0.002739726	0.9999649330
0.25	0.9963717000
0.5	0.9928670970
0.75	0.9893915640
1	0.9859285940
1.25	0.9824269620
1.5	0.9789298760
1.75	0.9754609210
2	0.9720042030
2.25	0.9684667130
2.5	0.9649190740
2.75	0.9613633300
3	0.9577996660
3.25	0.9542254040
3.5	0.9506429590
3.75	0.9470527090
4	0.9434548440
4.25	0.9399247180
4.5	0.9363982910
4.75	0.9328732340
5	0.9293496290
5.25	0.9258949900
5.5	0.9224502540
5.75	0.9190132400
6	0.9155839570
6.25	0.9121624130
6.5	0.9087486190
6.75	0.9053425830
7	0.9019443130
7.25	0.8984314990
7.5	0.8949147190
7.75	0.8913981110
8	0.8878817790
8.25	0.8843658310
8.5	0.8808503720
8.75	0.8773355070
9	0.8738213400
9.25	0.8703079760
9.5	0.8667955180
9.75	0.8632840710
10	0.8597737350
10.25	0.8563553420
10.5	0.8529461840
10.75	0.8495422340
11	0.8461435370
11.25	0.8427501400
11.5	0.8393620870
11.75	0.8359794240
12	0.8326021960
12.25	0.8295823020
12.5	0.8265959400
12.75	0.8236271340
13	0.8206757730
13.25	0.8177417440
13.5	0.8148249380
13.75	0.8119252460
14	0.8090425580
14.25	0.8061767660
14.5	0.8033277640
14.75	0.8004954450
15	0.7976797030
15.25	0.7951603550
15.5	0.7926846300
15.75	0.7902329080
16	0.7878050130
16.25	0.7854007710
16.5	0.7830200130
16.75	0.7806625690
17	0.7783282710
17.25	0.7760169540
17.5	0.7737284550
17.75	0.7714626120
18	0.7692192650
18.25	0.7669982570
18.5	0.7647994320
18.75	0.7626226360
19	0.7604677160
19.25	0.7583345210
19.5	0.7562229040
19.75	0.7541327160
20	0.7520638130
20.25	0.7497607730
20.5	0.7474491620
20.75	0.7451531780
21	0.7428727200
21.25	0.7406076890
21.5	0.7383579860
21.75	0.7361235140
22	0.7339041750
22.25	0.7316998740
22.5	0.7295105140
22.75	0.7273360020
23	0.7251762440
23.25	0.7230311460
23.5	0.7209006160
23.75	0.7187845630
24	0.7166828970
24.25	0.7145955260
24.5	0.7125223630
24.75	0.7104633180
25	0.7084183050
25.25	0.7061022050
25.5	0.7037607780
25.75	0.7014288200
26	0.6991062890
26.25	0.6967931420
26.5	0.6944893370
26.75	0.6921948330
27	0.6899095860
27.25	0.6876335560
27.5	0.6853667010
27.75	0.6831089800
28	0.6808603510
28.25	0.6786207750
28.5	0.6763902090
28.75	0.6741686150
29	0.6719559510
29.25	0.6697521770
29.5	0.6675572540
29.75	0.6653711430
30	0.6631938020
30.25	0.6610245410
30.5	0.6588623830
30.75	0.6567072980
31	0.6545592610
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','KRWZero'});
zeroCurveMktData = MktData(mktData);
zeroCurveMktData.params('convention') = {};
zeroCurve =  ZeroCurve(zeroCurveMktData);
AllMktData =containers.Map('zeroCurve',zeroCurve);

%% import and construct a discountCurve


rawData = [0.002739726	0.9999649330
0.25	0.9969051350
0.5	0.9936464120
0.75	0.9903030680
1	0.9868714260
1.5	0.9795532300
2	0.9717455410
2.5	0.9647073440
3	0.9576967790
3.5	0.9497533300
4	0.9415365330
4.5	0.9330818200
5	0.9243969010
5.5	0.9162108330
6	0.9079396460
6.5	0.8995762020
7	0.8911244400
7.5	0.8825883120
8	0.8739717780
8.5	0.8652788050
9	0.8565133620
9.5	0.8476794190
10	0.8387809440
10.5	0.8314116050
11	0.8241413460
11.5	0.8169353300
12	0.8097929820
12.5	0.8027137340
13	0.7956970230
13.5	0.7887422910
14	0.7818489860
14.5	0.7750165580
15	0.7682444650
15.5	0.7615321690
16	0.7548791360
16.5	0.7482848370
17	0.7417487500
17.5	0.7352703540
18	0.7288491360
18.5	0.7224845850
19	0.7161761960
19.5	0.7099234700
20	0.7037259090
20.5	0.6979117140
21	0.6921773390
21.5	0.6865071400
22	0.6809003110
22.5	0.6753560550
23	0.6698735900
23.5	0.6644521420
24	0.6590909490
24.5	0.6537892600
25	0.6485463330
25.5	0.6433614380
26	0.6382338540
26.5	0.6331628700
27	0.6281477870
27.5	0.6231879120
28	0.6182825640
28.5	0.6134310710
29	0.6086327710
29.5	0.6038870100
30	0.5991931430
];

for idx=1:size(rawData,1)
    rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
end

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawData,valueDate,'KRW','discountZero'});
discountCurveMktData = MktData(mktData);
discountCurveMktData.params('convention') = {};
discountCurve =  ZeroCurve(discountCurveMktData);
AllMktData('discountCurve') = discountCurve;

%% import and construct a swaptionVolSurface
rawDataSwaption = [0	1	2	3	4	5	7	10	12	15
1	0.241106940	0.250694550	0.270350310	0.287338780	0.305136100	0.315490940	0.322454660	0.322246670	0.322246670
2	0.271254280	0.276567820	0.283799570	0.291111480	0.297593970	0.293795700	0.301006020	0.301123060	0.301123060
3	0.296268100	0.295401400	0.298796910	0.298035420	0.294466020	0.294147210	0.300855010	0.304765110	0.304765110
4	0.309430010	0.309899750	0.302818260	0.295625000	0.289203390	0.288464650	0.297988750	0.301304040	0.301304040
5	0.328387090	0.322707770	0.306877030	0.290025120	0.278120180	0.282504480	0.300596690	0.307035130	0.307035130
7	0.314639550	0.306959630	0.297175780	0.283334630	0.270126590	0.283176560	0.312008640	0.319472970	0.319472970
10	0.307500590	0.306734590	0.316867430	0.312613670	0.292554400	0.319997590	0.352026220	0.355798230	0.355798230
15	0.448647240	0.475518250	0.472610460	0.461280170	0.440268510	0.431768220	0.421909850	0.404699510	0.404699510
];

mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
                        {rawDataSwaption,valueDate,'KRW','KRWSwaptionVol'});
swaptionVolMktData = MktData(mktData);
AllMktData('swaptionVol') = swaptionVolMktData;

%% create  a BlackModel to calculate market Swaption
modelParams= containers.Map({'modelName'},{'Black'});
KRWBlack = IRBlack(IRModel(Model(AllMktData,modelParams))); 

expiry = rawDataSwaption(2:size(rawDataSwaption,1),1);
tenor  = rawDataSwaption(1,2:size(rawDataSwaption,2))';
swptnVol= rawDataSwaption(2:size(rawDataSwaption,1),2:size(rawDataSwaption,2));
swaptionPriceSurf.expiry= expiry;
swaptionPriceSurf.tenor = tenor;

tempdHTenor = zeros(size(expiry,1),size(tenor,1));
for i=1:size(expiry,1)
    for j=1:size(tenor,1)
        swaptionPriceSurf.surface(i,j)= KRWBlack.BlackATMSwaption(expiry(i),tenor(j));
        swaptionVolSurf(i,j) = KRWBlack.swaptionVol(expiry(i),tenor(j));
        tempdHTenor(i,j) = expiry(i) + tenor(j);
    end
end
%% create a LGM2FModel with initial modelParams
% rawDataSwaption = importdata('C:\Data\KRWSwaptionVol20160215_15_15.txts');
tempdHTenor1 = reshape(tempdHTenor,size(expiry,1)*size(tenor,1),1);
dH1.tenor = sort(union(tenor,tempdHTenor1),'ascend');
dH2.tenor = dH1.tenor;

%swaption only initial parameters
% dH1.tenor = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];

dHTest = [0.0589827750	0.5430013770
0.0612349970	0.5674244400
0.0629610860	0.6223866280
0.0674903510	0.6496740140
0.0754105970	0.6393475700
0.0876639030	0.5935912850
0.1126691120	0.5013776550
0.1172207040	0.5377825290
0.1406929510	0.4448735090
0.1680270090	0.3903845070
0.1826096550	0.4105599050
0.1849111750	0.3279588720
0.2146213420	0.2973594590
0.2130823790	0.2682332790
0.2285953050	0.2435941850
0.2202045780	0.2410855850
0.2761156600	0.2028503890
0.2445818410	0.1349582550
0.3148535820	0.1208169770
0.3176845760	0.1097731310
0.3253685430	0.1002297830
0.3487236570	0.0734578150
0.3174713300	0.0330170770
0.3590499240	0.0065734210
];

dH1.quote = dHTest(:,1);
dH2.quote = dHTest(:,2);


dH1Orig = dH1;
dH2Orig = dH2;

% Alpha1.tenor =  [1;2;3;4;5;7;10;15];
Alpha1.tenor =  expiry;
Alpha2.tenor =  Alpha1.tenor;
correl.tenor = Alpha1.tenor;
correl.quote = zeros(length(expiry),1);

%swaption only initial parameters

AlphaTest = [0.0279411110	0.0062583820
0.0213271260	0.0066912430
0.0150092530	0.0065021720
0.0121886780	0.0077180570
0.0102319710	0.0090654760
0.0064374470	0.0094779210
0.0046466540	0.0128900280
0.0042037190	0.0242484380
];

Alpha1.quote = AlphaTest(:,1);
Alpha2.quote = AlphaTest(:,2);

smoothdH = 1.0;
smoothAlpha = 1.0;
smoothTermCorr =0.1;
targetTermCorr = 0.92;

freq = 4;

modelParams= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','smoothTermCorr','targetTermCorr','freq'}, ...
                        {dH1,Alpha1,dH2,Alpha2,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,smoothTermCorr,targetTermCorr,freq});
% import irModels.*;
% import targetFunctions.*;
                    
KRWLGM = LGM2F(IRModel(Model(AllMktData,modelParams)));

% tic
out = KRWLGM.CalibrateToATMSwaptionSurface(KRWBlack,'global',200);
% toc

%% file io
% fileNameInsFwdCorr = strcat('KRWLGM_','InsFwdCorr','_',valueDate,'.txt');
% fileIDInsFwdCorr = fopen(fileNameInsFwdCorr,'w');
% 
% fileNameInsSwapFwdCorr = strcat('KRWLGM_','InsSwapFwdCorr','_',valueDate,'.txt');
% fileIDInsSwapFwdCorr = fopen(fileNameInsSwapFwdCorr,'w');
% 
% fileNameImpCorrel = strcat('KRWLGM_','ImpCorrel ','_',valueDate,'.txt');
% fileIDImpCorrel  = fopen(fileNameImpCorrel,'w');
% 
% % investigaion on the implied correlation
% newexpiry = [1,2,3,4,5,7,10,15];

%newexpiry = [1,2,3,4,5,7,10];

% newtenor  = newexpiry;
% calTimes = [0:1:15]';
% testLGMKRW.InsFwdCorr = cell(length(calTimes),1);
% testLGMKRW.InsSwapFwdCorr = cell(length(calTimes),1);
% testLGMKRW.ImpCorrel = cell(length(calTimes),1);
% 
% testLGMKRW.InsFwdCovar = cell(length(calTimes),1);
% testLGMKRW.InsSwapFwdCovar = cell(length(calTimes),1);
% testLGMKRW.ImpCovar = cell(length(calTimes),1);
% 
% for i=1:length(calTimes)
%     for j=1:length(newexpiry)
%         for k=1:length(newtenor)
%             testLGMKRW.InsFwdCovar{i,1}.quote(j,k)= KRWLGM.InsFwdCovarTest(calTimes(i),newexpiry(j),newtenor(k));
%             testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWLGM.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
%             testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWLGM.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
%             
%             testLGMKRW.InsFwdCorr{i,1}.quote(j,k)= KRWLGM.InsFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
%             testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k)= KRWLGM.InsSwapFwdCorrTest(calTimes(i),newexpiry(j),newtenor(k));
%             testLGMKRW.ImpCorrel{i,1}.quote(j,k)= KRWLGM.ImpCorrel(calTimes(i),newexpiry(j),newtenor(k));
%             
%             fprintf(fileIDInsFwdCorr,'%f\t',testLGMKRW.InsFwdCorr{i,1}.quote(j,k));
%             fprintf(fileIDInsSwapFwdCorr,'%f\t',testLGMKRW.InsSwapFwdCorr{i,1}.quote(j,k));
%             fprintf(fileIDImpCorrel,'%f\t',testLGMKRW.ImpCorrel{i,1}.quote(j,k));
% 
%         end
%         fprintf(fileIDInsFwdCorr,'\n');
%         fprintf(fileIDInsSwapFwdCorr,'\n');
%         fprintf(fileIDImpCorrel,'\n');
%     end
%     fprintf(fileIDInsFwdCorr,'\n');
%     fprintf(fileIDInsFwdCorr,'\n');
%     
%     fprintf(fileIDInsSwapFwdCorr,'\n');
%     fprintf(fileIDInsSwapFwdCorr,'\n');
%     
%     fprintf(fileIDImpCorrel,'\n');
%     fprintf(fileIDImpCorrel,'\n');
%     
% end
% 
% fclose(fileIDInsFwdCorr);
% fclose(fileIDInsSwapFwdCorr);
% fclose(fileIDImpCorrel);



dH1Calib = KRWLGM.dH1;
dH2Calib = KRWLGM.dH2;
Alpha1Calib = KRWLGM.Alpha1;
Alpha2Calib = KRWLGM.Alpha2;

modelParamsCalib= containers.Map({'dH1','Alpha1','dH2','Alpha2','correl','modelName','dH1Orig','dH2Orig','smoothdH','smoothAlpha','freq'}, ...
                        {dH1Calib,Alpha1Calib,dH2Calib,Alpha2Calib,correl,'LGM2F',dH1Orig,dH2Orig,smoothdH,smoothAlpha,freq});

KRWLGMRisky = LGM2FRisky(IRModelRisky(Model(AllMktData,modelParamsCalib)));
%% LGM model building end
%% CMS CMSRangeAccrual valuation start
%RA info
rangeAccrualParams.payCcy = 'KRW';
rangeAccrualParams.startDate = H_Date('2017-02-17');
rangeAccrualParams.maturity = 15;

% valueDateH = H_Date('2017-04-06');
valueDateH = H_Date(valueDate);

nominal = 10000;
coupon = 0.0362;
couponFreq = 1;
tenorL = 10;
tenorS = 2;
lower = 0.0;
upper = 999;
rangeDays = zeros(15,1);
%rangeDays(1) = 1;
% rangeDays(1) = DateDiff(valueDateH,rangeAccrualParams.startDate) + 1;
rangeDays(1) = 365;
rangeDays(2) = 365;
rangeDays(3) = 93 + (iterNo + 1);

callYN = ones(15,1);
callYN(1) = 0;
callYN(2) = 0;
callYN(3) = 0;

%resetDate startDate endDate paymentDate
couponScheduleInt = [20170217	20170217	20180217	20180219
20180214	20180217	20190217	20190218
20190215	20190217	20200217	20200217
20200217	20200217	20210217	20210217
20210217	20210217	20220217	20220217
20220217	20220217	20230217	20230217
20230217	20230217	20240217	20240219
20240216	20240217	20250217	20250217
20250217	20250217	20260217	20260219
20260213	20260217	20270217	20270217
20270217	20270217	20280217	20280217
20280217	20280217	20290217	20290219
20290216	20290217	20300217	20300218
20300215	20300217	20310217	20310217
20310217	20310217	20320217	20320217
];

% couponScheduleInt = [20170217	20170217	20180217	20180217
% 20180217	20180217	20190217	20190217
% 20190217	20190217	20200217	20200217
% 20200217	20200217	20210217	20210217
% 20210217	20210217	20220217	20220217
% 20220217	20220217	20230217	20230217
% 20230217	20230217	20240217	20240217
% 20240217	20240217	20250217	20250217
% 20250217	20250217	20260217	20260217
% 20260217	20260217	20270217	20270217
% 20270217	20270217	20280217	20280217
% 20280217	20280217	20290217	20290217
% 20290217	20290217	20300217	20300217
% 20300217	20300217	20310217	20310217
% 20310217	20310217	20320217	20320217
% ];

%resetDate startDate endDate paymentDate
callScheduleInt = [20170217	20170217	20170217	20170217
20180217	20180217	20180217	20180217
20190217	20190217	20190217	20190217
20200210	20200210	20200210	20200217
20210208	20210208	20210208	20210217
20220210	20220210	20220210	20220217
20230210	20230210	20230210	20230217
20240208	20240208	20240208	20240219
20250210	20250210	20250210	20250217
20260209	20260209	20260209	20260219
20270210	20270210	20270210	20270217
20280210	20280210	20280210	20280217
20290207	20290207	20290207	20290219
20300211	20300211	20300211	20300218
20310210	20310210	20310210	20310217
];

% callScheduleInt = [20170217	20170217	20180217	20180217
% 20180217	20180217	20190217	20190217
% 20190217	20190217	20200217	20200217
% 20200217	20200217	20210217	20200217
% 20210217	20210217	20220217	20210217
% 20220217	20220217	20230217	20220217
% 20230217	20230217	20240217	20230217
% 20240217	20240217	20250217	20240217
% 20250217	20250217	20260217	20250217
% 20260217	20260217	20270217	20260217
% 20270217	20270217	20280217	20270217
% 20280217	20280217	20290217	20280217
% 20290217	20290217	20300217	20290217
% 20300217	20300217	20310217	20300217
% 20310217	20310217	20320217	20310217
% ];

manualScheduleYN = 1;

rangeAccrualInfo = containers.Map({'nominal','coupon','couponFreq','tenorL', ...
                                'tenorS','lower','upper','rangeDays','callYN', ...
                                'couponScheduleInt','callScheduleInt','manualScheduleYN'}, ...
                        {nominal,coupon,couponFreq,tenorL,tenorS,lower, ...
                        upper,rangeDays,callYN,couponScheduleInt,callScheduleInt,manualScheduleYN});

rangeAccrualParams.params = rangeAccrualInfo;              
% rangeAccrualParams.payCcy = 'KRW';
% rangeAccrualParams.startDate = H_Date('2017-02-17');
% rangeAccrualParams.maturity = 15;

rangeAccrualParams.isExercised = true;
% rangeAccrualParams.isExercised = false;
rangeAccrual = SpreadRangeAccrual(rangeAccrualParams);


%computePriceMCForward

% numOfRandomSteps = [100;500;1000;5000;10000;20000;40000;100000;150000;200000];
% testCallableRandom = zeros(length(numOfRandomSteps),1);
% testNonCallableRandom = zeros(length(numOfRandomSteps),1);
% 
% for i=1:length(numOfRandomSteps)
%     numOfRandom = numOfRandomSteps(i);    
%     numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
%                             {'AMC',7,numOfRandom});
% 
%     numMethodAMC = NumMethod(numMethodInfo);
% 
%     % weekly step calculator
%     % pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
%     % pricingOut = pricer.computePrice(valueDate);
% 
%     % daily weekly step calculator
%     pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
%     pricingOutMC = pricer.computePriceMCCallable(valueDateH);
%     testCallableRandom(i) = pricingOutMC.callable.npv;
%     testNonCallableRandom(i) = pricingOutMC.nonCall.npv;
% end



numOfRandom = 40000;    
numMethodInfo = containers.Map({'numMethodType','mcOneTimeStep','pathSize'}, ...
                        {'AMC',7,numOfRandom});

numMethodAMC = NumMethod(numMethodInfo);

% weekly step calculator
% pricer = Pricer(rangeAccrual,KRWLGM,numMethodAMC);
% pricingOut = pricer.computePrice(valueDate);

% daily weekly step calculator
% pricer = DailyWeeklyRiskyPricer(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
pricer = DailyWeeklyRiskyPricerExFlag(rangeAccrual,KRWLGMRisky,numMethodAMC,true);
pricingOutMC = pricer.computePriceMCCallable(valueDateH);

outCallable(iterNo, 1) = pricingOutMC.callable.npv;
outNonCallable(iterNo, 1) = pricingOutMC.nonCall.npv;

toc

aaa = 1.0;
% pricingOutClosed = pricer.computePriceClosedTRZ(valueDateH);

% daily closed form pricer
%pricingOutLGM =  rangeAccrual.computePrice(valueDate,KRWLGM);

%pricingOut = rangeAccrual.computePriceMCForward(valueDate,KRWLGM);

% % out =1;
% newexpiry = [1;2;3;4;5;7;10;15];
% 
% 
% testCK.impCorrel = zeros(size(newexpiry,1),size(newexpiry,1));
% testCK.InsSwapFwdCorr = zeros(size(newexpiry,1),size(newexpiry,1));
% testCK.impFwdLiborCorr = KRWLGM.InsFwdCorrTest2(0,newexpiry,0.25);
% for i=1:size(newexpiry,1)
%     for j=1:size(newexpiry,1)
%         testCK.impCorrel(i,j)= KRWLGM.ImpCorrel(5,newexpiry(i),newexpiry(j));
%         testCK.InsSwapFwdCorr(i,j) = KRWLGM.InsSwapFwdCorrTest(5,newexpiry(i),newexpiry(j));
%     end
% end
% 
% newexpiryFwd = KRWLGM.dH1.tenor;
% testCK.impFwdLiborCorrNew = KRWLGM.InsFwdCorrTest2(0,newexpiryFwd,0.25);
% 
% 
% newexpiry2 = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
% 
% for i=1:size(newexpiry2,1)
%     testCK.impCorrel10_2(i)= KRWLGM.ImpCorrel(newexpiry2(i),10,2);
%     testCK.impCovarl10_2(i)= sqrt(KRWLGM.ImpCovar(newexpiry2(i),10,2)/newexpiry2(i));
%     testCK.spreadInfo10_2(i) =  KRWLGM.SpreadInfo(newexpiry2(i),10,2);
%     testCK.fwdSpread(i) = testCK.spreadInfo10_2(i).fwdSpread';
%     testCK.muSpread(i) = testCK.spreadInfo10_2(i).mu';
%     testCK.sigma(i) = testCK.spreadInfo10_2(i).sigma';
%     testCK.corr(i) = testCK.spreadInfo10_2(i).corr;
%     testCK.mu1(i) =testCK.spreadInfo10_2(i).mu1;
%     testCK.mu2(i) =testCK.spreadInfo10_2(i).mu2;
%     testCK.sig1(i) =testCK.spreadInfo10_2(i).sig1;
%     testCK.sig2(i) =testCK.spreadInfo10_2(i).sig2;
%     
% end
% 
% aaaa =1.0;

end

aaaa = 1.0;

In [ ]:
%test_CCS_BasisCurveBootStrap_USDHKD_20220727_spot2D.m

clc
clear
tic

valueDate = '2022-07-27';


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;

% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);
%% Calendars Start
%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end

%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };



USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% HKHD Calendar
HKHD.name = 'HKHD';
HKHD.holidaysList = {'2021-01-01'
'2021-02-12'
'2021-02-15'
'2021-04-02'
'2021-04-05'
'2021-04-06'
'2021-05-19'
'2021-06-14'
'2021-07-01'
'2021-09-22'
'2021-10-01'
'2021-10-14'
'2021-12-27'
'2022-02-01'
'2022-02-02'
'2022-02-03'
'2022-04-05'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-05-09'
'2022-06-03'
'2022-07-01'
'2022-09-12'
'2022-10-04'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-01-23'
'2023-01-24'
'2023-01-25'
'2023-04-05'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-26'
'2023-06-22'
'2023-10-02'
'2023-10-23'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-02-12'
'2024-02-13'
'2024-03-29'
'2024-04-01'
'2024-04-04'
'2024-05-01'
'2024-05-15'
'2024-06-10'
'2024-07-01'
'2024-09-18'
'2024-10-01'
'2024-10-11'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-01-29'
'2025-01-30'
'2025-01-31'
'2025-04-04'
'2025-04-18'
'2025-04-21'
'2025-05-01'
'2025-05-05'
'2025-07-01'
'2025-10-01'
'2025-10-07'
'2025-10-29'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-12-25'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2028-04-14'
'2028-04-17'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2033-04-15'
'2033-04-18'
'2033-12-26'
'2034-04-07'
'2034-04-10'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-12-25'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2039-04-08'
'2039-04-11'
'2039-12-26'
'2040-03-30'
'2040-04-02'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-12-25'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-12-26'
'2045-04-07'
'2045-04-10'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-12-25'
'2049-01-01'
'2049-04-16'
'2049-04-19'

    };

HKHDCalendar = Calendar();
HKHDCalendar.name = HKHD.name;

for i=1:length(HKHD.holidaysList)
    HKHDCalendar.AddHoliday(HKHD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD JPY CCS Schedule Generation
% jointCalendar_USDHKDCCS = JointCalendar({USFDCalendar,HKHDCalendar});
jointCalendar_USDHKDCCS = JointCalendar({USFDCalendar,GBENCalendar,HKHDCalendar});

    
%% index fixing for USD SOFR Index start
    tempDate = H_Date('2021-10-25');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.01530000});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.01530000;
    end
%% index fixing for SOFR_index end

%% index fixing for Hibor3M start
    tempDate = H_Date('2021-10-25');
    irIndexFixing_Hibor3M = containers.Map({StrDate(tempDate,'str')},{0.021415500});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Hibor3M(StrDate(tempDate,'str')) = 0.021415500;
    end
%% index fixing for Hibor3M end


valueDateH = H_Date(valueDate);
%% import and construct an CCSCurve

dataType= {
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
};

dataTenor = {
'1M'
'2M'
'3M'
'4M'
'5M'
'6M'
'9M'
'12M'
'2Y'
'3Y'
'4Y'
'5Y'
'7Y'
'10Y'
'12Y'
'15Y'
};


% dataRate = {
% -0.008918500
% -0.014830000
% -0.017892250
% -0.019945250
% -0.021551000
% -0.022000000
% -0.025740500
% -0.027959750
% -0.003300000
% -0.003200000
% -0.003100000
% -0.003100000
% -0.003000000
% -0.002650000
% -0.002550000
% -0.002550000
% };

dataRate = {
-0.00891800
-0.014830000
-0.0178920
-0.0199450
-0.021551000
-0.022000000
-0.02574000
-0.027960
-0.003300000
-0.003200000
-0.003100000
-0.003100000
-0.003000000
-0.002650000
-0.002550000
-0.002550000
};

exchangeRate = 7.8493;
% exchangeRate = 7.84929990768433;



%% Construct USDSOFR_Curve

% forward & discount zeroCurve
forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_Hibor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_HKDBasis = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));


% rawData_SOFR = [
% 0.0082191780821917800	0.0155115111614577000
% 0.0191780821917808000	0.0231662696955027000
% 0.0383561643835616000	0.0231854374869727000
% 0.0575342465753425000	0.0231792732753092000
% 0.0849315068493151000	0.0231906397268176000
% 0.1698630136986300000	0.0238691847203881000
% 0.2575342465753420000	0.0256878985528601000
% 0.5068493150684930000	0.0291408833611058000
% 0.7479452054794520000	0.0303732043830678000
% 1.0054794520547900000	0.0306924948323285000
% 2.0027397260274000000	0.0294471091663581000
% 3.0027397260274000000	0.0275475183356320000
% 4.0027397260274000000	0.0263250282980152000
% 5.0027397260274000000	0.0257083517441060000
% 6.0109589041095900000	0.0254825579206226000
% 7.0082191780821900000	0.0254138307616813000
% 8.0054794520548000000	0.0254370370414108000
% 9.0054794520548000000	0.0255576866198680000
% 10.0082191780822000000	0.0257764383248286000
% 12.0136986301370000000	0.0263497096020728000
% 15.0109589041096000000	0.0269875965721440000
% 20.0136986301370000000	0.0269897620310076000
% 25.0164383561644000000	0.0259901024750434000
% 30.0219178082192000000	0.0249311602136004000
% 40.0328767123288000000	0.0218547725322423000
% 50.0356164383562000000	0.0186810030223569000
% ];

%2022-07-27

rawData_SOFR = [
0.0136986301369863000	0.0155115111614577000
0.0246575342465753000	0.0231662696955027000
0.0438356164383561000	0.0231854374869727000
0.0630136986301370000	0.0231792732753092000
0.0904109589041096000	0.0231906397268176000
0.1753424657534250000	0.0238691847203881000
0.2630136986301370000	0.0256878985528601000
0.5123287671232880000	0.0291408833611058000
0.7534246575342470000	0.0303732043830678000
1.0109589041095800000	0.0306924948323285000
2.0082191780821900000	0.0294471091663581000
3.0082191780821900000	0.0275475183356320000
4.0082191780821900000	0.0263250282980152000
5.0082191780821900000	0.0257083517441060000
6.0164383561643900000	0.0254825579206226000
7.0136986301369900000	0.0254138307616813000
8.0109589041095900000	0.0254370370414108000
9.0109589041095900000	0.0255576866198680000
10.0136986301370000000	0.0257764383248286000
12.0191780821918000000	0.0263497096020728000
15.0164383561644000000	0.0269875965721440000
20.0191780821918000000	0.0269897620310076000
25.0219178082192000000	0.0259901024750434000
30.0273972602740000000	0.0249311602136004000
40.0383561643836000000	0.0218547725322423000
50.0410958904110000000	0.0186810030223569000
];

% 2022-07-27
rawData_Hibor3M = [
0.0027397260273972600	0.0091349856869783300
0.2520547945205480000	0.0213579080125783000
1.0000000000000000000	0.0312428052857285000
2.0082191780821900000	0.0318168128986144000
3.0054794520547900000	0.0305712232657243000
4.0027397260274000000	0.0296389020644710000
5.0027397260274000000	0.0291752849094337000
7.0054794520548000000	0.0292267244134564000
10.0082191780822000000	0.0295951888372656000
];

% %2022-07-29 Ver 1
% rawData_Hibor3M = [
% 0.24657534246575300	0.0216265436680857000
% 0.99452054794520600	0.0313638714111886000
% 2.00273972602740000	0.0318785023907728000
% 3.00000000000000000	0.0306101308817079000
% 3.99726027397261000	0.0296668247492547000
% 4.99726027397261000	0.0291971116434191000
% 7.00000000000001000	0.0292423466607353000
% 10.00273972602740000	0.0296063232581489000
% ];

% %2022-07-29 Ver 2
% rawData_Hibor3M = [
% -0.0027397260273972600	0.0091349856869783300
% 0.2465753424657530000	0.0213579080125783000
% 0.9945205479452060000	0.0312428052857285000
% 2.0027397260274000000	0.0318168128986144000
% 3.0000000000000000000	0.0305712232657243000
% 3.9972602739726100000	0.0296389020644710000
% 4.9972602739726100000	0.0291752849094337000
% 7.0000000000000100000	0.0292267244134564000
% 10.0027397260274000000	0.0295951888372656000
% ];


% % USDHKD Basis curve zeros
% rawData_HKDBasis = [
% 0.087671232876712300	0.010646998847663800
% 0.169863013698630000	0.012895080109643100
% 0.257534246575342000	0.016826404961270400
% 0.336986301369863000	0.019244052620072800
% 0.419178082191781000	0.021367993117414200
% 0.506849315068493000	0.023577262003047900
% 0.747945205479452000	0.025979946126825500
% 1.005479452054790000	0.027142310579774100
% 2.002739726027400000	0.028609608753372700
% 3.002739726027400000	0.027437455620890600
% 4.002739726027400000	0.026595117863201600
% 5.002739726027400000	0.026125907619111400
% 7.008219178082190000	0.026276122443267700
% 10.008219178082200000	0.027019915336342200
% 12.013698630137000000	0.027125218038061000
% 15.010958904109600000	0.027113043010524100
% ];

% 2022-7-27
% USDHKD Basis curve zeros
rawData_HKDBasis = [
0.087671232876712300	0.010646998847663800
0.169863013698630000	0.012895080109643100
0.257534246575342000	0.016826404961270400
0.336986301369863000	0.019244052620072800
0.419178082191781000	0.021367993117414200
0.506849315068493000	0.023577262003047900
0.747945205479452000	0.025979946126825500
1.005479452054790000	0.027142310579774100
2.002739726027400000	0.028609608753372700
3.002739726027400000	0.027437455620890600
4.002739726027400000	0.026595117863201600
5.002739726027400000	0.026125907619111400
7.008219178082190000	0.026276122443267700
10.008219178082200000	0.027019915336342200
12.013698630137000000	0.027125218038061000
15.010958904109600000	0.027113043010524100
];

forwardCurve_SOFR.params('rawData')  = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

forwardCurve_Hibor3M.params('rawData')  = rawData_Hibor3M;
discountCurve_HKDBasis.params('rawData') = rawData_HKDBasis;

%% SOFR Rate Index start
sofrIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

sofrIndexParams('forecastTodaysFixing') = true;
sofrIndexParams('useRounding') = useRounding;
sofrIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
sofrIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
sofrIndexParams('historicalFixings') = irIndexFixing_SOFR;
overnightIndex_sofr = IborIndex(sofrIndexParams);
%% SOFR Rate Index end

%% Hibor3M Index Start
IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','HKD',0,HKHDCalendar,'ModifiedFollowing','Act365',false,forwardCurve_Hibor3M});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('historicalFixings') = irIndexFixing_Hibor3M;
HKD_Libor_3M = IborIndex(IborIndexParams);

% tonaIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                             {'1D','JPY',0,JPJNCalendar,'Following','Act365',false,forwardCurve_TONA});
% 
%                        
% tonaIndexParams('forecastTodaysFixing') = true;
% tonaIndexParams('useRounding') = useRounding;
% tonaIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
% tonaIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
% tonaIndexParams('historicalFixings') = irIndexFixing_TONA;
% overnightIndex_tona = IborIndex(tonaIndexParams);
%% Hibor3M end


rateHelpers = {};
        
for i=1:length(dataRate)
    if strcmp(dataType{i},'SwapPoint')
        % we are valuing on settleDate
%         % so settlegap(here fixingDays) is equal to 0
%         swapPointHelperParams = containers.Map({'exchangeRate','fixingDays','rate','rateTenor'}, ...
%                             {exchangeRate,0,dataRate{i},dataTenor{i}});
                        
        swapPointHelperParams = containers.Map({'exchangeRate','fixingDays','rate','rateTenor'}, ...
                            {exchangeRate,2,dataRate{i},dataTenor{i}});
        
        swapPointHelperParams('exchangeRate') = exchangeRate;
        swapPointHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        swapPointHelperParams('calendar') = jointCalendar_USDHKDCCS;
        swapPointHelperParams('convention') = 'ModifiedFollowing';
        swapPointHelperParams('foreignDCurve') = discountCurve_SOFR;
        
        swapPointHelper = SwapPointHelper(swapPointHelperParams);
    
        rateHelpers{end+1} = swapPointHelper;
        
    elseif strcmp(dataType{i},'CrossCurrencyBasisSwapRFR')
        
%      % Libor6M Leg
%         floatingLegTenor_L = '6M';
%         floatingLegConvention_L = 'ModifiedFollowing';
%         floatingLegDayCounter_L = 'Act360';
%         spotLag_L = 0;
%         payLag_L = 0;
% 
%         % Libor3M Leg
%         floatingLegTenor_S = '6M';
%         floatingLegConvention_S = 'ModifiedFollowing';
%         floatingLegDayCounter_S = 'Act360';
%         spotLag_S = 0;
%         payLag_S  = 0;
% 
%         calcCalendar = jointCalendar_USDIRS;
% 
% %             domesticFixedLegTenor = '6M';
% %             domesticFixedLegConvention = 'ModifiedFollowing';
% %             domesticFixedLegDayCounter = 'Act365';
% %             domesticFixedLegCalendar = jointCalendar_USDKRWCRS;
% 
%         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
%         spotLag = 0;
% 
%         singleCurrencyBasisSwapHelperParams = containers.Map({'floatingLegTenor_L','floatingLegConvention_L','floatingLegDayCounter_L','spotLag_L','payLag_L',...
%                                                         'floatingLegTenor_S','floatingLegConvention_S','floatingLegDayCounter_S','spotLag_S','payLag_S',...
%                                                         'liborIndex_L','liborIndex_S','discountCurve','calcCalendar'}, ...
%                                 {floatingLegTenor_L,floatingLegConvention_L,floatingLegDayCounter_L,spotLag_L,payLag_L,...,
%                                 floatingLegTenor_S,floatingLegConvention_S,floatingLegDayCounter_S,spotLag_S,payLag_S,...,
%                                 libor6M_Index_fwd,libor3M_Index_fwd,discountCurve_Libor3M,calcCalendar});
% 
%         singleCurrencyBasisSwapHelperParams('rate') = dataRate{i};
%         singleCurrencyBasisSwapHelperParams('rateTenor') = dataTenor{i};
%         singleCurrencyBasisSwapHelper = SingleCurrencyBasisSwapHelper(singleCurrencyBasisSwapHelperParams);
%         rateHelpers{end+1} = singleCurrencyBasisSwapHelper;
            
        domesticFloatingLegTenor = '3M';
        domesticFloatingLegConvention = 'ModifiedFollowing';
        domesticFloatingLegDayCounter = 'Act365';
        domesticFloatingLegCalendar = jointCalendar_USDHKDCCS;
        
        foreignFloatingLegTenor = '3M';
        foreignFloatingLegConvention = 'ModifiedFollowing';
        foreignFloatingLegDayCounter = 'Act360';
        foreignFloatingLegCalendar = jointCalendar_USDHKDCCS;
        
        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

%         spotLag = 0;
        spotLag = 2;
%         payLag  = 2;
        payLag  = 2;
        
        crossCurrencyBasisSwapHelperParams = containers.Map({'domesticIndex','domesticFloatingLegTenor','domesticFloatingLegConvention','domesticFloatingLegDayCounter','domesticFloatingLegCalendar',...
                                                        'foreignIndex','foreignFloatingLegTenor','foreignFloatingLegConvention','foreignFloatingLegDayCounter','foreignFloatingLegCalendar',...
                                                        'foreignDCurve','spotLag','payLag','discountCurve'}, ...
                                {HKD_Libor_3M,domesticFloatingLegTenor,domesticFloatingLegConvention,domesticFloatingLegDayCounter,domesticFloatingLegCalendar,...,
                                overnightIndex_sofr,foreignFloatingLegTenor,foreignFloatingLegConvention,foreignFloatingLegDayCounter,foreignFloatingLegCalendar,...
                                discountCurve_SOFR,spotLag,payLag, dummyCurve});

        crossCurrencyBasisSwapHelperParams('exchangeRate') = exchangeRate;
        crossCurrencyBasisSwapHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        crossCurrencyBasisSwapHelperParams('rate') = dataRate{i};
        crossCurrencyBasisSwapHelperParams('rateTenor') = dataTenor{i};
        
        payLagNotionalMaturityFlag  = true;
%         payLagNotionalMaturityFlag  = false;
        
        crossCurrencyBasisSwapHelperParams('payLagNotionalMaturityFlag') = payLagNotionalMaturityFlag;
        
        crossCurrencyBasisSwapHelper = CrossCurrencyBasisSwapRFRHelper(crossCurrencyBasisSwapHelperParams);
        rateHelpers{end+1} = crossCurrencyBasisSwapHelper;
    end
end

% forward & discount zeroCurve
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','discountCurve'},{rateHelpers,dummyCurve1});

tic
yieldCurve = YieldCurveCC(yieldCurveParams);
% out = yieldCurve.BootStrap();
out = yieldCurve.GlobalCalib();
toc

outInfo = struct;
outInfo.impliedRate = zeros(length(yieldCurve.rateHelpers),1);

outInfo.diff = zeros(length(yieldCurve.rateHelpers),1);
outInfo.NPV = zeros(length(yieldCurve.rateHelpers),1);

for i=1:length(yieldCurve.rateHelpers)
    
    if strcmp(yieldCurve.rateHelpers{i}.rateType,'SwapPoint')
        outInfo.impliedRate(i) = yieldCurve.rateHelpers{i}.ImpliedQuote();
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    elseif strcmp(yieldCurve.rateHelpers{i}.rateType,'CrossCurrencyBasisSwapRFR')
        yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.domesticSpread = dataRate{i};
%         for j=1:length(yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.domesticLeg.couponRates)
%             if strcmp(class(yieldCurve.rateHelpers{i}.crossCurrencySwap.domesticLeg.couponRates{j}),'FixedRateCoupon')
%                 yieldCurve.rateHelpers{i}.crossCurrencySwap.crossCurrencyBasisSwap.domesticSpread = dataRate{i};
%             end
%         end
        
        swapOut = yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.Calculate(yieldCurve.rateHelpers{i}.discountCurve,yieldCurve.rateHelpers{i}.foreignDCurve);
        outInfo.NPV(i) = swapOut.NPV;
        outInfo.impliedRate(i) = swapOut.fairDomesticSpread;
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    end
end
aaa =1;

% 
% %% import and construct a zeroCurve
% rawData = [0.002739726	0.999965207
% 0.25	0.996569895
% 0.5	0.992648282
% 0.75	0.988507257
% 1	0.984021343
% 2	0.965105185
% 3	0.945510165
% 4	0.925938679
% 5	0.906326256
% 7	0.867854044
% 10	0.809835931
% 12	0.773180526
% 15	0.723742908
% 20	0.649044875
% 25	0.582936139
% 30	0.523634856
% ];
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,H_Date(valueDate),'KRW','KRWZero'});
% zeroCurveMktData = MktData(mktData);
% zeroCurveMktData.params('convention') = {};
% zeroCurve =  ZeroCurve(zeroCurveMktData);
% AllMktData =containers.Map('zeroCurve',zeroCurve);
% 
% %% import and construct a discountCurve
% 
% rawData = [0.0	0.0
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'KRW','discountZero'});
% discountCurveMktData = MktData(mktData);
% discountCurveMktData.params('convention') = {};
% discountCurve =  ZeroCurve(discountCurveMktData);
% AllMktData('discountCurve') = discountCurve;
% 
% freq = 4;
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',1,Calendar(),'ModifiedFollowing','Act365',true,dummyCurve});
% 
% KRW_Libor_3M = IborIndex(IborIndexParams);
% 
% depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
%                         {IborIndexParams});
% 
% depositRateHelperParams('rate') = 0.03;
% depositRateHelperParams('rateTenor') = '1D';
% depositRateHelper_KRW_Libor_3M = DepositRateHelper(depositRateHelperParams);
% 
% %swapRateHelper
% % swapRateHeperParams =  containers.Map();
% % fixedLegParams = containers.Map();
% fixedLegTenor = '3M';
% fixedLegConvention = 'ModifiedFollowing';
% fixedLegDayCounter = 'Act365';
% 
% 
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% 
% swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
%                         {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});
%                     
%                     
% swapRateHelperParams('rate') = 0.03;
% swapRateHelperParams('rateTenor') = '6M';
% swapRateHelperParams_KRW_SwapRate_6M = SwapRateHelper(swapRateHelperParams);
% 
% aaa = 1.0;

In [ ]:
%test_CCS_BasisCurveBootStrap_USDHKD_20220727.m

clc
clear
tic

valueDate = '2022-07-29';


useSOFRAsDiscountCurve = true;
useSOFRAsForwardCurve = false;

needBootStrapFlag = false;
% needBootStrapFlag = true;

% useSOFRAsDiscountCurve = false;

% useRounding = true;
useRounding = false;

useSimpleTeleScopingFlag = true;

useLastEndDateAsPillarDate = true;
% useLastEndDateAsPillarDate = false;

valueDateH = H_Date(valueDate);
%% Calendars Start
%% GBEN Calendar
GBEN.name = 'GBEN';
GBEN.holidaysList = {'2021-01-01'
'2021-04-02'
'2021-04-05'
'2021-05-03'
'2021-05-31'
'2021-08-30'
'2021-12-27'
'2021-12-28'
'2022-01-03'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-06-02'
'2022-06-03'
'2022-08-29'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-29'
'2023-08-28'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-03-29'
'2024-04-01'
'2024-05-06'
'2024-05-27'
'2024-08-26'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-04-18'
'2025-04-21'
'2025-05-05'
'2025-05-26'
'2025-08-25'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-05-04'
'2026-05-25'
'2026-08-31'
'2026-12-25'
'2026-12-28'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2027-05-03'
'2027-05-31'
'2027-08-30'
'2027-12-27'
'2027-12-28'
'2028-01-03'
'2028-04-14'
'2028-04-17'
'2028-05-01'
'2028-05-29'
'2028-08-28'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-05-07'
'2029-05-28'
'2029-08-27'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-05-06'
'2030-05-27'
'2030-08-26'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-05-05'
'2031-05-26'
'2031-08-25'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2032-05-03'
'2032-05-31'
'2032-08-30'
'2032-12-27'
'2032-12-28'
'2033-01-03'
'2033-04-15'
'2033-04-18'
'2033-05-02'
'2033-05-30'
'2033-08-29'
'2033-12-26'
'2033-12-27'
'2034-01-02'
'2034-04-07'
'2034-04-10'
'2034-05-01'
'2034-05-29'
'2034-08-28'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-05-07'
'2035-05-28'
'2035-08-27'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-05-05'
'2036-05-26'
'2036-08-25'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-05-04'
'2037-05-25'
'2037-08-31'
'2037-12-25'
'2037-12-28'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2038-05-03'
'2038-05-31'
'2038-08-30'
'2038-12-27'
'2038-12-28'
'2039-01-03'
'2039-04-08'
'2039-04-11'
'2039-05-02'
'2039-05-30'
'2039-08-29'
'2039-12-26'
'2039-12-27'
'2040-01-02'
'2040-03-30'
'2040-04-02'
'2040-05-07'
'2040-05-28'
'2040-08-27'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-05-06'
'2041-05-27'
'2041-08-26'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-05-05'
'2042-05-26'
'2042-08-25'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-05-04'
'2043-05-25'
'2043-08-31'
'2043-12-25'
'2043-12-28'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-05-02'
'2044-05-30'
'2044-08-29'
'2044-12-26'
'2044-12-27'
'2045-01-02'
'2045-04-07'
'2045-04-10'
'2045-05-01'
'2045-05-29'
'2045-08-28'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-05-07'
'2046-05-28'
'2046-08-27'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-05-06'
'2047-05-27'
'2047-08-26'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-05-04'
'2048-05-25'
'2048-08-31'
'2048-12-25'
'2048-12-28'
'2049-01-01'
'2049-04-16'
'2049-04-19'
'2049-05-03'
'2049-05-31'
'2049-08-30'
'2049-12-27'
'2049-12-28'
'2050-01-03'
'2050-04-08'
'2050-04-11'
'2050-05-02'
'2050-05-30'
'2050-08-29'
'2050-12-26'
'2050-12-27'
'2051-01-02'
'2051-03-31'
'2051-04-03'
'2051-05-01'
'2051-05-29'
'2051-08-28'
'2051-12-25'
'2051-12-26'
'2052-01-01'
'2052-04-19'
'2052-04-22'
'2052-05-06'
'2052-05-27'
'2052-08-26'
'2052-12-25'
'2052-12-26'
'2053-01-01'
'2053-04-04'
'2053-04-07'
'2053-05-05'
'2053-05-26'
'2053-08-25'
'2053-12-25'
'2053-12-26'
'2054-01-01'
'2054-03-27'
'2054-03-30'
'2054-05-04'
'2054-05-25'
'2054-08-31'
'2054-12-25'
'2054-12-28'
'2055-01-01'
'2055-04-16'
'2055-04-19'
'2055-05-03'
'2055-05-31'
'2055-08-30'
'2055-12-27'
'2055-12-28'
'2056-01-03'
'2056-03-31'
'2056-04-03'
'2056-05-01'
'2056-05-29'
'2056-08-28'
'2056-12-25'
'2056-12-26'
'2057-01-01'
'2057-04-20'
'2057-04-23'
'2057-05-07'
'2057-05-28'
'2057-08-27'
'2057-12-25'
'2057-12-26'
'2058-01-01'
'2058-04-12'
'2058-04-15'
'2058-05-06'
'2058-05-27'
'2058-08-26'
'2058-12-25'
'2058-12-26'
'2059-01-01'
'2059-03-28'
'2059-03-31'
'2059-05-05'
'2059-05-26'
'2059-08-25'
'2059-12-25'
'2059-12-26'
'2060-01-01'
'2060-04-16'
'2060-04-19'
'2060-05-03'
'2060-05-31'
'2060-08-30'
'2060-12-27'
'2060-12-28'
'2061-01-03'
'2061-04-08'
'2061-04-11'
'2061-05-02'
'2061-05-30'
'2061-08-29'
'2061-12-26'
'2061-12-27'
'2062-01-02'
'2062-03-24'
'2062-03-27'
'2062-05-01'
'2062-05-29'
'2062-08-28'
'2062-12-25'
'2062-12-26'
'2063-01-01'
'2063-04-13'
'2063-04-16'
'2063-05-07'
'2063-05-28'
'2063-08-27'
'2063-12-25'
'2063-12-26'
'2064-01-01'
'2064-04-04'
'2064-04-07'
'2064-05-05'
'2064-05-26'
'2064-08-25'
'2064-12-25'
'2064-12-26'
'2065-01-01'
'2065-03-27'
'2065-03-30'
'2065-05-04'
'2065-05-25'
'2065-08-31'
'2065-12-25'
'2065-12-28'
'2066-01-01'
'2066-04-09'
'2066-04-12'
'2066-05-03'
'2066-05-31'
'2066-08-30'
'2066-12-27'
'2066-12-28'
'2067-01-03'
'2067-04-01'
'2067-04-04'
'2067-05-02'
'2067-05-30'
'2067-08-29'
'2067-12-26'
'2067-12-27'
'2068-01-02'
'2068-04-20'
'2068-04-23'
'2068-05-07'
'2068-05-28'
'2068-08-27'
'2068-12-25'
'2068-12-26'
'2069-01-01'
'2069-04-12'
'2069-04-15'
'2069-05-06'
'2069-05-27'
'2069-08-26'
'2069-12-25'
'2069-12-26'
'2070-01-01'
'2070-03-28'
'2070-03-31'
'2070-05-05'
'2070-05-26'
'2070-08-25'
'2070-12-25'
'2070-12-26'
'2071-01-01'
'2071-04-17'
'2071-04-20'
'2071-05-04'
'2071-05-25'
'2071-08-31'
'2071-12-25'
'2071-12-28'
    };

GBENCalendar = Calendar();
GBENCalendar.name = GBEN.name;
for i=1:length(GBEN.holidaysList)
    GBENCalendar.AddHoliday(GBEN.holidaysList{i})
end

%% USGT Calendar

USGT.name = 'USGT';
USGT.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2021-12-24'
'2022-01-17'
'2022-02-21'
'2022-04-15'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-03-29'
'2024-05-27'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-04-18'
'2025-05-26'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-07-03'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-03-26'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2027-12-24'
'2028-01-17'
'2028-02-21'
'2028-04-14'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-03-30'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-04-19'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-04-11'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-03-26'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2032-12-24'
'2033-01-17'
'2033-02-21'
'2033-04-15'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-04-07'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-03-23'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-04-11'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-04-03'
'2037-05-25'
'2037-07-03'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-04-23'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2038-12-24'
'2039-01-17'
'2039-02-21'
'2039-04-08'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-03-30'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-04-19'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-04-04'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-03-27'
'2043-05-25'
'2043-07-03'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-04-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-04-07'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-03-23'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-04-12'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-04-03'
'2048-05-25'
'2048-07-03'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-04-16'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2049-12-24'
'2050-01-17'
'2050-02-21'
'2050-04-08'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-03-31'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-04-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-04-04'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-03-27'
'2054-05-25'
'2054-07-03'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-04-16'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2055-12-24'
'2056-01-17'
'2056-02-21'
'2056-03-31'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-04-20'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-04-12'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-03-28'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-04-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2060-12-24'
'2061-01-17'
'2061-02-14'
'2061-04-08'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-03-24'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-04-13'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-04-04'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-03-27'
'2065-05-25'
'2065-07-03'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-04-09'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2066-12-24'
'2067-01-17'
'2067-02-21'
'2067-04-01'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-04-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-04-12'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-03-28'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-04-17'
'2071-05-25'
'2071-07-03'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };

USGTCalendar = Calendar();
USGTCalendar.name = USGT.name;
for i=1:length(USGT.holidaysList)
    USGTCalendar.AddHoliday(USGT.holidaysList{i})
end

%% USFD Calendar
USFD.name = 'USFD';
USFD.holidaysList = {'2021-05-31'
'2021-07-05'    
'2021-09-06'
'2021-10-11'
'2021-11-11'
'2021-11-25'
'2022-01-17'
'2022-02-21'
'2022-05-30'
'2022-06-20'
'2022-07-04'
'2022-09-05'
'2022-10-10'
'2022-11-11'
'2022-11-24'
'2022-12-26'
'2023-01-02'
'2023-01-16'
'2023-02-20'
'2023-05-29'
'2023-06-19'
'2023-07-04'
'2023-09-04'
'2023-10-09'
'2023-11-23'
'2023-12-25'
'2024-01-01'
'2024-01-15'
'2024-02-19'
'2024-05-27'
'2024-06-19'
'2024-07-04'
'2024-09-02'
'2024-10-14'
'2024-11-11'
'2024-11-28'
'2024-12-25'
'2025-01-01'
'2025-01-20'
'2025-02-17'
'2025-05-26'
'2025-06-19'
'2025-07-04'
'2025-09-01'
'2025-10-13'
'2025-11-11'
'2025-11-27'
'2025-12-25'
'2026-01-01'
'2026-01-19'
'2026-02-16'
'2026-05-25'
'2026-09-07'
'2026-10-12'
'2026-11-11'
'2026-11-26'
'2026-12-25'
'2027-01-01'
'2027-01-18'
'2027-02-15'
'2027-05-31'
'2027-07-05'
'2027-09-06'
'2027-10-11'
'2027-11-11'
'2027-11-25'
'2028-01-17'
'2028-02-21'
'2028-05-29'
'2028-07-04'
'2028-09-04'
'2028-10-09'
'2028-11-23'
'2028-12-25'
'2029-01-01'
'2029-01-15'
'2029-02-19'
'2029-05-28'
'2029-07-04'
'2029-09-03'
'2029-10-08'
'2029-11-12'
'2029-11-22'
'2029-12-25'
'2030-01-01'
'2030-01-21'
'2030-02-18'
'2030-05-27'
'2030-07-04'
'2030-09-02'
'2030-10-14'
'2030-11-11'
'2030-11-28'
'2030-12-25'
'2031-01-01'
'2031-01-20'
'2031-02-17'
'2031-05-26'
'2031-07-04'
'2031-09-01'
'2031-10-13'
'2031-11-11'
'2031-11-27'
'2031-12-25'
'2032-01-01'
'2032-01-19'
'2032-02-16'
'2032-05-31'
'2032-07-05'
'2032-09-06'
'2032-10-11'
'2032-11-11'
'2032-11-25'
'2033-01-17'
'2033-02-21'
'2033-05-30'
'2033-07-04'
'2033-09-05'
'2033-10-10'
'2033-11-11'
'2033-11-24'
'2033-12-26'
'2034-01-02'
'2034-01-16'
'2034-02-20'
'2034-05-29'
'2034-07-04'
'2034-09-04'
'2034-10-09'
'2034-11-23'
'2034-12-25'
'2035-01-01'
'2035-01-15'
'2035-02-19'
'2035-05-28'
'2035-07-04'
'2035-09-03'
'2035-10-08'
'2035-11-12'
'2035-11-22'
'2035-12-25'
'2036-01-01'
'2036-01-21'
'2036-02-18'
'2036-05-26'
'2036-07-04'
'2036-09-01'
'2036-10-13'
'2036-11-11'
'2036-11-27'
'2036-12-25'
'2037-01-01'
'2037-01-19'
'2037-02-16'
'2037-05-25'
'2037-09-07'
'2037-10-12'
'2037-11-11'
'2037-11-26'
'2037-12-25'
'2038-01-01'
'2038-01-18'
'2038-02-15'
'2038-05-31'
'2038-07-05'
'2038-09-06'
'2038-10-11'
'2038-11-11'
'2038-11-25'
'2039-01-17'
'2039-02-21'
'2039-05-30'
'2039-07-04'
'2039-09-05'
'2039-10-10'
'2039-11-11'
'2039-11-24'
'2039-12-26'
'2040-01-02'
'2040-01-16'
'2040-02-20'
'2040-05-28'
'2040-07-04'
'2040-09-03'
'2040-10-08'
'2040-11-12'
'2040-11-22'
'2040-12-25'
'2041-01-01'
'2041-01-21'
'2041-02-18'
'2041-05-27'
'2041-07-04'
'2041-09-02'
'2041-10-14'
'2041-11-11'
'2041-11-28'
'2041-12-25'
'2042-01-01'
'2042-01-20'
'2042-02-17'
'2042-05-26'
'2042-07-04'
'2042-09-01'
'2042-10-13'
'2042-11-11'
'2042-11-27'
'2042-12-25'
'2043-01-01'
'2043-01-19'
'2043-02-16'
'2043-05-25'
'2043-09-07'
'2043-10-12'
'2043-11-11'
'2043-11-26'
'2043-12-25'
'2044-01-01'
'2044-01-18'
'2044-02-15'
'2044-05-30'
'2044-07-04'
'2044-09-05'
'2044-10-10'
'2044-11-11'
'2044-11-24'
'2044-12-26'
'2045-01-02'
'2045-01-16'
'2045-02-20'
'2045-05-29'
'2045-07-04'
'2045-09-04'
'2045-10-09'
'2045-11-23'
'2045-12-25'
'2046-01-01'
'2046-01-15'
'2046-02-19'
'2046-05-28'
'2046-07-04'
'2046-09-03'
'2046-10-08'
'2046-11-12'
'2046-11-22'
'2046-12-25'
'2047-01-01'
'2047-01-21'
'2047-02-18'
'2047-05-27'
'2047-07-04'
'2047-09-02'
'2047-10-14'
'2047-11-11'
'2047-11-28'
'2047-12-25'
'2048-01-01'
'2048-01-20'
'2048-02-17'
'2048-05-25'
'2048-09-07'
'2048-10-12'
'2048-11-11'
'2048-11-26'
'2048-12-25'
'2049-01-01'
'2049-01-18'
'2049-02-15'
'2049-05-31'
'2049-07-05'
'2049-09-06'
'2049-10-11'
'2049-11-11'
'2049-11-25'
'2050-01-17'
'2050-02-21'
'2050-05-30'
'2050-07-04'
'2050-09-05'
'2050-10-10'
'2050-11-11'
'2050-11-24'
'2050-12-26'
'2051-01-02'
'2051-01-16'
'2051-02-20'
'2051-05-29'
'2051-07-04'
'2051-09-04'
'2051-10-09'
'2051-11-23'
'2051-12-25'
'2052-01-01'
'2052-01-15'
'2052-02-19'
'2052-05-27'
'2052-07-04'
'2052-09-02'
'2052-10-14'
'2052-11-11'
'2052-11-28'
'2052-12-25'
'2053-01-01'
'2053-01-20'
'2053-02-17'
'2053-05-26'
'2053-07-04'
'2053-09-01'
'2053-10-13'
'2053-11-11'
'2053-11-27'
'2053-12-25'
'2054-01-01'
'2054-01-19'
'2054-02-16'
'2054-05-25'
'2054-09-07'
'2054-10-12'
'2054-11-11'
'2054-11-26'
'2054-12-25'
'2055-01-01'
'2055-01-18'
'2055-02-15'
'2055-05-31'
'2055-07-05'
'2055-09-06'
'2055-10-11'
'2055-11-11'
'2055-11-25'
'2056-01-17'
'2056-02-21'
'2056-05-29'
'2056-07-04'
'2056-09-04'
'2056-10-09'
'2056-11-23'
'2056-12-25'
'2057-01-01'
'2057-01-15'
'2057-02-19'
'2057-05-28'
'2057-07-04'
'2057-09-03'
'2057-10-08'
'2057-11-12'
'2057-11-22'
'2057-12-25'
'2058-01-01'
'2058-01-21'
'2058-02-18'
'2058-05-27'
'2058-07-04'
'2058-09-02'
'2058-10-14'
'2058-11-11'
'2058-11-28'
'2058-12-25'
'2059-01-01'
'2059-01-20'
'2059-02-17'
'2059-05-26'
'2059-07-04'
'2059-09-01'
'2059-10-13'
'2059-11-11'
'2059-11-27'
'2059-12-25'
'2060-01-01'
'2060-01-19'
'2060-02-16'
'2060-05-31'
'2060-07-05'
'2060-09-06'
'2060-10-11'
'2060-11-11'
'2060-11-25'
'2061-01-17'
'2061-02-21'
'2061-05-30'
'2061-07-04'
'2061-09-05'
'2061-10-10'
'2061-11-11'
'2061-11-24'
'2061-12-26'
'2062-01-02'
'2062-01-16'
'2062-02-20'
'2062-05-29'
'2062-07-04'
'2062-09-04'
'2062-10-09'
'2062-11-23'
'2062-12-25'
'2063-01-01'
'2063-01-15'
'2063-02-19'
'2063-05-28'
'2063-07-04'
'2063-09-03'
'2063-10-08'
'2063-11-12'
'2063-11-22'
'2063-12-25'
'2064-01-01'
'2064-01-21'
'2064-02-18'
'2064-05-26'
'2064-07-04'
'2064-09-01'
'2064-10-13'
'2064-11-11'
'2064-11-27'
'2064-12-25'
'2065-01-01'
'2065-01-19'
'2065-02-16'
'2065-05-25'
'2065-09-07'
'2065-10-12'
'2065-11-11'
'2065-11-26'
'2065-12-25'
'2066-01-01'
'2066-01-18'
'2066-02-15'
'2066-05-31'
'2066-07-05'
'2066-09-06'
'2066-10-11'
'2066-11-11'
'2066-11-25'
'2067-01-17'
'2067-02-21'
'2067-05-30'
'2067-07-04'
'2067-09-05'
'2067-10-10'
'2067-11-11'
'2067-11-24'
'2067-12-26'
'2068-01-02'
'2068-01-16'
'2068-02-20'
'2068-05-28'
'2068-07-04'
'2068-09-03'
'2068-10-08'
'2068-11-12'
'2068-11-22'
'2068-12-25'
'2069-01-01'
'2069-01-21'
'2069-02-18'
'2069-05-27'
'2069-07-04'
'2069-09-02'
'2069-10-14'
'2069-11-11'
'2069-11-28'
'2069-12-25'
'2070-01-01'
'2070-01-20'
'2070-02-17'
'2070-05-26'
'2070-07-04'
'2070-09-01'
'2070-10-13'
'2070-11-11'
'2070-11-27'
'2070-12-25'
'2071-01-01'
'2071-01-19'
'2071-02-16'
'2071-05-25'
'2071-09-07'
'2071-10-12'
'2071-11-11'
'2071-11-26'
'2071-12-25'
    };



USFDCalendar = Calendar();
USFDCalendar.name = USFD.name;

for i=1:length(USFD.holidaysList)
    USFDCalendar.AddHoliday(USFD.holidaysList{i})
end

%% HKHD Calendar
HKHD.name = 'HKHD';
HKHD.holidaysList = {'2021-01-01'
'2021-02-12'
'2021-02-15'
'2021-04-02'
'2021-04-05'
'2021-04-06'
'2021-05-19'
'2021-06-14'
'2021-07-01'
'2021-09-22'
'2021-10-01'
'2021-10-14'
'2021-12-27'
'2022-02-01'
'2022-02-02'
'2022-02-03'
'2022-04-05'
'2022-04-15'
'2022-04-18'
'2022-05-02'
'2022-05-09'
'2022-06-03'
'2022-07-01'
'2022-09-12'
'2022-10-04'
'2022-12-26'
'2022-12-27'
'2023-01-02'
'2023-01-23'
'2023-01-24'
'2023-01-25'
'2023-04-05'
'2023-04-07'
'2023-04-10'
'2023-05-01'
'2023-05-26'
'2023-06-22'
'2023-10-02'
'2023-10-23'
'2023-12-25'
'2023-12-26'
'2024-01-01'
'2024-02-12'
'2024-02-13'
'2024-03-29'
'2024-04-01'
'2024-04-04'
'2024-05-01'
'2024-05-15'
'2024-06-10'
'2024-07-01'
'2024-09-18'
'2024-10-01'
'2024-10-11'
'2024-12-25'
'2024-12-26'
'2025-01-01'
'2025-01-29'
'2025-01-30'
'2025-01-31'
'2025-04-04'
'2025-04-18'
'2025-04-21'
'2025-05-01'
'2025-05-05'
'2025-07-01'
'2025-10-01'
'2025-10-07'
'2025-10-29'
'2025-12-25'
'2025-12-26'
'2026-01-01'
'2026-04-03'
'2026-04-06'
'2026-12-25'
'2027-01-01'
'2027-03-26'
'2027-03-29'
'2028-04-14'
'2028-04-17'
'2028-12-25'
'2028-12-26'
'2029-01-01'
'2029-03-30'
'2029-04-02'
'2029-12-25'
'2029-12-26'
'2030-01-01'
'2030-04-19'
'2030-04-22'
'2030-12-25'
'2030-12-26'
'2031-01-01'
'2031-04-11'
'2031-04-14'
'2031-12-25'
'2031-12-26'
'2032-01-01'
'2032-03-26'
'2032-03-29'
'2033-04-15'
'2033-04-18'
'2033-12-26'
'2034-04-07'
'2034-04-10'
'2034-12-25'
'2034-12-26'
'2035-01-01'
'2035-03-23'
'2035-03-26'
'2035-12-25'
'2035-12-26'
'2036-01-01'
'2036-04-11'
'2036-04-14'
'2036-12-25'
'2036-12-26'
'2037-01-01'
'2037-04-03'
'2037-04-06'
'2037-12-25'
'2038-01-01'
'2038-04-23'
'2038-04-26'
'2039-04-08'
'2039-04-11'
'2039-12-26'
'2040-03-30'
'2040-04-02'
'2040-12-25'
'2040-12-26'
'2041-01-01'
'2041-04-19'
'2041-04-22'
'2041-12-25'
'2041-12-26'
'2042-01-01'
'2042-04-04'
'2042-04-07'
'2042-12-25'
'2042-12-26'
'2043-01-01'
'2043-03-27'
'2043-03-30'
'2043-12-25'
'2044-01-01'
'2044-04-15'
'2044-04-18'
'2044-12-26'
'2045-04-07'
'2045-04-10'
'2045-12-25'
'2045-12-26'
'2046-01-01'
'2046-03-23'
'2046-03-26'
'2046-12-25'
'2046-12-26'
'2047-01-01'
'2047-04-12'
'2047-04-15'
'2047-12-25'
'2047-12-26'
'2048-01-01'
'2048-04-03'
'2048-04-06'
'2048-12-25'
'2049-01-01'
'2049-04-16'
'2049-04-19'

    };

HKHDCalendar = Calendar();
HKHDCalendar.name = HKHD.name;

for i=1:length(HKHD.holidaysList)
    HKHDCalendar.AddHoliday(HKHD.holidaysList{i})
end

%% Calendars End

%% Calendar for USD JPY CCS Schedule Generation
% jointCalendar_USDHKDCCS = JointCalendar({USFDCalendar,HKHDCalendar});
jointCalendar_USDHKDCCS = JointCalendar({USFDCalendar,GBENCalendar,HKHDCalendar});

    
%% index fixing for USD SOFR Index start
    tempDate = H_Date('2021-10-25');
    irIndexFixing_SOFR = containers.Map({StrDate(tempDate,'str')},{0.01530000});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_SOFR(StrDate(tempDate,'str')) = 0.01530000;
    end
%% index fixing for SOFR_index end

%% index fixing for Hibor3M start
    tempDate = H_Date('2021-10-25');
    irIndexFixing_Hibor3M = containers.Map({StrDate(tempDate,'str')},{0.021415500});
    for idxhong=1:10000
        tempDate = AddDate(tempDate,1,'day');
        irIndexFixing_Hibor3M(StrDate(tempDate,'str')) = 0.021415500;
    end
%% index fixing for Hibor3M end


valueDateH = H_Date(valueDate);
%% import and construct an CCSCurve

dataType= {
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'SwapPoint'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
'CrossCurrencyBasisSwapRFR'
};

dataTenor = {
'1M'
'2M'
'3M'
'4M'
'5M'
'6M'
'9M'
'12M'
'2Y'
'3Y'
'4Y'
'5Y'
'7Y'
'10Y'
'12Y'
'15Y'
};


% dataRate = {
% -0.008918500
% -0.014830000
% -0.017892250
% -0.019945250
% -0.021551000
% -0.022000000
% -0.025740500
% -0.027959750
% -0.003300000
% -0.003200000
% -0.003100000
% -0.003100000
% -0.003000000
% -0.002650000
% -0.002550000
% -0.002550000
% };

dataRate = {
-0.00891800
-0.014830000
-0.0178920
-0.0199450
-0.021551000
-0.022000000
-0.02574000
-0.027960
-0.003300000
-0.003200000
-0.003100000
-0.003100000
-0.003000000
-0.002650000
-0.002550000
-0.002550000
};

exchangeRate = 7.8493;
% exchangeRate = 7.84929990768433;



%% Construct USDSOFR_Curve

% forward & discount zeroCurve
forwardCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_SOFR = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

forwardCurve_Hibor3M     = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
discountCurve_HKDBasis = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));


rawData_SOFR = [
0.0082191780821917800	0.0155115111614577000
0.0191780821917808000	0.0231662696955027000
0.0383561643835616000	0.0231854374869727000
0.0575342465753425000	0.0231792732753092000
0.0849315068493151000	0.0231906397268176000
0.1698630136986300000	0.0238691847203881000
0.2575342465753420000	0.0256878985528601000
0.5068493150684930000	0.0291408833611058000
0.7479452054794520000	0.0303732043830678000
1.0054794520547900000	0.0306924948323285000
2.0027397260274000000	0.0294471091663581000
3.0027397260274000000	0.0275475183356320000
4.0027397260274000000	0.0263250282980152000
5.0027397260274000000	0.0257083517441060000
6.0109589041095900000	0.0254825579206226000
7.0082191780821900000	0.0254138307616813000
8.0054794520548000000	0.0254370370414108000
9.0054794520548000000	0.0255576866198680000
10.0082191780822000000	0.0257764383248286000
12.0136986301370000000	0.0263497096020728000
15.0109589041096000000	0.0269875965721440000
20.0136986301370000000	0.0269897620310076000
25.0164383561644000000	0.0259901024750434000
30.0219178082192000000	0.0249311602136004000
40.0328767123288000000	0.0218547725322423000
50.0356164383562000000	0.0186810030223569000
];

% % 2022-07-27
% rawData_Hibor3M = [
% 0.0027397260273972600	0.0091349856869783300
% 0.2520547945205480000	0.0213579080125783000
% 1.0000000000000000000	0.0312428052857285000
% 2.0082191780821900000	0.0318168128986144000
% 3.0054794520547900000	0.0305712232657243000
% 4.0027397260274000000	0.0296389020644710000
% 5.0027397260274000000	0.0291752849094337000
% 7.0054794520548000000	0.0292267244134564000
% 10.0082191780822000000	0.0295951888372656000
% ];

% %2022-07-29 Ver 1
% rawData_Hibor3M = [
% 0.24657534246575300	0.0216265436680857000
% 0.99452054794520600	0.0313638714111886000
% 2.00273972602740000	0.0318785023907728000
% 3.00000000000000000	0.0306101308817079000
% 3.99726027397261000	0.0296668247492547000
% 4.99726027397261000	0.0291971116434191000
% 7.00000000000001000	0.0292423466607353000
% 10.00273972602740000	0.0296063232581489000
% ];

%2022-07-29 Ver 2
rawData_Hibor3M = [
-0.0027397260273972600	0.0091349856869783300
0.2465753424657530000	0.0213579080125783000
0.9945205479452060000	0.0312428052857285000
2.0027397260274000000	0.0318168128986144000
3.0000000000000000000	0.0305712232657243000
3.9972602739726100000	0.0296389020644710000
4.9972602739726100000	0.0291752849094337000
7.0000000000000100000	0.0292267244134564000
10.0027397260274000000	0.0295951888372656000
];


% USDHKD Basis curve zeros
rawData_HKDBasis = [
0.087671232876712300	0.010646998847663800
0.169863013698630000	0.012895080109643100
0.257534246575342000	0.016826404961270400
0.336986301369863000	0.019244052620072800
0.419178082191781000	0.021367993117414200
0.506849315068493000	0.023577262003047900
0.747945205479452000	0.025979946126825500
1.005479452054790000	0.027142310579774100
2.002739726027400000	0.028609608753372700
3.002739726027400000	0.027437455620890600
4.002739726027400000	0.026595117863201600
5.002739726027400000	0.026125907619111400
7.008219178082190000	0.026276122443267700
10.008219178082200000	0.027019915336342200
12.013698630137000000	0.027125218038061000
15.010958904109600000	0.027113043010524100
];


forwardCurve_SOFR.params('rawData')  = rawData_SOFR;
discountCurve_SOFR.params('rawData') = rawData_SOFR;

forwardCurve_Hibor3M.params('rawData')  = rawData_Hibor3M;
discountCurve_HKDBasis.params('rawData') = rawData_HKDBasis;

%% SOFR Rate Index start
sofrIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                            {'1D','USD',0,USGTCalendar,'Following','Act360',false,forwardCurve_SOFR});

sofrIndexParams('forecastTodaysFixing') = true;
sofrIndexParams('useRounding') = useRounding;
sofrIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
sofrIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
sofrIndexParams('historicalFixings') = irIndexFixing_SOFR;
overnightIndex_sofr = IborIndex(sofrIndexParams);
%% SOFR Rate Index end

%% Hibor3M Index Start
IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
                        {'3M','HKD',0,HKHDCalendar,'ModifiedFollowing','Act365',false,forwardCurve_Hibor3M});

IborIndexParams('forecastTodaysFixing') = true;
IborIndexParams('historicalFixings') = irIndexFixing_Hibor3M;
HKD_Libor_3M = IborIndex(IborIndexParams);

% tonaIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                             {'1D','JPY',0,JPJNCalendar,'Following','Act365',false,forwardCurve_TONA});
% 
%                        
% tonaIndexParams('forecastTodaysFixing') = true;
% tonaIndexParams('useRounding') = useRounding;
% tonaIndexParams('useSimpleTeleScoping') = useSimpleTeleScopingFlag;
% tonaIndexParams('useLastEndDateAsPillarDate') = useLastEndDateAsPillarDate;
% tonaIndexParams('historicalFixings') = irIndexFixing_TONA;
% overnightIndex_tona = IborIndex(tonaIndexParams);
%% Hibor3M end


rateHelpers = {};
        
for i=1:length(dataRate)
    if strcmp(dataType{i},'SwapPoint')
        % we are valuing on settleDate
        % so settlegap(here fixingDays) is equal to 0
        swapPointHelperParams = containers.Map({'exchangeRate','fixingDays','rate','rateTenor'}, ...
                            {exchangeRate,0,dataRate{i},dataTenor{i}});
        
        swapPointHelperParams('exchangeRate') = exchangeRate;
        swapPointHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        swapPointHelperParams('calendar') = jointCalendar_USDHKDCCS;
        swapPointHelperParams('convention') = 'ModifiedFollowing';
        swapPointHelperParams('foreignDCurve') = discountCurve_SOFR;
        
        swapPointHelper = SwapPointHelper(swapPointHelperParams);
    
        rateHelpers{end+1} = swapPointHelper;
        
    elseif strcmp(dataType{i},'CrossCurrencyBasisSwapRFR')
        
%      % Libor6M Leg
%         floatingLegTenor_L = '6M';
%         floatingLegConvention_L = 'ModifiedFollowing';
%         floatingLegDayCounter_L = 'Act360';
%         spotLag_L = 0;
%         payLag_L = 0;
% 
%         % Libor3M Leg
%         floatingLegTenor_S = '6M';
%         floatingLegConvention_S = 'ModifiedFollowing';
%         floatingLegDayCounter_S = 'Act360';
%         spotLag_S = 0;
%         payLag_S  = 0;
% 
%         calcCalendar = jointCalendar_USDIRS;
% 
% %             domesticFixedLegTenor = '6M';
% %             domesticFixedLegConvention = 'ModifiedFollowing';
% %             domesticFixedLegDayCounter = 'Act365';
% %             domesticFixedLegCalendar = jointCalendar_USDKRWCRS;
% 
%         dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));
% 
%         spotLag = 0;
% 
%         singleCurrencyBasisSwapHelperParams = containers.Map({'floatingLegTenor_L','floatingLegConvention_L','floatingLegDayCounter_L','spotLag_L','payLag_L',...
%                                                         'floatingLegTenor_S','floatingLegConvention_S','floatingLegDayCounter_S','spotLag_S','payLag_S',...
%                                                         'liborIndex_L','liborIndex_S','discountCurve','calcCalendar'}, ...
%                                 {floatingLegTenor_L,floatingLegConvention_L,floatingLegDayCounter_L,spotLag_L,payLag_L,...,
%                                 floatingLegTenor_S,floatingLegConvention_S,floatingLegDayCounter_S,spotLag_S,payLag_S,...,
%                                 libor6M_Index_fwd,libor3M_Index_fwd,discountCurve_Libor3M,calcCalendar});
% 
%         singleCurrencyBasisSwapHelperParams('rate') = dataRate{i};
%         singleCurrencyBasisSwapHelperParams('rateTenor') = dataTenor{i};
%         singleCurrencyBasisSwapHelper = SingleCurrencyBasisSwapHelper(singleCurrencyBasisSwapHelperParams);
%         rateHelpers{end+1} = singleCurrencyBasisSwapHelper;
            
        domesticFloatingLegTenor = '3M';
        domesticFloatingLegConvention = 'ModifiedFollowing';
        domesticFloatingLegDayCounter = 'Act365';
        domesticFloatingLegCalendar = jointCalendar_USDHKDCCS;
        
        foreignFloatingLegTenor = '3M';
        foreignFloatingLegConvention = 'ModifiedFollowing';
        foreignFloatingLegDayCounter = 'Act360';
        foreignFloatingLegCalendar = jointCalendar_USDHKDCCS;
        
        dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

        spotLag = 0;
%         payLag  = 2;
        payLag  = 2;
        
        crossCurrencyBasisSwapHelperParams = containers.Map({'domesticIndex','domesticFloatingLegTenor','domesticFloatingLegConvention','domesticFloatingLegDayCounter','domesticFloatingLegCalendar',...
                                                        'foreignIndex','foreignFloatingLegTenor','foreignFloatingLegConvention','foreignFloatingLegDayCounter','foreignFloatingLegCalendar',...
                                                        'foreignDCurve','spotLag','payLag','discountCurve'}, ...
                                {HKD_Libor_3M,domesticFloatingLegTenor,domesticFloatingLegConvention,domesticFloatingLegDayCounter,domesticFloatingLegCalendar,...,
                                overnightIndex_sofr,foreignFloatingLegTenor,foreignFloatingLegConvention,foreignFloatingLegDayCounter,foreignFloatingLegCalendar,...
                                discountCurve_SOFR,spotLag,payLag, dummyCurve});

        crossCurrencyBasisSwapHelperParams('exchangeRate') = exchangeRate;
        crossCurrencyBasisSwapHelperParams('OriginalDomesticAmount') = exchangeRate;
        
        crossCurrencyBasisSwapHelperParams('rate') = dataRate{i};
        crossCurrencyBasisSwapHelperParams('rateTenor') = dataTenor{i};
        
        payLagNotionalMaturityFlag  = true;
%         payLagNotionalMaturityFlag  = false;
        
        crossCurrencyBasisSwapHelperParams('payLagNotionalMaturityFlag') = payLagNotionalMaturityFlag;
        
        crossCurrencyBasisSwapHelper = CrossCurrencyBasisSwapRFRHelper(crossCurrencyBasisSwapHelperParams);
        rateHelpers{end+1} = crossCurrencyBasisSwapHelper;
    end
end

% forward & discount zeroCurve
dummyCurve1 = ZeroCurve(MktData(containers.Map('asOfDate',valueDateH)));

yieldCurveParams = containers.Map({'rateHelpers','discountCurve'},{rateHelpers,dummyCurve1});

tic
yieldCurve = YieldCurveCC(yieldCurveParams);
out = yieldCurve.BootStrap();
% out = yieldCurve.GlobalCalib();
toc

outInfo = struct;
outInfo.impliedRate = zeros(length(yieldCurve.rateHelpers),1);

outInfo.diff = zeros(length(yieldCurve.rateHelpers),1);
outInfo.NPV = zeros(length(yieldCurve.rateHelpers),1);

for i=1:length(yieldCurve.rateHelpers)
    
    if strcmp(yieldCurve.rateHelpers{i}.rateType,'SwapPoint')
        outInfo.impliedRate(i) = yieldCurve.rateHelpers{i}.ImpliedQuote();
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    elseif strcmp(yieldCurve.rateHelpers{i}.rateType,'CrossCurrencyBasisSwapRFR')
        yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.domesticSpread = dataRate{i};
%         for j=1:length(yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.domesticLeg.couponRates)
%             if strcmp(class(yieldCurve.rateHelpers{i}.crossCurrencySwap.domesticLeg.couponRates{j}),'FixedRateCoupon')
%                 yieldCurve.rateHelpers{i}.crossCurrencySwap.crossCurrencyBasisSwap.domesticSpread = dataRate{i};
%             end
%         end
        
        swapOut = yieldCurve.rateHelpers{i}.crossCurrencyBasisSwap.Calculate(yieldCurve.rateHelpers{i}.discountCurve,yieldCurve.rateHelpers{i}.foreignDCurve);
        outInfo.NPV(i) = swapOut.NPV;
        outInfo.impliedRate(i) = swapOut.fairDomesticSpread;
        outInfo.diff(i) = dataRate{i} - outInfo.impliedRate(i);
    end
end
aaa =1;

% 
% %% import and construct a zeroCurve
% rawData = [0.002739726	0.999965207
% 0.25	0.996569895
% 0.5	0.992648282
% 0.75	0.988507257
% 1	0.984021343
% 2	0.965105185
% 3	0.945510165
% 4	0.925938679
% 5	0.906326256
% 7	0.867854044
% 10	0.809835931
% 12	0.773180526
% 15	0.723742908
% 20	0.649044875
% 25	0.582936139
% 30	0.523634856
% ];
% 
% for idx=1:size(rawData,1)
%     rawData(idx,2) = -log(rawData(idx,2))/rawData(idx,1);
% end
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,H_Date(valueDate),'KRW','KRWZero'});
% zeroCurveMktData = MktData(mktData);
% zeroCurveMktData.params('convention') = {};
% zeroCurve =  ZeroCurve(zeroCurveMktData);
% AllMktData =containers.Map('zeroCurve',zeroCurve);
% 
% %% import and construct a discountCurve
% 
% rawData = [0.0	0.0
% ];
% 
% mktData = containers.Map({'rawData','asOfDate','ccy','name'}, ...
%                         {rawData,valueDate,'KRW','discountZero'});
% discountCurveMktData = MktData(mktData);
% discountCurveMktData.params('convention') = {};
% discountCurve =  ZeroCurve(discountCurveMktData);
% AllMktData('discountCurve') = discountCurve;
% 
% freq = 4;
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% IborIndexParams= containers.Map({'tenor','currency','fixingDays','fixingCalendar','convention','dayCounter','endOfMonth','forwardCurve'}, ...
%                         {'3M','KRW',1,Calendar(),'ModifiedFollowing','Act365',true,dummyCurve});
% 
% KRW_Libor_3M = IborIndex(IborIndexParams);
% 
% depositRateHelperParams = containers.Map({'IborIndexParams'}, ...
%                         {IborIndexParams});
% 
% depositRateHelperParams('rate') = 0.03;
% depositRateHelperParams('rateTenor') = '1D';
% depositRateHelper_KRW_Libor_3M = DepositRateHelper(depositRateHelperParams);
% 
% %swapRateHelper
% % swapRateHeperParams =  containers.Map();
% % fixedLegParams = containers.Map();
% fixedLegTenor = '3M';
% fixedLegConvention = 'ModifiedFollowing';
% fixedLegDayCounter = 'Act365';
% 
% 
% 
% dummyCurve = ZeroCurve(MktData(containers.Map('asOfDate',H_Date('2018-02-17'))));
% 
% swapRateHelperParams = containers.Map({'fixedLegTenor','fixedLegConvention','fixedLegDayCounter','IborIndex','discountCurve'}, ...
%                         {fixedLegTenor,fixedLegConvention,fixedLegDayCounter,KRW_Libor_3M,dummyCurve});
%                     
%                     
% swapRateHelperParams('rate') = 0.03;
% swapRateHelperParams('rateTenor') = '6M';
% swapRateHelperParams_KRW_SwapRate_6M = SwapRateHelper(swapRateHelperParams);
% 
% aaa = 1.0;

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
